In [1]:
import pickle
import pandas as pd
import numpy as np
import glob
import os

import networkx as nx
from networkx.algorithms.community import k_clique_communities, louvain_communities

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

### prev_dict.pkl (data/upload-check folder)

In [2]:
with open(r'C:\Users\sarah\Downloads\prev_dict.pkl', 'rb') as pickle_file:
    prev_dict = pickle.load(pickle_file) 

In [3]:
prev_dict

{'Key': '453683_453687/output.csv',
 'LastModified': datetime.datetime(2022, 4, 8, 16, 2, 34, tzinfo=tzutc()),
 'ETag': '"782350954c470c8430be25575870c388-2"',
 'Size': 5670159,
 'StorageClass': 'STANDARD'}

### entities.json (data folder)

In [4]:
entities = pd.read_json(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Data\entities.json')
entities.head()

,id,name,category,user_score
0,1281,1000 Dreams Fund,Donations,NaN
1,637,100x your coins,Others,0.0
2,646,10x your altcoins,Others,0.0
3,1532,1517.0,Donations,NaN
4,1364,18eighteen,Others,NaN


In [5]:
entities.shape

(1683, 4)

In [6]:
entities['user_score'].value_counts(dropna=False)

NaN    1020
1.0     581
0.0      82
Name: user_score, dtype: int64

In [7]:
entities_fraud = entities[entities.user_score == 1.0]
entities_fraud['name'].unique()

array(['1xSlots', '21dumpstreet', '24kCasino', '777Coin', '7Bit',
       'A1 Quality Credit Cards', 'ABCC', 'Abode Desires', 'AccMarket',
       'AC-Team', 'Agartha Market', 'AidosMarket', 'Alibaba marketplace',
       'Alice with violence', 'Alpha', 'Alpha Cards', 'AlphaOmega',
       'Alterdice', 'Altilly', 'Amazon Cards', 'Amazon GC Store (One)',
       'Amazon GC Store (Three)', 'Amazon GC Store (Two)',
       'Amazon Gift Cards', 'Amazon Warriors', 'Angelpharm', 'Anonymix',
       'Apollon Market', 'Apple Place', 'Apples 4 Bitcoin', 'Apple Shop',
       'Apple Store', 'Apple World', 'A preloaded debit card', 'Argo',
       'Asean Market', 'Asia Transfer', 'AtoB Mixer', 'Auspride',
       'Avaris Market', 'Avior', 'Baby Bitch CP', 'Babylon',
       'Babylon Mixer', 'Bad channel', 'Bankor', 'Bao', 'BCGame',
       'Beautiful Girls', 'Besa Mafia', 'BestBuy Cards',
       'Best Financial Market', 'Best Financial Service',
       'Best pictures and videos', 'BetBTC', 'BetChain', 'Betco

In [8]:
entities_ntfraud = entities[entities.user_score == 0.0]
entities_ntfraud['name'].unique()

array(['100x your coins', '10x your altcoins', 'AAX', 'Alexhost',
       'Altcoin Trader', 'BCEX', 'BETTOR', 'Bibox', 'Binance', 'BitBay',
       'BitBns', 'Bitcoin Duplicator', 'Bitcoin is King',
       'Bitcoin Postage', 'Bitcoin Private Key Shop', 'BITEXBOOK',
       'Bitfront', 'BitHacker', 'BitHumb Global', 'Bitkub', 'BitMart',
       'Bitso', 'Bit Wallet Shop', 'Bit-Z', 'BKEX', 'Bleutrade',
       'BtcTurk | Pro', 'Cex', 'Chaoex', 'Coinbase', 'CoinBene',
       'coinbit', 'CoinDCX', 'Coindeal', 'Coineal', 'CoinEgg',
       'Coinfield', 'CoinFLEX', 'CoinMate', 'CryptalDash', 'Cryptology',
       'Cryptostorm', 'Dcoin', 'DigiFinex', 'Double your BTC', 'DragonEx',
       'DSX', 'Felixo', 'Gdac', 'Gemini', 'Graviex', 'Hanbitco', 'Hoo',
       'Huobi Korea', 'Indodax', 'itBit', 'Kraken', 'Kuna', 'LakeBTC',
       'LBank', 'Liquid', 'Max Maicon', 'Minerlock',
       'Multiply Bitcoin Times', 'MXC', 'Nisarsoft', 'OKEx', 'Ovex',
       'P2PB2B', 'Paypal Coins', 'Poloniex', 'Private Key S

From the literature review process a few entities related to phising schemes were listed. Check conducted to see if these entities are present in the provided list of entities.

In [9]:
entities_nan = entities[entities.user_score.isna()]
entities_nan = entities_nan['name'].unique().tolist()

for i in ['BitFog', 'BitLaunder', 'Helix']:
    if i in entities_nan:
        print(i + ' is not included in the list of fraudulent entities but part of the dataset.')
    else:
        print(i + ' entity is not part of the dataset.')

BitFog entity is not part of the dataset.
BitLaunder entity is not part of the dataset.
Helix entity is not part of the dataset.


In [10]:
del prev_dict, entities, entities_fraud, entities_ntfraud, entities_nan

### input_labels (data folder)

In [11]:
input_labels = '3G9FeknNxN5nfdRgxiBwuheAB59f9JMA7c”Œ"34X5Xa1KsDryqM4vTyKNSLwjCXAtoRbYUS”Œ"3NTm6uxwvqP2L5Rdze7DEaLeUGbuGPLHZw”Œ"3B2VSk3aTtLMZAtgm2SQzYbt1SDmvEEgAK”Œ"3BYWwQMhZGtkG5gVxCckpwUF9RBbgzof7q”Œ"3MLY2rX5EVUBp7CRJxgb3KkgmxzKNgSm8D”Œ"39rXjCd9AQd54WAjxKhWq322orU9n89dBE”Œ"3KVuVJ3TeaQ6bFzrehX1dhyCevvQopsPTo”Œ"34Vz1KtMZggtWc1QazxFcLeaSU68Zy3uQH”Œ"3FBB4vzfenLV3DMxC9yRgrUB2eHAodmXXc”Œ"3GqRZvhQQUxzeZ5FAcNs434FR3kAmbhLmd”Œ"3BFtHxDBQFb6A8nNdha1A82wmmeZyMnr29”Œ"33m73hFYG81sSGBv3ccE7QaVKEkJyWtkS2”Œ"3Nqifw78gH2W7Cd6QzgMvYLvDhtz68i3Xz”Œ"38SCKk1JpLKcfwLRrN7i8iRic2Ay8JHn8a”Œ"3GhxefsHL3hoPu21dTgFdiyom3TxfsUoaP”Œ"3M6XutkHHyNV6eoYtwdc6MnuXDFAc1NGoa”Œ"1ATf54CxQU9cNQVjg9RmypFVSbz62TU2rG”Œ"3FUzoFg2uTZEFuvgYqia6gFFaGZMSDrYa3”Œ"3HrY4iSs3n1aGthMwpAtF68WDRnmCPJSW2”Œ"3P7Vr2FqYV4YmhLVku1JWbQihWPNmXurPn”Œ"1E7oWnXRBo9gJdr15CktcRknuHCmj41qPT”Œ"38FHTK3S6Qx58PMpLRbwWQHx7ouvvweCVU”Œ"33QugePLaoM5YMy8zTGD2bvUzJ6XV62Hqm”Œ"3G3PewwadBa7EkmHCnfDZYaFfbghUDX9uU”Œ"3QDvdh5ixWpb7vVPESAPFW6L9iE1bdjhUS”Œ"3AMnU8EoU7GitQa426dgM2hSLwa2VR3dxp”Œ"3GP2wQCtKKDFxCSx739hdhCwprG26mLR3a”Œ"35zJx7bBKV7wYbsPLuYBxPjUcoegzmQEiv”Œ"3MUASNkoCsMEgeYV1R3X92CE9X9oJatp1m”Œ"3G1nNTTyrpZBwyEiG6U3bJvZdu9sW3rbUz”Œ"3MaEfBQBykdXhrdNJadhZ5PVExybC86NNB”Œ"3G4HTfadPmUpn6FeQPKFSNQ1fR4UvRwPYc”Œ"3NjLNtaGE7yNed9uHQw83X7rS7ccy1oJpk”Œ"17Gde8xfHoqmWJ9iZLmySEPJcQPYnPj3Ye”Œ"1GRDUyYdkgpNmXYUZDhifNZLqLDEdWwkKk”Œ"35nLaA2yEWU4qJapAEhk2Lxnq5nQmaoHwX”Œ"1BK6oaNb7hkym8evHGF8W7fEMmK1TJMCMW”Œ"3MuwdSGCFLseVPZkPTE8vLju6jbAE3kRof”Œ"39TxAhMr2Stfi2cFzuGnZ1uJ14DySUXxS4”Œ"3AQC3QMmVFRsvo4GaWnrvAqmnfYk5VVrZG”Œ"3HT12fZfq9DoeuiPdFa3GudZFAJYdpNBRk”Œ"3JAx6MmhUePfvREHXZW5HnAHXtCMKL9fQh”Œ"3DjLqZdACnDmeV8ysYbJEfiQuhuBmtQ8hN”Œ"1M7jtkTeRqWrq4rG3UbKiNGZtcg6voADSV”Œ"3CHFoUWtJ7aUsrWxmPDAprAV1WrUkFfBKY”Œ"39PGq3rHnRSWPAhVBZ74So5md4VyyNtfGG”Œ"38MhBApDXmAYDvYPqAEYZJhtxrJvNAPDJK”Œ"39w1CoQxhcQcN923s67GRPb9KPz9pT8ELd”Œ"3GZpSJBH4ADRokxTjAXQ8Kg3o5j5FFrURL”Œ"3HqVz57AUvfbXXeJjeEs5n7Nk4iufjtHC4”Œ"39sc7X3i5Hx4MWJ2Dk7fKRrz3KL6a6ZEH7”Œ"36q2dLtFub3Lbpu7gh1B9nYgb1PBaj6oh4”Œ"3DqN3oBsRkTWPcdB3cCrvmHYboJAzbQCy1”Œ"32PC9jqbo5ie31quV9CrMst4eFpbdQpgLF”Œ"3MpVjVzRcU75wPoVj7ZArDCQoVpAhDnqik”Œ"32PSg1HnzwEfV2gexhoMiUvNEzQCXs2EDQ”Œ"3KCaJVKKeycpn69dsWgb9R1gzsQm9oCeHU”Œ"3NFvkWDPEJeYX2gq9T4Gcqh6z8MCaNn1Xy”Œ"1MQ9Lwh1rxskjqyjydx7vwuTnrjNYTWwAy”Œ"3JkRTf7EkYXccuQ9vZTU3JgJqpuWN6fuY8”Œ"3AKk1SvSRA63JzhfQcCCWVJxgPaFaHBEmn”Œ"3EiHgNq5UddAvDWPVh2K5JCt2XDwHGfzuB”Œ"3LNoVmmdpzN9V7EbbeNTS4DktUv9Yvt4po”Œ"34gP8dVZVAsJFwjuBnivKbJH9rqPxctBza”Œ"3EXNzrpHvzw3PZt8SFZrNDhKCngJXfgjhY”Œ"1PhmMsdwamJA6soKw5mNMXxzGomHEHWY5P”Œ"3AFGyfTQn2hxsRY8Xocya8EgseDngNKEeN”Œ"3MGdD2r5Npfv8HDMKtoiSQxEHjiLiYmAxC”Œ"3Bb3xq97aDKFVCAk27PQJU6HnBHZBG9dNc”Œ"3K4MZAmif6JbiBdMtdHY3wMd5TMPdYExCP”Œ"3CabX11ryy61sGbREWrC8YY24n8gW8sitT”Œ"3Ks62y2fxGgjg4iJgWbVh7L1z1KReL7WDi”Œ"336tZgYw4BfwNEsRY2YqSx94vWEpFdNMQb”Œ"3NR3Xntn4bNe2wkvPv2XSVuPa9yieGAMFm”Œ"12wEcuChavEhifGCMTwrGbgEGnQEH2FxNL”Œ"3NsJwGYUPzJURcqpEhHH8GSzBntcgso53t”Œ"1J9SJYNVrnwhQkTXcSN82NkzumvZjmDiY7”Œ"33gnWbMhxdPg1rVCXSzkGYR4JoGfJHRnAB”Œ"39XG6s2PEx6BVGLESTXHx9PJXEim6pFw7j”Œ"34g35Qc3XaYKGsarD2yoLEwoMFTQnjBRRc”Œ"35PRszPaLMrUz8gMBcmZm47X3uziC1FUa4”Œ"37VDRHYH3NysMyN6WdZggHeAPz2zrTvnPp”Œ"3DjAt4g3WP73qS5uT6HCLHBbLEVmC8brEb”Œ"3HxwENApwctdUx6QbocrcW69oRaKeAGU7x”Œ"3PyKEyiH3tJ3NbvfTSCZJaa3JiyDpnpsfu”Œ"35U1pHdqFAE8nbzoCHLb29oKb8zYUEMrzY”Œ"35tCnb29m1S9NmHHAT7jWYec3ekwYrHNkz”Œ"35VRfGTkVExLmBcercgfHQJYRAa4QjYQCw”Œ"3Eg9PsUzgE9JC5PCij1hQKgg4VdCmkh8zY”Œ"1F3jCjSxSxJeF5k4cph7WvQ3kkeDTKRPjd”Œ"1NAqP3gTX9vNRYf5NywdDdbT8BZk1Bj45J”Œ"39FCUnwhuRqcnjyrfe8aKBjghADFFguEbx”Œ"3AHqbnAewmB59cpzjYN54YHKm7RasUnsN6”Œ"3DC1n6L2GNJonn28fisWNpGJAPYU7VsKLG”Œ"1HFRD8o64RdxZq3EhKYqoicoyopg16AwNR”Œ"3LYEworkgE7nD25CHRDDveYHtqQspeMZSm”Œ"1GKmCumeG8QxvneT7A2Pe4jKKEdyCQTo6w”Œ"34RTnERzXraruoD94TaJhFSTDLofx4TBv2”Œ"3JyCT3w2VkfNWdxiHmPHuwKqNySHYPvLFc”Œ"3MUxJkTjnFBJY93SaUaMU8YSEWm1YAuC7a”Œ"36PVY8PDB2CnDvgqbTASFvduq18gfQn2dn”Œ"3EbExhbQSLTab5S7u9emosJRpFQRucjANN”Œ"38STgd3FxSCaG2N2AU4791knQGFdRRZX8N”Œ"3DmwugmzAGaqEUsyNyzyYFf8MpskYn5j1d”Œ"32CYKh86iaWpRTcnCaYChhDHS3N8omVDav”Œ"3Ajk3dSCJf8Ltsx1wCJL3McZcuvvUro4BZ”Œ"3ML66ANqdW7J9bsQmGz3qdT3Gq6uWv3Fsi”Œ"36oXKJbg6YQaTRA7gTZY6UFkx9r1u3zX6i”Œ"34bjFkp4RKJTALzXeQqouLNffsQwZdYhFv”Œ"31i3U6tPNCy8XS14xAzchuMwAfnzGgv2Ny”Œ"3LTeZGibBbwweZ4otZSAxhFT3Pk2EcAoj6”Œ"3C2LQY3DLxVe6mqJaZ5u2NbGKziQeeucPF”Œ"3Dfo6VC7STeHXzc1gDjPrSKMsoyTVNksA3”Œ"3FGiB8aLopZsrQM9aN3Q7CRksghASdy6St”Œ"3CQkqZf9XkC3dLfz2xJ2JBTJyXGHSaR4Js”Œ"3At5ZvNLF9dij98ctastNv4ijnZ2PsETt5”Œ"3AuehfkrxfbgEPdGStekTFizVicbT3KG8d”Œ"3EvDFxjVEhhtxEUkDqAg9oJWt8Hr77BiDn”Œ"34k5WovXLhaYgrzpNZLjjSfC5jJai3nNHt”Œ"3DaUzKpPeaLNC95fXnjaQczoykJCsPqUyT”Œ"3E7yi56M5uhBshzfZZec5b2FbUVjCSFLcH”Œ"31xTgLfafceqMjMwp153fr6K2eFtXXJVDx”Œ"3NbkY1cNzJpMGmVx2sKQEUBzCGJnHc9bX6”Œ"3F3qWRBXkAWBBghq4kXaSkxHGS4wW2Gajk”Œ"3HY3y7pdF4ppo8SCtMrpro22J7gTpd3zj3”Œ"39bRnDbuZqauhNhnwXsPa6iM9njRVMdYaz”Œ"3QNwVRiFWZGUdN4orqtURcwiTem4BQywJx”Œ"3JAJBxoZYkZKaDLUnMMNureCQ7bgBREbzL”Œ"3A4ucRiA9Yc1ktcJ6ZZSSNmGuyfFMW15uc”Œ"37xbE14NNRp6hJWKE7TtNPYGthkSe5vsWw”Œ"32UbdkxXSZi5ArVDUoSAptGpDf8tYG1Ma6”Œ"38GPg26ifprR5TEPemdcjQHrSCzcGSAvhH”Œ"34ypzdP43TewwQ4z4vz6pktHNZFwQPdKgv”Œ"3HNVSjbYi8wySz9f1DwAxq9Aywk3srxW2q”Œ"38cKiKpXbkqGUcgWd2gKLkDfuUJmHMtasS”Œ"3GAqGycxEww1Mu4RnqkVBnuJjpG7sUGHBq”Œ"3KfQ7ZL2eCGpZL63q6C45CE1YidVfFQBbn”Œ"3AbvuJCr6V5HbFsRFUxocz8ef4y5HRmRFy”Œ"3MDaLcTnGmFeQeGFP5B8gTgyKVRbKXGzBE”Œ"3P1REYYgzdSeK5MmArq9v9poUseJT9URs7”Œ"3CZHXdnSUK5EwoUFf8A2pQnhzJf6vgAbVH”Œ"34CUMwbucC9cNYop2cocAbuR3R9YrFC71m”Œ"3QbqdsAefHg9495kYh1TMnipW9SqCGWsNa”Œ"15abPPunU5BvLGWrx98bxha7m79GtfDGDA”Œ"12VpNLvUEBZvHL8WzjRcvoZKJ4KPdbJHko”Œ"3Hmsg7VFDwKR4Mq2BZtQD5YpUHS2xdy7f6”Œ"3QrLsmUogMnoAHuu8bsRP5iVoVQ5peQ7Df”Œ"34ZEG66qaioy3U1nWFFwR2NUHm7d9VZNCj”Œ"3DGSgxRZkBrkzHyy7QWuX3aJ3kU5cddGJo”Œ"3AcrxsUbxjqBz9wrEb9koEMxzh3ntktg9z”Œ"38c32neMNTDF7DqaJZgpFwpxkfB6mcd48c”Œ"3F2Vc3eUQAf6Ycy5HEyugBbgYXS3ZBpm2A”Œ"3HKuZCGiuuho1BYXWEyWMFmZgdTAEXppiD”Œ"3EEStmCszvXc7wTPYoeFW9wH4meY3NUzYi”Œ"17PvBqJy5TCQ6NNceLPzaS5nYAUFriErmL”Œ"3PNoWdkPRLwW5aWdGm158NAYp3u8s8rqsd”Œ"3HqmSqxQpQQAEfUJspGfiCKgkyn8e5dyh8”Œ"17eK8iMpNwMCmUm7DUE2GqWtm1VupPGTCW”Œ"3BoywQWrznpdswB4rJ6Sgpt7LcUyGhQHGK”Œ"3MqLaey4LqVYW2bHWLw5Je13Z6P7HnH7rd”Œ"3JQaxMmPL7ku3PMM9PStmRNMa1TApfMEZw”Œ"16bqDKzqnV8j2wU755uhcv5Kb6ynQdoyoD”Œ"36xam61DjuijnphhU1vnNjtWvtAH6L99SA”Œ"3HzsFq9GwmP6FpUrNLwnpo3WcnZ8rst8pj”Œ"33KPMR2U4HFAjpo9J4csZCGeERyEktcrXT”Œ"39PwsQiNAGtsq9h4L2HB6UFFUtqBfWwods”Œ"3QuKtnwWGthrTQUD99ynMoFmRbHBPDwHAd”Œ"36x2RdXkTvHKiffupTWLs2LMd34qAVWhui”Œ!1amZAHJLqcJn9wivVpKSyVVw8pT71n51z”Œ"37UWWzTTgtDce2rQ8b19tXiFmaxRfeqiPA”Œ"352n4BVUEtgAyH1LiCEAfJufh51AdDhkAc”Œ"3Ck4swnsDrResb1xum1LqcpTJJNnvKH3Ha”Œ"34FQbVSkPSyYyHWaH3ApQY1FLf5iD3NZAP”Œ"3L8tmDAf7ptXUSeusrrfcDWEPNee5Lvg1U”Œ"33oCptiThwZ9CZ7WPd4r7CrEpvmK18Xaba”Œ"36EkZpxuuKbxrNeurkWpVS3iuZd6ZySREz”Œ"3F1ZG4AHbFgtfR944GrEY7TqcyzPaW3nTg”Œ"3NzCKvLKoSgdjdndfDKqS7TkrgBYiipry6”Œ"3AJKDHkwoS4CNcLCtcfKkpyp6nqPUVcbTX”Œ"3BwaQ4PmaiFyGzpEbhDErNRJRpkRNYqQyU”Œ"3Csqiwk7jADijqQQDUYTwRms2j1LxBuYuv'

In [12]:
input_labels = input_labels.replace('”Œ!', '”Œ"')
input_fraud = input_labels.split('”Œ"')
input_fraud

['3G9FeknNxN5nfdRgxiBwuheAB59f9JMA7c',
 '34X5Xa1KsDryqM4vTyKNSLwjCXAtoRbYUS',
 '3NTm6uxwvqP2L5Rdze7DEaLeUGbuGPLHZw',
 '3B2VSk3aTtLMZAtgm2SQzYbt1SDmvEEgAK',
 '3BYWwQMhZGtkG5gVxCckpwUF9RBbgzof7q',
 '3MLY2rX5EVUBp7CRJxgb3KkgmxzKNgSm8D',
 '39rXjCd9AQd54WAjxKhWq322orU9n89dBE',
 '3KVuVJ3TeaQ6bFzrehX1dhyCevvQopsPTo',
 '34Vz1KtMZggtWc1QazxFcLeaSU68Zy3uQH',
 '3FBB4vzfenLV3DMxC9yRgrUB2eHAodmXXc',
 '3GqRZvhQQUxzeZ5FAcNs434FR3kAmbhLmd',
 '3BFtHxDBQFb6A8nNdha1A82wmmeZyMnr29',
 '33m73hFYG81sSGBv3ccE7QaVKEkJyWtkS2',
 '3Nqifw78gH2W7Cd6QzgMvYLvDhtz68i3Xz',
 '38SCKk1JpLKcfwLRrN7i8iRic2Ay8JHn8a',
 '3GhxefsHL3hoPu21dTgFdiyom3TxfsUoaP',
 '3M6XutkHHyNV6eoYtwdc6MnuXDFAc1NGoa',
 '1ATf54CxQU9cNQVjg9RmypFVSbz62TU2rG',
 '3FUzoFg2uTZEFuvgYqia6gFFaGZMSDrYa3',
 '3HrY4iSs3n1aGthMwpAtF68WDRnmCPJSW2',
 '3P7Vr2FqYV4YmhLVku1JWbQihWPNmXurPn',
 '1E7oWnXRBo9gJdr15CktcRknuHCmj41qPT',
 '38FHTK3S6Qx58PMpLRbwWQHx7ouvvweCVU',
 '33QugePLaoM5YMy8zTGD2bvUzJ6XV62Hqm',
 '3G3PewwadBa7EkmHCnfDZYaFfbghUDX9uU',
 '3QDvdh5ixWpb7vVPESAPFW6

In [13]:
print('Total Number of fraudulent/high-risk input addresses : ' + str(len(input_fraud)))

Total Number of fraudulent/high-risk input addresses : 182


In [14]:
del input_labels

### output_labels (data folder)

In [15]:
output_labels = '34xLKAZwB5DXLrTi4BXksuKGL6psA4Z4jN”Œ"12KyRXPfPwDmRy4y1K1Tbw7ib6eMHxVBuQ”Œ"1D5sGvM9tRt9rx8CwSCcNAZtDFNhtYgRmZ”Œ!1dD47b63nBV9ABks6SQU1PC52BaVi9Lix”Œ"1PfWy4BCyRwxp8wYajbpbQC18szdKYCq9q”Œ"13DruKxiK6v4MGARkVKDFs7Wpj6u2WoAA5”Œ"1B98ZUhDg5h8GB8n3pNHb4p1w1r6YcCcQx”Œ"1ChjNHDxkT6tQeoxVbtHbtJr6bbACt5y2y”Œ"35vxmN1162cfnuab3DVhaMdmfa3jXL6njJ”Œ"18UMFna2bMzP1G8it5WiXpHzJVMrruXRib”Œ"18hNKMTQrUWzdtDs9MEp7FKp3Sfs66Qwhd”Œ"1PBmfaUui7ataQm1mh8E4pitrdoAMChWrM”Œ"1NrmFsmLzVN8YBW1ZWapYBPbLLYM7WvAQj”Œ!1b1y2ns1Ht4K9y4jA1c3L6T3sQuWY4JN5”Œ!1y6tuJiuYeRuFE4JbJXGPtnNmdqZAUDvt”Œ"1Ha9DmRDWznzrnBdURGFmyU2PzbNSihwb5”Œ"1E6DNBa1hVV58ffmf5a8a5R49g97J5eLsS”Œ"15sM2vWoMWW3UzTZm8mxLXKSjwXPnp8bZL”Œ"1GtvGLPasDq14xS4kTHfmVpEwMEHZcpWNz”Œ"3A9au92PaNf2t4Wrfsg6w3dEkNcxGrNq2X”Œ"3FUzoFg2uTZEFuvgYqia6gFFaGZMSDrYa3”Œ"1KFYAVFduGqLXcDx3zEyGnTxm71eLgR363”Œ"3BBdZ9Zjom17GTgmq8RX3f84P2Nf6L7SnV”Œ"1MP8AgRq3E1z7V3vemursb7Rsc3rTCbTbh”Œ"3MKaTNDSnbzqqYPEF6ebGQ6f9PnNLMAoQQ”Œ"1PQGnnmtseXfqBxagyTH7RKkEuXcovSpwp”Œ"1BzV9ueajYBwaNRUqRtFQ4bBFmHkkv3Kca”Œ"14AbgH75XZCLVYsNmWqvbupwuxLpprJTqB”Œ"39Ni8eKBTgbEuNLnL7RGoGRKqgYvXRVMyS”Œ"1Eu88VWpVFmUbaN9Qkf56ErZFFp4hJEieR”Œ"1E4gTnYcz1SBwXyhfz36PC6QaXGv8c26ra”Œ"3CRTUd9vmxKSmPxsVf2pzaNn3szWwh1bbJ”Œ"17gY7165k4mggE2muw9dxi5TGK4LHtYgPy”Œ"1D3EhMaXa42rvbHLkvHQQHADztS7iS3M2S”Œ"1DJvrX4wyU7QsJyghUfdz1VPSSADHA2FAY”Œ"1EjUG3brkvPr9eKKtiYYorsA8bUujMxqDw”Œ!1VTZCk3tu7MUF8kqTbpzAD7BC3reK7tQz”Œ"1DJsvbeaP8JDTiXQbgG8qBzGQMLFbZHH2D”Œ"1KSXs7qKDhEtLpFtr69MkDV9wpku2CKHoF”Œ"1M6PV4XU6gkdNySTwvzc8UQ6Z1gkhsfcGu”Œ"3AhZXXD6kAzcHd9qLNifsS5oUrML8QsBB6”Œ"1CxawqyxKH4WKUE4rxJVvnbEho7pxC7L5v”Œ"1BCSjCktatsf7DoAXFYtJHAkHPRdxkr66p”Œ"1FuHfT64u2AkGA5qptmK3tM39NTMHfnuPN”Œ"1GxzhNg7YTLkgaFqpq7KB3o2mxgT49APfC”Œ"1MbVu4c4XNfjnUxCSVdmbeyM5tq2WYrSuE”Œ"354VHna2LXzH6hrewRXn6RfJeG481Drb5Z”Œ"3R1DGVcG3w4gNsUQiL11SBEPCEtBFKDWAM”Œ"3A4HcUHE55TLG8bfDbYGGzk7EijfpwnB1F”Œ"1MkdVKJpMXmv5guTsiFN4f5QFC5NqSnqq9”Œ"1Ap394yYJKwYcYnd654aZcY4S8LYjmJpw4”Œ"1DxnbEy72STTZCuQDQtU1z5byKA1LjznM6”Œ"16n281vmMFzayGCc69Cc48fBeFJQazryTV”Œ"12ScRTiRsAKUDwaUkrPHqfe7JzgpDhx4xa”Œ"14jFoekqStRkroCLoRXbaCwsvCcBcHSwpA”Œ"1FacnHYhLWxACPRJQApPm58KMd3QdzoCNy”Œ"19JsgZCgav1CHRx5bZEPpFMnwkZznsQV5c”Œ"3JwFSRyUTc6kRDZZhuztzmEwas7xY8HLqv”Œ"1E5BVRsLe7H3fGjo7JxGBh5ghwn4L9nspZ”Œ"1CHk18nJo1QfGcQ52jnNkgw8NrwqeWeKpQ”Œ"1HH8aWivxzJCQbWtzZ4GRbeTQNyWuafnmr”Œ"19hEBMwdxfU4CRDY4Zvc9HycpPHm2CMqZv”Œ"1Fc8bbmYRELovkJ8LVWWZf69FezpbYAFps”Œ"1NMwWQzaEgoZ7jo3FYVDQeJQBeyWAYPT1d”Œ"38nqEKkDEEkjMyFPCnUyBBSat7adX2Fh4M”Œ"3LvS6r1mFvxjK7dc2zTfgL3Jq7hUxpjsmw”Œ"17wim7RH1mpSGgnXoEcF62neB4DTvnsgqK”Œ"3ANcVfDnqnMk2ieyRAnmCxA5Pgqn1ykk6y”Œ"3B1NGEPqJE9e3LWBQn49afaqM9zqRC1ihJ”Œ"1Kso4wG9tcP1b9HSY12wxfHxMbnJbNr27S”Œ"1PZ65TEcvPSv33Ni9mMRbaa9HUH4G3rRat”Œ"17gFeqhwMNYjqRHGLBP6iaKyGiK4nPepmD”Œ"1Fyoxnq1Mkz5A2pymt4CGYbZVudW8K1awC”Œ"18ytPj99PYaZ5tqkyBE2fSRtaYud9VuVGb”Œ"1N8qY1bEpGknVcr54oXEnXtNj381FjxuEY”Œ"3Hx6Bq5cA32nCZbCd8qm4A7g6wTbHNjbcu”Œ"3QxysE1KvvuZRAyT8j8TKvvdq89CNvWi5o”Œ"1K91ds8Mp9VqagVDxFJxJdGG4F8QzwUy1T”Œ"19HCLrwwd1Qca6B246SjEMxR8SAdME2vb7”Œ"1C5NqSxp4mVHMja8QjP9B1iGAWNJMYoSCt”Œ"1BohAuXoeGUfxe1oNK3M3WSYLCu2L6uBBq”Œ"1FBiDWWU8uifoNLD5g5G5z1qBb9Afeiihs”Œ"1PQQPHAybpwRBS89Cddjqr1vnsEQ7pJ9Wo”Œ"3DyyYNW8Sfw7vzjkhs9f7aDbk6d6CMPjoH”Œ"3Eeh23FizqeFTFZDyC6HpsjQpYv236agfR”Œ"12sPPRAgQnVfbYRjMkqsHB6oyopcbwRBJ6”Œ"3QbpjKX3j3bi2pq1HuXqG71mHJhHxMkwhM”Œ!1V56BNRns1raBZmPKvsKnQvRJGDyzYH5K”Œ"3MMDaCjVyrNf4ireENVVcvic1bb8pmHjPq”Œ"3QrxzQ9gkvDcRFfHjRHBNSSB2n2dLLVJwb”Œ"1Kxb8bxM2xo3Tz9uHmDrGboWgeNiBNVCF8”Œ"1G9wEgXip8W1YauFpjGBtzYuYeVsJsDUaE”Œ"15rcjDdMfJvAF3FQ9FdbPmwd9hRdtoCVih”Œ"1JeagRyjZwSdDX2u5SfquLPdaudsA9BM7V”Œ"1CYkLfNKJu3x4BaLBgZgNWLoP4WiBfDrkk”Œ"1GzLAKFkhhb5wQqmKZcBPKm67iwC3Lv1uG”Œ"18gv1oiCuVJRjQMHBLgz6zRuKXn5qa8ioL”Œ"1MKvi5WUmcRb9JZpasgsuvkC5XC3YjQHkJ”Œ"1MkNTmheBstiUGEdBdZ5FHQc1x4D5SxaE8”Œ"19xipZVf6MtmoRyZavdgER6YSB4119C6TW”Œ"13EAUE4XuJUBgNvs5y3sDNDiTMzUAfyGXz”Œ"3HaKhk1mEezkLE9bmQxAf8GTDDYuCiK5aM”Œ"1H4Je3EbkPUwVQKy695BMTfwuVRUWf5t1Q”Œ"3Jxsnme1Qr5TG61q1CFUHtN5k7ALQxhry8”Œ"1FKLE4V3dYwFeKX9vvDhJN2hTYBqR7BNqH”Œ"3QCLx4qptF8ojr2Rs1RWqCpaLmBn4FdBjb”Œ"1Mj4pJRfWXQbt5zi3h1KfmW1rkuY1j6N1S”Œ"18rdKmjrg1EawxgiVT3ikLExj6GWS2MNCk”Œ!1RLKmqAcsbzRrYGkLrFEFPr1QFJUsGx2k”Œ"3CcL1548tJJzPhoBdn6zoSBATgQWUgJnth”Œ"1Kx1ZjnLexyb6P1Z3a4H3FYuhrLei5Cvfw”Œ!1Zhj6zWQHfnjcydz48VdgSqUDA4kG5GFQ”Œ"15iTpErzGsD7wvJWKn91d3VjwbRBHjrCbo”Œ"1JKNWj6RWJPiWh96ag1wjJJYwYjRShBxh8”Œ"17hPstr1mGv2ghVKuvMfCEgJDADEq8m7x1”Œ"1GuznyFWSHKQvCcVhnZDRLTJSx8Ep4SLZ5”Œ"32ozWyAotP8AqR4is7TkUxxaNhQ71MvSPL”Œ"15vEqV466SzoweQnU9bde87ch1GQFqA6z5”Œ"3BCPkZhk3MHPHU7SM8f3y78wAsmQjzS1BH”Œ"3QEhuTwUQ84YrtPQhXK77p6vqzWP9vrufC”Œ"3JhEsGcP5WEzxwAjHCaSy3fJgLrze4Sdmh”Œ"16MogchUCgd9g3Dz4DtNyCtGLnuY1bjQzx”Œ"3DztqkQ4dnWs3ryTPxdcam9GdAWBMgVkC8”Œ"1DJFx7QZabbQQcYMvDB6K9La4zKQPj3TxT”Œ"3G4gHK6w6iYFTT65pFBLDhcNnfrgB9bkqo”Œ"1Ei4ns8dxXnxZZQyDKdnpVK3Fzrh2iUvJ7”Œ"1JGrifgA29nM5Ud4g4GK98J5uPbJhYjyQt”Œ"1G5HfF4tQoTsedM4P5cdDkmNBca9V5Mieu”Œ"1Lte7sQWSkUpJdwh5wEePCmLimV8QEPRJh”Œ"1EytW7911rQZXm1Ad7ZJUELDE53x8gCEch”Œ"13BvZG77hNwKRxjWdNG7VBgmXotWVK83hK”Œ"19iJuQjhpHno1BSMu5ZpMe9B5nMogaKmrd”Œ"1BTxCFa6KzxgaPGv2kwKRyCBG84rjReBWQ”Œ"17kDVxDMQwo1E1vjBSGjD4MCqa8czMqCkY”Œ"1F2Wdwka8car4oUpS2FqfSuktasbp4gLNh”Œ"1N2w1HXKyz3UMHVDyUxHSNngqdHTkn5ZGh”Œ"13qPLVbd1Gf5L8vBKpfBPfaz4aq5MgEdrm”Œ"1DExc5dA4GZs94GJ9t2VT8zH7C2oSHipVU”Œ"13bqmwC2B5u5EprQn48rkYZawLjQHDofwL”Œ"14tEa36Vk234mEko9x72GjSYU9Ex6AvEMg”Œ"17GvXVzu6pMgig58eRzfSPKvC3VBAHk97y”Œ"1D1qdd984jc4195ekj1K6mazMkKjQFtjte”Œ"1ANYM5MWeM7SQPmyw6gA8vczXyVVKbUx98”Œ"337uW841K9rEqCQ8jWWg4ck4ZYopsivF4W”Œ"1J31JggCrik1JAbujJ58zL5ueQZVQZ7qn8”Œ"139ijCsTEFMhoTPP7v13DF8vqh5NMYVvKe”Œ"15RfvxSAQ5KxRj4V4jtoheCdnmyfSEoXyo”Œ"35ARmU1APvRmiZ1f3tYLVsHNNjPhnvAMaG”Œ"13rHMH88e22rF6NwSLtQxCZsYHxf2WW9bq”Œ"3ActdHhsdZbvjiASRW7Gjw1QG1csYKZg2C”Œ"1BhfN2NDd8kQKABJu2hwZXZJMntEsRArPD”Œ"14qmBPpLtuLLLuWM5ovwNRFJeWWGmjvCN6”Œ"1EvqJ6NUZ1fpSoyiJGSTG3Hzd3irDoHwtK”Œ"3PsVcnYm54Ss8N1w5HcCkzYnvkzTUoGayw”Œ"39jvZ1AHNXUiGx3urf87jm46RP3Y9r8fjQ”Œ"1JJNtpTB7xk4TJ1GDrzp1jvDW9P9fT1tXD”Œ"1BVmw1FQHuMoH4dvmxJ2RymyswLz3nNBUp”Œ"1G4zYpKAXX3wjjpCy9meiv1dy871rbtjJH”Œ"1KTXFP5ZrC5U3jxXZ9o73vpxyRy2We8p81”Œ"369STxceWJJhW4pHX8QdEkapLcBeVp6JBH”Œ"18T1TgymHJmX1TSi1GWSQaaEGo7QdsxTuR”Œ"1A4ULFRea9QDL64UWb1Ui9usio7KSJM32X”Œ"1Jyncr5pFtF3LxL3paG2iF91EUziYXTeFF”Œ"3Nh69ViUVyoybaDG5yPPqX2EH38fLZnTiJ”Œ"152APvWSMX1PvmaN4Ajijn8wjQEigTu2zH”Œ"3BMEX8wPEEwd13UFcUcHupHaEXYnpBpD3R”Œ"1Kt1WmU1HYX78LR8bJ2YYrUWpMEAaFhZTG”Œ"1E47y5LWaKXfnMfe7h9gLrK1o6iFq8k9sy”Œ"1HQkMKrnyAwMw3TqSGkvB9t9zhFBZ8kSRo”Œ"1Dd7MTySsXaACLLwECpgWBDCxtMhXSQ1MR”Œ"31sxpE7sK94ySzbvWUTcWNF56eEVja4CHC”Œ"1D64Q6Z83ZVNDQLmQGX9pKB2hrBMZy2wSK”Œ"37Ti3h21GDdAma4ymhPsttCWqaDsWFpLNG”Œ"3EgU198V4BWAmxxKwUDVJ7aKtPZmAFcD8U”Œ"15bfcTxetY1JK2y6AdJRrVT7XTLMxDsjRp”Œ"1FgK7DD8Zubu1mYtigz96mZFRyPDUZ57h7”Œ"1HHa9L9Umyi1pUVmssMHwzqUX41vLfoh9P”Œ"3M7W7vjTjMjNTa1X38yHR1aseGXMo2Ukbe”Œ"3QH9hbkP5chABngdFDq8u44zbXP1XVu8N2”Œ"1L1grYhGPNLB5tyt9PSjh257CBk33vduME”Œ"1NNFJMd26bE2vyiGFdRcQrEQJxKFCMAGo5”Œ"1EVGxk2DgzHzGRhCBkroHHwUgQ8B8FTWbe”Œ"1NpHfyzAM71GEytsBou18FgUxFa4uyGuPq”Œ"3NgznW9s8HDcLgvnskxYQ23Y6dJ372rtaY”Œ"33VVJZ1dUEZF5zR7VZStNedvfiha63acK7”Œ"16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ”Œ"142kwByVvq49jerTpPd6cGffg5SN5SmSD4”Œ"1HtRBT7K8PqA77VqZEZ8EEVGbAnyUy1HSy”Œ"38Ys4skEZjSxm8LjEnQSx7rt1p5mVFNxdC”Œ"3PM3HtNBb5u9Sr9oXDZNZ2enjzJa6NYJXt”Œ!1cabfMQWbgAdVS4igfDv8HvEMrnRa9uz1”Œ"1KvoiqGVPGv5a2qGGqAeFfsqtLfxPMWts5”Œ"1PqyLtsKoYRHJMvsQ3Pem6Wd9iuNyWKnko”Œ"1F2G3y7cJ9jYy2W6gHFE79wACk9N8paiyj”Œ"1FarqfMtm7wLJUCRyHuRiHuxf6RXf8mrE3”Œ"3JcwnwsC87iKjyVmPEhXYHxcrmyMoTm6UF”Œ"3KtfBUdjLQzmBebE7t9rHVmbLu8aoNL2HN”Œ"1ADzBd1QZjAXCeKX2sPvsmh8YGcHQ5TJcR”Œ"1AU3v4L3bNWh9aETxf4VCWkFkfXg58L5u3”Œ"1L2dR3VqZN7Z5crCSXqGsyX3rkSxsbnhFS”Œ"1F48q2tLcoukhDRG96sB1cn6M4vxgzcbZb”Œ"1GAVTztrAyG79GgQCa5hU1isD9v6cZo3KL”Œ"1MJh4ZTZK99k9F6Jw1qqADaG5BMw4CJQ75”Œ"1PeKz1j9RnBJAMwXGhkHFTCon2W22j6qw5”Œ"32NiEd57YKhT8AsU4ttnPvHAJsJygZpYYg”Œ"1NEVoibTF1Kes9UUE9QbPS1oJz4Y3w4Nyg”Œ"1P6TCoFfh9Wus4JTPmWMivLCohvP55Qq7U”Œ"1451GCR7uHtT14M15H3GxJ2F4wDDrm9c9Z”Œ"1EToWoUkMoWwwCj6XwUL5r1LpqoBSDYzjT”Œ"1PbAJwE4yrtfqMjdN2RKWynkA2fFNRvJCP”Œ"34pu4VLF3CWViikBBbY9gyq9ej8KP4hgUU”Œ"38DPyo8BK1NzsfhcMGeykTNJjqDQ8sG2w7”Œ"1G7YU3bDuhrWpNpY9Nc3v9afQvAk19Yg9K”Œ"159LJ3fHdrckZpg8J53CmkJJ5ZbA7Q5nQz”Œ"1CCoepQFZhTYBAPhPqdqphzw2Kq36amjHL”Œ"34PfyMjWNXCF7RAWToRAfYWZ4WpALsa6TK”Œ"39ZH8bhgckunSNUPYLxwMua6MXZ9wPXN3h”Œ"1BZ43MED2XuTy4EkC24MM8s814p7knfotN”Œ"3MWcdpoqQ8JMrRaJ5anW1VXdhMoB4N2WN3”Œ"1DXYzcAfZwrcr4Yptjn3kxno9bUQVUm3Bk”Œ"1GffWnauwq6QUTgFPd34eK9v4v1967WobT”Œ"1PjYBcEWbFHDyZu47EEE4wRvjJv8WzYcAD”Œ"3MHZUnHKaCEmrB88KRsJJesSDA3Bd9rUpY”Œ"16Dbn3nRNmre6f1b3X2LS3csdrFYVnbwzt”Œ"14Xz3HZVNkHgan6NESTRqZFqtF9Vj5Fgem”Œ"32fKxwL36ZmGXJjTa5pCzQXD9T9LhBXEMC”Œ"1GoX9itEyvWi8aGnFsbYK7gm6FDSxXoFoq”Œ"32frVocqd7bbAJJiCk3nG1HQwBu3zDJhcf”Œ"149Ehbfy6FH2uD95PEGZTxm1WUE9qXDgvY”Œ"1DZx1J18hMa9odEf9damcfp39oKTaY8JkM”Œ"18YU18r5nmSLTUX8e46GPcC3AWArjHzbty”Œ"16yLzcugYoHce8NNU8ozBdfCj4wvamsUqv”Œ"1A1osKjokr6DXKHvm9ydi93vBCaJfywM7L”Œ"1HCgNTr9hUZ8Xbwg9jQUsCcj7BJnYAgQXW”Œ"1HtRzbTWMZUdZg4K64upPzRukF7meGKAuL”Œ"1CXqzNdxxhD3UzDxHQj3y9rMnJMieK24nq”Œ"16KoZXSM8pDkfXeKE3VT87Up4jV2ribcqf”Œ"13F5fZBPmW3wXv22dWmCKq47KVzMqdjxeh”Œ"1JQRjJBPdfsuWKA6cV9LZDuyzJHmnUXtsg”Œ"32my6L8hCr8uvKut9CMt3BLAoH2mJnFSZq”Œ"35MzJR7cRCVp1uH1LCUoqkjGzHVMUBSKHd”Œ"36LnAwCi6qxmdypm1cHzEk6fsBNjiBFq9g”Œ"3DE52MTFJ2gebY4urqw63H3bcSCr3tpGgx”Œ"3A9LQvFAsy8oyEd412ePje6teWuJqSFaZm”Œ"19C8oZU566a1cX6T5zdnkFSF44nCTkvrLG”Œ"1BuWj8Lp7PKViaWMXAKrw5WkGobpWaMnW6”Œ"1K6462LiWYVUXwfQzjAV7RfbEc4x83yjxV”Œ"1AJR1h6j4ct186EbXjZ5WFn76bPbuc7gS9”Œ"18q7CY5JKPQTnxFqvFSjMfgMXePPHPKzxY”Œ"1D4Z18jD58eMxbzLmWTU4jePhCWJeSUQJ2”Œ"1FqCYJPzJYos1cZgVqS3N6mrZJRY3zQeba”Œ"15GNpu3ArDf44jLQZhQwqcgu6zsJT4KGkX”Œ"1Ez9kHGc2dPENBwVpX6sDg4QJkiXez9BbQ”Œ"1NZEXGmz52Y5CqcvvWz4uQ8YQZRDDFL6nK”Œ"1BDkA7KNZU8zWMdCnY3hLEPDpCuEkvyLYj”Œ"1GFP24QmWTNtZ4F7Kn2b62uHKimxwxdNVc”Œ"3G1pswa4X9gbE41GFM3XNskXgPcUJGdu79”Œ"3HjuLCLXvoaU5CwGFeGy8S94dveSr8KSof”Œ"1JD3trhn3dYkw3U811idphGMeUAHa9NshN”Œ"3A4es97qPi7eU82pQBCCnz5zbJKmhTEqjE”Œ"3JDk7SN4zDEWWZev6DPAHw6WRHiEpx5fp9”Œ"1NeJfG4WT3dv9GkD1C8WyiGcLgxcAebS55”Œ"15SEfXs2ypSEDwuZQjg1xZem1Zu49SXpht”Œ"13qktUXHibfsX5kLYZ8erJyAvJW5bb9R2r”Œ"33EDWqnjekCkNoqcQU1guMQoNUDaJxahvd”Œ"1F49DKDKGdzGmjznFavN5wfXdYsVrSFScy”Œ"12JZQUmXXSjAPnDHpP9tShvJma6Lq4Lany”Œ"1D6pyzBVn5Kykh3VJxS6y6Q6m8v8Lx7qGm”Œ"1Ge4SuC6awxWmyq1Tu2FmjSDnhDKcSjF3V”Œ"396ePHDz9qsGYwC9rktTutMnYWvorToLW2”Œ"3PipxagTL6fsTAnnD1dQYuaRP78N6yUrw3”Œ"1ARBGfiMMgDq4K83AZsx4dtEkSntn3LyE8”Œ"17LeeoDqgEv4troq6B3kt4Jw8EppX8TER3”Œ"3H4YjZDCxjP85T7qpEf6jMqQ2Vn2dfFcXj”Œ"1BADek9RtPsaAZUN76K7gWmcY4ysNjnxDC”Œ"13v4NfW3kycEhSoFWg6d3UyDsSKyLQ8kpC”Œ"1QFZcJ76i7389ocDQHVjuHn4h5kATM9jUJ”Œ"1MHPKonSRh6XrHcoxQBPj2Mren9f8G67uX”Œ"1FvStTeKLUA1UPcjRE6fW7NVUYRB4peMNy”Œ"1755m5pF5fy2whKAkzMagBSzkJrASjTTBU”Œ"17hbCWk8eYgqcr6SXzGcppS1FXkpPMyqFf”Œ"19S8eSEPLHoNHyTweVa5NFyt6i9fKmXDTF”Œ"1HikASJhXb4nX9JCdkbvbTG85gTFbgQ3U4”Œ"1MqhkViqFjcSDF6vhovMdPaCas1oyRTEED”Œ"1Lw65XDbsQMJsCXsLmmwXmAbLYR9o4VCKS”Œ"1EhJ8WtvXfhBQU7gGs8FPi4uyepr21eujd”Œ"17Hixa65rrJ3AF2sZpwjXUmGdMnP8TBUnk”Œ"14oHnMj32VJ4ycPE4SK1QbyLhxjizM3toQ”Œ"1GCjJzUTtePvx5nE3fm9sMJtFkbH3Kfb9U”Œ"187q5chDrtQbfUqXH5sWbargAAgux1BGpi”Œ"1CwQPSfLEVCXABMSj44U2XqJT8Hooh8P2X”Œ"1HQcWQqJ1hYJRnVVUKiZTNJpzh3XBD3J8X”Œ"1MrsRE9vf9aNTQVVMvPqY6mBYTiR29ynxs”Œ"33GnjZdTTMdzzY56ExKc7Y1jf7zqyEojhb”Œ"1AW91Ygxz9HAmY7ystgwDT2hnTDF52NWvD”Œ"19R1siUJsRq6gGWfzHeGMVRdRCtZAd8qPK”Œ"1Q1eBcGkSfDEuNJzDfbB6ryBMTxRnr2KnE”Œ!1QtFUhYPuUeRLzpJaPuBACjAdi2djwnX4”Œ"1GkJJbS2K4CgwxW3w5wpe8Vk8nPYngQDmF”Œ"1CeYwzz14eStTqL6buX4yrniBWRy2VVxpU”Œ"1Lfgtcsgb3CqVDd2mdinmWCHHZnCteyssj”Œ"17UM3VRrdoDfahNARPXe17GyTMLsAEin8h”Œ"1E3UgiZ2Je2wZvMxMiwfrzBJFJXP38f6qr”Œ"13rHUzsY74sjGfNzyTHEP4mV2UGkdue5sP”Œ"3Nw96PpfFoiM863pEvWXyKpee9TFdCEV5G”Œ"1MzjQgF9NRGkWuXiGP1oAihr7yVMBa1fG5”Œ"1FQXZibcmKQ2P3mfRR9ePji9KVo7cL7SBw”Œ"1C7aqD6J34ZujNWqrS1yrPcnvpJri1VG2u”Œ"15gCabm4SS39165uB5UWV1JfHsBd5gsSRB”Œ"3CL3koLC3zzyQnTMdLHHsQCRJCBbwGBusx”Œ"13MbEGVegxx2kcgXqEGHNQHjMn42NqvDja”Œ"1JofRtDK3m6PBi879yV8kmUoyQ6gw4nd4r”Œ"1A1M5sXqSRbkaNiR4VxTNstt217wZVtsYk”Œ"1ALUcZGNe6zcGZ3rdHo11EZWZ683DVZGDY”Œ"1M6kVYqaufwTKHj2ALw9XcKCj2GWMyU5vS”Œ"1JR9RGTyicEXM4JduLmmbDNM8FkEF8t22P”Œ"1F7MsxTD3ybH5DvJkRDp8LTFRRoWnhsD8Y”Œ"1JMmtAQpevPAzaBBkQ2wpNVxm9MTsYvT7r”Œ"17A16QmavnUfCW11DAApiJxp7ARnxN5pGX”Œ"3D5W2cbnBqH7MYsxKZD23VBAkKEbFw7F7o”Œ"1AGukT9GJLSarovf3KDTgXLcELgu84RdLj”Œ"1CG6yQJt4hh6Qqp3zYsdW6NZomRR2hztUM”Œ"3EuybJp9uZERQ7fUPpv5sAaD6zQEYu2RNy”Œ"13HoiVJBj7XFpqzr7TLWaHM6tar8h2THXg”Œ"3344hSdR2qFZcPcqNTe47MCJATG89NKsHt”Œ"1GzmqGDxmRCMjzKE1M3tuU5GtwLCgY88a2”Œ"15HE8yMgdh26eFRfsWAJ1LgQCDQ9qgGA7Z”Œ"18bFo2FnSSw5yE6dt1UbmMMsztprvprsBh”Œ"16BTEvu8awBeBGDKWwSev2QmQkaeSJ8ukZ”Œ"17zP7LFyTWgnjZym5euXqWwVyxYfG1SPdr”Œ"1EDewqUCWrm1vJkseBokUPjSvGYKtHVuzA”Œ"1B3VHuHiCEoKJMovSVpeZ4y8iiPhf6gPFH”Œ"1HnUg3T55qBujyWhq2oRHVJhHsxPTq4BH7”Œ!1CU5ha6Rhy1e4rsctsJa5GyBsVqhfwYVV”Œ"1N4q1AF4gzeMX5tzQsd7p8BNj21fYMcUKK”Œ"1NSoD1bYpRs5oFPuLPJdcWQx7kFgems7HM”Œ"3BpHPH7Mcy4V2JHgZKQF1mc2SnCMcX5fQE”Œ"1NSzvEdvNMHsZZuFsRD4gQymByKWUPNNBB”Œ"3P9Se3foAXbPrQ1FYtfEUTnujLyBjLd1VB”Œ"3HMziFUapkU15nztASQi4C7DRrBMepbZ41”Œ"1BjVU2ckvWPebQmoBuyYbgEWMus5qtAniF”Œ"1KAYMnAdC5t8cE7atW2j1fA42Y3JWvYVAW”Œ"12jypgHRduLVAa5MdQkkDYyaPXDBvAG6Kg”Œ"1E5x89KKS7dWWxBCufxBJg6DKm51TeoF69”Œ"1Cn5VpcdDpwxprdD7VJoZCpXzDnGBJ2TMP”Œ"389Z9WskNXTPJqAsjQ8FBxqCkZ4bGB937z”Œ"3KbDCdkT91pfMoj5wuA3e53AkczAHKGhVP”Œ"1EaQuaDE1hw8iak7jP3EKHattGs7Sk5mrr”Œ"3PqmvyWdeDnBVyCzzjnRfKQUbPFFKcsE5f”Œ"1FVjpt1Qq2s8KC8d5E6vyRr9aWdh71qKM7”Œ"35tBxaHToZSfGNBJGEo5sXCyJb1QGW3mxp”Œ"1HgcjDJEwe6KBTnEoXURL6UNscdpw9Vehm”Œ"3JXSNkk6yY9Ueoq4Er8BSHrB7h74yugQpG”Œ"1M9QvqUhB8ZuV5cxKfiqPZManADos4hMdk”Œ"35cP6rxAekMPAS2LH9oFNbiq16niyDH8ci”Œ"1BHugMP8L1ysTNWAph7FyLoynSM7HUMzDP”Œ"1P6QkKAbLrEY2PxhHD6dzbHYw5B56ffQDc”Œ"3BJx5pAFmEa5KkZY9pGqVS4HxBVyqNAME8”Œ"12hM9F5HSg1a4U9toXZ89oUbgDZgUdNew7”Œ"15P4uTht7CerSHdAvTCFxkVe3T7davES1V”Œ"18KtzGxSJtrgDfx9QmQhY7VTTDsFpHgPqG”Œ"1H2q6R5CAJeqLssoVazDZht2om3LnBy78Q”Œ"19zPtAtqCBo5DoLCZKpV3Zami499sYaheT”Œ"1JFHaWGQx8HY9RUD3Aspz7XznnakAdjk7C”Œ"3HxdXaqcy8SSGXc184zbgaq7k3qBQparVU”Œ"1MLkh6d1BAS6HGoVCz4cduHtBbNwmVzwqK”Œ"329fKp3j1erNwaQJ7TywQHzNjXCWDfJ68Y”Œ"1CYit3BeNddmEec7LL6ktVwwzBMYvxFvy5”Œ"37YZGEQsTBjUSyQf86if9wRp3sAPg672E8”Œ"1DgDvmFnm62fufsqFBU8e5qfnBazqQXwGq”Œ"3C82wy6AD6Rw8mPTfMR1A9Cq2ywjNpcVFu”Œ"3CfyWbiuP5Ax1msUUCsWxjew1cYiFPLYun”Œ"1LzsWqrGQyjQS9LABvChj12bzUvz97t5ZD”Œ!1WfhmakzPdTookGWJprub8g5FWfyeuZdE”Œ"3JQ8eVt97pn8XbzoBJwir441Degm8PBBzd”Œ"1FiqxXmGKYb7wu4tvyu1wSSHhnHGeZr8xj”Œ"13KHqUuzoj2qjesiwH3Bpo719nYRCCgSQL”Œ"1N9BgF4C2RD7BJ6BcCfeWrFfHWeGZ5umon”Œ"12oNWD7KRnAZLh42VNkgN5EBHT9cqfoqNW”Œ"16y9K6DYveBBYesaHzjFpTr6fH19fgnUHF”Œ"1PhsBsutdiQMbAf5cUNdgtcTKEMy6zYsqd”Œ"3J5gMDXZtfVR6Wqni9mhaaX8k4oy35LBpr”Œ"1Af7wBWyXCY2xFS4UHSZTVqw3i7vZz8qVa”Œ"1DGev55WdwXEbSN7o62wVaiNjKBnn2iGFR”Œ"1Ggggm8hcuVGkT4jwSTkj1tdPZUZe94Mcp”Œ"1LhQJsTM1dzwmWrT6v1JAWUb72MpNLugha”Œ"3GQryd2AwFVyYdAZpgaeLb4aTV5MM7zvQk”Œ"1D57ubwU6w3LMquha8merygsiTUmpcpTvq”Œ"13VZoaTf3swXRwDUxVneKGkcLusqG8H762”Œ"13fk2JFYbkVymLCJJ9YXH3xogFELk5YY7B”Œ"1Konj5Z2kV3DkQpuQ1jF8eTycfTpqCXP4R”Œ"15W7PqWhsdKCPMkCbpT6a9yuCLKMraPBSw”Œ"1BTEPNRocUBw968Sri6souCpLFExszCeGe”Œ"1D17JmZZJwoMRdXieRXV4oKmc96vPwjk7v”Œ"394uqi1meuoM3yKwswWBbZCezeGnTq8Xt7”Œ"12riCdnATSWPu6o1yWhRiEqLt7HpAnHKqt”Œ"19YJw9UE9vvq851Cwaz1v3JAG98rckJw5g”Œ"1Ax3NnVwkP17XXtAQGi8joWo2BKBmYXCUn”Œ"3LBiyk5bMjmY8dZXpaTgRABtpkiwbkim8x”Œ"16iV82b8rkM7HQXHhabLZYybmwgR4G6m5w”Œ"1FcDtxaAuuw1QtCzJnAHsUdDWpf4odPT1n”Œ"1MqMuZMxjrT8JLXHNf4qVY62LYFaVYcr2G”Œ"3GHe2CzYi1f5YZYjBj6oCNDFbttuKxxz6r”Œ"3My952NAJXCEowPMPDiRtGQnYdavbtKTws”Œ!1RBSyj5XHQvXc73LdxExLmJ8Vd38KDKYS”Œ"3HEKNJEvwL6aMjGYekogqjPFYuJtV3PHyK”Œ"1MZo18CFhnmtroGifMh8qYVjQvT9MgTJ6C”Œ"16c3njmbb6tQZmrNraxPTLbTbErGorNpWb”Œ"1JxuLQU2Dng88AurCAn6M1YpQgMfawAzcq”Œ"3C8mTPb13sws6wjWxRJNZ3UhKeNbBqGdKr”Œ"13u28cNdxavLHdRoBoZgLJ2cEHcuGMNA4D”Œ"1DDZsozivDKsGzLY6ejN7rwnuZVBB3Vkte”Œ"1LM7XH89P7kfAmfYGaDkqqE47tmJr6xL5P”Œ"32nuywsd63RXT3CnwqQkbfPyVhpKtJ3A7S”Œ"15dEEnhMhcXKQWefAVSuNfsR1tgbV9SB3A”Œ"13d3mf73RJfamJU7GNjnU9vSNXZqDDgh54”Œ"1D4f4zjHZsz2Av6oy2th3Hyai2jbmmq4XS”Œ"3LfgpnJmhsoiEZNt2TsNwFervXAsrCub8M”Œ"3NJnU6KWYQ6j5sFn79AS6PjtMWyYoHh1aD”Œ"3Axt8aXzL3mSfYmtm7rHVBHpUk5bDhZZAm”Œ"1Po1oWkD2LmodfkBYiAktwh76vkF93LKnh”Œ"14skfrfZjTu2WPatTuEG14KMNDpUVzZBr9”Œ"1AKPEiZRki7XYbojq9jU2RbFro2gpohCa9”Œ"12aDZTkjC9abw1UP84bhc9wtJcvcaYC7MS”Œ"18HmaEUEFWrxZcxToRWTg1dNoms2X6Qves”Œ"1GXgMj7X9f83JQQGaHwvkLNisNesUgLqvA”Œ"195tQKcN9jPbPj15r5FuzABgshF1kK3oKQ”Œ"131xe89dHFgZ5ZRSwHexGEPrSubnVGsCrY”Œ"3LPvmLAvetdr9kiciHPG9UVLBTMsYSbC1a”Œ"16jnwcqKwd3UKyfdC8TWFzMVAXqo1MbRkQ”Œ"1KkaHCxCn1nFvuVHLNMcrdPuU3ykvBLHjW”Œ"1NAXoGXkTTLAWjetrSHk5FBonCEGEPzpDN”Œ"1FpB3nzgeJgS9XZdWoU7Y91ZsnWrbtr36Z”Œ"1MGKooYQrJ6UiDbkiKJYE8k3reuLJVseVS”Œ"1DpGyD6PzskhoK99rQMygFFi2eRx33d9JP”Œ"13RYW3pB3ka3unhJ7NQB8gSoUSDJGyiMja”Œ"14y3ccUKkjAEip1HJ8rf5uZLKkGBSJ9BPZ”Œ"15M3txSTFshFHDsBq4AdESpk2bA9w3oq8r”Œ"18LbSeTB3ttCw3pGCp6FJkJ1NNq13mLjaU”Œ"1Aokx5SHhnUduqPps3oFEJMbnTVjShjZo4”Œ!1JdYKKV6TVq4qibVDEGp9EGUV9JtPE2o7”Œ!1cZpeYsqPRYegAcmbxSE78ZgZSqFdXCtA”Œ"1F2XnUhNCVhHyqkJPbLH3mcPH1BkndUQ66”Œ"3EALczWUMhx1VoJkSSJnKDmZyUKZpuRMpq”Œ"3L6LL15Sw2dGS6VLzdN7JUz99egTzvFDXu”Œ"17APQemwC9jA3NTc1xexiu49Dnp613NhS8”Œ"3LvukKJCnKMQTF7Ft7cSYzfwPT6vWeE721”Œ"12yV1YMoJwV66ZrmjZUwHdMhCw2GdMG4m1”Œ"1665BLVYGBfav5KfCRqxBqkCMRMyUKXAzs”Œ"3QTEKLfM78J7yETRRDkt2AgDTirSj4SGjS”Œ"1J8FqRt7f2mTm8MxsJLvWL9RHrS8299ysN”Œ"1FTnabPZM5F5yzC3p8AjjhUzwDQzALyDQ8”Œ!1rz4UjvDPXLNQokNjpb54ViHtLCeeJkYs”Œ"1DFqcP3MbpVkY4u2YeckYoBPmuVs1VqKmn”Œ"1BuJeJvGRBs8gwMMtaiNwZvWAxqDeJWSML”Œ"1CZ1Zhq947jtrDVHcpvs5NDBhGP7tSRdts”Œ"1GPQs3eNfUE1C1cNMUm5CqodJY69juyByK”Œ"3KhRzwzzPbu6dfqJE9RfEVqQLu15Tx2U7f”Œ"39ERHehu9pEzkuBEtbFJuw36oKMw51Q1pX”Œ"16PJDrvBcBTyUF2dkkg8MqMZnaTZrP1L2J”Œ"17oMboAkxfeXG6Rc9FqjH1Yi4rGbsiq2Jx”Œ"3GBTPsfRS9hZZNps2aGVqY4Uw2k7vW1jtz”Œ"1KhSNHZc4NfKtC4wi7t4RdAibytMKWE3kL”Œ"1JkYaEK1vpr2SpzWpMdHukSZNfiWSs3ry8”Œ"17sNCfxevoxfcqSGqb1KtejPQjY9efFeJa”Œ"13smn9L93zC7qmdQe1nesmuCPDgcLHhWcA”Œ"1KcT4p5cXWqKie3taWFYAvQyixFntZp9sm”Œ"17Z6KiDo5mzMD8bspT6DAq9csjp9cXPRT6”Œ"1HgwFhhs5U6UvhobqMbyafuJcxb54vRrpR”Œ"124T3auJsqykwi4sABBySu84FkNRhYAVuM”Œ"1MUvWj37mLTuXNoxv2A7joBowJChWSQiPe”Œ"1HyTW6N2m92aRxajXh67arrZp4vdZV1ZpV”Œ"1GGHsCosV5j6L8ca34MdivNDZyrdDzHsDu”Œ"1KeGqdkqF1ZH6eu1BN3z2kpL9S47x3TCKc”Œ"12YhfmaRQANwSHX3R9o3Cd1yvuZufPARYE”Œ"133P7GZapzbQLeQPqzSBTH37wjuLCcK7MC”Œ!17P2H4aqcTqZAMU56NbYzWke8DpfZgAvy”Œ"1EJmo32LKehUGyYpZi5UENB4zL4TyC7WQQ”Œ"1JgJXshjebPUs2sxK7g3XjmXhoAdN9jt4Y”Œ!1LickNZM1aRWVPYdpdAkAyFk7BLqFm3Ls”Œ"33PJN5hbfpE4aybAZSqjmNcwRR3ojXxAhE”Œ"3Dej6deSDt2MiYPQ2rHt4Uki37YxsrZqkB”Œ"182bUbkoe245a2ZcLY1jryaKm1W95igBHw”Œ"12vzwjoNyM6D71aEA5sPtv35tedxeSuuaW”Œ"1PkZ9dLMUMpgS1EbyUKWr83JogjiJ5kGBK”Œ"1Crq4Z3VKoNcFHsQSk8qj5T7QmE71kiEo1”Œ"1LBjSLfUxGFMgbY2jjCJHPZzZW1nK3RhRG”Œ"1HPAjGtdw8jgMZ7HdyfJhVsy7wSU2nK9xY”Œ"1PSgjAm3H6RutUHrYinMSojDkVo8YUqRmo”Œ"1KgKYkQerd8cWgf76tmVG5YjKKcE1DoDu2”Œ"16P8ZAAL1zwxeFToh9fBL2JmHnFVW9bxHK”Œ"154kg3K7wr64BwgsFQFBufATREtHGNg3Mb”Œ"18yzRcUh3jGTTsewN2jXzvQHHveVgxf8Ac”Œ"3L2dFBVs4MRkpRcjsw8YCfVx4uU5yS7hf2”Œ"17yPeDCYmBhCdLA5jZhdazDaLy7vA1dLt1”Œ"15DtHFRTS4iYhdTEVsTaLitM4PzKGzSVab”Œ"1JB5WHrjf1rEbdviLTXKHe6EBV4QP1jqdj”Œ"1JneRu5iBGW9nKJ3QQwvrKecP3DGg9F4uD”Œ"1MLuAYxqkqS6zqAmtf4kk7sGd3xn8pVwGJ”Œ"1BZxLADGc8VDXBG5FL7ksPHWpAQrpaxYs7”Œ"1AvJ7n91jLYivtuZSd8AgZgMc3if2UU7xN”Œ"1HxtzEb65kyyRZpt4WBMu4nR5wNMNjz8e5”Œ"1HpQsgg7qfxqJWrXmCWtKJVXEf3h41vBKs”Œ"3MsCHcxuySRRrxBRpah26TFweVZpwKN931”Œ"1KvjYz3QrKZYPT1empRb6PbfdxBfVtnh8V”Œ"177a36k71SvcoDAKq9HTKwgTCpqaRUetY4”Œ"15m28wBz9CAYibPobWoewQBADE6DL8ZeQ6”Œ"12narkwE7zSq2RwRuLdHwE1b5vL6nQ4Cmo”Œ"1DcPnj2UEH5sAzER35bh9fwp4FQ9AiHwBf”Œ"17jRFRJb6oUEn6N44h2varjo6GCvTYXaaL”Œ"1FV7fw9uumJ9Ljr24Hrp6uft5PbeZqRcQL”Œ"1CMk4ZQwt3x1m4B1MyPBk9ExwF4TrhiBcr”Œ"3GrHFmQjUEuzXmyQBFtMCbakgPvz8MezXX”Œ"19ugaktHY7asfYUBWSefqY6D1JZrnvDbiS”Œ"1CvPtG44yjiLwFjMx8zWzEzG27h8aLZiyX”Œ"3L76AFcyeHZFc4ADRxV7J3KZ5ZHzKW3Dxq”Œ"1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj”Œ"1GEBb4VGNG2dtCkoHRN1AGZKDo5CiasxeD”Œ"1DHVyTKa2ara6hohJ4qFc4zzyu14KxmN4F”Œ"1AtTrAUqxMqqARZK4PemCgQzYo7pNBLPm4”Œ"1GDLSVUfc6midDmba8pqcwu8DHHDvSJRZy”Œ"1Q9HsfHzKaLuQd9buyCAcARduuXA1MmTim”Œ"138swHsSqaJBFt3CHmmB1y6hpD2C73Z3LM”Œ"1KxMYrei696SjYUnLd8PUpLBypGHpsyo5o”Œ"19JHmo9djkjXbDdHFJkiWtbNvwxkBsBULq”Œ"3GowH9N68pzXZHoUkrZd8Mqj2visXKs3mP”Œ"16LEa79mBXzRkYzNw89xxsbsoCvQXkv5Pt”Œ"18HeiGsnq9QSj8EgXY42eguPRv2fVEFVyM”Œ"1PZkE4JVSU4Mi5ZNH6mtUgy7h8HoZ7myCo”Œ"15ENnFspfiZoEP4dfip5DVamnnYvPcG21T”Œ"188EYfmyk1Ryhh7wetJpU6oZN5nPZrNq98”Œ"1EYDthjjXqjrByeXGE2GfG4ZJnrZGm32wh”Œ"1LPvRVqYdZXF3ATpr2569XqBMLFufjWMzB”Œ"15UdZaMqDcHt7pSp7dgF8PZfFbj3NSjqeh”Œ"1CCqesHcGVn3UoJpDkNo8aqBTjwNV2DNZE”Œ"1PY7ZPUzZVj4soVPCHV91avBgweHArsR7Q”Œ"17FuoeR8wf53RsmZMFctcU9bxL1m6akkjs”Œ"38YXNHRDYci8NnS48xa7ueTYxw2HyS8uP1”Œ"3MEtq5QSMnY2skhr1Kp3vjStwPgHczH5LC”Œ"1C91S6871QRTTTMdhVsQEv9LbdU684eamg”Œ"1DXaX49xe3s4XyH1zHy7Mzs3i4pAf2QWZw”Œ"16k57xAZ2GBf9mgr6dQ6aNJ5NitmfusU5F”Œ"1KndtTwXGD8wzo92xaTLdK2UB6RUymSBFV”Œ"3Ad5nchXqu6UaLTjpMgX1wqLpi51nzNr8n”Œ"1D9WN94jjsaZQz4PFXb5ypkFyGNApDjsNM”Œ"1M3vjbtXHMuzceqXQoC6Yb74NpKLnqG3oL”Œ"12rWQ2f6uHDXyVqpcSGjc4DE32bN87vLpM”Œ"18Z1occFn9d8KoXxHBkLsc73Qgthxdf2tz”Œ"35aq6eoMUjzyj1DZcdhPXaiyEay1GUeJCV”Œ"1CnqVppZ1gV3s3Djii6Ae6hBC3XMvU1z7m”Œ"1PAcn81eXKEXvPSSUQLQcGDs9GvpjURV4Z”Œ"3HgVg7YZZFGzJuE932vK49PNtBzLaCwYMZ”Œ"17vrEdxZDUQFxp6X17Ezeq22zinysdw6tA”Œ"1L6jRCwSdRoU23feiUwGNefgYthY9juTJy”Œ"17oKAk6AwpvzUS9wP3UJRV8fVUwimm7ETX”Œ"15qe8v7bZxNxJTsTXHSX9iS3WPksc5TtDM”Œ"35jECoQ2H73U52W3nuYC8xXx6T536rrMxG”Œ"3HYkYDVLg1HjtiEqMf1py7b621dZmJbuHj”Œ"3MzD5LnMxTorUh353paJBFxmd51FQauGJ2”Œ"13rHFcz3mmGBijVfKygFmn31c92L9239kT”Œ"1Pw4C9FDRUujeTThPBT94NjGHndVjEpeu9”Œ"1KdVzmXzYA8CxQjzPHrJj2TzzFsqmJNJ7j”Œ"1EwFWrnkzH6S27noaprpnF5M6ctc3fdpPY”Œ"1E61XBGcZyRDxCLZxbd3qwDiWKuvkmGKt2”Œ"17kfri2RjU6u8QV25z7AtwPb3GYa1B7UwE”Œ"112dhkgEqMvknyaY9KiSVxpHeyRYEqfvbw”Œ"1G5fpV6GNEN28QVa8j1pbDJd7NFhf43uFX”Œ"15w9LMCXF3taFAto8qeGz5GsVUWJsQwJ1F”Œ"1PnsVJ7HTB1Cjk8KAfm5pnkNK92FdDAoAG”Œ"3Cuz2ong5prFuLEg3KTvQ9VtxDfuwuwRqf”Œ"3DPuQ9sntyQDwRLbScco4dmusuMubjX6yc”Œ"1NwEf6TQVAFg7AyfjH2zJQ7XeJvN6nMRt8”Œ"1CWEsxT4JPC7DGqQGE23bfsu2LiZr3Dkwi”Œ"1Cz6JeSFSZoiLdTffv68pT8iqhrPgoBX5p”Œ"1B99NhB9gWGNduxtFqNULusPB5kia3Yvf2”Œ"137Kq2SvsRNJarZGkVqrFyuqqfGNbjtYtk”Œ!1oZkTfVHfbaRAuHBpxLQoG46xfcz7YhYU”Œ"1CivkBzRpkgupgNV7jxP5ngBiyVbSVfVBv”Œ!1Z7T1VqideE9xZ7KuHDvQ23MMrXqk8EG3”Œ"1NHsYgZMpkobbvdWLpFhqxsZ5tDdZzHm4t”Œ"1JMBVuFYNZ6mrGQtaEsoXAgydwHdZjEkcr”Œ"3BEDAN97QAkiry6EA52twHLzBsdpdGn6MV”Œ"1GNXAQzQ377MeXUhyq5wvveByv5VFzcx6h”Œ"1KYaxsL7VZrVP41rZjiYa7HhMeVZuDEVbk”Œ"1D3KFa78ZSruqMZGg1i7wrrsK37hF6KC1W”Œ"1H6EQfS4KQ7Ydw7vULEuZTq9hq5nsskZv5”Œ"18N3GufXTGfCTnaqeHn3KKGb4fRRzZqm5i”Œ"1C5ZtYsLDvWUyj9P6VkZvB8NU9JPdEEUGs”Œ"1LBQvAJRvWtzqHhbeFakWK8fdJ5m9EYxT1”Œ"3LCmUaicp8gW5XMG8vGtVkbk5KVSBvgenW”Œ"1M2Zevya6BDALcYsZDfcb2dUoQrjUq1U3X”Œ"1F3daFKBGHKhWcs44Ahxaovz3DYBVySos1”Œ!1MZWtitehQbi41JUAtsysuPQrPFRToA19”Œ"1AqM19LtRmdzi1rxNrNPgLHPLRachF5YBq”Œ"3JSBSB4ZZ1NapbuwuV3ZP4BEiMwXYpq9PN”Œ"14m3CDwSaQ8TKfcaGtfqnssdkFZY9B7zTn”Œ"1AfUrCHzjtaErrvJCYxxc6SoGJKLYdCrYn”Œ"12LCD8Rp5KXQpnmrpbeMjZm2CrAPdN6Nx2”Œ"1KbvYBTkJtGoi4PfmJGFtURhWRKDSViPPZ”Œ"1Mtm4kgqJmRLGUcc9i1khd3hH1aMb9a8dx”Œ"1D72k4jdEpNj64x1B8V2yPWazo1PuW66su”Œ"1Bi3vMGnaumQFKocHWTykPUJXAVnHDy4Bo”Œ"3AvWh2s1jXw6KDbchYc4pKjTVAQCetWL3D”Œ"1FMgMQKb8GtKZtbHTw78mvDU6E5FN5yBRC”Œ"1KnVfBdwba3nsBP5h1kNWCQRTbDjSmSg4A”Œ"1PBkv6eASbc1bqUBAqkVnN8KrDVRrSpdfp”Œ"1HebCCwJres9PsBBVQrmoPPn9BirRGuhUT”Œ"32BoJ2tGYoDYEZy9fQNsr67wUUdYWpVDZo”Œ"18aUvAgquqhZXLvWLP8MJozoLGDPbHukM9”Œ"18RNaCtvQgrkf9foxAb6EabfEzDpjkxkqm”Œ"1GbCkq6EKSaWfmciZEBYwaBmBNkBxm88eg”Œ"13MfnQpCbCHnednWkrWmXWjsU8UJXHtgKn”Œ"1E6FmWXERj6cAXXQJwLBB2hKCNbGLA6mvY”Œ"33rzf1W5U1Zoc41jakvYsXtBzv3fVEuQ8F”Œ"18XfGHYgrzp6wDfTUjydyHbmwy7eBjyzy9”Œ"1M3UECzMKWXDBrXTFmu7EckTSSZxcgNUKG”Œ"36MjvGwwUucc9dabZCxsGJ3Jgrg5ULJkbq”Œ"1EnsCVK9qrDZBPE5mVgpt5CPRKsKMaUcZJ”Œ"36v9nXEran4UBPMxuWmPt6poUdSp9TdN2d”Œ"1G6zyoPaJMDneubBJ69GZRjqS13ZPZAT4K”Œ"3PuBddBAowuyW9yJoKWVihtcVdxi8mMPPB”Œ"1EXJwZZdKhKY67naAJxzoSmJruJAvW2LdE”Œ"3982ARiSvuLrGafv5gNR3HkYAcUxN2G7x7”Œ"1NxM2W5SJqFZ5Txsed5ya1kfHoGYtSV46n”Œ"1CcXLNnRt7nPjAzNJGQx3aNrdHpVke5WLp”Œ"34v8g78xJvhkYydtbcVaJUAgCCuyroH4hr”Œ"13o1KH7DKDijdUMmWcmJq1fp8RzzdGciLF”Œ"17HdBxFWHpfzPXHMs1Bb2LHdFCHg8bVmnL”Œ"1FBuGBHRnxwmFtbXrNBPRCTe7b4BiLSjVK”Œ"1MJardjyaSp2fiJuLR8UK8ztNUdfpfmQEw”Œ"1C1Fh5jcQaizwyERLRtneT5j6vodLE3nr5”Œ"3DdrEpDwqmvBhkycckgF7YGFemjooN8YE1”Œ"1LYCBWRYdy7V5K5XtoaofxGaKyDTQ3XUE9”Œ"126m8NCxb2jbBcdx1NnzMArPgtHRpqqR4i”Œ"12oEhnYYxqja3Mo7YkubB1wTk6NkGbmnWW”Œ"17WXAiLFNAnPd4bCFbsqcrhjKMnz8K1AwJ”Œ"17bHcQ2YnSDQ1dz5W5QuPGf2kyJyDWyV4s”Œ"17sXSk52DQgbKYVLhm4ujDg2wTZQXUGyaL”Œ"14qsiofFEPKpxSkZFQaoud3hZmzTUCpzNR”Œ"1L6Jbzn77m6P9mrXhy47KYbtNsTHWzpU96”Œ"3M16e11suSdfHZtY5Tr5ULfApijfrXqWz8”Œ"17ErgaQtbUp1kFqDNAzHgcgRVo8u9oRCca”Œ"15D2rT2AiiU2CFYA5XPDgYWmyEi82Qh7g9”Œ"3AEEetk58fo1jtW6bCo6tTW85FaH6XUVB9”Œ"12LTvvGtdfmsXY84XFYkmBCfKF8vuv85dU”Œ"1LauuHma4c5u7wPVjWLgLvw1C6NJfziVR6”Œ"1AyQCqmXJHciEa7JvKjsuFk5mZHuLGWxXp”Œ"1F9XT8rjS8QYKjDGJzpeTyy5R2wfc3VhpG”Œ"1KBKcxkXBkb2ebSX5UcpwjWWuaVTVUZXci”Œ"1F65gfUHsUfgELynobwZ5NwhJpu7h6Eipx”Œ"1JadUzUqfyBrpin167yDrqyN6MB3BXyMoU”Œ"3LQbbbUheVizNNtfoGpaFinUaxnN9jgZ4a”Œ"1ApzMvgM6gkTYFw3DLUEBpEofSzv75fp5D”Œ"13FAqxnYbBWY2WGK1CnUXULxBPCLs9SADG”Œ"3EbiWfViNSpBrixGYJ1ZcKY1L9Auet53gC”Œ"3MHABP5wwPzKoVSLw13CrSgmdhhubPS1S1”Œ"1ELbM1KpLVur1hDd7Uc7qB1YgNsCT87CtM”Œ"19yssmQ5En9B8TiNWScKSDCAnbqNMhZTZL”Œ"1Kou3uaKdbf3hqdvy8yRyXDE5pMha4aUN7”Œ"32FXUmeEFzuukP6h3z1Vy4ZpKx47zT9kwp”Œ"3QNVT4s7FcjqKiFDkZikP1D4CVLgcjFns1”Œ"17g6gfyEAV2SGXGgkMyZ4G1ziU8ZjiysyL”Œ"3Qe2EcELTSDsjak7v7iGo2YNhnxpLXtJDG”Œ!1MrQi2E8iCbhojQEynZtwfrDSCnDp696D”Œ"13dc8G6rSi1Y2Uh5zvA9yzpfJQUyzC1Ucb”Œ"12sEFdBzssJp8z7eZ56K49WCyxn8G9kvQF”Œ"15iZ3s5LkFT44nrxRQN2Dz8frnpMor8Qac”Œ"1219HTuCNvdduqTNQ2StBnbWr4EKGthU9E”Œ"3855zCRftJoPH3GbdH8BRotiSmA7J2hoyy”Œ"17u7JDzanyMbPnfCXy9xiVHsUL7KxmmUyk”Œ"39VWFsa1rAE8PFqcw5ukQHoq3xfkDz2YvR”Œ"14vUgQYAoFJ2QQ6SskYawTAcD3oeXoAgSY”Œ"1D9RZJswza31fkKuQHusfjdjhTVBioSmp5”Œ"1G259dSkqrRk4tZ5aL7mjeRGCNcKUapSAE”Œ"1KkdUwLKLK2YYAz5MQ3j1LH2mzJg6cwedP”Œ"1NFnr1wimXvFFWnfy6GMmd3iBbLJ7qWoxa”Œ"1EA9DMnoopvMHhVgvaRruDEhNRz1kLkSVs”Œ"16k2oABHe1aREbp5SYtuZV2dThgFL1RLMJ”Œ"1H7Gqdw5pJH8W4roHq5nF5cTtdaZ6ako8B”Œ"1LTWu944ueyVhFZsuT7MyDWNzhtQM7hDzZ”Œ"18K2YcNgpXKQdZnY3aEjGZvz4xJKsTuVpP”Œ"1Q5Ra4yn5i5WgzcUo6BDhQBZs2LGA8yr1d”Œ"3DyfDsycF7JbVQCPwiHf95RbL1rC3jxzKx”Œ"1C1scDxN3AF3BeCv3kGhPPA2oT1ywUoW1u”Œ"1Eo4GUN9Jyon1WAwC6rRPbZnNy5xAs26fd”Œ"18LYymit6kNkQJMS1EasNygTF9R5isL4fF”Œ"12dryXDV3m52s8cedtQ2qThSaKzB6dNnMQ”Œ"17E6zh5DRcnouxNM7T94h52KyqGPnH2nvk”Œ"1Mod7vDiDMQhjxzDenRabUkqKSMJD48DWK”Œ"18t1Ae1wDR44wAivn36NcgynsX5D17VHv4”Œ"1Ae8q1XSBhSqePHykApXePFTpdUzpsEQP3”Œ"383HEBySBuwiLodb4aMjheF2EzaoCrbnPB”Œ"1HqfBV8MYASFeS2485jRxjW7ZFLHcfHn1H”Œ"19g4QAgMijf5cn44vc4FpM6LQ8KWsRX6E5”Œ"14NeYLiFeru8TzpkGyaK9hJb29GFYncVh6”Œ"3ELYqQPymucYcWNnwHC2yW6o6icTdvSYWe”Œ"1HgbiKVRLvCto7p9iZYgwpAqhrUJRVcL6b”Œ"1EuJrkDKDJmNXyBRdMdwWwzFZM9VbZXuo5”Œ"1DbwuYNmdtJb2bHJEiTU1rkSCYKZwwEHzv”Œ"1NpspBFNAf9JM98uqBXwqTwMygbedGcvnZ”Œ"1DAj2rN4EPheCVvzddM3t8Ehgei3GQ3nzG”Œ"19zhrsqKYX7TwDoGxkMnYNKgtTVgupt1pL”Œ"1F9rvW5QYi5w7ZLbHAo1Bj1ZythnzJjBM6”Œ"1JBYLgraRfE15rsiaBPZLDnkB2x2EBaCca”Œ"3PAKaB6RS6g8sZtYuBfm779SVBn4JGpKXr”Œ"3JLNwRksY4iqHEki9byXAGTrZBvvbpooVP”Œ"1NT2oXAByDZwtNKyiK2yuvKLBs8i9YoT3P”Œ"38uMFx32hUvTEy1kDzsMiEgCMUUjWdxyw7”Œ"3EKFHYVzdsS9oAL2Pb3fjEmw9pKQuoASYF”Œ"195rT9srfryKxDHPKYwWzGxKJS8Y3bGZSw”Œ"16rE9DQTHwzPapZCFpAmXgjezxW3ShSmF3”Œ"1LnD5LWAMP215JgR248cezQ7AnHJQyHyQr”Œ"1CXJBsXFop7j7JDJA2tKnabCgteRzKtVnq”Œ"3NmofPmzUeuj4YhPi825yhJ5PhEc3wNyT3”Œ"19bgF8p3QwsVzL3NzvMovfcYJAz7G1UVfp”Œ"18uSr9WNRWDagmvj3DKhZf9pARHbDGRSgn”Œ"35rp7bVy3KLueZRazYnHQLzwawo1EfDcgp”Œ"19TA5bm3t3p1GGZuebvzPEs4rf4EHgsqHm”Œ"1QHxoNazPegUG7uxAqxhkHhK89LiyxNwu5”Œ"3DWbrxpvbDoCPi3tN7jEbNBpLuJzEiN2Qs”Œ"1F3DwRKwFJqMLEJTKozrV7Snq7K6SHhdSh”Œ"1Pkg4DgT26GNf4bgAcUSXNC1188YRHcmEd”Œ"1EkNP1yThac3C5TxwecqWrGKUCAPLLJ8vF”Œ"1Dfu3nNKwpoeChb1rphrk8muxJLwf822hD”Œ"1NtqJJcrLxEjTkxRpJiKcMHFSomcfXcQZJ”Œ"1AjpdAgCVQa8DaZVcRFPzvh5qFZCZhfcdZ”Œ"35i4M2WbM8JvdivErT9bdBJu1WwJSCY1kU”Œ"1JoCTaR47KE61qqXvpKHK4q31UsrJsZTKP”Œ"1GEs8mYztHDh43HCiyzAofx5QDmz3mBaAA”Œ"1GTcv1KRucZjS6VUdnaLvgTSeoUNdK2bbo”Œ"12q6xX8tAQyK4N1v2rAXVrFkZVmiJD2Bnq”Œ"1F4v9T2ykuTAAiZUv7hyGUq4ChZEL7G3oy”Œ"19zycnprVKXiMe9rhNRY2Acv8D7pzhHWMZ”Œ"17wGpJAoKuqMLXwyVmGuQ96rxEAXbi5pdW”Œ!1LhxSS5SbXvUnmjYUcAFxR9vLyjajA7xn”Œ"32imWafqESKDn1xDVNUyv6fKCrbKx7YhmB”Œ"1CxWWHzR6MQ3oFJ3S3gBJNEk6V3YudcGiF”Œ"1HoUXXFmeuZes8dpPQsacuy7gLNiQ7gNTx”Œ"17KQ6MSG8wpP2miBZk3foXnz3K2phYt7Bq”Œ"18KqupRuN9WqhoDL7nSihdgLMxfoEHmje3”Œ"1JY3xZkzwoXrVghHCeWFTwqSz2tcmVmuRv”Œ!1nBkfQmB41eMJfb4sqCNnBBtsW7YtJv3L”Œ"38E5JLWH8gG9tuoHJKrgsJF9kKQ8Swjnmo”Œ"3DJmSrPpKw6FsbTdg4DAWxJRDbjGbUtjNe”Œ"1HsvgRKSGiECmD95GEZELyApBYsrAAvBj9”Œ"1NmL8SJHxeBvmA9Gc7dDgTmPNda5bXz5yH”Œ"1BaqQJjyXHV2eYx96PWUsANiUQEgn8PZzN”Œ"35qaV57awVTCb46UjiAkzBGgJGLZVMkvDA”Œ"124MTNepSKqDayKkikF9rGKd8qVPJBnxqW”Œ"1CFeB6Fm2YVpsZT6DaCDxp2B78bZBsfVdo”Œ"1JQZM7dqFAq9j9eRUNmhdAzMQ1ikD2Rgrj”Œ"12Gra369ji8jLqnCEG9DsMHEWLdDna5b5g”Œ"1BXCfeNvDC6y2jVthfTa8fj74nJmc1PZu8”Œ"1Jq6iV58ePbmVfHk6hDrmcWWNeQdWYvUK2”Œ"1JNicqKFYLcDnAYvrj3neTH8pCXacWWCVZ”Œ"1NUAMMW5g1iBCUyGqNkjCjDUbtPLGGfnjN”Œ"15Rk7TbBbd4xYxvq4SoJ3e4zw9Q2f5nugt”Œ"1FtoQaYu6rB7xFPstJvmSLAHC3J6Estevu”Œ"3JCDaKgvQ6gYYSC4DormVyYs8qGYifDY8j”Œ"1NWJ2BEAc7dnvL7YasjMW7zHgKYhkxDq3C”Œ"15xpskAUFxmS4XD3vjEULnGA1TcWCr3xdG”Œ"17Uxbn9tu7rPBNikNvbdAC9yiGDcQvh6Qg”Œ"1Le77kH4Me4Kqj2hmeSRAXpadGusWEenLT”Œ"3KBurpC8zMUmTFyvbwPhN43M31zeVaxMKG”Œ!1dgYznmqiKT1AJGGy2VFdpozVM51ektGp”Œ"3KHWrMo8EnzCh6uhXrozHUhBtogqiWpux4”Œ"3NbyxRMfAbf4aGrg1cTMfvWjbbBR7xKJPk”Œ"3BQjEhqd7ug4eoA7g1RTXNztwp89c9Qyv7”Œ"1K9k3fQmLRxPQzuWRsVvU96PeAqySNJvqg”Œ"19nW3iAbvkF22Zi2NNWTNbqLqdSRNCJcWo”Œ"1J4aNFPXNacjbMVnN99sMDuuTTQRPkc7Xs”Œ"3MsTTzZWs1pH2qkkRfTXKsbu1fmL9SE6W7”Œ"1Mhvr35oBWb1vsGd3a1AqG9Th4f79RbdkJ”Œ"1DZfFAtxLmxG88XjvhbYsUughMbpwPTbpb”Œ"1BhSWwQ7XkzM36rmDeQC2CJrfeDHEncd1P”Œ"1NyqpamyuRTDmSaytMCuoR34575LsPnX3q”Œ"1M8Tf7djzvh8fRcC3R7oMdnxTza3eDMBjX”Œ"1KFwuXznFBSe4Kee2pLSift5J7XjNX1a2q”Œ"1NFmh2iMMi9fbSK2CjExFFQSNcAyvCvKT5”Œ"1Ay9sGkwguym9dqucLxA5Ejy5pkBXVki6S”Œ"1L9KiRTEK65QxzQswdXVf8NGBjMKFmyn5L”Œ"12SXr74zka9QLdQAMCNAw2PqAV6eKoX7jd”Œ"1QHQr6VQZuh1FW74MvPNxMb1WR1SSrE7BY”Œ"1PioXkGr4okY8z1mmR5i3XCRioEMHC9eza”Œ"3D9CYWA1JU1iqgHrRaxC1j2qptNS4EAawD”Œ"1MZfBj19nSVG5ee3jEQMUDtXrca25xZhcj”Œ"17utNe6N4JjhpStDzqgWvvq2s8EoQVM6PB”Œ"1LXVs8Y5fCDez4yx55EnTXxyko2K92i8BA”Œ"1AcCcgQ8pquwpvm47dLEGA7c4MWZqikbP3”Œ"17XKBBY2jq9RipCd2r7aDguCKpLzHjXff5”Œ"36ugjVXsAaE9JMAWvf1y6gB5DyCXm5zpio”Œ"1712x1k7geHa2pLouA2TAC1PLHW7rMAerR”Œ"3EmtSeEEJEGaX7JKpmuupf5N2F4pQ7vDuM”Œ"1Pv79dEkNFztQSCDnTWRiVFRF8dUSDhA1R”Œ"1Gc3biG5hsHAoj4ZNAtnT4sCBbtF9sKEEL”Œ"18j5W9bdsebWcNUumVvCX47B7T2SLt3cxq”Œ"3FjoiHpvwmTaaFaownSxvA9JvZwpMpDyzJ”Œ"1Gr9FcTFPb8RG8qkKr2ibCbHCMxz5Vcig8”Œ"1Q5eFCW5mf9agATBKnJJHNWHrhbQ61NPjy”Œ"1HGr5ZQcBJk8TRL7SNtg4V2MhGXM9RTQZ1”Œ"1MDetJpzLzbUahUwTfZRPkVpowrUNJX21E”Œ"1PyPvaMM6PYbnVu1HH2oeUf5cPEqYLcpXZ”Œ"135tZfUX8cBG8qysGgFvD5YvLnzYvRNG57”Œ"1Gx19UDqShkkRDqxF5VRZHoggVvhZHHo9n”Œ"1M3d1aUbb1odFmgb5H73XGZtH1fCVjFTaR”Œ"1Mry3Lf5GkZ1FYYYVHtAoNafLEuPntonDo”Œ"14Foywxbib8iT2gVMiCTJ2ebEk3p9f4ngr”Œ"3GGgpwSGmXDzmHyGeZQQEm9EuZuvfGvFJL”Œ"3EXoppPzsEBh2HEXzB16BTTd5S3fxbA8z6”Œ"1DUmAYJxCJYG8HQAJUMbTFJcJfecVn5LMq”Œ"3DSL9jwgtj68p4F18Cik7ddzKi5wg1QLi4”Œ"1J1CvWXd5aVEVbyVG7SwTVuEC4NvmWH6EV”Œ"16aHjo5wrBKCF7T5nA6bSpEFYC4VyK7jDn”Œ"1MdDcHySA9E6jAiWRBpaGnRwQvrhnrZzYY”Œ"1BenfjATHyKoMmWXXWZawmax7jbDWe1rq2”Œ"13xbfQFggSNxgSddNaBYV5FtGJeLph18SY”Œ"1BCTCLMiuXPT2jT8usET1dEpzDtuvBJKm7”Œ"1ESbkMgvdYzyDQRUiV4PEsZwfonM4aJhb8”Œ"17NH8XzF8RCNTRCZjBLz3oeDKUZSLZGBPk”Œ"34Tc2VQPmLfryYtCKuhcLA8z82ApoUcaQM”Œ"1LVFK7mPYfmrx4MpeRwPjF1LstfijTP4P1”Œ"1JxM7mWWp6mgDKxS8Y33y75qmtX9dRrUYX”Œ"1MnhT9rkEpCEMPvzoseM83DJZc1euNx6i7”Œ"155QqsmgqxEhaC6H7QFxPsEVEL4QrnCrG9”Œ"346ywn56fw1cVscJQvU8NWCF3nPYWUJdxR”Œ"163MAwcijGLiTciSCx5PBBHwzT2rocyDJi”Œ"13XguxhT3zbqoJbB6fugG6aPym2WhBeRE2”Œ"1JXZ7z1MLQik7BKcNc1WWQ46T8LDDNY7AF”Œ"1JVoaGyLQgiPGXnADMPRG8zrVxvSTVxFQi”Œ"12QnjxuRjkwchdSmJczCRZ1U8Ampn1kVBj”Œ"1HWhsfP39p8hxxvkoWRTW5dQUnqQNPYriV”Œ"1KzSuzttbYqaAzE2Mwm54aZfANZdC9fVaC”Œ"1CfUewGfBJ1uyUdSi7gBCXx7hu3eZ1Uu2e”Œ"1DKKXFLn5FLSUem3s4fA4WpCWBBbYYcUS2”Œ"1B4apZhqoqUXmGkWUmgAf2asKLBogE9dzk”Œ"13oZ72LjtUT1Hyz7vdvuAUqUXkmuJzJdMF”Œ"32j1Q8fpvyuoJmbpKidWB5EtggdGTLtx3C”Œ"12rc1fK5khGionvJXjC9MowrUnSyVjexqL”Œ"17Qvbw3gd3tvC7J7m9xahHGgypxhnsWJ41”Œ"1Dd3e32DGomvUwoPqLnZuBVoXrH7w5aeXR”Œ"3Ac4hHDhdUHhircYu58rdyCo85Q94YYBXw”Œ"1CpMtrxQbAer6avhEH6nXxJCPN87Kj3Li9”Œ"1JGDAZeeprMYaeYmpCMQwiMmP4k6TeQrYH”Œ"19mPfanAWq4jWVB2BRs27tS6EPy2M32zmv”Œ"1Bq4fASadCEMRGDTrDxNVtgMJ2HxebDa3t”Œ"1Q3t3vxpNnjKwBaGHf5V8rBHniTLS5nNq8”Œ"1AtgMZUaShUycFEDJd2uG9G7tjmF9E6zRE”Œ"1Nf4LgpXpXdTJDzQxH4JyypLdsG6AuBfLE”Œ"18YCFgNopnwDd5jMfNyoAvBLk9fdzndfop”Œ"17XKM9cdbJZfLqSdERBNNtsPDNBBx6wqDp”Œ"1Ay92uhTwF4VSJbuBanTHcq3wTUqdTL5ss”Œ"1HV2JK9wf9yzGNb1L3kptZsvX4F5LJr7r8”Œ"16R6pwtYHjYKUWyDf1jJigXd6Ky588WBCU”Œ"1AywiAVvARnTmc64Ao9zHPeBJcQnbCdfrX”Œ"1KKvYoPta2cmdqNZ6Lm7gxhKtNWWyAWUMy”Œ"1L1B4i9VRdyDreFx9V76JQMVkt97zarAoJ”Œ"1L7p6Tui7f3EYXkEKaNVJNFoa7fVBr1DZP”Œ"3Dt3sCkX4X1YdEsmrsbicbYQ432HhELVEQ”Œ"1DHnT8esuxVgQ1ogRigrqMVejHibCx8S1M”Œ"19xdeiQM2Hn2M2wbpT5imcYWzqhiSDHPy4”Œ"3GD4VrPeEE5fcBwNU34P5QrKPCbgJSTxRa”Œ"3GEaT8ZRXELcjMSFvGro6eZcC5S1LSLZuN”Œ"16L84GUeY74dLhZiZsvYzgv71FQZx6tzvp”Œ"1L7nesY47MZs15xWyZ6ZTRUu1JkX8SyJvU”Œ"15dx3p69HWdCJ4t5EJiFA8zkpgNXFSqp89”Œ"1NWMBv8k8AZW8uy4Kc2RBPBkXxeDBTipt7”Œ"33HeKK7ancRkPQuc6fzLB82iqZ71Qu1L5g”Œ"18EFge4q87Ez6AnHyzUD2FHSyczcYVVDxC”Œ"1LEaANDbfWNFfQSZs3fbmiyuTaqxk23n96”Œ"1Juga9cZjQ7iqpVDsjmjEsyJhfEHHysNEq”Œ"13ZoFUtXw3PJGFvvVEJjDxScr7nyKUQmdT”Œ"1966oeqNG1ySPQPrLQwv5UkBBCgWVvzCCD”Œ"1DehoL4sndqF323eJXjE75EH4RS62MmAFF”Œ!1TLroXX13KSuUzsT722pXHpqqrnXTn9U1”Œ"1BjmDut3JhzbQ1ik9gzm9uzTQNZ9veNFUD”Œ"1Dgu9Lu8kCkiBcY9BQR28e7tJWsvtDQLxX”Œ"1MPzRxT74PrUG3AWJHzU78qSmKCALNVqiu”Œ"3KWNHCjUAvPysvk3VUfmrATXGu2B2hvnKg”Œ"1LEeRxcLqskXAozQib5zscz8dYWAp18WMk”Œ"1MPdZivJJxgpqHPTeb2e7CLJVDnGubeNzJ”Œ"1FiFX2hGUdSJ11s1EHVQevmiqAn94LKMMj”Œ"17sUpU5oHRnaZ4iU5QzYnkgm1ajp9SX3VD”Œ"1C3Do2AqdA2kvGhDmewgZkCiCnL9DqfUTH”Œ"3KiB9kG8sbVsZUQBT5qh89axu25hi1UYci”Œ"1JYMCH5cWgYBUmyyw31Y2jCgPaRgQq2sb8”Œ"3N6Xkt58h4eFR28jp1QH1sCGsupVr7me22”Œ"1KZtsoUd5spbPArSVidbuTQKMKmY3tHr5b”Œ"38gALXhXU8fMd7pHXzmujZbbmThZBBG528”Œ"19dS7mG5BAqwWcxSDF5hevovumjwbAH5iu”Œ"1D4G1hrKgSUzE8aYNcT5ojEnHQ4QKX2qyz”Œ"1F2hPTXp9Ra4K4J9VEfJYuRQ7APHhgSANG”Œ"38wQ26n1sdPf4MRpsDy87SPh9RazKm3Y1u”Œ"39RUH3w83iz7JqZD3gB33HBMv3jMSn6moL”Œ"1F86Exvkx1HUbD6bC6kq2WXwgH4q7YnQwk”Œ"16B8Rb3kJxQEBCkUnphrYeLbhj9Kiu5HgA”Œ"1FuD1D1sqZxuX7gCUhBngbse5ts3fAj6FP”Œ"1Gr9RJFzHnxo4Z54QLWPvP6ixRsZUQERdq”Œ"1HtstgKR7Wqk68jqREApiz94cNs2z5KVZa”Œ"1Maar1zAjaA73gsyhDTg9qGddYTnGe7dyX”Œ"18b9ZYAsnc6zQc8n3rssUzWF7opgtiub6h”Œ"1K7uDqQYvoNzjQWmCzZC1Dm5TbbeTaoCjf”Œ"1HGKFWTJaNx5Ek8NBvaSviexG5FvPCM4Ld”Œ"35jPWpYAsxsr3t6aH9ZSWLCRsxEpjL8gX4”Œ"1BGnjAoGJAuCxQgyXDXvRsT54EuQsuyF2H”Œ"19QrynZvXJx5CgmVZEDa2XBtabQFgTuVMJ”Œ"16DzHWJq3a9BEvR6qXKA8q4nayKNnuFp48”Œ"1Nr9YJZao4nZ1AcRykBpvjaig8aYKqWCC3”Œ"19UfZdb6nZkX77Fjt5ZoqDL7jGR6iAR1jg”Œ"1JFp7pofNkckH1SwdSxH2hdCPvJZHJXpWY”Œ"17ZTaFHCPyyCiFFFeNoguRsnfVi5Enth23”Œ"1AwB1WdtNHC6X2eseUD3y8SNCAodionRN4”Œ"17ngjosQQRFkvQQhTTq6VRmbMoieRwVReJ”Œ"17H9V2MkSEdS323ygZWK8cZvXwDAjNRpqj”Œ"1P5MVFBVKjkHqfTBtbAxaEFpny2FoUawRR”Œ"1AqabbXJaKjNvDgybSsKBNuyC5XxaNt2Hb”Œ"1EGWp8xTdkBVViYXw3i3TGJ3zpyfVHyp1t”Œ"1GxHTmffVQJBW7dy9JrTCNQq1k5x28GAZ4”Œ"32ZSW1XoL7vjKhwqzyL9NUmPFbrCS8rKp5”Œ"3HP2n2rBhSiDxkLj5BNARhTBMCpq9AGcLf”Œ"1Csax1hBge2FLVGJH6ncaNVyxWQfKvGUCT”Œ"1NGQU5p17FvgVYYoosgP7iC6rG7W5PXvny”Œ"16aVEhFSdDGDTNcbRtyATm914dA21QeLw1”Œ"3G18v52iSqLBBtwKf9rst4B9ygKR4VAhbv”Œ"3ADS6muoRevM6gVTdgc1SEt1Wtdy2y4tHY”Œ"185Xnv1SzhGdjbSpYq895Wf7Q6zeztdQhm”Œ"18oqY9MPuEb18u94Qkh2jgx9XTBp9vXjQY”Œ"1KCgvwG9hPbQBZJaAbpWnJbd7uj9GMXeKw”Œ"1EGznTrYoKsFNjz2MEapV3h4bySjdKWiZA”Œ!1hqS97YWYrQyDkJ5D6XXaBLPZvy21RaBJ”Œ"1FMUQZHqKnCqCoHaTwF7BHaCcyz6oCHvq6”Œ"37fJjdPaQmTV3KmAkVGazpGG3LC9F62MCX”Œ"16Aed4afA2W3ud9b5Ayyut1xM1qqbgRREt”Œ"1AK1BbC9CZjwjLSz72MUwcNV6VPULqDRza”Œ"12cKPv16B7iT1T9qcRbKGfnasdAg9r2Vo2”Œ"1JcgWPWqtnwM7kja6JJdeFSMEfkdA4homc”Œ"15pnK5zvsFZ8TEC3dVe8xEdNyVkQjaoWyc”Œ"1NKPUPRSFUhNmEHMfBmX3N3EaHA6MgU2bE”Œ"18fT8GYjumeaskUb9JYj9FrYc61Xo6L8wy”Œ"3AX7T8fd6F2LsZyPdk6e6M1ddbzd5ZPbek”Œ"15CrgsKEg296zaHpnDEXRt4rTg8qZZpanJ”Œ"3DPkSyKDrXY5dXjpdVt3kUYGxvVPoMnKwh”Œ"1P9RQEr2XeE3PEb44ZE35sfZRRW1JHU8qx”Œ"1HEnfug6VUUuMpG8S7mTEN8d4m7EaxbKdf”Œ"3Gb5SAYZ81VQa1MJdyLhrGGcv8PU815eKG”Œ"351kqTaiC8vPzgww7aUWLvx7wDcG2BKVLd”Œ"3MTzZykUzdcLvwquh5RubecBKre7RQUiXG”Œ"124nszmyqDWXdgwGoGvH1uRcf9BgVLaV2F”Œ"17qq8XR9oyqkqWiyrV5S3Bpc44ftee9hNK”Œ"1EKA7D9BnidLofGTPhcqEKseHdeqAxGwfN”Œ"13ZdLVY85oTLC91N2Hqpaz1m4XCJDVrYT7”Œ"1PxrGYbLsPn9gsDeEcvStwHdLtJLgg6kKb”Œ"18S9v49oSrP9uap4893VPRfYVX1R2HJLJ7”Œ"3MMVTVsEkCrES8bRvG2PEBw8jfMFCYRnuQ”Œ"3MTpNe5XfVgceCPkXZccctXePXgqfo3961”Œ"157VbVVYTex2fykqd9ZvhEPSowj6bia8oh”Œ"3LDhDqsewTPk2hnsi9NGUutkGt91rKfA4o”Œ"1DkPphTdGJfrVR7ALL3geFFDCSZbSp2x3Z”Œ"12cvKhDQCEHFeiaytJX8YNJK1dc3qyRQi6”Œ"1JYV6sGhLFbArgcb8HAhGmtbHecqpFFgQr”Œ"16FoXkkh6z9pZUEdve8g2dhvzfhZw9ddd7”Œ"1NTuCMAzqdrHnViYjYdZaDZ7CJE34w53cz”Œ"1PAjUxJ7DfT3NaVmmcij6McvnTZGHewHyx”Œ!1tGq9MW9VBJnzS5vFMuHPPrup6GiuRbB5”Œ"16ARy1hjoeXZWtW1seThueBJxUAytvukwJ”Œ"1Le4M841f1q8mcZMHssWvqc1riEDZHC4Z1”Œ"17ix4DtihRyhNpPWRjCjoPVuqdDUc6q6eF”Œ"3Ph99DMMoAGZqQzmdRZYNGMinV5wmB3sxu”Œ"3HZJYDXAx8coUNAZHW9i7TG66VRMKcC7Hm”Œ"1QDSi7YZjj61wCfmSAmXARjLo4CmEkxyp6”Œ"15kmkYHaW8bxQ5DqknRo1s9S3a25jgH2NN”Œ"1EToMbaqR9uJCrTybYv788Rr6nWhH24AE8”Œ"1G2EGrXuPsnYJFj5bStPiqjvSE71NNeQL8”Œ"3Ln7gf52tiohftdFpJifomGdyogD8JBjTn”Œ"1FMsoDv1kawCMv9qjtjSaYa7SUKYcQJf1U”Œ"1GZDexijjytnfV32UnST3voUMS9WXeTr76”Œ"1M8FC42MLj2G4mcqzMcKKk2ppxE5Sd4amL”Œ"19LUEtZYzXfwRpGhT4u2PUDf2wxKQGTZUo”Œ"3KrBykcsVezJj5GMbkTQMknpYyTxUwSs3t”Œ"1LxFxNubknUCDbDQeooFDL4Jxfss643Tck”Œ"1HEf26geAv5eRqKYBpT5rNMxvbzwgefrkL”Œ"19z39NkcxaXSDEDcckQRGiVmWL7HKwSPa4”Œ"34ozhvx7Vq15hmCYK3adpcX4nWz3NPmNER”Œ"1Ad4L5K2iPC4TW1UxeCmGdt2hKmtspj67p”Œ"36ZoffBvdQaFwT8MarbFEnZPfcNZDnC3Ld”Œ"1BcLEf6e9Gx3ieBiM4o7kdzackC9PA6pMu”Œ"1BddgbfpxT7883QUMg5FBrDmYcsSg7EZAX”Œ"1LcQ4X9PHM5TZCnwvwXGMc54Rr3MeoNrg6”Œ"1BQuZNY1E8KoC3EbpSuDReJUDf2xCeWEBk”Œ"1BqcRgre7UNysexSjQh78t6knJY6xKyHFR”Œ"1Ezguz54JZmaWD2EeRQFRBGLSDQU7z6ny2”Œ"1HiexxhpfC7CkRd2ic81mNgqugobMXrDkE”Œ"1MLhdVLSyeHhFx2H8h1n4pxQDS152rw3WD”Œ"3JJWHdMoWHyoZvq8gVJnswf2YUwoDBYCLm”Œ"15qw1YC22ygQmcEL3p6Ee1u33ggf3sfzdi”Œ"1Mb9E3yy99sJBjtLKxvDXLc84v5CDXW8vb”Œ"16n7VRcLd8pFdXNuZZTyYVV4qu4N325ip8”Œ"182mQSQEVJfxpf6vGseSCJc7dMB185X4iL”Œ"1A2QVrHvErvXJUYTxrAmRVGHm73sTvk6HY”Œ"1MZP3RkMsffpABMJFG1EX3dN1bVfnbSzX8”Œ!1G3i2yW2bL5spPvu1jasQ296B7HUujDup”(Œ"1D7t64XqExhmJkzJz3HwhAS4tUfsb8zkP1”Œ"1FHREjqR1p7VzTmgREKpQXX4hu71MhEG7h”Œ"1H2G7Bkv5SAopGLo3CWrGWrWpQUKEgcpUG”Œ"3LApe9gvwyuWgsmYeWXR2nTfHoPCd6gW48”Œ"1HvRcbzn6ngoCL1ETqnoDoRB37SrtDKNiX”Œ"19h9SRr8qyiYzZDM4PrEE24JyZdRbfed3i”Œ"1LHrMLFbLWwPHL41pKhfujViQjZwUxgyb8”Œ"1N9DY9aKhYFjgABobcWpzwrsy9tRJky2Pk”Œ"33Rsu7u8QBe3CHjGTojKBfUhwKZv2mHsfQ”Œ"16KZM4PRC6Sjs4pfeonVRfZF9xssJnYdTS”Œ"3MKn11mSNTYgZMwATgjzhrbCByx9712sbf”Œ"3QEaGZNDTBqPSU6h7mdJNwPM1Hd9ujCCS1”Œ!1Y5kytrNomUzPoVn9f6LGtZcAoC4GbPCn”Œ"3Ctha213DsBucqPfkrgaPnkoLBF7rN96mK”Œ"1F5CPfVu9JnVLRWzvwCCHDxG9wdBcX46oo”Œ"1LWRLZ9EniACn5i48U1ioZ1n5Xt83ffbGb”Œ"1LWRrS2EGjdvWTgDDS3KUBWFEKrvFntdNG”Œ"39RZAcy94Xia59FQEXey3T2xkpdXyPUiig”Œ"1Mmv3nEFrnKaksJRNzxCXtaokRnwyUspvx”Œ"1NWw8KHwiGZUsAdnk1xAX71dLMoiQHCTRd”Œ"1BGtuyaUNnUGC39m8BSHBeBMsRhEgL4moP”Œ"1GpPKuQuy6tEDWYnhC7eWV7xU24tGcbUiT”Œ"3LF4HW5ryggrEmuinWVbxqwfhJjsN3F48F”Œ"1MVsYLyu8Knhy1gQfYuKaB2swSmDQ4iM7s”Œ"3N9CmKQHeoAcjHsgo4V5xfMCUK8VoGdVX9”Œ"15c4vGRHndbZWCepxS8HGvYY5fEGwk9SFA”Œ"1LjsVMyxWTPA2Xh1RX9TCXRBST8i7zRXDV”Œ"18aPQ1XxijJXC2UGXALw8VFpNUp65ghj1K”Œ"1JNburcpyo2KvuZ4RpJcqC9xsjnPLE1pa3”Œ"1AxH3ANMVHs2ApmCDV5swA1aB59eBDSUi9”Œ"14QJERxUC9d4fcz2BW9PXRvd4zW91FGHM5”Œ"15JufP8oPNyqRSmi8kkrRDRT4rAvSHYQUf”Œ"18h6VB9S8WQAZqVhED3NqwdT6HHh3YnGaT”Œ"1MkvnvnH8L9ARDBCsA5xNYxUnmgSSVm2kS”Œ"3FNRq8E2fHRnU4Kj5wUfdr1UkbLdnKQ49Y”Œ"1K2TPjkVyaCwZfdkchTazN2xJdW6ZhCUK6”Œ"1EjGLc1pWuM9G4raBQBhVGZnmCtJf1M2RS”Œ"1PV5HAqYTU1h72YUNdwzP9USPqyL11UXa7”Œ"18NsGYmZJa3woZ7zE5kzLVqfCg1CcgSP7K”Œ!1R3n31VJdQLcBBK3sMTJfTBKmFSvdcjCu”Œ"1A6aXdtDfa1UaEeNzsCv7cvQT2tvsoJFgM”Œ"3GTr4z1o6vB56aFqku5Cf1zDsJGySszJJW”Œ"1Chc7v5MXhy4ZW2z7Uk8y42FsaKmyBXsBZ”Œ"1Bs6fG3FeacfCHJueUHWcBr4oQ97P7z5qC”Œ"355L9TEymw1ijn9JFPXzvX7J9dEpcBrUMS”Œ"1KKL66ZidhXdYp9tLyWrBrrYLKUgpRcENT”Œ"1JSu1F22ay1usrZbFXecyskj6GvQiKwjCP”Œ"1NEUXrx8RuyD1oYYN33QWoiewKoUZEr8KW”Œ"38Fcw1uHDF4AuhVSG5dR17YsbnH6bqWFcW”Œ"16taeZSuUFwzzSKFoSevMJMb9gxwgm4ZcM”Œ"133XYyGN6FzALnpDXuy1pARecnWHGRgPSv”Œ"1AuFmG4iT5V9LAWJ9atnLGYu6MBzrUeNCq”Œ"1FoxBitjXcBeZUS4eDzPZ7b124q3N7QJK7”Œ"1GZTZiWsZo7wGJh2ddc7jZWzcuMrFD8Nt2”Œ!1M1QN6XxtR8i62WsjzQ16qGZQkFQqkgFx”Œ"14SuSAduHF7zVejz8TKt4W1zb4h6J2PWAt”Œ"1HHd8vbLfSVgf8uwe1cDhX4pPA9MMUNLNi”Œ!1t11DXPgE8N6ZfG8htbBGYUP4KYQPcFbT”Œ"13jsX8xyeJNnE7vs51vg3bYmyhuRpMMFDe”Œ"1BSJ5gynhAtXni5mPp28GabydQfJ668eXd”Œ"13KFBFUbuvQrsrU9ke4hj5rKgvAaw8xb5V”Œ"12eJoCUJoJXDXum89XDTBqEKFBBn61aMo1”Œ"3CVTRxtvzFttHAAkG2DitcUzWGCTW3j9cm”Œ"148fJBPFXGyoTjStGtdfJDE6vaAoztHFpa”Œ"1DdsLnCsECYPUnDjmEirhoWVb3VCEa2Bnx”Œ!1UEhBHwSp32fCnoDfUB1CKHxwu8mHX9cm”Œ"1LrXXBtEC97C6b3StQkaibisRKzQiHisbZ”Œ"32iRjH1KC9ABNVi8nWMRiSfJLT2VT2fdCy”Œ"1MkQ4FASsFtbyG2LU38srYr3jJ8yDPcLhH”Œ"18pHsqaNfePcHD9WLwVHMd7aq5yPSwq6zE”Œ"3CfzWWvn5UVWzspvDrGN9mxKwUfvTWAwLE”Œ"1CMhkrRaBTpnFERGDBsbqZg5CBbGxSFKak”Œ"3Hi9TtQscdk8gKPcn3tYrHVHAqtvg7CvrD”Œ"1PECinHfDkCygPfmZDx1mZTSyYHPDte9gu”Œ"1PD9FFsSVVs4LfWsiXcQg99bE7rG7QooaK”Œ"3AnjaXFo7ZSyEzQffDXBZUPSsGGcMQQ6x5”Œ"13UdpEWLZozifbzVdZ8pK5Dgpm2JnaTSns”Œ"19C6rCXrFt6SeLrKkAQQVsJeHtocXTE7BN”Œ"1793FfdMZDtQxHE9xuyrgKYTvwNMg4GHxX”Œ"14uMeb7pZRFJrxVb1FeA2QPAKoEExqQqnz”Œ"1MEPWNmrSC1LwPY3ZEHu29Uitxgi86mSRc”Œ"3Eta33V5qyzWEy8CKcUVPKX8dtph3Pk9hi”Œ"18fPoaJTNh8RRzZTYaeRTVCFCf6z2LkPJc”Œ"166N5hj4ZzWfumRJ3w65CWYXukxJRmZrUb”Œ"1LPMgMZTzV49u1NuLFFDrFZxWWTQ1Mgtum”Œ"1LU6otSadvrCeVfeUvnS7PKGgWTbrFNP6J”Œ"1K7Fw8YwTZke15o5M2vZetLB2tPF5XYFAf”Œ"1L2GDfpokn4C7YNPoJs7PL8tGRTxSoBJJs”Œ"18xoWo4Gev4QeKU4Btu8q1sWnpF2r91znV”Œ"1LvcxmoWsn7PWSr4ZMmJHUkEmSDyzqzEqs”Œ"17RYYq7EwRRv9YVXcjm3aLb7zy2M4ChwpT”Œ"1A26FHMnZZVkaFQSkoEFxwM7KcaXaXDh5A”Œ"1FzKYMA4VWBXHGMceTkMob36LaKUWLtUxK”Œ"39yaHMTWko16cWVLBMzjboYKmrwZdLwa2a”Œ"14naNXRgXoKFHG7TKuT2P5CP7tLL9kVL12”Œ"1APjfNYjNUCeMDCgwb5RmnE8YVcuCmkehT”Œ"1Dh17dGyWDovKtK7XLhxgcbmHMxFrWvyKS”Œ"12ZQWFt7GHsNK3Rz1kNLHjtYLoVB326s5m”Œ"1LZYu9booVXzv3yFTF1CudsBtzL78Vf8Bw”Œ"1PtXa8C6tzmKiUZsxKSGTqm93UE9QsrVoj”Œ"1336LwZaPaMdxgnc1Lgwr5yMVJCbwuYgW1”Œ"3PC5PjC8RHzZ1vhHeDsx6HtH1VdkiNNU7J”Œ"18LsSWfEYgj3J2CzJqzX4yAWYedGYbF8nW”Œ"3Nc4cS58WAbY9W4eTq5Y5rxwHaSC8CTCfL”Œ"16tgXnb6fYfEdpiKdnnR9aADc5q5yJx9bw”Œ"3CBpkcEQdNikZqHZqH7efpT53dPesG2mhG”Œ"1HxaSZQAKg33KNpH23K6XLLPhEYGr7ei2z”Œ"19KDgUEhUWTSUecTRnmJvhw2NZzhea9AZm”Œ"1EKv7UqBdJEsnSjtwuvoYVQ9FVeJQDZRaL”Œ"3DKFBkBj4D2mUXyPzFB8DGKNNcpcAHcefN”Œ"1JvCAqsT31rg46PZe6GkH7URCf4ELGCALP”Œ"1CBKdx6oQbdDTh1BcpAWWT8k21Reh8ukhV”Œ"1MyGFRBswNY21xARH79oArYJ6pHPsxtLF4”Œ"1PxExZb1aKmEHactxx3XsitExdFYs9T3ig”Œ"1JgxkfTBCnQoa6Yg7EfPbLjyHMTTnVMQj4”Œ"1PGH6bUey48WNqHjRBNcgwiWFWJuXvicbG”Œ"1NCz1nnn9uovH3CVGZjBB5e141YXRgkHMY”Œ"3JcTa9Q4oT1ReTZAzj81AAWHRPEuZyFuzT”Œ"1JmcUJgt2X8ztxFrj5XVw7tNReBTPGvGZF”Œ"1GNC2RwKn32X213FVUvqf8T5DANp6q4qh7”Œ"1JYxKB4LHWnTQkRwcbyqA6Tggktusz47YK”Œ"1DycuAy5HKTxkD5dKosBmetHRWiug6Cmpi”Œ"3ANFcAKU9tLwcEWcp1nqP952bBqAKdXzz5”Œ"1KKkXvFQkiNkTkLfWiy7URfEpvWjHa8sC6”Œ"1MFYP2vaAySnvYSMs3qtx5XMMi9qNZmvCE”Œ"37ppvrphPJGmboMbFjhKywCrrDAy2WsrpK”Œ"14uxhxy22vCPWyaktkV4b87B71zGCNQHMY”Œ"1FE5zL1eeiXUTjYMy1ziidk2zmhEvrTVNj”Œ"1FMkXiw9NJFXuzN5eeGyCEmPBLKm2nm6FC”Œ"37BsD7ohWcEbbgMPkChPAuFbdhWA9ZTPHS”Œ"1C4VrkAoqjTcYSDdH6ydGLX775GGoxEo5L”Œ"1Fp6t5883ZiPcgjosF8cnGWPoKPaLkmXnP”Œ"1MSQc3L3XnceozRHMY2maMd529T24b6tT2”Œ"3Ev3ovRN51WotPdJqBQ8r2ztqQa4WN4mqi”Œ"16kq9RTDMHRuxxJe7bt5Qp4Hw4PBVFxs8G”Œ"1877a8RnXY1o1Kph3jRv72wCmRTyF6hjgc”Œ"1GCk6pRxwYanf5PjZ4oBHtm8G2NPnn1YhZ”Œ!1jEhxEzUyYBvMRVh15esmGpGHWFPvV1Bj”Œ"1JQ1mDpf27Z9yZYfGDf1141gAtFmy1JFYF”Œ"32ma5G8Q37idSG84YstroBGEWxHrz8WKHe”Œ"3G4WCDiVvXa6hrL7veo1Eu2oqNody9DyAU”Œ"3GN8GXRi2cQirgKaKGkTnumh6Py3rBTWPD”Œ"1ETsaB8P8Tpzybmr2EBsEUc852kTDB5qtN”Œ"16rbYsUPRHYBVibXsoGHvNSyAx6Yxjebvk”Œ"1ADTN7iVXdbJJFMBobt2PrRkDEXWuPC82q”Œ"18gTxprVf71Tk3D6ACjyBAEBHLGAKBgeR7”Œ"1JoJndpA8zDmhtTUvTBHpGZe6fpupMm38P”Œ"1HVzEacJGJTV13ghvoWh8HQGZUzoMJWXJx”Œ"1J6aJz8GFwV3ob2d3EoznHc7iZDHMRRfZt”Œ"1Kit1EB7JrKdiKtKnW5wbP4RfarMrVYk7G”Œ"1MoAvnkrZo4emzXbNZra6cb2wmRcrjaxf7”Œ"1DpJzWhY9KuP2UwEDaCmQpBC6GHRmwKuoK”Œ"1PhW4SGQVmAejfzH5FdHMVhw6ogM2C957M”Œ"16GUMNsqZz3vRBPKuDCDW7sU8ftFCGkeNZ”Œ"1Ps2yCj9fjvVuAHjuWvucUzDAR43ypf6Wk”Œ"17qajKRJNmvQbSZ3tPJqtyDHku6UdkAtmK”Œ!1oGpD7NRe6aDZ8FvWkiupLVjAEP2gQWMB”Œ"328R5fCgE2m4TJPKPD8PdAgRMp4CmoZyzs”Œ"34gP8dVZVAsJFwjuBnivKbJH9rqPxctBza”Œ"13XVZUpsFKQSpbbwPSuQFQTXtHeayH3PpK”Œ"1FRRwsF5m2oT8bwYNmMRWa8LwHF9SwP7Vk”Œ"32QzH7uk3fmbYhkzxL79aysM638uTvQWyQ”Œ"14J9TZ9PtzDsq98LeucR5FQC9NZceonrpW”Œ"35CR7UjrTm7z7Z7SZRWRd7JpC3ncixRxuQ”Œ"14gcafDb9xFjmf64xgC8QnKbCPPfWmshDP”Œ"3N43K9e5Qdkui6tqEF186jM35oWibMBUbY”Œ"1ERnxho7a2RZnyd5Sq7oerVepXTGDQYBS2”Œ"16uyqZfB4ySbxKLQ7nwBgYpftuKfxisatZ”Œ"1632iT1AxCQHqPV6vV6gHRSJNQm11SkfQx”Œ"17pv3YRJUucok6dR5MRbevBS4VwTPBWwb8”Œ"17uh65Q8xbrYqxh5pMMyJnDX2Df4FcB1a2”Œ"1LSP6N8UoCMPnjbxph9HnEhFCH13eUm7Sr”Œ"15ud7BKUmkWeqdVcJ3W3VUtADBXZJ63kBs”Œ"3QvcvyYb4DB7pKL15QyK4Ra26ifCiP2eU6”Œ"1C5TtDMKk6Kjg1PNfnjsmAz4xz1HZs6fvN”Œ!1TipsnxGEhPwNxhAwKouhHgTUnmmuYg9P”Œ"3M8ck4N2K4WUqE595Vom93e7EoJq2JmSo6”Œ"1PFb2bAoY9H1udzEmMCTGkybnfg6DgGeFw”Œ"1PUuXFxAZPLGmoMea34t2Q3dKvjNGkYVdr”Œ"17G1fBZmKZ1r2xav7cWfH8rDHV6SSsJwhE”Œ"1AmsniZJxWi7LJLwdGwx6o33xeokw7Vzix”Œ"1Jwm1fF8ASNFZ6pSVDtsJuh5gaRgajqqLB”Œ"3JxVMemEmsan4CrYjR7CUYcwzcUhvdzbCm”Œ"17i1AmD1ADSvqWmDcTKXxdCNNXtfv8XkJD”Œ"17py4MrWM8kp9idPg4Hc2sFYCFWMz1z5Ty”Œ"1FEEBhRhZo3xTAST7CwWyjH9TEWGug4N2f”Œ"1HkB5ujWRFYVkN2p93JAocipxVhFwFA7Wc”Œ"1MAxee6yCzRwb9dJEUbfTZFVLRvp27h1Et”Œ"19t65WKHzd8ueCEATaCHNwADR8PVgZd1zv”Œ"17WVoKym1J4UoG9ZkRkLRcFeXvcSuLVFKV”Œ"1ECDUu6BthW8NXD8wV4o3tgaSRmvjx5nJJ”Œ"1JsmCqwhCSyMt9MyDv4YHf2vM5iKuhnXvM”Œ"35XNscx4GJW8PxadVyk5DtevERCieGWuuJ”Œ"1DJVJMQ3YnX3rtZrWhyYGgPihPUyaatEsW”Œ"1JpoxY8CiiuHaGe2tuNXzzuxiyMjFnNwwm”Œ"1HPMs6bBKfXXtqAebAHN7NQV8hwFqDpj7M”Œ"13LxDGKa4sAp7D7KXTPP48CLeQxPbdU3Zg”Œ"1CHdTTtakRuXmF9f7D8MpHRNVLazawwSUZ”Œ"39icaRMGmnwF3WT24RMv2AaVyYH3NLhKcq”Œ"1BEmYFLZuGNRic2PLjPzFLYPMbqnBNB78h”Œ"3FKgxKfkqMZhG4Vr9c8Bax8hLFePrk4BG4”Œ"1FPfyxdQze2GY2yyAr9gzBG4ay48wpahA9”Œ"17kfNdv2qsCdSPcaKojKP18xB8dau5DGzy”Œ"34dwbuLrRSFKv92wd77z9NxMwZWEW4LEX3”Œ"1MxZEzcRafw6Nf2E9yfsCsyX5pYazTaKnf”Œ"1BT55sTe5xZL3midBYwfNum2ddRrJzq5h5”Œ!1G449VL19JfVy66D67SaMyEwfMLJqWC4G”Œ"16rghGQjggwbpbKAFxkXWf3igFGMTm5fzQ”Œ"36me6g7pAv4GMZpuxX1Frx4dQSncv5o42y”Œ"1DKDyeL8QQZ1yMbtPWDFu3KjC7DFs5LABM”Œ"17eXE9KtRrFcL5BeFxuzuJx4sHSbca4vor”Œ"14DQx5jGA3g5S5V2uW5PgQbDzpLNGsh6m8”Œ"14eUc5rKLLpfv4TVZpp9QscfveF5cwiHtr”Œ"1KV6y97tkf4z1ZTg21qF7LXDviqw4AtALY”Œ"3MVtU1w7yo6UE6gtMvhmrDV5g4B8VWNHb7”Œ"3PXyub2vNuE1bXJftTX2y5Z4oQHVimyMEJ”Œ"3Be28KYjmKkFxABDxUQEGpmqmGBiQ7Sgn2”Œ"1EagvF15Hbq4u5pyaVyWLScTzTJ1gYvrtr”Œ"1AEqfZm4U6WDEu6R9sZvUocvuyPVvD1T8S”Œ"1Gphgsb1KLNXdKYhnUUJ25USNeAKzZxYh3”Œ"33gw4miw6FLy2A2tDMDMj8MCNvpkTo2juu”Œ"3Q6XiL3A9WBoLngra6CfYMFduuxD9Eja5z”Œ"3MfN5to5K5be2RupWE8rjJHQ6V9L8ypWeh”Œ"1CiE91h3tXMdQkoYb5xhy4xR94fhrcbjAC”Œ"15TWPWn7W6ouLWt8EfLdg6vmimmp6aPUrH”Œ"1BtGMuYaWi7yB2mWWHguDLzPJsx52dBhJq”Œ"135TGWmDmYSoniS2FMN5wTvQWb53nzDWWN”Œ"1LkgsgPRuYqnbe5dsyhc4zDN1sBg4ja95H”Œ"17xpZNiUxqVgHkMWVyVNXfP2TdupLjfC4P”Œ"1N8hW7RFsyfxwva6J4mt2JqhSnLqVbaDRh”Œ"33M9q4MdTw9JZ6qg4rLfC1ri7cWVuz1JPU”Œ"1CfPdoHcqJ9X6kA1GMSA9opqMRwHr8SxWo”Œ"19jWjVukGNJHqBRx4AA6cEzxa6FWS7JJtR”Œ"1MWNoZiAtfxkav9G1zaYVXD7unegcMhMrF”Œ!1t2XyuXDZULQg4AgAS3cdXoGtud4QhwMz”Œ"1Q8kZUxmPHW3hUN8eBdidJ6WhXNi4Np479”Œ"12asrBs3cvehPfD9mSpGyb4NZHAnK1hhKF”Œ"39gaib5C3DfSqR9xr9A97JQUUxsk31ZZQc”Œ"1KYzK2d2CXtSJi1SzFhAYpzowENRs3G9q5”Œ"17QWAonVvbNXRwt1rboEn4biabT3Pj1T6D”Œ!1uoA8xQmpgUsEtXuqoitjhNsm7vhdC7iq”Œ"1DibkyG192E1mgak11Ys9gJAW6vjHWnUdZ”Œ"193TmiKhvW2SjasYm2Ab2v4FZxarauYuqr”Œ"1HC2qWu4QME8JQat5twjuqAAPWpTjhPhLi”Œ"3L1aMoVbtJUZLZFUynjVcso6TEYpLHNjRd”Œ"1HTUvPXYYyh6uNjx9MeteZCrT6oZS91t3Z”Œ"1Ht7w8zVoC5AmZqqPem3XX1d6n93s94LZR”Œ"13nGBdmzdxfh5uFG3dWWqamxqPn9w2iF92”Œ"1NAjS6ix3vq2Krv4MsMJiuXgbask65tPv8”Œ"1AQ4Njqj2ntPEKXu6jkivu7khXmgF5N33B”Œ"36NJU3svUkCbzHqf3V3nZ2iuS3P9zuEnXg”Œ"16SXNV7e86LSisjWxc1sLVXHwNUptufTuN”Œ"3JbmJpsWZpDaXbattXbgnpbTyFSPvpFUA8”Œ"3KQC3sTCxnutRgzXK9fPSSzmMEPLuieXE1”Œ"3B7Mx8PKPAChjeCNshXPmaS5uu6drqobFZ”Œ"18P6wdcMZy84z53N7ZvnaAHx5FYszQexgj”Œ"3GFaVSyVhD5tg2JFE6Ftueek3NHE6XsZKi”Œ"1AjVb6Pz8mCLJE9NcYGtAoZ29YktK69uMT”Œ"1BTMNPbwK6xDjPzFYtBDw5kWHZNdVspJXi”Œ"1NBrqZR6Bo8EK7SzUXihnquasR7ZkxbZcn”Œ"1J6LWpAqfV52ZM4Uo25ZPsUiFEtZEuqVqq”Œ"1Pknv8QWbEWfm7WxxRcwBixUkT1jc1ZdRK”Œ"39uqxJqkeYvLn66WGbpdkHGz23n8JwMiVr”Œ"1K1soGM9D3PK2EZmcESarhRw2YAcnNPtBC”Œ"1Pq8BihcnKmnyBHVjeBZQB77DpdC8uJ3wQ”Œ"1HTziwPyLHCDdb8KCgEqJhNQgyxsSx1iaF”Œ"194zMcwXswZtkMdpoX9BYDc4T4iGqW1G61”Œ"3AtbwzRW7D6DJLgrM2FiAmR2JnUHpuSEM4”Œ"3KbCCeypeZTkL7MFVMHtYLQqbS5VJ9FLBT”Œ"1BEah7FxNJpqif33DeUEz7Ft77KwihG7Tk”Œ"1GtQDzzzhwEfNXeD62FQF3iUNXctAFtYTm”Œ"1FoRy77XWfxq4pbjeHMwBqXzE5C6YmYftQ”Œ"1MpLojgNQKZ7JdLWVfdbqYTB2seBYaRkpJ”Œ"33K3aLNBsy1j9g4iHC4wZ7ezvotrVJRWU3”Œ"1G5UYdjtGNbdXh9jXJRj5Rs72gUzjkPjdq”Œ"1PdvqRUySpfhgiZuD1piwu9xDwYTykdRM5”Œ"1KQSR2gKQwaKRF7kMG82w1Vh7BGqjShwRt”Œ"1Fxoins8vVYXuBjcFpP9rpF3Tmoogr9hTE”Œ"1FAG9ko4Ahyuiz2Kz4seyyFwhq2PLnqHdA”Œ"1Hpy7XmWwdE28q6C7VTyewQ4QbjnQuxFrk”Œ"1BTe5YD75optEBesAiyxhSzp3ZKij9ACLV”Œ"1HK1spLXh7XqPxJQ4TiWzE4nMLhNZTsQJz”Œ"1Bs6bxenJwGSJnpKNU5GbAwV2mKjuFrPcK”Œ"1KqwcYKBN9WiDojbgCYNsz4itXgERJUX2C”Œ"1PV3Qh2ciJLy9d2UMoHCXtwLRYoDid5mWE”Œ"35JbuxZXc7vRcBJZT3eWL1zgxnA71S325Y”Œ"3Q5KheH4k7PLX15QP1JzwrguD22ZBhpBTQ”Œ"13vStM4u6MDMqHVTNNk9415xshcc39ufhp”Œ"1CUt36DjLeLM2PP5sj66NPgVB8tKUbKfKk”Œ"1QHa3GCzReX7CeM2WawhN3k2L774PZCiHs”Œ!1cLBJczfQjDpiZk1ZTnsVw7psJDKGxujw”Œ"15tEF2SqoG4nV5pigkjq1eHJ5VJjzxeiRF”Œ"113Z1twUMf7yLCS8yKAxTVRwrVRiKBgSX4”Œ"3EPUhCGRRaRcMyKjCRY5uSKCd3ZVKDn9DX”Œ"1FFDjnG4RCs6Cuq126tbV6SwRSf8TDGrmt”Œ"1NLnoGZam1pkGQJDXKBNZLpGkPWMXLpC7u”Œ"14S3mZxAb9NRiaZaywBjCFxx4JvNfHsSSy”Œ"1KVBvdu5z9hwA2VcK2k2zpy4WhnWkXiiFm”Œ"14JNWv2Z5nBFHA78fXbaR9XRnpryFf2KWB”Œ"14qHG8KUkmShty6tekGrNNT5A1Hpa6pc1N”Œ"1MkZAcijxofwWeoGtAVgJhCC576jXsTsJq”Œ!1SvKBkra2EmLTbG4QbypS5LemeV7j5KLo”Œ"35oB19PNB39SXUs13KpReBeVkXtf4zM7Ae”Œ"3KpQUcj4ww2wehBVbFmRTCHaQkCR3dPBBG”Œ"173EcCxfuhE9iFbTRV2khCfyw2Wp7YNLcp”Œ"18vu2saCLoaa8RGaib5yR8ExCCggNeJ8ks”Œ"14sUMW1WE1dJtWe1mfG4kAHXRgADnZwMDz”Œ"184mB9c7kQSYUZd5vVudHMsuWTfLjAM422”Œ"1JDTDDtWb9rsbC4wKsK3ooS9p4ARaHVJkz”Œ"1JGV7vncWjDKQvN8e2vzEUL2JePfNkUNdT”Œ"1LwbidB7su2cYR3Yfse1Ahw2iUSyFPjtMv”Œ"1EmdBoW5BLbcd5oewQ1kdZ4Djzhb8hm96f”Œ"1PZbEDrJjTCbe5cNg5esRwsXs1B1gYvd3s”Œ!1Yc8V3HCDhdZGPoLKYcQrpy2BsQxEmiom”Œ"36BMQvvjhMSeZeS7ZaGtAnbDv3dao8VK3K”Œ"1KTEhY6R3safPpJ7AX74bS17Mgo6rhQ2f5”Œ"3FVHHe2YFZeeMFwPw3ca8kSKYuLJ6jPkKN”Œ"18qi6rPwpgdBaPTFGo73mGeuLb1D9vHxRJ”Œ"3NLKqBanDTjabEH6XNuds6sKh2r1LnS7GY”Œ"1Kfp14oKkPZN9pMxQDKdhqcN5q9jjUiJkB”Œ"19fjbFMFLh41iAjUvJmhqr3AK4bdbgh5rk”Œ"1H4AeChhuMzSBspSGNfTWQXX7vSgAMGPhF”Œ"14xBPh3ndNkiSzu87ctzMp9ZPMNBTm66if”Œ"1H6crfEXkCgsjfZwe49kgkyrbbak2sXDuV”Œ"1MTpXkWJAtPZZN84Y1wiUEmFjpwYuBQm6N”Œ"1MqF3v8f6JnrRKGokksbczYriiv6boTA4L”Œ"1MaM9fmnGG45GNeLndcqXfzWHW5tSoU6iG”Œ"1NAj56DVZ5Fuqipda2Aige3jBr1UW4cXRX”Œ"3BWkoArw7LUc65huv9nRDXuoJarszZN1Bo”Œ"15AXvtzywH5SZb6Aq6EF2hafsq3GcBq7PH”Œ"19BvmJAvM9nzDr494ULoZhg1PkNEttAsbD”Œ"1GKyr8GGk7dgZYhEJWkZiJRmGZ6KT12Kq5”Œ"19SRF3U8SQdYPCdegU8jNebW59WRHZPsGo”Œ"1BLuwZMeazWNUn5JVe2QxAWxLg3WsaUVTi”Œ"1LuckyR1fFHEsXYyx5QK4UFzv3PEAepPMK”Œ"1Dcz8HTCaJqSZJcXN9GY6ZGf4GcZmxthn2”Œ"13ADqG2eGcJVtMgef6pHQ9PntdQWgEAFwV”Œ"15P3EMct6EvGt1BUFGYpvzp2wmZSdLVepD”Œ"1CMBrjSqpJoigEQTDTmermreoPUwB3C8qQ”Œ"1PZALRryEGtgYRfSBYCrkzxwpiuJzuR8NK”Œ"18CuhtqhQDwY48yG1duFKoYi9kDxFUjQ45”Œ"157qWvx4187ZAcyt35F2WhNKG6pTL3jsTa”Œ"18Y4KgS3eM9TTwiyG4pfbt8Ex4ofJUxz8T”Œ"1GGowKvHYKey7H1FSKxUQ1cduNHUxvnUTu”Œ"1AHHZm1XuhXLPVo9Ru6HSGgUcYrsHsRpZd”Œ!1jpQ6iMXPufAxvPpaAU2kZBTYqWzEkFsg”Œ"1EAxFQm1mWbAT9q2YjJZt26LpsEGJMWwMU”Œ"1PZ62yfSEL41zEgKpJu4YZQb38gb3qSVmD”Œ"3MauXwnQSi74VohGXBoetig4Muo4LeNGWb”Œ"1MqKCQS6RoAX7soe5dwEBoGTjy6dPDZqFv”Œ!1tf2NEXTZbKnudKYWYA8GsQEy2ssxkpAe”Œ"1KRphbNRLbJFYRWXt6qxUNUMHaQt9T7ycd”Œ!1X1yUefb4S4hsmown8vnQbwmZBx8WT6MT”Œ"19kbgxiCeFVxpbpge3Cu4esTqqaPtqprSM”Œ"16khUQCaFB1sBJHjn2A6PKrWUWYbxLjDsd”Œ"1BzSF26r62bHoqHmXJSbvEHSJ6FcaS527K”Œ"18uVUc5XzS4dQHmqzARJBvrZEw6jJ3sLRT”Œ"1MtspZRe7kPRutXtgZGQDozpPivnYZK4DG”Œ"16pKn6yq1XQk45SnBSm8eBurzWi6SwrWpp”Œ"1HPfTXugAZWiiyVdMBBmPLs8CQDqe36La5”Œ"19qd73aG2UbVK1uHgGnKeqdVqnUSEfGG9g”Œ"1Nc9T3b7CGtaEVsR8W5cgm3xdpxmqr3aXX”Œ"3BM7KqfZh66RKrsp7AdC6iuNN58tnnouzc”Œ"1LszLCZPrMgVi5DXV2iD5FE7vCpbJesTFr”Œ"39zcodiARChh8UQkb63ahzHjtHsE26biRc”Œ"3CHALNEQEgAtRqXwaY48BBF9iBHVVtLExY”Œ"3JnFBLxDCutY3bZEZsPTkHAaUA1bxmEMX2”Œ"12t2MUX6mJgyo6iaF1o9QgeuAChcsbCkYh”Œ"34atXCSCZEGdzGpHgWSkiCkEQ4VThcEJc8”Œ"3BGQf6iKsuvv2RukZ7fhdoFN41gj3bwsqg”Œ"3KM9BNAi3dqNoqU7MAVfvhgL1YuBi28BSG”Œ"1NhtwaRL8nkUb8UGSAsyBgTpuLFEobxMSD”Œ"3LG2YJ3jVsSR5cTXMNKV3MqkJsV1rC4Stz”Œ!1Ve28Mc92wMGxBfTjJmGYDaDqA9P6Juv5”Œ"1ALdjcUXbDiWGeuP3ifQuQGJYnTEQUdeD2”Œ"1LTyzPUdD1DFfvcpb7vHRoYKHcuaXJ59r6”Œ"13ksWw337h3wpSEc48wa8TCHKj5P2fxrhp”Œ"35JDE2BmpBE4zjuLena615JKobQzVG3p8P”Œ"1K9D3KkVL7Rtry2mRNHbrmUytnsNmS9cFk”Œ"1MhYWuKdTdaR3mJZbuzpiAJWh224ei7xYz”Œ"1QAVVCqEJSUH8kxs4VUqoBGXKLTvjnDW4A”Œ"1B4Z11i1ntNxGYP4Z3UwxrSoRKwUnxr8qT”Œ"16DJNkL1FefG7MMTZ8s7974nUBV7LefwDo”Œ"19ZEzgSDW8ZQsNBxZDUr9K5nLb9M58dAio”Œ"13ChAdR6eqwzterJq2TeRiGUQo65SMiZmF”Œ!1tzozh68hfHiioEgD64BgZVu4P8H6M6HC”Œ"33M4FTa5kimdBtr3QJzxpqW4ZsxyeiJn46”Œ"1CTQJKfJ9XGEb34RZQax6ZAmYaFreJk4Do”Œ"3CQAaxyga24WKsMa2YGeTMvoktmVGj3okA”Œ"3JfdZdtidtgA6HShQsKqps9SA2ESmHo7KD”Œ"16yJdBmptfhhMXNxpaxWoCBFHC9UaqXtkP”Œ"1FCijomoweX5ejwy8QSokUCP1YL6RbxE6Z”Œ"17dudJekEqiNfW9xS3sc1GuhZR5pPYbY65”Œ"15zTN4nS8B4yZN9yoBWDpLPjBMpzaNLrTW”Œ"1K3SVEDgr9P7iW8he9hQKpSVeSvNt1xxpA”Œ"375fvLA2eDfUc8SfC92iMRNH6xtvL6bdAh”Œ"1ABAH1v7nkXV2fiqDoSsjhxf7hEDU8RibJ”Œ"1LnNsUycA3iAWhqKj1L3vEH1GV6nJE5NtF”Œ"3LkGKNDqdtrrTb14hgw58xfBDwpYuJQyvB”Œ"3FuCEUV8qfNPzs8xVqzHcyTDm8M9Z9TW7o”Œ"1FYb2AGmfY4BugDPEYRphDTehXPmDvEP78”Œ"1PJGcRRHgm9YUR2U5k7aVfY1uUNA7P7nJP”Œ"1LjtXnjsvLo1oqSdGBKz3LyFX4GSxfV9vR”Œ"17xqGo8KAqmoTmbZAyxqT6eRxWqdFDG8Z4”Œ"3HG3tzCJYdD4APFvfrPyEidCXxC5YfHpJW”Œ!1rFLsyGhN4oxmeNwqXdDPJEvYmwWYpfpx”Œ"3EiZxPkJdCTW1stDL13dtfNzQhoeV3enYp”Œ"3M1BNXu1o9w1S3LMNUSNu1nw7qqLqfygsx”Œ"1PJv1cjngTeAWXRD1PQdGxJN9s7MMb2VHi”Œ"171tfDWR2sHJegGML5c9AMcoLXi5ZV6XuY”Œ"1Np8QEJ3f3AT21L5TC5gcMp7YyUFzZ4HUg”Œ"3JhFqd6HtzFLfAYeXxfnR7qnkZujes83L9”Œ"1FQPFxp8tGmXQg9wqnDTR4ezCMiJg658Ak”Œ"3Q3EbtXXgvc1YTFTyYCYGNXw4QYyGG9s9C”Œ"3NjooQceBnBpVLk2kkF6uC9A63KMbGrFVj”Œ"1Kwzcp3eycWRFYNe2jNNbqBiMVZisu4QZM”Œ"1CHSFZo1w6VGshkzAZSJNxaUyXwACdfmXg”Œ"1GzZDcC1z4s3N3JqBDfjEktyGnQN5puzgL”Œ"3HCLjE16bxKd48qsNTjFvUMKSwmwSCw18K”Œ"1NfLUCeagGaULwE4mAS5pbyvEFXvtnt8fC”Œ"3LxTCKSJ9pSZESuQSo6Wx7dpT6ggCs4XK6”Œ"1MAd62SRLVkW1A32dbYgUiCoSar5RFpbab”Œ"1MgyyACta4i3bXGexQDVxNeq23ZVccn94R”Œ"16YFTEMhRj64xeqSoLau7MrWYkAAVswqo3”Œ"133b3zTf6uVH1Pq5oYxDGwM7kpy3kzSBw8”Œ"17Y1Resp5Fxp1JpS4Zgry3qQx8MmYL8xmU”Œ"361shH6XRc5kyPdKKpxN8r7oS6hvv6pyBU”Œ"3AP3ej1isQGTMVTp7J3RTNwPzJ6PzKXK9u”Œ"3CYi8XsQJWZnWWYQMGjakhj756ZARnLPeE”Œ"1Hkhe6ZKjWLRuu76h7gKUFEi5P7fZrjWSt”Œ"1FGo2L3ndgRJgcZmbB9nFj3EZWhZzQaS9U”Œ"1JjjnR2duoAUywHrRedkGSMuVhd7qGgwoA”Œ"1CpgfVRXJic8CF25DaUvDFDsDhSbvwEAfA”Œ"1Hin5vb7QwGgWcMmmAFZ6vg8H4ABfrUfMv”Œ"388QdPV6XKDe9TALTqRjvM2bf7fR5Pw5KQ”Œ"15n6x5AarEKX6SBjd9PjAKiPgbW6SrpKAP”Œ"17vtVxWSBCqj6ss5LRVkLdHEnBUzUXhuRm”Œ"1AeDefRJmomzeD2Uu7HvUr2pp6srrinvxU”Œ"39vSkGXEWuTzMmrySHT8bBVECtbPZMUEY4”Œ"1AzxCgdDVQiZn9wA1aUNfnYpPp8Aua7cC9”Œ"1BjNXmYyHuWUeF23BP41H3dhYVZGcm4uvg”Œ"3D1Sy8bvj2dC9b6ANEMZbH77mwasXaig1u”Œ"1HKh1cDQRo4PswxG3fbbz3Lta5Wdjd45hT”Œ"1353xUpRBPHospjYc81GNrsfNftcut38PQ”Œ"12Lz84K6iHJgeWXBFjUX3ktdCXb1ybnnc3”Œ"14JbjrF6FPDMaPnENcELQuKq23SAKs6cyt”Œ"1FGCG8piuwLTpou9R3HFvPqw619AiRq8ij”Œ"1Kb4suwrVChWYeNvQ4n6sUwq7CNc3bCAYd”Œ"17227EaKoRGtL6mufrH9enMYULNX4QJdzv”Œ!1YTTR7LowFczpd7vQSQ2P5B4WUowQodTv”Œ"3GFtNbGxB2dfZyCAmXGkG4R1s5CoAAKZCM”Œ"13RtxCzj9gCmJ3a3RS2qVszG7TPwFcJjy7”Œ"1PVidHanfSP5xhfuFpRTW8N1qL8CVp49Ry”Œ"3H6BYhxuZsS7u2HxbKui1DxRnJVTfGT3YQ”Œ"14Eiq1M16ZowgMD3hJNzu2Mno4qP6kQcCQ”Œ"3N4ZaaTSAnyYtScBAbTy37gd7BGd4xwiao”Œ"32viZJhwxSSL2u25sQnisncqLFYH6QkZmg”Œ"3CZ74NDJfJkDk88PWWhreimjYws6ccmvYX”Œ"1NWEeBqtoS3ow3N6fYoS8or4f4RwK3Md5f”Œ"3KNwSG7nHTyGSeF1E3tMpxgtJc1W2LsaW4”Œ"1Q2u91u5e343Fg7bjJ87rtUdry4KEm921q”Œ"16KWVBA3L7owLt1uMuVKNjZG6X3XwB1jis”Œ!1JsyvqVm4ApC8ZLGhoGa63K4VQLxUvcA9”Œ"1Md9oigYsKGwbC3b3L9ufXAvBggqAMaw2o”Œ"1NjCRiwUbs3qgeopa8iV4wMF4CGvzPReiU”Œ"3Qqweq4JcwiMwzgMTr6bEShEoH45h5HGSb”Œ"3GJAjSsW1YAWjE4ePF5Ntxs5dnBnqLCGbW”Œ"15txPcw5pYZfNAoyH5XsYpRxH6dxUyfpc7”Œ"1QAR5ytDLnWnNszAstxZZRVxqsuxrMSYFn”Œ"3PUyWFEidnfE9YZNCSUsBLDEEaLxHuGwa2”Œ"1Coinima7BE7uARUWgsHFZWQ4ipKFtqDBi”Œ"3Bpwij7RTSo7eTgS2msxTNKDTCXyQHrm5J”Œ"3CcyakJVZA7ZWRGDDauw3gEFjSpv7D5QUJ”Œ"19YFeUWRxwNjcsA6aKFkrsPZKid9KrW296”Œ"17YGpvVK7iYsSunbjf6iAMyqESff9D4yMy”Œ"17zNuPzdQrK7HicinXpLJ3wi1EW9Wf7gHx”Œ"15wPNJSsH9dDoYJrYdDpda23cshjzMnDAQ”Œ"3Ea7XTBT6PAm3rS8qQW7kBsjEzfzrPAvuq”Œ"1HuZnhYt1XxREsGmsAsRpc6TdCpoQGRaJN”Œ"14GJ6WpsM5Kjgki3JVbf9PLuekcsgzG6TX”Œ"1PKtfUiLMrtYqYEt6RSdqtJ4CVLrZrkVJu”Œ"3GhH8znSfJfK8sK1ZovANEp6btFJ8pQyff”Œ"19QQzHAdPCWXd4g64oAYD2XDaKC7YMP9kZ”Œ"1BDfH6G5UNchmcxuTFv5hFisGHbDoAMza5”Œ"15LzPAzChW6PVpFBQU9DH9SPPtkfMVXa94”Œ"1DrUxsJSq1FP1c55RujKrZT7inLgutTC6n”Œ"1L7eDYNBM7CYkwKxuCB45YMKrXqHDgKLEG”Œ"1JsVv6mPxzswDH9wCCyCys9F5T3fb9yjMp”Œ"1ARjUhzCRWzHXnYdVBW51gghsP3fdhppZy”Œ"1HiT6ff6XxDd2HAJLcgxjNkhz1quf3yYAX”Œ"1Ejy2XQT7vthyeeyXDjqqiaMaBZ4GPeAN2”Œ!1rmQnQF67Ea6UojEXvDFEgXsTybiYpeR9”Œ"13BaSN5LrsbTFMg3QBFLiEWhdJPRC1EfK8”Œ"34ooQg9DDiTaiP8dsPJhrjM1Zj4FgBH5AQ”Œ"1AdPan1YfS7KeLV2azNBUC4CmwCYmEnR1g”Œ"1LVa8QFs69A5XQpeCdb8HKUDVYj63PSvBn”Œ"35RAE7pdnRpM9orvxxryLB25U4Ye33vAtZ”Œ"18heXbo7z87DyT4oJSLAWARcP2W2GJxfAH”Œ"1EsLE8JmqHvWMPH1FQgaft7g1kyGTTQKHf”Œ"15ZaCowCF9Qo2TeQozy6NgtESGyzVwkTfq”Œ"1ABfPJYwQrgU68fDX8pthwjYo61Rfur4zp”Œ"3Pk9FXW9wKxkh8zxAxtVJexkMWTtMkDQfu”Œ"1JMP4zxK9aRQbJs6Lg37CKzPPjFMcN7GRL”Œ"1KQ85Zf5VBXsyDbdWmzoMgcLRu41henLC4”Œ"17GcmMw6V8um5Z5ESbPJi8vSthZq1eqHGC”Œ"3E8R3sVhFfxM8jVDw722AddLM3r8A6vY5X”Œ"1HVtNBtGT4UQ8t3XMf81ZxhZYV7rfRrMYx”Œ"1KTRcaRTxjxwitNj3wG7SCF6keho5TQ1md”Œ"187Cw3xWyaswcKbFjz2XEigaQBB1XmQeHY”Œ"1QG7iHFiZJCFL5yRE2UbPzhLBDvWdW6g3j”Œ"18JyuyTzixbTREhDfcbxEv2nWaieQjscXG”Œ"1PgEGnzuxovKwbv1MHorWU7TPd4coiSyZS”Œ"13KaE79JfVc1jdk38MB3p87JhMhPv6c3sZ”Œ"39S3ADqMxsB7DLYyZnUmnMVMRLC3TLDoRM”Œ"19FRjqsvPRqen1BrVtpLdpqq4eorrb7dQK”Œ"1EqfsuS4ZRHGkBQ2c7QG3HVEWEetxGK5C5”Œ"1Pvc5gwHFVgrwt9yofmELrLEMwGUEq6ze2”Œ"19S79KkM1wxxq1kQVLfLXuGivtoCx4Hd1p”Œ"1PoFGmsBvAyxDEudrtstgxCK9ytUNswq5w”Œ"1K8rwWtV6wg4GjLeKMN1i64MuLaEDHmFBN”Œ"1CMbtvxY9ZUqkEhiFNahjmjvukktDKCeqt”Œ"3EDZox7QBrocU7uR6i7u2NZ5s9MzEAApFg”Œ"3QLTGM6BSA1PenPQ8d93PLoLuqC3tFVvpR”Œ"1AtcLELFgaadMmoLiZPrKB2j6cAQwA5SqG”Œ"35P2xeYiRhtzyjmCC1grmSN4UqqFqjuCwJ”Œ"12tX3utb24CduwP2cWNCr2opA2Aq1qSQKA”Œ"333bGvZ4vuDSea9AfrwpRjSfdz5787BpvP”Œ"1GwbqegGurgvbkztfQpmjfK4bRZneRWLcT”Œ"3HQ7NpYU1RrsCerzjiuA5FU51y1epiydm6”Œ"1FfsmqJS1LvJnH5dG3Cr3cnFFKjVZMYe5S”Œ"1DT4TnYDS4SihWYzpjq2y1iJXuatkxupYY”Œ"13jf7Rsn14SKvbkGSiNFjsQ56npp29VWcS”Œ"1KowyYccf41z8ESRTBSUVshJTbK4oDPQZV”Œ"15fredgtsR2YmXpQ5QYuhnupKTcbk9LGoZ”Œ"1Ngqao228BMZ2MmytNHoFP4TuMWp3gxrGr”Œ"1EVRQQXJqMFVV9FUZgUcLgmxHnevAhV4em”Œ"18Q3VEzt5Bbv1yY7qcKcTZyaGAGvRubhA9”Œ"38zpeUK3zgwqTc7YeLLfVh73VC9mNo8oVe”Œ"3FF9BoESYsRtRgWzEvNBKeuTrqVzXYqYyy”Œ"1Mp5ccKmHDX1uZnmis9yVTGq5AmNaW2a1c”Œ"1QH6ULjgQvHn8Jp5pBbN8kxG8sZ8ZSaNJX”Œ"1NhRhpvXys3ot4v4m7o7LQ7e4sxwFzGa1G”Œ"36DfpSZBztZ9XKysuXfFqjtDMb8VD7a8nb”Œ"1EXrZfthgmRBRMzrMqso55zmaHTmvAepVm”Œ"1JCHGD9FCQwQpYhYNSbiB6V3T27rrsMUtX”Œ"3CD1QW6fjgTwKq3Pj97nty28WZAVkziNom”Œ"17npb2PDQwhudHnTMpnp8bpDBaVro3dmnF”Œ"1ANa1gL1JEKUmQc1ZaSQLokF5pVPNpyi5j”Œ"1HbDwsTSPNVGVBoX1zzkfvjXJU9QdtSEET”Œ"13LNzeaY7AmRox6joXGnnKhbz9Vo26HPU8”Œ"1Hz6kYTHyMqCwDvvVKzepctDQ8GH2LbJ4A”Œ"15oycQTVB1gfS9h9pZYnLmRb8XVxM91XCB”Œ"13i7cUt4ZRQT34Cuxa5YGD2f2imu7rDqhh”Œ"39AWrujueDHifVzEuHQSF6kzm1Gcz7BPwS”Œ"1GCWrjysauieEdF89WiXnEZWJLt1bxWTwg”Œ"1PcRNCU6pJ1UPq4aaaMarBb6Wy18xJcYu8”Œ"3N5XB5VFDtoC2ovfXXPLUFULyZCGpzV9kZ”Œ!1nfNy8LtyKRSgEp4d6jookaVFULYEoM6P”Œ"17pkJh5SZ3QmjHrdy2ogCqFhUnBtPLmaA2”Œ"1FoeCTuUb1LeDgXbzVunsLDjCA6fCkVi5b”Œ"1PTQVoTxJax4uutwxa5mhC2Rpzyg6BqLrr”Œ"1F1omzKUTkRvkmWhe6p7oeH7FvTe9pPBbM”Œ"1H5gtqCCzUMZkLA874jv16d1Eviw7GJf1E”Œ"37BCcPRiSHiJiRYXgU19dgeucPRfirAPGf”Œ"3PUuiYu5cFMsagkffArrKZzQFtWdHttU3x”Œ"12sbwYv4yz5jnTwUM5GzfvkhUnSh9YHNLv”Œ"18rbYcCLYVK63xM39N7AWKf86B1J7xbVXq”Œ"1D5jVYPnCJ8KCk91LxbrDFuobnvP6xsPK5”Œ"1PJVpUWHUb4Y6ony5eVqES62rTBLWssMpC”Œ"1EgDgLgwzVh9ZgXL47PnXoWQ8YXgosq6CQ”Œ"121CoRxNiPgwKrRDhY7Hy61ztXhVpgERLW”Œ"1QKgyMo23aSyyvBa9bVsmMPVNBtzRpZ6q9”Œ"1HpoTyAM4AGRyipLo2xSNBGqLuzQeda8Aq”Œ"1BJqXU7U53rUsAZhZwwJkBrxNZJxXWA8UT”Œ"39AiNUD9Ygi44TWs2BygPdoR6BWq9tnN4V”Œ"19oNcsV97DFN6d9XwtuL1r6EE5gyCXpcy3”Œ"3CkkcUzgMFD8mZ5835jF7i9jPofnZYZPWp”Œ"1A4NacGPh2eJPeQNWP5rwmQuVFSJrPxxwD”Œ"3KZ8HdcAKk7ajQHQkBBhKBfrgtksQUKWD3”Œ"1AAq1Bf1M1S8WDwWjj3nrEGpHRwF2pxELe”Œ"14ny9uAPuyoeEmB3enhK962nqMUbhv8j14”Œ"3G7Ytn1eTGTgFEmcrrVW267Uxg2RhM2Ggu”Œ"12zwcdD17paTbcpnckgxHLyYipN3CWucJq”Œ"1NphLzSc9iS7nMqFPKcucQDtBNeNuUx5Rq”Œ"1QCocM6V7evfpSC1Ynvt56MNZuNC1FCywP”Œ"3MsVw9QYacKAbjnNZRhaSMAE4Lq4kKVguu”Œ"154EtzxGeqpwnLLcr2qcb7BW9Y6dCNwr7x”Œ"3LQxUzikM2rWBhUFrLFNiuWPbPLLJn84DB”Œ"16pAD9RXzShWRU8i7ahAVpD27cCA8pq4yJ”Œ"1Edo9eqNSpxVnQvejFffYA1JakCKvNwJFY”Œ"1MXMRVpp9LcU3hid2mWMZY2ziSxrfao2Z8”Œ"33AA6qgGPjXWHoiGrpfx9H4GFvpdrqK6CJ”Œ"16WsBCHJRpJJ8SskNJjF9zQ6MQ4pCyek1a”Œ"37nciijVrrgdqAMi2KrYF6hsdUJx6fqkSC”Œ"17gnNkunbALw6ogJy4bKAx3o7nfs67JGQ7”Œ"17AVhTterUreNATxLLXRzTzpUhADfL455E”Œ"167uqeBFuxu3MZ4YpKZ1Fg3FK6noPKbF3H”Œ"1EDoRQ2Gq9c1Yq4zwWoBABo8C6iWiVJ4wE”Œ"1BrUstj6gJwXSESk8288u53yMoxxyRdNdb”Œ"1HCmgWSB3wBA6YU7yYDxYGwk8e7rpwr2Ui”Œ"17HTRqeJYxKjzmvKEaqD7ynAaKFvnt8Gju”Œ"19Xv2GjWs7iqj11ntcRCwR1HHBeKg6C2t5”Œ"19Pt7BcdkCLjcuxDJnpm6t7bxskGZp6tNG”Œ"16et55wNR5sELGXh2FDQqAJxcK53Fgvjge”Œ"1Azm1TtAGBv2DnefCu6uktCtwC1UkfPNeX”Œ"1EcvXqdxAgXcceDbyNMhnbhtuRRP3wXhPs”Œ"1DrCPFUEaaassxWpSY4vR8VGfhigBDk6FU”Œ"1EtAqxVxBwLC43B5wDS7tif6SdgSmALPhW”Œ"1K5apFAj5oo8hoL6xhdv5dbePbyL4MnXUe”Œ"1GMUT3X7tWYsAfExQvd7kmYgPyMFdwsecN”Œ"172FYpDLUkKgNKXoRV79LaP2HmNABQ15WU”Œ"1CBYTkF4RoyhTswCkzm2G6LYG2jL8Kn6kx”Œ"1D1JwNz4UmDHRDD5SeoRecEACvXWrEVs4D”Œ"1HMM5eeg9q1aKhvjWRuef72axMTCXPoygZ”Œ"1NmnsyszLDK83qqMvPMzmjcqiF7XiUjH2u”Œ"13vHWR3iLsHeYwT42RnuKYNBoVPrKKZgRv”Œ"321cfJKWzoTXzzNMautzvfT6sW3Z4dFy1Z”Œ"1H5WYVjWRpdtdirgY9uCSVFtbNSXG87oDb”Œ"14NMSk55aZ8WmQR9CGLw9nbjf7zCM8NLM3”Œ"1DEcTtkrbdXxS1wbrMvzub5X4VpByYmAwo”Œ"1H3nUMXSiLuUoiCu5ZfJjB3fL5rxpJYvFK”Œ"3Ff7BtfXNyxGDRaiXXy8h6DC6n1pruYLge”Œ"1JTu4E7Ln6uhXUMq4hBUxcvnqGeKApyUj2”Œ"1GzHpEhtAgwPC2nBkBdxQ3TsQyYD1Tw1YW”Œ"1G1TUDSa9fzzHkLyFTmAwsLvF8knQ3UKUb”Œ"1JVbTNJptVgovMo6vUvGBUSCKpjcihwoTZ”Œ"1P5zg89To83CfpM9xRuZmXTf2ASwGjnVyr”Œ"1Ghb1yTzeqd4wwDHB2a8vyQZQVy5Wp6fiA”Œ"3QReSeg1nb6MFj7vXb6DV89yfcMAw1ZSJc”Œ"19nBisr2NCpstJYw5S91bDjFkq7dh7ZV2S”Œ"18Exf9if3StCMnXLeGzxZe4oYokCR3dGcM”Œ"1EyjaU7F16eCHPbzvHat3mA7MGmLoZG7Ac”Œ"12nEJsnWP5sKvLDwruxUg1qF1fFEw6Qe1j”Œ"1GLZSQWmkA8NBYno1eWnT1987Z6oRaRtnr”Œ"1KXuKZDjAPRT618f4RmCBqpVbMYgm6eSPN”Œ"1LB4nQesQAho5VpEuZwWG3qCLQH7bj5C8D”Œ!1XDujgLVh6QvkmnpDhSXRz6gxFAWKA9Cx”Œ"12X3oYMzsei6tT6RwEiMhoVFfSz9dqYp2s”Œ"18Cx4rgANvuaLrNyqfxfxU2pVCvqheLFSE”Œ"18Wbd3XryKEAW9oQKLwg5tYpBSXyKp66Ss”Œ"14dL74wLcMnnSB5ZjdHUAwWU9qjc9GVmWH”Œ"169QypouL33vGBZoWexkhzRGo3RoZy7rL8”Œ"18X6evYD1Hy5mraaT1P3XBwsPXayZPqSF2”Œ"1EDucm8WWkbqPwh5qhm6SCknPHLuc8a3b3”Œ"1Mw3EkVo5rTfMjgmXSbZogNdbpM4RGi3b9”Œ"3QqeGYdvsFNzPPQEsT6aMh2pBoUU7tvTpW”Œ"1Lsjx4V6igd9k6UPDy7eXUPpKejMHeDCGb”Œ"1JoeHEDH89XFjbQH4GQBgQcPZ24yErQADd”Œ"31qEjEdAbCFYf8A6FCMoNeVwLKkFxhtma2”Œ"3DgJMKbTcCp8QnWZ6XBNubM8tUQfTa4H6W”Œ"1MWduiatRKB8A2LrtjarBK5eDY49bfaWFD”Œ"1KqS89ToiFPYTFJEKBirkS1bJLzbsequCq”Œ"35HVkqdZD7g7F8hrcgdFjmHwPQQpvcUQW9”Œ"3895vNkraMusM7rst9a5tz8LGes4DuH4t7”Œ"1P5yuBJ6JLYaCsq8e65cxCP5pZqgZUXo5h”Œ"1QH9VZ113jMaHvZ69uR4RroShBLVDSq3X7”Œ"32XG3cHDkZfZPpVhyhj9yeeH3TJC5ZWdTc”Œ"3BMEXAfsJbdJDuLdkTQfBrAYbd8WwujjXj”Œ"16EwUeVPSKjFNE2tjuVRJ8ua5K3j1G5Vqx”Œ"19F53i7HFrX5NpJtx5mzM33qocVBNMJXgD”Œ"1GmmUq2t2qzd5brHqpB2ec5oG7Tkje622R”Œ"1N4qNYgDmgg5drtneXicZpFpSKehVn1Xg6”Œ"35janM1mM7RkBDtCESWN3YKoAma8bFKHjr”Œ"13vamp61UTm3Ypjdk12yQyndAWFUddVHgN”Œ"1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd”Œ"3DkqKdjKU5hc1xG86WpHh2d6vXUTptWPBn”Œ"1LTJSgyeW4WwwK5twfvZwmU8K53fuQ1YGF”Œ"1NxugmYq3h2zh55Ydf5u79GhgcT7Guogce”Œ"1DJWK9bkSExAZY1x9ke7Kk31ZMR1xNjF9q”Œ"1DSuR9De8G5Q7w8PdmBTydmfFmQxE74vzB”Œ"1GSPuHSXFoNJBHXmh6dJDdEoPNKS8oqbBR”Œ!1Wua8BUG6ahTeuve66S8VK8yPjuUnxCeX”Œ"1QE5gqjHhdwUAu43fD2y7NGTCjumMfmTPS”Œ"12DTTKdNU4G4nN5HUNAKfL7U2gxTHxzBKg”Œ"1ArGety9H7wR8YmbDnN5bT9hgtjTS4j3hm”Œ"3F4S1aNNMuVnRdiKEMTrEzpt69qDbyfrfR”Œ"1PF34ZJeiudfUR6cZahw2uBotbpzKM6UQB”Œ"122J4WspFbofjJHRap4pYuMJB53K5iizyn”Œ"1KzQfkjXVuG1yXvvmD6Y92tTnJDCnEeaCu”Œ"19q4kiynL8r2uUfHfw4wF6CmNQ12H3nuEZ”Œ"1BDe6sFbg4tY7UcUwheYMY9FJkWrxrzvrA”Œ"12ZH7wXkH8s89RHMRq2qGsSeWgkM5N4JEC”Œ"1KQijqHgpPhojd4JdtpNiw3864EuxtEf75”Œ"1ApZrYnug7x3BXKhS2L5gfpersdSHpagTM”Œ"1G9DCQ5kWqaV2LbKUL2oUpLhoR46TndDda”Œ"18PoNeydSwzS2oFkJChNZZRXxUQWAMvypx”Œ"1N3Cg37Q6TW4xU9XF39eS1EgnjXB7b4B55”Œ"125DkPZhggcHVnwKqwqory7p7Ve6FHNBYB”Œ"32J7fgZtFtFG3eKvCmny3HPFnF5M2vXiXG”Œ"3EBV6k688N8HGynb4Tr9qhLai6CAXpEWaV”Œ"1HQocbnE4kojmCA1jG94E9zjihKZpPYoQA”Œ"3CWvgMJk6ibb3rbxVVmTMbVDgUaiL6ZvB9”Œ"13PxfE31nTP5oxmiTGUCKgXTrsHXo6Dwzq”Œ"1QDFNXnB9CygWFWyQzdUSrD6kVV7UwRfNm”Œ"13oaDo2jPHiNdhd1tzbYSGSt58rKKPXcDB”Œ"1NUdt8iiuNUui4w4dDQ92hJq5vgcZj4NQE”Œ"39sk7o7UVUwdxEsqiFnSG7YNrHD6cNoSDq”Œ"126JJ7ZFDjYQrReNqoMLHTtuYGxe4RPz18”Œ"1CMkea3MSMhF36k7g435eZEzPB4XqK6118”Œ"1PRbwfTHXMKgrm9P1juqxo5mZ3esAScyHr”Œ"1L1CLrZBQ11VTncpqdzDwAsgrXrqzQ6Q1w”Œ"17YXanYrRu7HXVKq9fYPBSLj6Mdbqqb6nt”Œ"1AkETAeSij9FHDYivvR9oePVWnvx78EAuc”Œ"13XA1XewN4FHcUiJvLotVDLHYSsFvLFPNw”Œ"1FdzDRdKhUSm5ELGWW1MhvpVrQcSTCSC91”Œ"1L97obSZB8tUqnTVEqbej3YgLJip5iKnff”Œ"1PKYAn3GSxiFHMLCcY8XgzKGeWeQVDN111”Œ"37ghTQbqYtn4vDdqvc6MBj4tgyE38j3qYf”Œ"33fwNKjFKMzNdBs3FUZdrPU6q1KMYAFNAt”Œ"15aSboNGz9oT44x3Pm7MdFwKVBM8LBcqQ8”Œ"1PfXJKVJ543MSZPmABuQzvkukLUrcL9RV1”Œ"1HF985EowA6upnFwxBG4b7s9rdgVQjn8bu”Œ"3M979GyPiBnuYeMqoPBSFBDzYPt9bNP5p1”Œ"1DVb5gxWDq6DDZTSwZRPHMVHibVWQtq8mZ”Œ"12biZqhmQKctodhrapyg3Aa34xRT1XHVWf”Œ"1LJpKKWUvCA8RgKM7qT3RgUNY5TqdPUrmY”Œ"38TWGC67Bos4upWLdCn2bXsFifeNxjb6FU”Œ"3C6gnsdB5VXy9ybqeyG9BeHW1MU5DJpAFf”Œ"38LCRSFQ52EBjr6pYewPbvZetpdetVoeHp”Œ"1EbyGC5S6SX224vzPW4oBtVGPRcx1z6DwJ”Œ"1Av9DjqRWDmn4c6xCtGcTDs139Cu78bTEc”Œ"18X5uKXSKBVSqeWrcLrEwKHfDQ1nAPWAVa”Œ"1D8aBmtF3pZ5iHdQsxUm1NzieHLRuV7mYz”Œ"1GUmz1oXTt74UgTqtE1rtRsZKRpZMnMNrY”Œ"3P6Bz9qrCM2vTotCTbQTmoxAtE83zXb8T1”Œ"19HdtN2Vm2kKv8aPQpo4hmppknRzrGTYA1”Œ"3JGq4ZkLcRfijQVamAJBFjErCFhgL2vaex”Œ"1HJgxbyQYMbBYJ4Ffcou8c4doyy4wzEK79”Œ"31qJ6qLzKhHjiy2znpLPBQEES7bPzGdAiP”Œ"17zCj8PCqcZLLpoE7F8CP2P21AASamTpif”Œ"1DF1WKTUXjjymhPmGM9NVHsFX1cJxhXF6K”Œ"33LGC4pDbHgfB5BRzigVKvc7GqFhhyzWWE”Œ"1FB98mdePE3D6oS8G1ZbLTnjuW7544SL3B”Œ"1PgEAbgDCpCQDkKvgnQfh5mqvX8JLxWcGW”Œ"16D6T2CJLPEQiVTC8yTyaBRZk5DvMCJS9p”Œ"3EhCm6fkHLrpEGEoCYAMVrDLDAN8c3KUtz”Œ"19MZSLwtYTCrEviXkDgv9fjAfBWZerM5SQ”Œ"1EfGbyVshcoUdiGs6X8VojG5y1ZYVoLGHr”Œ"1PSmoefgLemy1zJ2pTSQHYGFnEpbuoJ1A6”Œ"14srGYPAiBTRYJyw3XfCEd8vZUEcgyZBVg”Œ"1B3GQBn2mPx8MqTH9JvK6LGF9iwwqtvqcF”Œ"3K29vkU1ogq69otLfgZSf3BZhnttYmGWfk”Œ"15n7d8ViAW7eckfbUR5y3VY9HZ7cDuYHoV”Œ"19wvqp9eRy3j9wqT1v7FftKTzDCkmdWKyZ”Œ"1MHX7eo33uM2mUNo5WWUrbxRZocQzF1MPS”Œ!1gZQmEfdNPDTZUutU9r74HL5GrMmA93rp”Œ"3Abd5RVDJdD2TVLkYYktqPQVgXMisA3iLE”Œ"1AGyExYn4TT9PhyMiLqzKJehdQPCupzhKF”Œ"1ABfezfPDRpiSQhVhtpeDnD1xka7tfR1pt”Œ"1JUHRhVNwWpXKt2khV8GERLVsuiMmobEqi”Œ"382UkojLHMrh3dgm6mUyKuDn4XXbpWxQWH”Œ"18JJTHQJeaK3a9vVgdVZCmRU8eKawA4vup”Œ"1AKUGJSb2f1NdzMWTzKcyaaXCQFRPLuMbH”Œ!1xXYYdv6awgm49mALoPcHdPLfvkUa9w2z”Œ"1HrYWxLnWY5BksL6u6DVVSwnC1h7zqpSEo”Œ"1QD5fKDYRv3NQBMFusd8EkLTiPzcS7o1HN”Œ"1KDAQ62NDfE72VDmUD5sj5ymtbJt5xj7rs”Œ"15Yk5Jb5eVf25K4ngNMmVUwXVbmmxxi6eJ”Œ"17EASkMiQBNxPiHKuueCirfUuUqCR4Raqn”Œ"1Dt37f28D4dfmjye35cutH47RNDmwPkpzj”Œ"3FQmHqGtQ869YZmL44dbmVXxF5Gev9imKK”Œ"1DFNZ5Eie94xodBriuC8QkknPcQW55n6WB”Œ"1HGsi1um1oShzWUBkfNjR3y5gqrRhE8pTt”Œ"1JgvM1ZTDvntWbDjkewq3Dn8oPUrLHMvqg”Œ"151yWacQubM6D4KGC78nRsazXnUKBQbZsf”Œ"1BwWqkE9L6XUfrSNbZMrPPYFVbtszZxqdE”Œ"123mV2YYJJYh3pGBvBJJ6Yc5gf13qyyhit”Œ!1E7EEqHRHyX4yWPUWyiYZrNj6fahLMAt1”Œ"1M23ERJsafw7SMZBsTNYVJZdpcZU16vjFt”Œ"1MyoKZQpfy6w4XJQam6wzdPyhqt7VcLaG5”Œ"1GNpnX3xYrct3vsuPBdGCCQvy2yeagJqRL”Œ"3BeNDor45Pt36hWJ4pVqJbuUeuzKCjvQdi”Œ"12ZRb9siHhJdZeQGjCwHtxSvanFYjta5ho”•      Œ"13aXy9BtaQ8fvXDCXKPYstjbFGahSiiuRe”Œ"3D5bozc3zJrE3zwf7Vd1W6zTi69RbGDpCz”Œ"3L8DwRQmGZPFm9TpTkQJH6idLzYwoAKuFy”Œ"1K3fsa9ZczzXgNcXuCvR9XEi5g8f7R2QoG”Œ"191vT42SkrsYnr1Aae4woVDWX2t2ZeHRXM”Œ"19FJDTVZWbUw7ZmQwmCRFQ7ui6B2qD6Q1P”Œ"15PTNxhFJwWbojhaKo4gP73JrnJCqoZD35”Œ"19xFJgEPVnWesUeDaxej8VwZpbVhLjbrPg”Œ"1CEKLo8QugTeburE1ULuZLevCmmE6gYXw5”Œ"18HVK7Uaox4Jipi13SB8HUufCuW4pwC3LP”Œ"1FieDXY16Rs5YL5m6AxnMH5tmtBecsNvoD”Œ"1HzDRUB31Myd4NtpdQGpnvzTVayiLQvyFM”Œ"14YAfvyw9BpNyqg3q681jw4Pxnno78aNRw”Œ"1PTadQoiVufNeV4ryugfKg4AYTahjvusoq”Œ"3792k63LZroUYg3BSfNvK1eSmVGmM5HpZs”Œ"35RA1Q4rZKbyY2SqKc4F4229hua1hP7TxD”Œ"3M2diFBbDfuAoCvARneQAuq3KaQCyqnpWe”Œ"18ouGbDZtTTWJrpeg8ztS6ixwVRKcKDm9E”Œ"3GSM7kf8bXg76CEuASw6cgjTW3eJhb4YZq”Œ"12BaJ9Dg3ezfwMQRp9sTgRuoC3rLA2Xbvt”Œ"1NJzBDrSns3SyexA6eAhttynwbHyA35fze”Œ"12q1RCBDwnbaGiU9TyS83H9Ne6JsScyHcZ”Œ"14cumpmTAgq28rRsrSRrnQwmEghf3bWVSm”Œ"1DZm23LfUyfD38dY46jgjsETxNmHN65cxs”Œ"1CTQLuHSEYRj43uyVnq9ZMLiuCSzCucG8A”Œ"1FhE9ugcYH6GvSBK4UummvymZgQoRid3gT”Œ"1KdAWdzo12eR6eSmwc7Abk9g6xEqkx3HwL”Œ"1MTDo6xp9cmUS9HVRiGx6bJEv3Xei7aru8”Œ"34RnuBNJgXxBHMY7CUJ5QNvHtnBQsQMzrz”Œ"3QvaZ651Zq38egbMN7wdTrGi4eEWLfaq1a”Œ"1NYPq1xGYqXokWNoUBvzVhZEFp9Ta4mnMy”Œ"1N3NXoheSr9zZe3GMuGGxNPYn3whtpR9pM”Œ"14CizjsaTyLZDhj9UY9BkqPR15azzifUrf”Œ"1KVCjHt9fBc3kimztuGW81RHeG4xUEi6Wg”Œ"3P1sThUfzBZKm2eRqR94Xgyq9tXtiWwFiH”Œ"3ECJwvx9VgfotcUuEJMVNvmWnTGVMk179L”Œ"3BTaNM47n2bQGFiUqs9DnmgJzD1WRebYdJ”Œ"34UWdYKu82fc7MS4EogcGSsgAkZ6Cxdryg”Œ"19SW8eJkUvpawE4JQNsFAZJfae3VU4mEgn”Œ"13UTabXghJ4GmSQ6qbqB8J87sUL9UrnWk6”Œ"16ea3S2hAxugfCasy1U8xkhdhHqXQmvhEE”Œ"19ohGoiLKAUFy8xVCngwWGt6Vd7Zx5K6WN”Œ!1V3UuvjvoG36RE5efCxJz8Ujng3NTjgL1”Œ"3Eu57uFzBMuqA6ujhaD5C6MYfjUAc4sGnM”Œ"1LmYoaSZ33DyB8Vw7x1W3ddJRBbRqhyNzy”Œ"1DFNQgfM8xMbWF9CdDo5367tMKXV3VWApB”Œ"3L9y2p3S6Rzh9vPY3JsYifpmhHgdv748aV”Œ"1BVZ1RsfNMWAXo6Re9YCwMqNRyFazhYhSS”Œ"3AjoV5H8Frdvynh3GTkk5PxqKrhP4opw4z”Œ"3L114EGe1L8JjiY2Rjg7MioLQuLftDjGkB”Œ"12VpNLvUEBZvHL8WzjRcvoZKJ4KPdbJHko”Œ"3JZGz1APzYYVh5tBhdRKyigt8J2B2K4mtE”Œ"1EUpJXEXzFYjoLMzwReuN44PeFRDK3dgfg”Œ"134WPd2UGRjK1Ynk1ePRboqHoKt3P388zu”Œ"15sKNC8PEvvXgX2ho8usLB68pzAQZ4KovB”Œ"17cvmRqjVAsdggtLngScHVEZGVDPTiPV5H”Œ"1MNLCdRFehGn1XgwaZ4rePSKDL415FcGzx”Œ"1EyZtePPKp2KA4cWYxMzUWtYA7XHTCNkri”Œ"1Prg6mMwau8VQQt8em9d2GiXsKzkNgXHvB”Œ"1NjzZtMokp2jxQdw5MafiF2QRpMZJSy2mN”Œ"1EiquNStjwwVvKpFRefoGfveLzovaZoCfB”Œ"13m2hiCDPSQHaM4QwjHvEz6d9VCExLPoVP”Œ"1FAowKuXszJRRNmLr8MELtm5suEmnR2dWd”Œ"1918NqHHTpGZGLDctXhSrkBEBE4dFCD4sX”Œ"14MDgYBXuE6G6ER42qNkrFzPHqCRi9QFzd”Œ"1Dm3pTPShbxFBK8VL4wUF9aWbyS21MLtzB”Œ"1PKsoK7uh3vHbX9nif8xp7WntNs8VHzYJ2”Œ"3KX7kC6o388W88r4AkQPz9YnQysAnFXiDu”Œ"17cXsrcKbmMQYEb77jMbQd6PB7171gmpFu”Œ"1Dy4gyven7MEESJuCbsqJeHuLwQsqEej3i”Œ"13avgc86aPZGcUvqCFYxNh1DAAF7H6wbv5”Œ"1DWZYV63Dpqa8q2KmBVubNkZHwvPcWPs5A”Œ"15yWY85HPhVphWrWPPqSSG8MkgCgdNJDWX”Œ"19sZUMzE52PgrMy6BvyRjFjpEUtjXAkXoz”Œ"3GSovGJNXktELdYaWSYGbyEQ4WYwY8MFru”Œ"3Lc5wRfxU6uqNDNb1bRHRXN8vgJMR6uFW4”Œ"3LiuuJA1TX1iwELzaGFVxWJ2v92pzzmSmP”Œ"3MUxFQinz2eZhtTZKozbWSG1yrGvLRoCdG”Œ"37SivAqUERUWGnCZuDyDNfXFZVqYDrSrYD”Œ"38vH8PTSU6BcT3HBL2VYDM5H3hm9MbmoQo”Œ"1KMNAKCcXUYjs3YGDQM97HNcxYWxdmGP79”Œ"1996urpGdkQwRAJ5kF2m6QZcNCNbEY3rRq”Œ"18GBLVX8bScqFF9KQdfcd6ZmfCQ6oEQaey”Œ"3Coibegxo39FXbtH96Wc9vzkLqsCBDQ5Cb”Œ"1FLpu5bhuEod5p3CDuSToAtdfQgqCh1aoS”Œ"34LxvX15nrj1TW9pnsPC1uLPewoEb6dYrF”Œ"1DAZpyrEWosQ52wdYjTamNjC1FYXjWAehg”Œ"1Fbh3XC58hcGuHgwpBtaY6kSYtWwj3SHB9”Œ"36Sqg4oHo77zQ8ermkmaveYec3ijDboXhn”Œ"3EABbLkUA5M3xVbLRAS37dsuPUkt6TtuJK”Œ"15DoK9EDTqUkhg1DttZFwkkbpsB1F9LLNv”Œ"1DqFrxf9vL6oM9b3cSWZwEhRJ5UWvuEAvY”Œ"1MCP1SAPfszNVdkU7LGqYVxEShCGuWCKE1”Œ!11DVjGTPbW8MTZVvsCGV4cc4xYiUuyADs”Œ"1P4vcffVtbjv8eFbsf2yXnHQoMb3W6dpfJ”Œ"1DBfg3s65uspAtKCDSbAhMERMjUzxG85KZ”Œ"1H6XFWUNdwBS2MrY86neRKCHpmeWMMSbJJ”Œ"12Pb8Nc5PPPQJZTFu1szw4FvBbKXXy1cXj”Œ"37FubjxknUTfaczmrToduBzEBubTNujdMW”Œ"1GsqQ5npxN4CwMpCPRWXJKL5C7rfkoJzew”Œ"169P23m9ns3a8s1RfhVUuGBTRM1UU19yj3”Œ"1EkzKje29gaFbawozuQ2ZFQikcugiJUUjS”Œ"1DZQJx9njvyv7hc14armCbVsyG6558CcLp”Œ"3E8DZ1KPDfEfg229FH6todxMZtUefCsAwf”Œ!1FfHgVBzGEx8KKPLohXsthNq2zUAZvJtS”Œ"18iSchp5t3pbqfwjqKENWjTMZW9vHju4X1”Œ"17zLjj8FkcbxvpeAdb4zovmtzcv1nCT4AU”Œ!1oxagkWGydJc7nrqV4sAzR18vdywxqW7C”Œ"17dtxZB5ydMiYT463F2DXzQ29ijZT1YnpJ”Œ"1GxTtmH6SXKkf9mXorXZTqNwP8cikGtH36”Œ"1H6FXzCjiCcJyFYa5wpfPCjE27aJKT8h9u”Œ"16ZuXmkgY18Crehkd4KvC4AZo1PAGDCxLq”Œ"3LPs86iz4RmgzNgYBXFF2tj9HvWQG4ZEih”Œ"32kg9p72v4PPkmXgT4tg8VxwzrwvVxgWpM”Œ"18B7ywBWagkJdnx96MgA8DUeW9cDMPaxkk”Œ"1BTnG8VPSvBvht8ieXix8FSdUJQhJykPZE”Œ"1CqbBLNph3GFy3YbjjG3EPuNWCxKTauKUH”Œ"1CgrcsWg1wLdfAfSDwWsYdwSbrXQfKCZqB”Œ"13gk3ULWVwWpQ8XpGe3LS2ZA7ynVavL2p5”Œ"33FZEyhvauxyNbMMfwnriRDmdi3puJCHSd”Œ"1PchTGtQ7uBXhPan9dDnHMjgAcv1jrpNEf”Œ"13WUSVwqrPGE93y2BCC8we58BW8fMqaZDv”Œ"16nGbpLJkxnvdBQMeEbZCWvzv2KRofRQkL”Œ"37PhsBn5HCQyYF3m2ZM9BpiouGYDCn1tGg”Œ"31kF9tmoqghGe4Upoh5mwGHnMRZA7Zgpk1”Œ"329yLywaEkQtgR1atNzZkoqrASP66hmqQA”Œ"1LAJQbt66ALxm6tiyvoooLPrfs7e9yDhDx”Œ"121599rjUawvyrf8xUQnw6UDjUV8SQTiy5”Œ"3JKkeNEaD5fUCoDkLR7fe7SzCyVhUwpNGr”Œ"3BZ5GPLP81sj3NREyUjR2H9zBWhWH8VQwD”Œ"1GQcTHGzj9U2dAWeejYD2tWD6VFh3Bp4EC”Œ"1HNvwpa9d4TcAp2awEJSBHbBo3eSUagAnD”Œ"3LbRtBD2KrJq6ZhFxLTNHT9NWhY6ZxwUZe”Œ"155WWDK1wvefFLV7wzjvX1dZWayQFkVmcU”Œ"14i1WbVxTSqAYWE5yre4fwjdZAok7n1DzX”Œ"1NXsQLBDZhSg8Digk19V2GkXt6GFLnR3oA”Œ"1FsfJ4hhVmk9mCifGCdPCMTwwjtXmkaEDv”Œ"1KgMa2hyLQDcTZNoJ9muSGsV88F1zjvicf”Œ"34TYcHACxdJj8LJ2KmtsNQs2Z5XpbX3MXX”Œ"1Ej5Xs3VFXPAnaYwg6u2h6KTL1C4araoWo”Œ"1Aitu3VcMfG2chtDCJR6B3knHhzwGQe633”Œ"179Uar98K31TPg1BmNJurinpQwCuP3Jpep”Œ"16HBHtRoX7Vy1UT6JXna5hQZ4SJnWKf4Mp”Œ"17mZp1uhACeL8vN6LbMxiSc827FNizNjJF”Œ"1Bdjats25E7Da6fhHBPaaHUn32SRumbnF7”Œ"36dhGjDV9sPs6zZqmhdnMf8vj2JChWwhCZ”Œ"1L5zU1ih7rt8A5VZNePtSJ1auUBDCR8dTr”Œ"1Bgs3H9X3QMrxDZiiHBeGM4vRTzK2cMStJ”Œ"12ynCFPLq2qvPr3S2hVfXsRH9Pzorj2hQS”Œ"1NwVVrinaX1wjUEsoFVh4GjHEhSnPFe5Lm”Œ"14FGGebu56vv4XG5XmPW2ichtkN4SsABAn”Œ"32Q8W3PK6uNgPWTBWvmYr8XYHxJwdbiaFZ”Œ"1A5c5ckkKe1qAmDjiWLyq8Db4pRYGEpcy2”Œ"13NiRxyvP6LaAn718KQMRPtaXfGqvRzpDo”Œ"3JJZTUz14P8fR2tjMo9n8D2hCxafmfkn1Y”Œ"167KZasvpBbXz5E5rBczLGabzUa5zsAj8d”Œ"1ATQHTvkrBLALQyVXgbuqPA1hGh2jydeXW”Œ"16h5t1H1ZjveDFVSTKDR3wbqVjCpWeWd2B”Œ"1ERygaCP5k85rD5ALfp3CTVimc85rhyvjP”Œ"1LnF7bUivVvVY6vAXgobb4Wt4Ekq3Q7GDQ”Œ"1EHknnAf4ztmXZMT4uUs2HfZrYsMgoEqBy”Œ"1Jj45yn2u26LyX8kQdkBchX6NTkRPc7DbH”Œ"16c9BakdkE8JKR7xYpvkWsyBUerG4nR7bG”Œ"3Hw92T51F6giALtjsjMbJoL86eLT5JFTWE”Œ"18TeA2FmXdMJgRQ3gCuumNTrSdr76JfNiZ”Œ"1HUjW3cAbCXD2jLKAEox9myCPWyfJMTLHu”Œ"3LxrFtho3BuVc48sHqXbbv6hy34vZtuw1W”Œ"19nNVLsbddpzb17TPqTLUKiyTwcE6xfRpt”Œ"1C2xQwSmicFo4ow21d5ksCcTZ96Xdw4VVK”Œ"153zaxQgUtMriXrqwrVZgBUtpzdaQFd7od”Œ"3ChiCWdXotownkore5tyFYuzRydYzrBmkB”Œ"13xmfiJqH4c6Wv2EkQtxv2SN8Wm5tCQZqA”Œ"3QNNqx7ieeKWiiwv18JcoXnYmsVbqdEwTH”Œ"1G2BakQEpLyX1kejhDu6i16yXvy5JXtrbT”Œ"1CiyrdoaPT1Zo94eimrS8pmUGziAjuVMQs”Œ"1PskVb7uiKw5Jkrkmqvp7mwS6UsJRwa7JU”Œ!1fthf9pkPkPFSFZMVLSinihV5f3oLGnV2”Œ"1Hxhu93gwsRD8CnVXgcGzfVTjz7bT2ZAC9”Œ"1NkeHL2UZxNN9sLAwvk3BnyRn4GUDsmFpa”Œ"1KTkrSXuVtusA3rXpWmkTa8HoWrbm4d3kY”Œ"18pxcj7KFX6iCo1KeH12brL7qfv9wbkb79”Œ"1KSzipS3MHkVSwRGUoMBQZXB62ibrfnzgK”Œ"3CzWVdXbMgBzZTNiSWLS75N7azv7tk2Lvb”Œ"1MFPa2R8jsXxXuSkbBvm8sqNnwbNy9824q”Œ!1wcqeZ3zXiRYdVvYDsU4b7CAxENoLzRJ2”Œ!1hWs1kiwTbNT3CSVbtcbVJMJ9upXiudNv”Œ"1DofMqqnNo2wwm87WF7x7HZauRtgVyur2F”Œ"181fMeqLF4K6KNMiDPTZwFxE9hVvsNM1Jo”Œ"3N6ZTNZ7PcrnL1j9kMe1vfkuUfeFYeteSQ”Œ"15Ro1Z17Q5AmYUQMGLTu7hsDBLB5v8Bhe8”Œ"1CQpbYDNmMZWXpgHpwM9AdWcXKJ1C1JqxA”Œ"1JtSkjpX1scQMctShjKvp8bzp4Qq5p5mQJ”Œ"1EHWcKzpBdgouyy73338KbZcAbNdV7ZAfn”Œ"15CbHGmf3mgjc1ULRt4YAS3LCwxFwS9Jjw”Œ"1NaujPZAFegudJ5ydbD3YtYt5zJGDx4cxb”Œ"1GjwBL63h3ukJjrHWHLPSeuCXnWyEpUc7J”Œ"1EJ5iAQmUfquuYMR4DSWJy96ijsrCBeKQh”Œ"1Htbs89dmBuRxeRt3Ho1Cf8n8VDtjBnAvC”Œ"1CGGXjcWXdfw6fN9GHWXgqXKUwTHwvMeoU”Œ"1KrLKBCLhF4azXsZxiDxbGQoXfKXSujst1”Œ"13ANaTX6B3m22mrjCz1j1MQLBr3mC8MBdA”Œ"33sTAZVFsyrf6XMdzw26cQdZK84AEM1c5d”Œ"131bksrmGcPqJELeWEjbgEE1AGpn6ReoXY”Œ"1CWRgoj1VCMwBfqfoMfByR6Wv7Nt69kbjV”Œ"18m853FekB1dJ36UXBUTzNur6cLik7n3zv”Œ"1DgtEHtGRBAF7EtzcknnVEt7LJtPdiahGo”Œ"35VTBxWyAbuDk6tmRvvekx9UU9Xo6BhWwo”Œ"3CH1FdUtsD6pD8HTW4SB1jy66yr755SSEU”Œ"14TEPVscfVXzus7ZpVxJ3SiyAUuZceQ5BP”Œ"18KV9djxwsuFpZM4xbZ21UEn3sRAMj3L7t”Œ"13MLocsB5Fk3Rkn52A2rmBwpgmML6fz9Ma”Œ"1PVkYxPMbrxYfseHNTcpwyfjKwjdEdy9wS”Œ"19dgRfsFB5p7MeWn6PknVj9cW3o5xSPmsD”Œ"1Kfc2dkiqqqoke68EHaZL1CVDHphJfH7Ds”Œ"354HjkhrL9ebSQJeuAXaSHZkQhF3QrFyMr”Œ"1L4shm1Pn9C38dQzUWm6CmZUiiUaVQJfan”Œ"18gxzRDdjrgkGMrHuzw8XBcjMUHZF4qzAW”Œ"3LgDvxKbHzmowCTNVCyAYoNyXmFidRX9WS”Œ"18fEUynyVgLGFYPJijHvMw376PWQB4XCwm”Œ"3FuWaa4sJKsAWJWx2uJiBMipSmYCuYcJZf”Œ"33ZxNNBF7fTT6u8A6qUXqJWWnhcMncM5M8”Œ"18zw7oL6SvyTJNCN38NHGFy2SHgUjw4oyF”Œ"1Hiv7yp5o29ZCrGpqzdPpRK8o1UGPr9nFX”Œ"1Co3hoXSu6v4NeXoD1YbaVEHn1UXM8aqQh”Œ"1Mfgj1NWd4Mfis69ZfgSbmATpQ8jaX2FH2”Œ"3KDYnceqQ5Pp6TtTG1S8ctizi1vcVaQv7q”Œ"3Ki9RcJ9zw68tFyuYsZmighbwGNV3Bx9LC”(Œ"1Df3wKNXHDUFPhwp1kCygfKbRMdkXkunMg”Œ"3MKPmvthU7nsyY3pj877E64J4pLUHgv5qj”Œ"1MD4CzkW92RAgQuZxm9QzET1YTVxaU1c53”Œ"3QTZKbkwLwg6W6mbac7CqDXzuGqrgUBEKz”Œ"1GuXMSndejQLsvY4SGi1Hj8eKSSUsfnWdE”Œ"1NatzXjzKQap4uxTv8NC2auav2oy7Zksf2”Œ"1C5dk1aoiLXBTSSUKhKufrjVetLvwZWuXY”Œ"1ERVUHuiyqzRhYjrPNaSX59cjEDPwBgqeb”Œ"1LD6bdfCjvFQMbL7idncK29168cFCpKJxP”Œ"1MX2k16JBCR6np6giqiHrKnQwNKfPSXMVP”Œ"3DMvKK1oNkmPjy1f8hgD4Z9DxxxonXFT1B”Œ"3E6A3zZtoKnQK9S2FnFCy78by9u749E6ir”Œ"1DAKxXP57WRNqt6arCbQHyj9hPftYGSxap”Œ"3GCAiBYXqvrQPxzRtbEFx1AQW7D1jMR7U7”Œ"1J2G8KAdEjknjCutPmhJjZZaxVSXsBWT9q”Œ"3NHLg1nQZgGr2aD1hPqvpzMMb64HjQtqQv”Œ"16UwxikUqHyB2wsf79bAraFa2XeMc3aYPC”Œ!1ip4ZDdepxDjT3YFcpQWkHbhJ2fb46VCR”Œ"1D1nhyEEsvWKqjeJnA5sRfKWFt1FPPJ5KB”Œ"3EUQzCQs2RuMxHRPAD1faAMzhqS8oQ52Bq”Œ"32whze5nnAb6SuoQ2qv7EiAcfPnZDT2ezm”Œ"397JLNPg8XRdbhCDNJ7r8cLv44swJY2WoT”Œ"1AHriTGdEQSaNxsxdgyNPbFBSzj3q3wiFC”Œ"1NoiVCxVNdaW1hqSTxrYmFDUuyVvGuKZe5”Œ"1ABDXmaJesCvkwBaLBy89XBhf6e1AGTSiy”Œ"12btvJqCaufD94FTwVUUu9aUkRwD5s8YAS”Œ"19Xb6hGebcQvhfMyqN1LWsgtsHg5ixojUV”Œ"131HK4BFr1pfFwmdWez2tYhpdWahkrXAR3”Œ"122sCFqupHjX1QhXdffHoweEhMErVxDE55”Œ"1QHDj3ARUiHHY2DbEF9mVG3g5MXr3orKBk”Œ"1K17bzZVZtmPyyk1TeCGMsJiXpEB67QZ2E”Œ"1FUf7RM67mTpSAK6Q7dNyxiowFgobpg48x”Œ"13TfUTnXNRyoKarZXe6wVL64mF83UzLAuG”Œ"1GLkSAB8dEp5BbAx5Duirky8ievDrLJnWA”Œ"1QDHbJaWhWctNaGroLU7SmY1mMNQjcqhEc”Œ"1LXVkQE2gnFiiA5oAUGS6ZMsdPUq6k656Y”Œ"37vQmmLfThGCumry9nAapvtdbYc9gVmaa5”Œ"18CmyJQ48MhU3z3TJqEShhyPMCFUxYZDFH”Œ"1P2MxUTKF6mjRpvNPuWDBLCbrfU9R37Hhs”Œ"1AFzfsP6TZgCGPSjXQwhk2EGEV3WpqzsVV”Œ"1CcjGeTLCgmpJRYZU1a8dZUVYLK52Jt6kX”Œ"3BRRUuXq4qxVNSTTCwLGWmGUSuq2TKxNgN”Œ"1MMumtKaEweCZsdfBYNFQ5v4rPA6Wgni1r”Œ"1Ecr3J2bec3yFW5pYsHuAJWTvR1ic8BvCy”Œ"3EwJU75hZuRA6j7yF5R49N8h1etFQxTQ7N”Œ!18m2G772CFEm8h49uB9fwBvehGZeXN6e6”Œ"14H9xoxGMzDCmbT5ZcguXtXvEGfMUtbuyU”Œ"1GUt3rfwSgeSn4jLv4G4nDRPjfSnUAvZ44”Œ"1GgzcHLdG2sZiYHUYDuKYA2Ww43CXekovw”Œ"1K1PTM62AvNuYZs7JSTxkjP5XpTNv7MDC3”Œ"17vtxdjhjP9sNzj9k3zFbBRjf55hRS7kiJ”Œ"1DsJ1kXNWzCA2BLSZf1Mdh9xAkbpwKTUVW”Œ"12jjKFWRdekAHUaieJh3KXvqiVk79mepuK”Œ"1FH1WdFXX5qh1VRus8wZk9JSHCViCQmrUR”Œ"12moaD6KRZxYwXbBvCo3gQp8zWAutFeMRA”Œ"14eoxiisYpzAqZac8sQ5wx2Fcpxa8rhJYZ”Œ"1DsaEYUgfTiBLTwk1yk2SgosGXzBHMX7vY”Œ"16Em4AaiSMvPLT39zjHXca3Uw3yczJZrHS”Œ"138Mrr5iZmKF8QGcf8iqtprVX3Dk3jw5pH”Œ"17ELQcrmh67GWKmdmf9NVcHasxmo63BtWG”Œ"1JHTw1w4o8TdzpsceLv1fN74zHstrsvcZi”Œ"3BHaECiksGE62ZYdjVRm64jkuLjFgfNaw6”Œ"3QxbZcEGaEhNCa46QUQUCik1abwwDnUj7h”Œ"1QDgRK5rvYhiXZe1YCCkojLbkb9BHbFcgX”Œ"1NzBRifvYodqrmGTzBhVj9vPEncWUEJ8QC”Œ"343mB3AY4sFKNycHeVyX1yBF9wotvC9SAt”Œ"17BDsfGG2MACuEDPiyu7K8SjBLtnXxn824”Œ"1KCSgDevEzXrwUMfFunsBngjccoRrxrKfk”Œ"1FS1UbM78VxnBRt8eVxLB5CvAsY6NthEn3”Œ"1Dd7LmrsyRLTqDjs3kdsDMdKoqcPNhdZp6”Œ"178Ta6a7JkPeVN3LajfSUwRBsN2zTCfdB9”Œ"19bGUi172Z6igxgPVj9phNcjhmtvnVF6rW”Œ"1Fv2NX7gZvZoYdPgUAYEHu841Bbj2mmGUo”Œ"1P7zMaPAdf3Me7HJpei94mJaCB18t3PaYB”Œ"178NRispbeVTfkrLZinWPZZPPRMRVuWkQc”Œ"1Eipx2Cc5YhAgAkVHrZy87EDtevNmDbv2M”Œ"14v4DgSZQLnT2iiiU9NnBoifvCNHXGf4h3”Œ"1JZjm7JVkm6XAHVF7PEDcraUXgo5rd4EGA”Œ"1MXR4z4xB3ZEHsWpcYeCXGjb6jgErn4n1i”Œ"1DDsQHsq5dYtgSesqgiEorbcYCJQsU1Sy4”Œ"37dS1Kvp5iSoQRJ9tfaqVDWfy7j9yA7PFG”Œ!19ucKC6pxqL9y5ssRiXsGgbNDTacQqbsr”Œ"1EQKso9Xdo2wAYdn3PnQnYdwYheovPeovH”Œ"14ZZ9mujJKdY19FFJZEBmorhoHCiJ3ci8k”Œ"1JDphuQpaJsmKCDS5PzSgiDgiuQpkWWSey”Œ"38GKaU5uRBv8kSPvYqLQy2ikUv1bEyDF78”Œ"17NAxyqqb6JrVYdikaUvVg5W1XPmRfkMVD”Œ"1GqgoWvsaym1XV2w8BixrnkMM2BSVhw9B3”Œ"39PYGa2T7aqyKTQb6TvB8K2gnSkZn7zk2k”Œ"16pbnfEPZsLPLKtLwzstheCW2Ko8NCTFQC”Œ"3NArkWWKvaw9EhjVczFGKEhYUnh2cabGqd”Œ"1BnUMHtQiCmRVjP7Z7fPkiAwFk6vBRSBhb”Œ"3Q1miB6ahK2rFG7ZFXQ9sCXgU4s4fhZpBn”Œ"15Ddqo5DGMKfxYTct41RsoYDFKbzwcwU9N”Œ"15PfoLCnZoBakdMTWLaQzgd1rwjaDDgBTC”Œ"1C3C5LuMQoDMuPGkEdXkSu7YeWTBujmoSd”Œ"3KYCtVD3qPthbWn8kJku9WqWFK1RaLJNP5”Œ"19suFae77ShLvBPknWvVsLxrgsGrfd9esU”Œ"19DewfBLXKvtKAuLqBhxkJFtMvaPiPbifb”Œ"1ALhkP9engMqqKh5WVsfkrWoNyV9pxBezp”Œ"33fduW9rmNZtpVSHA5Ci8q3S6WYizqxFcH”Œ"1JeWQBsbYsrLNxkHLj1xjjDS1XLjbPuUC9”Œ"1D5XJV4hEGUz2yfApo8TWT94csfi9BJLPR”Œ"1DwCx1UXTFD7z12hF63JWtvQtfP1SjmXmB”Œ"3EJFuLz9dcruVKPbcEnWbdvLg3E2JKKDpY”Œ"1CDUDEsniDwAh9P8U3GY81nMSVrGmqMgij”Œ"1Gd5vH3cKkhXciACrm4pGciodVQYsqWt9o”Œ"31tT27QBEf3aWiPPGLfKjpfS9r9cMt2gwe”Œ"1HocXuvbqmru8npokSpt1dAZG6qpWNj83C”Œ"1GXHQikHjxsUCz38vu7dUToGSZnLqDUjwY”Œ"35K4Wz1HCcGWAeTRbNfJuJGW5zgfUn6cXG”Œ"1CAk4U3cKKYui3bXu4veTqcqrcUicezCVF”Œ"3B6jvuW7jz7mrgae2bKDLzyUnaF6AMNMuM”Œ"38mTS7njnz94YNSnmgcy8kz4tJL8HaAstc”Œ"17usmFdnRyxPYr3FMdHzaAtPGGtcYvb4sg”Œ"1Ps1BW6jfj1KZ9Yt6J5CfDbpz7SbbqaGNW”Œ"3DBzpudnP3oKe8sbqKWvvYF3tKtUrhVcqN”Œ"1KzPihbFvD6jNVBatHpYLfqpifwjoz5Pve”Œ"3F4Bg7bUPj9L5ramcE1qfFroCY3QUosnry”Œ"35v7v5BmWd2n8dcz1Kuax1ap5mHKy4iwYn”Œ"123pfEJfCYATvsgVvWsN5VbMsAGUKYDX7Z”Œ"1Abogu4d5WRsMWrk7hRNF7aVpGjdUQvU5u”Œ"17dXoJX5qrshMEYmUzyM4ac8ZJDGYYtbQu”Œ"17sqcc7qSLhz2tatgQKccmizVYz3LP8sRU”Œ"19xuKwgfphk3mMaN7TEYYYULLou671KxfC”Œ"1AbeQoAcVoFXPBBEyQ5QhonCzZsyK61AeT”Œ"1CVSXrHPTUTqFb9ggRKNMSagpB8izYtBrP”Œ"1Ci9QPnHedEmXbmu9tZJ6zKL8C1zK28mds”Œ"19wt8B39ppxhw1k1g6C8X7nMb7kUG2iido”Œ"1BW792qaYY255ky2Nnt2HfqZ9FTKvN5vUX”Œ"3HFuP6r5JAeaSsMSQrtFoKHzPTT9cEAc3e”Œ"17wgW4T9KXGM3zBC893BcCdcvATFwpL7no”Œ"1JhH49mhZX2Hz3VqbLuxypcPbKirtA2emT”Œ"1JSvxALp4rwinqh4rX7tihRcG4sJryxnc3”Œ"3MeJNxx6zqR5dZ5G89xJcmm2b3CdkSPbK8”Œ"1ApB7NjgPteXotjWoPqdSVRgG1vYuaMiv1”Œ"3JGqDLKvBm42NX3nZfGYHuCpgxGQWDCyAK”Œ"1MKM3h9ryR7gfeLrE2LSwqMMwemWgxzyy7”Œ"3HRR4jrpk3iL74W1umhPfBBqo6gRoMWmHQ”Œ"1JV2ryuq1hHEHEUYmq1ueKaLXkvCsP1DWs”Œ"1DENjFFtHV6bgVCt4cJCbToKKBpbMzdMw6”Œ"33zchVswCGqEEQibLFA8y1iUS5bQHnqV9y”Œ"1Bk1x15r3uDs5Gwt4k9uxJ5ArS3kQWLzfC”Œ"3AJ5BWuMKc12YWpgG4XNkdAcFnj9nW9ZQu”Œ"123XJpiWUo5e6ehbiMW1vTqPBARA6B8sod”Œ"1G3sE56cFGkv2sfNxnqHDurNN5LQGPNaGo”Œ"1PwrmdJQ5BKfReP7a2m9Kf6cAZyiJc6qon”Œ"15VdydfiiTkw6p1b3FLDPUxF68SjSXKxY4”Œ"1PixMukDjN7yaW6DgHQcZtdBPwCmtZgnnD”Œ"1FREUK8zDdn2WU7ioRAAPA5VDqWauPCeRg”Œ"3HvB9mbXn19fYCpuZPrvsmzNogLvCYFWcK”Œ"13VtDWKmiznFwRwgvgRkdhiddxZ6i4VMmC”Œ"37LL6M8KYvGdBdFXdYNPeu1WdbhaCKFTfR”Œ"19VuL6pr4aqNZ8MHRNZSCj8qQVk5hQqmNY”Œ"1FqNYTmjUEnvpwZvvLf37GCyj2kWrSACJ3”Œ"1LatdX6NBspkLmrwTjXwA33GYGJQx2kjkj”Œ"1GCJMrF8Vsy7rtC3ZeikdhYaq7cwTUQMRo”Œ"1QAXT3aq7T8FByTLVSBNYvuyK1YmE9TPAR”Œ"1GAGfamAuSFwcrrYscQ2VrUhPEfHWPMXHM”Œ"3Km7xDayoNHHpgy2LySfaTfskKKNjTzp7P”Œ"1PnTEmqDaAsZa215SZqu63zDwRQttQkXmA”Œ"1JjHgwT8qQGZPVepao84ANKxaucqRBCr8F”Œ"18AsoP18Cg7DLKJHZ382dqkiDSqTYn8xAb”Œ"1Lti2444mZv8wgTesri2ZX4iYaiCZUuzZR”Œ"3P2eoKr3vAeZhJcTzon3VFkv5r7DqSXW9G”Œ"1GQSnzh9JRgipxC7btKvD3rBS8Zj8SVnAo”Œ"18kpQjnYDSkvg2SQmmGTwZvSQoddAYfcsA”Œ"15CVgskHeV4BFmmDis4RSxLN7TnouMk88C”Œ"1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru”Œ"14EzdstYRjxU7tUP3ngq5ticwfuLwED9SM”Œ"19gYS2NdPT9PQysztsEgrLvHgMkZ5uek8r”Œ"1NHsiorLyAEenkNHmT8aMJGdBNUpyegmZD”Œ"3L5FbpCBq8kQKV5TZQbJVgUB8WuonY1BMH”Œ"1Bu9xAWaszJNj9pQtaJT4oNg7vJEz3YMNw”Œ"1CyjeERYKxRG7iBwBSzwQPAmj1F1BuQET7”Œ"37ZcVqmitwW6UEmhVKkJg6x4gtNxjiE4Lg”Œ"1FjPbuZ6outZHYxSx8UzHPL5J4itWorejz”Œ"1GWsnZX62KuBHp4szLUuwgmQ3CcUJ6aCPj”Œ"3FcoVowJ9jaHTX9CYFgbQiJaDP3uXHzYxc”Œ"3HmHWweGZw14qSgaeULtVJJNFvwB7kgrtc”Œ"1DkAfoHePwhzYuaDkprpAvYxaKbXoxPAQW”Œ"1MR6jxyxPswp3dQeAoQvW8bErMsnefQPqu”Œ"1BEq1HchTvcj7fCkmZncjKvKVkpFC6CCdj”Œ"31mmmKvGPggmUMvs5kBpFW5so736ZzDuPJ”Œ"3N9jGyMqkBBRkc2JmP43NPQMpM7Rdfhs7E”Œ"16NNqwapYrn398vXineC2aPYswuvdnn17M”Œ"1BVZqvkNLsL7exLU5ccqHut2xRhYLhTd5e”Œ"3CMCABMi2WgthnqzGcLkeR5Rq2BzyRmvjw”Œ"3QuWjqUXWqUtUkMgzy4AFyFN4KUiSsdCg6”Œ"18ctqiMFjbxPpcsaNqUjLYJE3V9ikmuCxx”Œ"1HFJiKApqhuztYVZpPbRsjNoTF1Pedv5aZ”Œ"1Na7PfNugT7LUAiuJ4BFKsfpXqmEWvSSVC”Œ"196qMf8B1yiLRe9mTg3ECocZj4yTaZxD6B”Œ"1EUpWQ3s24FiaQfqVrC2pygufSyyZmpwMo”Œ"1MD4LpbQ5G63pK5u4JEzwFaKVhKYuVicNv”Œ"1NaUYV8TqaqHGdKpA71ZCpDzJK3dqj3Ttj”Œ"1HL396rZCg1ipzHmquRm6ruxtFkLYDuEdF”Œ"1MxaeFzdYkyJ9dhzVPWEm6iwKtKW6srDpA”Œ"17yjiwvd4b1weKTpwnHHDNR4BG4f5LvLUx”Œ"1NVkBfjMZg3x9dFVRLhjkpGq1xeBDkZo8A”Œ"17SK5VyNm2KuoZj6L1UR9dwJmauePaQ8Wj”Œ"13CqfBqARS5cNAZGpoe7adbi2nsQUffjba”Œ"1GYMDEKiyqVfixRmkBXRFBVxUSZmY9BPJs”Œ"1Ky2NedHRUnJ9KBQk5A1Dgg2Qkhh9QbvYt”Œ"1L78icBxENVe9Puv8zdsMNLYVCyjy3v3Nk”Œ"14f7DNSe7XgdLiiWTUArX2tcAjpvBvcK8D”Œ"1Jb8cgr1RHrtLhgqTiaMG27ctpawXJjdn6”Œ"38KJE8BiEsrGvx7pZxTHxgjvWik5bGMhJR”Œ"13VEceULTjB6smHbyBPxdGRbAHrWS6iWTY”Œ"1Mx6mVztMe8mh8PgTStaf88nqcUdFRoyeU”Œ"1H3qLmskhzVSabb62KPYWAir3jenZ4aHkX”Œ"1JQCSWxspui8TEKm3ccNtFZv4yLFEvNNBf”Œ"1D3MAVHhpfw9ChCYY7kxqLpZyxxnr4ddpy”Œ"1PZkUjtsWVT9CojJ5aFXgVCPwnnuoahXQj”Œ"14q8vZWFUw6E8hJCeNmqhvz6osbDDSdnPg”Œ!1nDNiGUFcb312CyGX1WyoschbSd1Fb2RJ”Œ"16bE5MzpbrqxzmGAJyQk59H1ju2nNbefgf”Œ"192UfTzDs36ve3mNi3PQ1paEyGq8UiekMM”Œ"36CmU1BPrddL5YkTv7jDLfarzZFaeFHj7Y”Œ"13sqReiiQyFwgnPoP7SivNoC8qnkwYFwwp”Œ"19QMJ9ovp1NKjrzCSzeC64qqsg1e9pyHrV”Œ"1E8obaxyDGQduoU7dKUDL2xf74335gxfv1”Œ"1A5xGBsHXHbeuTD1XGj93zEBLAsA7BP88v”Œ"1LtpFiWCPm2aRi9cBEeQMxZDCo5xAKL8h5”Œ"18RRd5qKoCqmiMJ2d4FRvShUCsJBkZMTVW”Œ"1KpCQxasjpfoMa32DMcjnNxYbdhZhJ8iV1”Œ!1LsMtKbhrjyfyEjk2UMo5reAdFK3h11b2”Œ"1MdLZEAWhv5C9Sp6ZLedauEQNcLjBcG14F”Œ"1Fuc8qnouczRCu4rY2UdY8BCBrQdVoVAA3”Œ"3CpvASGPhhbZQecJCG7XSHegtep6EbbcfW”Œ"1LQyg2yonLPDPzLAEEAJ8pJrBmM9wVZxKC”Œ"1GcxaHZstscucUgSyGVXHmxoHwUrnrtSnh”Œ"1DehFQS5vUpyyhp6UqapFVcd4wpMZirAfc”Œ"16xaWnqACggjcNtAHspTSsd9w6J7MUQRsY”Œ"1KLFxfxRvUY2vruCSsSJTMtGvA7pTgRSdt”Œ"1NUHSwSEwRknEU1v1SYNdTvGT1peGhsURN”Œ"35578dydsmHDGz3KRqH6WhMe1kpBQfR1Xa”Œ"3H1shFSnfAQZ6VYSfzbbmX6G4hWvtHkyaT”Œ"1NXmMwv5s2QYn4ASCXQFXKNAc9g7cDS3dx”Œ!1PAJ9qNyzcDuxdWfmJ3BKnmcqpNJivoAM”Œ"1FRFSSrpx5L5ajS9mTGUSZ6NT5YoH624ur”Œ"14ukzunCgtc2VU1noV7HrdymNDRbZaoV5q”Œ"19EZqiWQaTfRFfhSMMTdmYxD721Gj3UhFg”Œ"1GwKKWTzUnaTo53X5VTHw9skBK3xCpXYd7”Œ"1NGPpsiDATDKYGB7XLhJLQWVo8xo5HxQfK”Œ"17D3L7axAGV79Ep3TAtZQrJbEeEdECbb7i”Œ"1BCnTBygUNT26cjeotfCTxiAqEpvHuYofP”Œ"1CDi8hGnTaer3CRmYspTyNav1jxBxLgiEC”Œ"1PWPDXByzCkVXQ3D4XVNB7nUarET3CK42m”Œ"114LhnRzAbZPniMmdBwqQ6QMYjxYFX8Qxb”Œ"1Ej4Jm8J83tKG4wUAbikNF3rQoGckH4Emp”Œ"1KyXkWaZnvfvEKZvBLwoN274iuYseBQ3qy”Œ"34Jc8GQD6VyvDs6NBKyeqz5q35t8NrW7M8”Œ"12ogfuZrw2XFnu9r5CeS7zMxregP9F7zpM”Œ"3E5LmwqxsrXxK6rpZiwgDD7BSokYDRnm66”Œ"1MCha5ryp6juidTbHT7wmdRCo4MSRrLxfs”Œ"13a5HagAdsphZSzsCEanW2fbx4PpcaUYcY”Œ"12Bz63AVGdo4QdPtTG6PEaJUWT5FiXvJT7”Œ"1DhHGzHFdkxgRCobMY9k8xmhJiURPvDnJj”Œ"1LnTxw3uNwUXJa9B7XMvtqaL4rxLRFAAtA”Œ"19heaak2JQFUsTpURePKYbNaqcJNqPdb75”Œ"1JNb6Kh8x9A1PZ3UdzsjocisAjU5SxmduA”Œ"1C9u2tYJd1uwudDV3WH4AbRmiftAL4Fas3”Œ"1CsFyJ22FKm7PuAK6judmvHuEm8GRsE6xt”Œ!1aXSiEunpzjDs9Ff471KAJpUirYhWrYgG”Œ"1KuQaDLLmGEdEFW2FenusAuzyaAG9HuZ1Y”Œ"3BeYSqGmDzkHMpXrS4hddprp52EwkVTNp8”Œ"3QyLjKpqe5ErvBZ1G819QKYhkTSF22wU7h”Œ"1N6dNaVTVUQJGhXmZwFHE6YNBc7SYdUGG1”Œ"1Kdf2pwVCbnSqcCUycs2cfcYrUyK69ZWPd”Œ"1JrCoFkRtFPWyDrcXvXeTQq8RzUw5bqEFU”Œ"1DgqMiLDEvikrifGqAGG8WCkMAgnDi8BHU”Œ"1LYEMYxpSFrQsmsQx5Ej5D7KR2Q2wCt9R2”Œ"1164yXRnZcKbNyxCxmCm3VJqjU27EpYQ48”Œ!1Ru5r73vaNjJmkJ5QYKTBsuhDBR37v9FZ”Œ"1HxYZsCNJCLLSQafKAPLMnhjx6ZKBzWt35”Œ"1PMkKQu52rm4f2od1WCjKyDRt1yHVJJjaF”Œ"38DdqhVuqb36jjmxTbvBkiFPXKA8EmW9Ly”Œ"3PeRQxG2Qq6PwNGRABc1fGoWxYd6nUZMx8”Œ"1E5xGaHM1kVD1tRxTRK4FLEWb1koAns6Fm”Œ"3Pp1DjF8ghYqTEji3uXLCtLCjh65zZXDzN”Œ"38hLHsHh8tkUNcLQoTZAMSniS2hg3kmhCb”Œ"1BTV1wj4RCUYECWdhuwAZuqcn9Rx83boeK”Œ"157bumGbisskuVBGUZdWRrMAhWTsrPcVnP”Œ"1NvvR4s7B26uEBKC89riJhRKPe82oEg4e7”Œ"1FjMAAFRfmSst2eMvMYGiQBZobqYoPczK9”Œ"1Ak4LrCPDU1eMscK4sHXyVgMDV5exVs73J”Œ"1DWD7EBqQLUgWwbQmEJ1tyShRSCu2XqycD”Œ"1GPfeSpEZ4RYGbw5GhYmJhuZLi2su1yeCM”Œ"1GwHTE5vW3rGwx289JTCHRmDFay1GbmfvR”Œ"16ye5Yu15BYpLYSBi9CWtngoedMgw3H5Pj”Œ"18nWmLPUBT6NxhP1YNf8W9wB6dxEQ67Lsn”Œ"1CRWAiXcHHW6VWejLife96MMGUjT92kk2F”Œ"16w9Ew8ano7igombXBj7tEN2a12rPwVT44”Œ"3Pf1pLw3TeydaGzJJEyDKLfL9mmJd337Z4”Œ"17AeVyg3W9Sp476vKeoXdEqQiZUprgbQVU”Œ"1K972iBm2grPV5agqoTBCmFQ6fY3mS3xNP”Œ"1MTSaETTtTuycEjJgpwmC9VskUdNBvwpND”Œ"1691EMnEqQEuZq3yRVGnmzDy4AsmJnwaHW”Œ"39pfYno1AtFo9xxo2a1bsD9pBduBffCj7d”Œ"1MXVY5GZT8vgn9eA4MPoGiSeXU8HZkCGk3”Œ"1AqWYXdZ5rz94vjWU7PXjvVx7RsAfPFiLC”Œ!1KgYwqkpQL8qkcFqvVav696Dm8jXBycFA”Œ"1QCHiktnCsubmzc42L9NziawgXYvszQQaG”Œ"14YeXMmTXYd1d36v5Ca5mVkm5Ev9YRzaZR”Œ"14D9iHFEaecYKBTyyTztHYhcNsNvGfARYk”Œ"19sdcwgCMJ51mdMdeFjVVqH2c8qUeh9V4K”Œ"3Bgco4jg4YZV7SUyHvzwYXMFCvLVkwj1Sb”Œ"3GEFaxBFLS9xuni7KieBb46zgBq8VKSiki”Œ"1FGaqGjJdXhXNhpVaQiEqpQHdPwYaQVHZ2”Œ"1Ld8Ek21CccTxXygQ4ALjRgx3dsdHSMmCM”Œ"1LtmRRUbUYKd9bHrYjEbUnaTxnFp15eiby”Œ"39Kb4DLpkW6hccBjhYHAPDJMd9B7Hs9xdn”Œ"13Mg2Yh9uqkfb4GRAErTbDs7BVQJEZ3cqd”Œ"1DP7rcYKXMKKm3wJ46Yyrmf4dXLEeC3Veq”Œ"13RmKSBPWXyzpvcUSfXe4skBcxak1vorJF”Œ"3H2jceuFXGcLnXUABhLhAbXygN6vjtuBqz”Œ"1GpYDDWEzepEco8qy7fSo16AxroQBwogoG”Œ"37MJQS838wdBfFP2mh7Nd1PVe4NvLUk2gF”Œ"1799nw5BWuY6kXMfkwRZCt58V6KzYaqodX”Œ"18z5e1i4gQrb1BpHbetnwPAU4wxrj3RRpU”Œ"3EP4xyWmB8u6ZDmMQHFv1oYPEETUjwMrQV”Œ"3CPYzD2hheycCYwGzMfD8iXjhw4tRTmWMQ”Œ"3K46gUx3RNwBiQaCFCaNJgCVV3UF8YAVFy”Œ"3KiZ1Z5evLsyxrjuZAxEPtLyLUNa4fy83s”Œ"15jTEydgMfs3gC49L554HnCg5jVykL5hQy”Œ"176KnxknEM7rfHGyMFsF4DgpEh7UV6pXdT”Œ"3B4ugysvGVLsNhkWb4BpUABc7LWRe1mp8K”Œ"12Ceka3AUArGCijiZQ6teCJ9tvPWoA4tTH”Œ"17LNWdgLv3d6R1y4VjHQ5ycufPrtnr1Umh”Œ"1748Fc1QCdcGQw89fMXxxXXGb4w2YuTp3b”Œ"3QtDiDx9j43JYyumug78ajV9EcEJf173XJ”Œ"1BSME5vZ41U44DYeKpfuR5zti13haPH5bP”Œ"1PyMNCiEBBdFdTbVAEu3ovdJqeb2fD1KzR”Œ"3NALGH1EebbSbxAY2nNxtDcA1WXBKxn83d”Œ"3B8nGAHwHwqf1emV2pbMgnhBHzoBB4dkA3”Œ"3PuofHjfGkibtkfoxnnTgChHjt3rM9pvWE”Œ!1w3uecoYwnMfLMkfBsjxYsCQjgWXEjgNn”Œ"1A62jBNy7AB959qnCqgRX5SdcxTRo2NguL”Œ"1CusDPp8NoiVuYyPbPfAvHx1oTaGZ6ko6z”Œ"12YTPwqrwvVYHEf86y36CH3ZRjiEPRGLP7”Œ"1H4ji2C8fZPw5bkzvCF8cUxBfadh89EorU”Œ"1CFdxcyuvtiDKUZbKKb62XaRwRTMHyw8oY”Œ"1NENWSoN5bRhrewtcwgv4nhvDN7jJSkTaE”Œ"16yJuT4qWAvGn5H9dFD3jyXfFVHzUhHCsD”Œ"15SXb4ueX5XBxVmXQZa4m573dPZDsANmxU”Œ"13SDTEY1b6UeqdCsZNKaPYGWEchqxAYuGf”Œ"114An6Yy8RhPymALs4pcD1gDEyTQ4MABCD”Œ"33CTMfSeUJEWDiNGdp5AGVu3cM7ANoaN81”Œ"1A2pPrQRKorAt1JCoiEj2fEN1zq5EZkpzg”Œ"19KsQVMZ9v3WC9S6XgNSsGqDnPUD19Fdww”Œ"1A36ogRkZfmGkYmWCchGEUv3ShKkREX5qY”Œ"3Dm5rgox9qpbw63wY9YdRzYFcPkpJn8ge9”Œ"3LBVcHiq3cJ2xKTG4bL9TN2barcJnXp8RT”Œ"1PjhJ2DQktBjW8GJKAUzSYZib2EE3ejQgc”Œ"18PpayZV53SjuZEWBtnQx67vaWUg5pDYAN”Œ"36h6CT4Dk1nXqfp18FiZH4noaXzurwtA8e”Œ"1JMuJfFRJFrWbc65doynYEQHrhahURbeCD”Œ"3Pz5peynFdQxQrP1d8A3dQuJoqyVfZMMjH”Œ"18J7jFkuTR3yVJU3vgJqn111Zam79L1uDo”Œ"1EZ7PLk4hKwTWhbKmrbevm7P186Aimoz1x”Œ"1FGv4RMbUL7gwfkbrfiBGVKdkdDYEKbiET”Œ"3AcMACuGvG6dhg3H4QAwAekD6mUvS4MV5p”Œ"1GXwzgt5fH1cxWMgBxZpq7rq4DAu6FMKWT”Œ"1ChA3k9ULfux6QecSoq1SYsrJYukv2dkZm”Œ"37p9pUugydmoLpQyFLLqGAgjWmUFERa1Pq”Œ"15jru7nhqicVPxP5yYHfkFV2njz89CPfi7”Œ"1EFw2B5VnFaKGPuRVT7bPJfnrLrj9JQAN1”Œ"1F3vBuxFDsinhp2aMpNxmvYnfoVDYz2hdv”Œ"1JQ88mx1oVnvFp6HBV2gWF5ppkxechraup”Œ"1KJ7CGtKrsLrD8HyCXydGAWFvxYLZM6ByP”Œ"1Da7ueptbsKMmqF7CK7Doig25Q9oWyCjyZ”Œ"1Mr6TRUPBsv6Azb68ndmDeEeRDJ6EtzPbr”Œ"1JEgjT3eT49hsxEx3kT39sS24eTDQduaZv”Œ!1kiRsu9XdhCdU7FYu8diEsLMdbRXbcces”Œ"1M3NJtFueegPi6hRimS6PDEsNpfHPbP1ct”Œ"3KxUvaLms31oFrR9JVcd95YdULXBxqyJD3”Œ"1FmsS7c4QX9BCMJT9Y4YgRyGR1dmPADi12”Œ"1JbtkD2LTgtMHLszMoiZJnsiYwDTt2RFwk”Œ"3CuceYa3921C247dPSJuH33MH2xKP1JxXH”Œ"1JEhGi85rwVyg72GKt2f6K2BZQEr8gvZ4s”Œ"15GfgewnQy1thVf65mnNXZ7WvPwckmyC43”Œ"193naAr3DSrzPZYHBepMcvF9wMcrgz2Vhp”Œ"12WSpNaaR1hS8Fk7in63rhVpK6hiik55TG”Œ"131bYwnnb9g1m9kBGD28RiqKp7vrnT3qqL”Œ"1G7nksMn5xMDBkFqPdnuAtAYipeBqW6PSd”Œ"13Msdq4J9csJ1L1FuSH5j8S9BJzogXFpoY”Œ"19vYqj4TEZeEy4Awit8nrJ2HDXy8nyEb5Z”Œ"3QTVdoojCF4scT6VHQTFTo6Mi4m3nb33T4”Œ!1r26FAP76L2dbUSrCnTMZBzx6dmovkKSZ”Œ"1Kx97iBFZiYjqnXkfTqBvPBevpj15AQLCt”Œ"39jhLCB1uvBuA1pxUMRZspCMAWAvtHeigy”Œ!17Nfk7KoufbjB6Nyom5b7ZiZMo1evDfaG”Œ"1LrRkKh9nivrdUhhqK6EZcEeU6ucDDp6uo”Œ"166axJCtEb5iV8gEvbh4LTPwNjGCY8ANK4”Œ"1C7Bu7qPgtxCkLPZSLzHk9cVyUW4RGGxH5”Œ!1nBYkQTUboJQTQ6GM54QWW8MMkRi3e8p1”Œ"1JShVSxf1ZPEAkXdW1fHLH89cMp7EtmzfW”Œ"1LXRyxUgPAeFD5riYbmkraScDphjcW2aaP”Œ"1167uucpHN9pzRo7bNw5m94tADcSBgs8Cn”Œ"139qJ3CrhRh7YTUypaDmsVpfr5QSg7qQge”Œ!1aNPXEb2fXMq5njzUiDMMhM9X5GQE8yBS”Œ"1LcjHtBUWjU4zTnoKoeKuVjNCyWH8Cudjc”Œ"14KppNHdBYvYmAiCdXJF7H1WFAHbUTSdB5”Œ"17htJwdkgg4EwYV48u8MtkmKC3zPr7zC3k”Œ"1L1Ks5bFPiK2vBrA5TqSgMJ1QQZrDCBtvx”Œ"1BpwkY6PxKX1gk8w2ffS4pTifTjsUG6Faa”Œ"1L3NqsAB8KDgwjCg8fHuxJgFHW3YKnssTv”Œ"1NGg6UVQGg93TAWwBLerVxeJEWfDx1Ufv3”Œ"1CK2EKvgM2DUkPMZXzwio54NUVLwswPxnG”Œ"14KPrZ4JT9yYf7GEjpVx47E5WzijZK8Jac”Œ"166jmuvxGKfAu6iaobqVUbt4rWaHrwHjNw”Œ"1GFEmqCZjiQjnFoqCN5KJq6JWc1nXVwr7r”Œ"1LZyGfbVV7yVVWqULW77P8NHeLrQCt3nos”Œ"1FE9GcSuj3ZLKrBiLDicBZkWWUZA5LuMgY”Œ"1BDAZBX5UWiLwNki77UHFr8gmhsrJGtC6T”Œ"1CA7UZHpGPsvNjfY2p6Z5FDK8DhrGQW9aB”Œ"1Dy4e8y2EPJgzxDnUCY44zMa55GmGbaaZs”Œ"1Lt5oNbwWLiCMhFr5jvBzTF7wWFUAVbASF”Œ"1EFSKpjv5HPFov5ZpHsFn9Xg17iCCpDZ5Z”Œ"1EpGNSCe8NntUtEHL4VP33Shw19F9dLD4e”Œ"1AeqgtHedfA2yVXH6GiKLS2JGkfWfgyTC6”Œ"1FmdXYwsSpm2Pkimp43fPNq8YQRHUcE2xn”Œ"1Fs4ECr5ueBWR7751VaVgtBGbiWQaE5oyF”Œ"1JJHGWA32vUwau2bgPA5sLLCZ9wHihvYXX”Œ"3EyuLZG7rjXidLeEuBpbXmZPqNC1i4JrrP”Œ"3QoHrQQPHSeQgobh99mJfe8RJynmUSS1bN”Œ"3EFqMV5srqj9ok8gbQg6RrHVt3gcXBKMyP”Œ"1QGwdTRP6bK8Lck5GMmXC34gXxBy62oh1S”Œ"15iaCFcSsVh4yAtPBZo3v1qxu2oWXAyXHt”Œ"3Au747mrpu4qqotA5TXahUw5jHGQHHMefi”Œ"1NEWWFRDG4xytgJxhq5LmtQHNEyhdNtihc”Œ"1MitZd7wxWVgXAP8xhUDvzmRRwAXjPbhcT”Œ"14dzs7mBLJLRC7mkyhkd8MFAjeE4SSVSAS”Œ"1NuHNc1WXb3K4TzchND4YvM8RcxzgGeoxj”Œ"1Pf3geoqLCavGr5PU4kEdttayaiiUKfABQ”Œ"1EsYbYtdMYzHNyHpJ4UgT2bKfy9MzAGuzb”Œ"14NmjggHSmdJq8d3B55QPf3E3Ny9haEiDg”Œ"17ynrNSQNMC396CZkqELTsLJBegmgxq4CT”Œ"13om7o49ngiAesTURVP5i9sDyhZxYyUV9U”Œ"185ksjqfKL7xwm8HMJVhi5qtsxJRgYfaqa”Œ"1HRp6fNo4pZZxeYAyZkp948MLXaMhpPNE4”Œ!1VTntrfhbBCSkjhGceE3iBBDvXoa5AqHq”Œ"3BQfnPUrYQMnCoLrLcvR5mhLWrXoCe8d3j”Œ"34j92rGasXrgxR36eFqE7HaB7UVrGLt1gL”Œ"186knLiXTmRSFmPrye19Tb89ppp6mjUpvu”Œ"38YEiZbMjMxbNTXCiiHx1dLndryL56s5xr”Œ"1CXwEdmgDZbScxmuL64PCZRiBzzphDPknR”Œ"1DFVEkJMvSvydALgTdm5LTySFw9R6PNj2T”Œ"1Jt4owQxbnmu5K4nRLTaV9zkHDAtJriuPb”Œ"1NKsN8P6h1XV2vRFJyqW2F7eSSqE96zcLT”Œ!1aiwM8ySCJnt7HBgDD55Ub1tkW6iH8RnQ”Œ"1A4mBC9QSBdX2CARGZzyhWVZ4n6gBCHYpa”Œ"3NRU5aAXxSJgPYF6A17ByXKs97UUmwvySs”Œ"1NXTwzJvKKXVTF7r7jPxWympVZCobRy5qG”Œ"1D3Qti5dPB4jZ634zP1QY94AXKawU54v5p”Œ"1CyAeXVScryCXc2ud5S3mUz2XzAZfayNJ4”Œ"3QwTa2E53psjA6QS93cT2PasjPgbj6zTCg”Œ"3Js5DNhJKSjecVRnginpL8EEoTkXwZZg4w”Œ"3CEqkkzEWw4qkDypNifGAW3VBvusSLT4Kh”Œ"3AcRaisLx69aLNgS3gZbaoeEUfPopMmNXP”Œ"36uivaJK8SyMMo4K1ZcDrTpaV4WiK2iTPY”Œ"3CTG9UAsSwq5GrZBQo854iY67bfHZEuAFW”Œ"12Nkx5KVy8JsJi37N2NNmfSmBz6K4LcjYd”Œ"18nQaEKduX6rqzWrsnH5b6XMNXTF1oYqaf”Œ"1GvnrUNPXni3e3FShjqpvdLSEVRwyNjqtQ”Œ"1GCM6Vkf9KHcZb23UQ46UfC8xJFrc6z6a2”Œ"1HzirnmtpLzK61YhHRL6qKjtg49ssDL55X”Œ"1E9keZVidCTPVEPYHUR9eJ1rn8a93ndphP”Œ"3NbhCueQshsmNMnD5QsbK3V5tQjPW8US2b”Œ"177vHAdQbbMx63Lr3BDV3Z6scfFS1kokvr”Œ"1ECZKfa19cTHtXh7uhkibNQmt24iz4VGiV”Œ"1EGffPhFCJ7Tbwxi351MR1fCPCU9R38Wou”Œ"12uR5fDssUPXSA7Ehv3dAS5Muzq4n4VBA7”Œ"1DoE6gACpmc2tQudsdxh8VF756oRhbc6SQ”Œ"1FWYRQGdxWHghtZjy6a57LjFpyW33QCX48”Œ"3C1XdGLYWm2ki5HrxV8ENqDb5fHQFAF7UF”Œ"15XjSojPF49EQk9yjSP1L416CUDPDYHFrm”Œ"15som9g9yeo67ryN5UG9WE5KVTPugnMpWh”Œ"1NBvFnXRSfVGDsS8V7PWR8DnC1ZHn4nPwa”Œ"13o5Uyb2HQz4x5Yn3YxRjWV1hq41K6Honu”Œ"16k8nYiVGPjWpVJyATcaAYRmWuUVytvkXR”Œ"1LfwS5Tv86wHMw1YJ8BJLcYA7FjhyqWCCm”Œ"3Eab4nDg6WJ5WR1uvWQirtMzWaA34RQk9s”Œ"3CPh9omf8CnCMgLbSgDzrWV7dH3WbDUyKH”Œ"14HA4EaeMXZCaAXRVXyqU29kkvsusshxHt”Œ"12iqNhSF8tTCaR4N9cmusXtxDDjKmqAqtV”Œ"12kyMfDyiRzQQ5seKmZEa41B4ETzA55NNf”Œ"1Pqb3mkP6vcY1PSSm4CKv7YP4VGsu8v9Pw”Œ"1Aa7v2LDtNYp2XEeKdg2AULGEFjDBGbNvD”Œ"1L8mbCBKrdwK5WZijUGa8KhiGNGtTEGjeC”Œ"16vx6YzyqAqxPCt1cxnPFYyNGAQhGMTXQK”Œ"18ujnaJBZdnv1f77xCR323GgGCqWEQxtuD”Œ"1Le7tz4CWrHDYQ3M4xnZzcYfvX2ijEphix”Œ"3LNBRmzphms6DPBDpdT2WDzU4YuiL3DkvM”Œ"15ehKyTk4vBgUU23T2Q1QqUKvDPnXT1KeZ”Œ"12t9fhZw33qirDA4xeC29tFsvrrNWfHzjB”Œ"19m5pnVpLKxEbmye4NAGLYgaihNBQha88o”Œ"1Nn6FLwoHJtCx1ubT1injHU4XgV73L6pun”Œ"3P39oAKVpM3xXUpryqmhesDjddYbA3v9NZ”Œ"1MDgmvJ1e7g4dx5mWbjLDNBUUErJCskkt4”Œ"1AMKUV1mUwNRrAty6SjNULntNpr6NRq4RG”Œ"3JPA4y3NmJUj5NvaazoQZ4DBZCJxTL669K”Œ"13X1hekHXy4gALisaoSAF7CXihtEXZKShZ”Œ"185itfWCyAQF3tPGL13dRo9GT6hXWHjPUK”Œ"3DpBDmbvdfFx22dRWDF9gzi8PvvWXW9Y4n”Œ"17HkofDzpvHb52Wotk5a559uvNryVaBVBW”Œ"17Adp5sVTsSYsz2B2DSPh3Ur1QJm6ULzGL”Œ"19UyNUh3vCYBX26d5u61erhfPqrxssTLnm”Œ"3KSKQz471QuPxm3jBePP6fE7TUaWUzWo64”Œ"1EtVf2nvVX8XdX7ewWgRzaaAdGbiRko1VQ”Œ"18n4VUVQkzvTaGNiJ6zS4LvVNgohkW2Nnm”Œ"1GtKnGiH29ScASmLpkjRwDkVjMhXpYsy26”Œ"19cEpusbkcfxfiGfRdGLZB9hR8WCbXYpQ9”Œ"19ywE7E3SYnYfZUaFKcbADeammLtZjtKsV”Œ"1HuVhLErr9M7ybAZNebpq4L1qX5vsFDuF3”Œ"1HMDsr1Rk8ewCGy5cxSwPczEwme9Xey3zE”Œ"1L1DLbFTTvS2xNdcYJyG1kJh5mu2Cc32Ld”Œ"17Cd1MFtdoPyrg5HBcJrg1SMaX8QA8KeXR”Œ"3DUFGeoVdHer7h3XRg6H5otHMRBtzGo2NN”Œ"185ANDEzSwHSDNGZqHg35qUr1eDVtbjzzR”Œ"1FPAwya6cGtj32qN15ZnbJhu65Rz5NcUHf”Œ"1NtHN8Tx7MSGZ3XNx5iyNSRqsmQVnb3Ab6”Œ"158GpuZ7oyA9N8iLsMnBZu41fZ1nxL14w5”Œ"14VKkXs4K4Vj3GTKomxkhe9PBsF3WEd3xe”Œ"1CH5qnScxmSnnejaVsnQmBB59s8TzLxqiU”Œ"12rSFwLToKP5fAn26sPgFUBSJDJH9anqB3”Œ"19Pq7h18WFCSZ7TL4JNUvsSXFps44gG6w7”Œ"1FW6RiKqR3khm38fZcEuR78qNeia1b5X1M”Œ"15yb8kNCYJJTMNFTaNFu86mti1Nm8Y6LNe”Œ"1FN4m8uCXckzdhXJxJ4B3emiGpPG62oJna”Œ"1DvrCRfgHiYnz8GDnLgdNnxizSrMBunX55”Œ"1NogKEUjxTwPRTrtBfpqCUPKgRaFTn4szn”Œ"3QkwZhZE4FJhzAK4UAm9Vk75yTsjQYcfWV”Œ"1EpWkDaPuY6W6rT4fpLALfgLUwzM6YH4mt”Œ"1MTNQHrd9rTDGxjeqjGvs1wdh3jt1wteRF”Œ"1BbvZ9Xa3DxTx1aSPzRVqtX6zw7d74Jkpr”Œ!1QVinwG9L2zxs2CHaLnhUktLvNDYzKVLt”Œ"1MT8ngYR7adRzrpbyvqQkNeHrnzh2YXD7c”Œ"15v8VYhXztyginZQSNoyBH5PFpSxVugNPy”Œ"392CXDuMUR188oUNrQnHBNrGzqdt5hxNdE”Œ"353v5npYiQ59ZoM12oNYMY7z4MEdD2xzPa”Œ"1Hc5kgz8ULR1zj68JGGKBaGsxFssLWPYmF”Œ"1MB8QZCGbYShCYLpszDoW493dajUPEsZQE”Œ"18CWPUytEEwG9xPMpMw7j1Bi2m3yBTuV5u”Œ!1LckcbBejEmoTb7iB3QNqzA26KYLjTuSS”Œ"19rAUXjxPifeuUD7vhgmbFx6GY2PYPDjYq”Œ"12hze2Rq98H4FStASnXPWTu9eCo6i76KCd”Œ"1FoivL7PqqabXWSLJbzELexd9zgbT8cu9z”Œ"1FASwAcTw1LLxq9jWrsV4eo2R2erLTrrFX”Œ"1FUL64qmVZKqfowshAmDPQeFrGFypmZ5R4”Œ"1JCXPhUWDdMDTrcDmWRpSFZbyiP8oAuNd4”Œ"1HpupvhiCBdtjDrt2qkAdJzS2p5xJVmZBW”Œ"14db3sipJtksjdbmEtYckvWFeHkQtKJQCB”Œ"13aMsDWBGbrAvpHELkHPW7r6crZggRw4mG”Œ"19eosQ8Qu4NNjrZDUD2PGabtHcEm3s3CYm”Œ"19bAsnS6vs4cDWJwo4i827yi2zGrWspQyd”Œ!11kptNa9C9wvMJRxVmep2ZkjFyRDbKzMZ”Œ"1PnEsVMQZqQgkW6Z6m7eDxqgUC87rZGHM3”Œ"1BNzmvyJKG45SBPLXDt6k3JMoDjmXiwL7p”Œ"1MkGwCzZXApm8PMoCqkEfhctaKajkmaRey”Œ"1GnixQuJznG1znTqafHjq6htTnp25uF57w”Œ"1722KDxuTvAZhfdcsJD7urddgXoeQBbGo4”Œ"1AUk797d7FXgZfLwiVNkrkSQ52HAX1yQbR”Œ!1prvxaTRteq3FXFtVavr8fZ7zFthTzKZU”Œ"16Gm2ZhqBzVuL86dsPQi9xPJv92qY4qrvP”Œ"1PXdAHLS5mep4wQwQuHiaVSnpVHnaNL2Jz”Œ"1BjmGptnWQ8ei3naoKPCFwZEqJfdTWH8HZ”Œ"1CLLgBvcgjYgSAMvFaQekWCwu9GAK5nXi1”Œ"1B9f6YjcNHxcyyP58gBJ7uFKpjvAnyBsor”Œ"1Q2LUe2ndhxy1trzPuWKtAXZsXkt8SE37u”Œ"37LqnoLC2Zf6XyZygEokWELoz9vLMC9rb7”Œ"38sP1e3jUBHCECxDBDmRsx6UyZsRyWwxz5”Œ"3NRGgB1J3dJ7sZhE6H2Gd4wBK6W3RUaXKm”Œ"3NhGk2q7VJuBm8XctCKZk2CZfosEeojJgq”Œ"1Daowk4wH3C1ZrN2nW6zzAzRTVY7BTNm8r”Œ"3DEpi2MkzcqRkYN9jiP7f1qioRqaudKAv2”Œ"1P1VHk9cApTApNJ1TfzGGaP9iT2TSDBFoH”Œ"1FTKcQaRiTAafcz4piqB1xckcFKz99JEHs”Œ"1BDLzVd6qqA5zmqhTWj7vSKhpGt7Md87hn”Œ"1KkfrA64PXARUSx89AYYrnwxRwzVfw5tNh”Œ"3HdNqNZstcrT1BvmSgDx51pABXMmzidMfM”Œ"18hQY1BRVk7KYAWF8AiuXLJrtkgrcgWdmW”Œ"1Mej4fp6vH2yLdgpMEnJPKgSS85Ub6iKSg”Œ"1GpEk6QjKTQ7mwkPAphwsARkAEfjA9guAH”Œ"1G5MCF5PpkrRnE39cuZDqMCrx6z8pv4nhf”Œ"18wayPeSUWRmhhibu2QY44kJemtphXYQyh”Œ"1Ci9tXwXdkFjEbEGkVgBtrxmwe8nMDdvuQ”Œ"3PR4V4M9Gqfn4NCskF1Lvwk8xP9mJggqya”Œ!1zcqL3nDpAx56Ddk1dedf5KMgewgDzUFy”Œ"1ERK69XDMDykWgEa88xnWSDqdCpGdQcgij”Œ"1A2Ua7hLzvb1ebLrv14NMjUcW42e7ihTL5”Œ"1L7p6TBo1B2bz2gL4QPY9bYtn97VjFnYNg”Œ"1Dk4QZADoSbxKKa75MqkLbnYEdtcyUSZi4”Œ"1BPAV8Q5dCABd1HybTkN61Zzv38ju37ibt”Œ"1K14TeLDMuQivqFmEJoHH14TfJQCbuwFx6”Œ"3Qde2TjLQMYM3z6d5Z3qCz8HNFrkxcfn7t”Œ"1BsEBJjfCiGCrzX6snUF6KdsH58Tjzx2ag”Œ"1MUY9Mfof2PuCrpZW1Hx7P1ibziSMPP5vD”Œ"1H16iXTY5iG9Ms3Z3WGKNv6c4X1E4W8iCQ”Œ"1MbipbLpZBvJcVkMJaVAAd2fGGcnGtPTTc”Œ"1Gj7pKEhg4NrMwQ87Erj6bFwFqz45QMSU6”Œ"1EummfUuXd9AJAmLweUuP6vV8kdsFi8qr3”Œ"1BCEEAdFrhpp7DaHmWD1qAbsispE8Y6DwB”Œ"1D2fxSLhHUfZC4nSR4uSsDuEy4wb1xH8no”Œ"18shphYZwPCFjhMveqpwDAeigJg3x7CUL1”Œ"18ecpBcQeGun3WB2hPkuBndn1phAAqQ4Xh”Œ"39QNJSgQg5JnBXAtbF8ezkDn72VqWdPZPJ”Œ"12qxVASo5fgs8FeCGs1kNekUTxNp3aQNW2”Œ"1LxADczfLnXXPsYJbcLnNeYwHEvFMBKAtX”Œ"16YWHHNGHisHJ54u6JKbJbe1jv6gLFG1vk”Œ"1471KxpBWUhUVCvGdUv2njxCMch2jRPNHh”Œ"15YztnmzVwr6GdxkqkCMPqrw9wR7faiK8D”Œ"1AcSptVkS8JL5KucwbtBgMW2y3gaFAVGih”Œ"34MQcAUBAFYd6zXL6zjHe3aQ9eC25u6fDu”Œ!1LVUUEfzxLWvDUJfw8brFhRqBJGT82UQf”Œ"3EW7yzcygZCEjqoUybbchSJw4tNceVc4xw”Œ"12PFFnUNQA94kHz12kjtovAJpgecQMoZwh”Œ"19vare6wBWKcPMdxXV1X4xpgcdk425mCqt”Œ"1EQKnzbmgyZe4rHF5cKtoBFaifQ5fWBGnh”Œ"32pDzQsZ9KyWahwh6qSLZd3hWuAytRPWhq”Œ"1CoKW15fdYCtVZnujzfURqAc5mnL3gYf9a”Œ"1DQmLyG8x4s7Fa4ozKe55CJrmY5HvLgVA3”Œ"1PnW1THTSU6PJTCDwoteM7JZ4MN66N6K4G”Œ"15oXFV69yETF4R26xafud8znnZuN3xf9gN”Œ"13hB6VxWn52YuzXJJ9FE6Y7NVnT5CtRjmD”Œ"1Jp6bcvRYsXitAKuyLqTRmBorFD4moPT3R”Œ"1NZDmUDvQR7ketApYWVk4VxCjraTecamM2”Œ"19EYVjyc4SgGM5Zprq1FJgCXThPe419W6D”Œ"14KxQdYNvLKV4XqWjwe8x3dnHh7gmhySac”Œ"1AYucGdNNeXd9Xzw2ttiCr7iyRffiUEkYe”Œ"344bDPUbCSeZUVDCgQFdoypYkQTxxtGsdd”Œ"1J25NB8PJmHtFbbyemcamrprDA98MjAHgX”Œ"1Kbcf9i7h9fgC2La3W1b5hZbC9KGT9S5DK”Œ"1Cpe8HhQCuMGQhXxk7iu6DrxNjedEg9qrA”Œ"1ESzZHzwWTNnvHb7s9rjemVYbgft6Ntp1a”Œ"1CT6v8SZxGgsWPKxd1LQz25vriypdPiEXV”Œ"1GwzYG2jRuYuWKsW5cFyitV4uCEC5ZVcfv”Œ"3DPnWdczhBypDXNrFvjmWiWu7HCuApZDo1”Œ!1Bt2BU7tTmBHTBeN9ULb1VtBzXiivKQZH”Œ"1Fq6VnScH2AkFMsXyD3mTk1uab6xJyEa6C”Œ"37fpC35szBzhzxFonMMoaBAaQ9Riu3bvZ9”Œ"1NYqXjHLU2qZzeqhiwTygynE6pdeUNAa7z”Œ"3FvZ6o5KLSRt3VkTpB4xne2yu5PnzrPSHX”Œ"3NGJjEZurWFeoC537CfRmpRtakYoisPf6r”Œ"1HEuLW5WQse2qErcwY52JrWUMJUKyYaGPi”Œ"147qjfbWTwPtxJthjMZM7scTUtkC1bYt53”Œ"1E97o62EJwhBWENE9qoCCMXhTXVuAJeQXX”Œ"1LgLEQMiuMH1Kh2qW1HTCQ3JLj1sjDZtfo”Œ"3MMWAaJoAVfatckp56yoLfPkY5K3iafMPv”Œ"1FT1hdMgCCGJu4JbwrCZy4DFAPucGfbs1C”Œ"3MckY1d3CQDEKpnW3oL6hyuaa6aAywS6de”Œ"1FQn2RE6pLcSGe9aCskJhrkPRp77QoYhBT”Œ"3PpznK3hUzyPnMg28zMyqFBRx1kf4GRz4D”Œ"3JwTtGG58u2YoSK1cUXPBKNtdDYyeuaG17”Œ"19m8Bu9zjisgsz9Y5P1vaG7Yokq2Hkyo1Y”Œ"1DustBryh5qacBgPiNDct9g6niiNAFgFPS”Œ"1HYxXFCFhFyqmUTiNg7xa1PMEAAqJDyB82”Œ"33JZyQhZgTvcZZkYUc6TZWsTEY5Jx4dQRq”Œ"19P5s83ygycai5TsGuJz8ehoD1fpizHNkw”Œ"16sqTLB3QGuhS1ZZssXZ1i2LLJaX2Ygiau”Œ"13LanGFCvzuZePgMBVfm2nraPtAfF7uiRB”Œ"1C6ppZ6B8u5SBF1JWf7aP8i24h19ixUi32”Œ!1czbispdv1UiQecYG8Vkof4WeJ1N76Xrq”Œ"1MK6R9WsJEjDvvLgpMn5ENKyefcKUkr4rQ”Œ"13HCRie2mMzmq232PPByaDS5t3QauVfEeR”Œ"13E4Ld7y4snnRgK1hqJx7ZGChxWzjDRwgW”Œ"1Fw63fsvjo1jKzocN5SuYfrYSEGT8jRVbS”Œ!1hWxP4qrjrEc6vajvtkrkVGRx3i4wUE8Y”Œ"3987YePVg6SSDSBFh8GnhartHYMMjBjhuE”Œ"1HKkMTZzNsNDiFYyeyuNABfq1TBMAwaPdV”Œ"3BsaSCn42GopN73q15XCSty6HbhbC9G83J”Œ"3HA9iE7oUyMb7J7cMyf1WJyB8VDa8zzRWQ”Œ!1srGK2n95UWsdDfPm7jmNSmZ2tDLvGRNt”Œ"17trQXYAZBNTvuCXFMdJXvcfJVF35h8P9G”Œ"3H2de3Sw4u48TaVrdotgLYb1tkSSzK4YSK”Œ"18YzXoqpbhnRQfvvBRhyS5wiP9p8P2DfUm”Œ"14yZzv3bRviSyZzQPEewXj8DzL4efLFVMq”Œ"11346znnyuwMCBG66UTCM6cHSmQYjgqXGt”Œ"12DiU9yUmEicuust1Dwm4GyghvoTTLpHbk”Œ"1P7s3qVhkFC8uydWLXZJxsC41AihuN29Gy”Œ"14XuF5SiBg6U3ieKSWqxirDyhoMk8xMiMe”Œ"1FRyWDc7f6WUgnTc1gka58PztuTfuc3FC4”Œ"1GKpvENyUxHiFeUUiXCsxLPX8hh2ZVKXVw”Œ"1MYKvhnkwgCttK52HVzaTCbJv9u64mTuFP”Œ!1zPeqFY5ZZ96m8zM4xSvhzwAhhvr9pZQy”Œ"13R13vTa4fUroC7u2APxZGfFoNWr2BryZq”Œ"128yv1d8gRXFL4QzwUkB2nnPp2sUGD5ky9”Œ"1ProDHiN1TTjikmgL6hyNGhoFnRS77WqnG”Œ"39PWf7kRMD77SKMj4xpLVbvoYfo22gLsAu”Œ"1H3UuyxNjas8wcQFXbvE8QqCu33UVY84Zw”Œ"12gBYWDG31o3EasHjnLdnwUP7wC5FMvnkY”Œ"1F5eqmUCrxtjbhBYBwqr1F9bZdXp1PDhCp”Œ"1PgfXnL1nJSjrmYKhp8Z7UKouuL8WujFgX”Œ"36ykJjn5hamKQWjcbKVtApSjrsRBZBBL2A”Œ!1vspg9EijWMc9m164zG9ARP6eHor7sEME”Œ"3JuNxoHUZEUY62v3ZwPcJbrAhEucgccGbw”Œ"15fdEUi4cfU8yMsvAP7Hto49xU98w7wS6q”Œ"3G53TnBXFnKCeBwzdvcgVs9duHdcdDiGbZ”Œ"1LGV4hD4q5pTSUkqhKPERgg3X1kLUij3tp”Œ"15JbRT4Z7NsbMPmmHh1tuV4QunEpH2tRhX”Œ"1HQehvpjtkoXKyg1wfgJuVkHMeQY4ZrbJy”Œ"172hSb9QNrPSh85gkyUgqxDR6znYd5x6py”Œ"3JiGNYDP36oTFCpby6a64UE313dneRDJu7”Œ"17BvbxvM2k5UTbY3funD1aStJhB4guCutn”Œ"3BgsfUJPU7L4BRsSpy1F6BwUHj3dGKNHzT”Œ"1LHajGLE2wpiceKVkZHh78uXa24EUqNqTF”Œ"1J7GNThWhVSVTq8ovRMDgoZ4fZVjZ3VZCe”Œ"1K8H1o1FryhAayik3TVuKa5fygQ1eNoqcs”Œ"1L8PaVUg5YH3pE2G34jS3DRbXzYfzNpGgP”Œ"169yUqgz5WbsL1ATvuNWopUUZeUqGjYZ1T”Œ"13VjhvSB428WTsJXYLQic26ytRgKrbZrJz”Œ"1KfubZEiUQaMgkXAMLi6PJAXcwdrV2hgqV”Œ"1PjY7riD2SiRHNdUCCXSsHvGgmoXr1gawq”Œ"14WhUR4MyfzzB3YHuUZiV9rWdhWCbt1PR1”Œ"16PJKLKbUw89XhUC2JER4VezYNjHPwFX3w”Œ"1NPKJ8yCyBbc7GQ2GuNkF8dMc6jC3efTS2”Œ"1LHs4TJr1LTAcmTFrWSnMCjJEmtJFope4T”Œ"1DyCnY4FPhDG7PbVA4JnipYPwFcLABEemu”Œ"1B7ywwU5G2F6MSqVm2YPTkpP9Dat2napyf”Œ"1DDxEwASoHqwwKZiGkgtGmGMN4b7ep1fwM”Œ"14geGAhg8WTpG1ikDFJJaB58LnDYZ7icGr”Œ"1N4tDCZeTYFgRhg3GfWiPHaMH3Q7UcTCr5”Œ"386wiLUBf73eAhCVReM94ueyzHgGm2dkk2”Œ"1FxoddF3RAqznL2CBnwk3ybAu3qRVNuTKS”Œ"35asaUFNvtsERCv1dFA7BSPvgQQaRwLWgz”Œ"1FtwpVmwMv11YnkVXCLJHV286SMsDXFvpC”Œ"33Xs8gDvFZBkPZRvV3H9jEBvo7jN4vaMqm”Œ"19WRSrziEqwUnZjUnChkxgx8ZQUagDx9pY”Œ"19YDVjUNCPdSRaeFkUa5rGpnWeZQAZWJ4s”Œ"1ARkPCFhW11UNnKe2Fn59J2MjfPYrCSMTK”Œ"18djTRWHoyK8M5BogEiC6gnCq9uF8RBKdG”Œ"1EMZ9qp2cp9JHznantcQ8FRBX3um24Ceqr”Œ"1JnY5H3gY44e5skoZbcoWEHeZ1hwJKiCsr”Œ"3PLaVccnzXJ1cJj61zjidQ6EYxgzgQirVv”Œ"3Fsw8FtHNUpN3Y2dWifXEUbg77zi8FGDxZ”Œ"39Q899y9EMXW1M27sHZJf3Equo4E4r12ut”Œ"1H62JrXmAJJStws2vhjLRRjgGDK7F2z1gr”Œ"1FCu54C8czFNKK9up261w26MkQuzUnDHTs”Œ!1mrBJfpC3hExkkajHwYxgp4bpNU3DCn6T”Œ"3EEwPZZ6pYRJSotCz9RBoVYPRnoWyGWEka”Œ"1GbWZ46g8cp649dAEsujpcMCHPXu3ayNeo”Œ"3KYDzvTpixsQYMHSf8e9xY1rBKqGqYvSbm”Œ"1AorUwCeqQrXfveXmK1ZfJfAtPmmVJBXa2”Œ"147VBXSZMKTbbAfu6F2VguNM9SdV9YXe2K”Œ"13ZW6NoRWgqLgJvVwjLPKFhFeUvTmWuE6L”Œ"16oLN8zxgUfgxCDoZtqLYYfGDt8rXXVNP7”Œ"12AE4kUXkSitKansYKoEJbgTVQjARNfWhL”Œ"1C5zpDbYh3NsNyXZ1AsS981fbQeioArCs2”Œ"15UMC9VpWvbpMQ8KGuypzQ6Ruif9bUuchD”Œ"181RCKEXuSUnGBNXA3G8uawvMaDyNGvUQ8”Œ"19SEiRHX9ErNSfV9muRtqt4fMHz4YXUnbi”Œ"161Yetj16yh8fxpFhqdV9sk1ebgZZNydDc”Œ"1MBsBbtaDGh6cgdxhhaFmTZSZ4EYP21CBR”Œ"12VzuD55k73ZaKhXi2zhKh9dLziUjb2VYW”Œ"1B5EWtSEefpq8G1r1sJquhX6ZLbnmLfRTn”Œ"3G1nNTTyrpZBwyEiG6U3bJvZdu9sW3rbUz”Œ"1PC98gwdgriRiZabjYuoNMm9PHT4rjHmcf”Œ"3FbXoqkbUjoMVxa8WmowD1ewfEXwt6wE6D”Œ"3LbSftatVw1WL3h9LPEBYD39DWXcazeyXY”Œ"33atGXLGL6B9upaWsgs7xgx5622VDBKsnB”Œ"3LZcCFSRjV7Nhqj2EFeAWpV62eVf31qKGp”Œ"3ETcKuAgCzD6KgALhVmrrNGAhLwpEEfbJ3”Œ"1CLLxjDrJTZQAseYyaAm4Tkdfqd2aSJhTY”Œ"1N8MrDbkDYDm96nLnc8cCcaUemSQ6YwK9p”Œ!1V4yV4wbbWd5ZGpPWm9dvcrEbs5ia6cn6”Œ"1KdFxZwWL2CCJ89J3fTUtmsynY3wEBqAb5”Œ"3A4U175prUGEn3B1gUDkz32u8fnF9Nx3Ly”Œ"1DR6bbH6bMS9rBTJbgQ9y6meszaYhZrZRe”Œ"1NFdMBQzcXbj5HrBYYx29JyqWCMRRsE4x6”Œ"1M3t4w8DmDBDv2YfVY7fVkapySjeP45boh”Œ"3Q8Q6QznyidVKQCs9LXb8XgqTJZSHAxRA3”Œ"13oWHLKafSjUTVC5cJ2PD3bqDXTq7Cd5R1”Œ"1NhjEWSZHAh5JBAcLsPH5V7Xg1TeVgkrLe”Œ"1D5JDpq5vP1PM5WwjZcoFeUGGvfv9hjsnV”Œ"1LZjLSo8ixjHzP5MR5XoHPSZrCp2wZaHbH”Œ"1NN2qKhsWwQ8H4d5rd1xwuaSdMM5eEsGJV”Œ"3HsMG3eD8AtFCY3J7TNu6v17fV2xUv87Tp”Œ"1B1rBdY3RnP6Xk8pHRCsCF2cJtZUjY2isf”Œ"1B7sHwkdkFM6Kq5vwSFfaCKKmocu5wJB9R”Œ"18Hr4GCDq9WH9MMn1bbajoVB9DPAUARKhp”Œ"13y1HpZ4nJrFPoorpKn33M6WrQG4fDtiWQ”Œ"18UHeNE7hWytaidyF25BR1uqW43bcHNa69”Œ"1Mjx3hBkkVscf5bk7Ckp6JpX85tcNoxuB6”Œ"3BMW1RCBJEKf9DeMEnv647iajGfJq16TFq”Œ"1N1zu1S43yrYHoj16o8u1YkQWYfhpnF9u8”Œ"1LDYzxbidPSFw4An7LqLPXUZ1SMhkVpopW”Œ"1Jz53Sm7hNPQGUQEQVBU1bEVWVhmtFEKhA”Œ"1CcQ5wiYiUG9uPkabC6urERuWzMUowYJnv”Œ"1Pf5KNGFZWVmSoDaKBLvg4SDbCu8XfuxQb”Œ"14t76pmBmBDk4eKacBRKYcM3hV7amMia22”Œ"12hCMwtuX8zzeMYxyGVgMyr2erEbi2Pchz”Œ"1A58NXjeMRvq5mezFPdLGKGXtwsgEXinKx”Œ"1LkY2wpvArdCckDvF5uTJx9Bpr9rX3Egod”Œ"1Nexeb9ZBNjbrsayHxvQKdvVPbzMc2oAZs”Œ"16EKFFn2HCSxvXtrmjbJeWixvJYwUeS2kw”Œ"19VWUEwUtWjLipzRgqggQsdFDocBQd3Z9D”Œ"1JnnwSfJDyMikpYmW2uEWMbUwEr1FaNqAj”Œ"16g9SM8EKj5i2HBhRVJnYEu3NVu149vxP1”Œ"1L2HbUvEL45asovjj7oZz7amGkQttgdGvd”Œ"1ME5db8FumZMx73GFcPH7rvyVubvnbNVfS”Œ"1CcvPsNSFa5jTtBquNbB4tyyQrZZwGc1aL”Œ"33emc4imuP6e91p7GEuN6EUbEH5snxAXs2”Œ"1CDsH8pEeMupuZvuBcNDYRDfG2UA44THCF”Œ"19z4iBKiRpmkydgMQLc2vjcwH8RLRZz9rd”Œ"14DC9VE2e3khCjXRhwYcDeUUdYJarfTEev”Œ"1D3cGAWXGQbCU2kmKbdJgbFWpG6aaQtNYK”Œ"1J6ZL35QxDR9C4grK3Y8QvYGDHaq9PeqDT”Œ"1Lsi1ECbvw1WfuVMsR6YfEFug4H6ywp2mL”Œ"3F5bnEa9GY4mEhvHMeVdXgSCeAcTzG12t3”Œ"1L5eEH2Ypq46r1FGohgM9oEzJbNx265TS1”Œ"13BiaQ3jXtwqEk53HPnu4wEK9VmCXeUHDp”Œ"3PcjYHiNwJAW2xEnzg64MXNHywt27Zf4e2”Œ"16Vo4cqAMDKXWoQ8ZerL2wZqXqwU7HAXfV”Œ"1FB1cA5V2SjJN9G8zFZSdpQxET3QmTd85Q”Œ"1Btc2UrZTqcbjLrr8trx66K9TozQmKeDGN”Œ"1KmDqxda7pv78YbtsfjzmXBYhUuYp1BF2S”Œ"3FiZ4Phy33NNFwJRB7279TrxZdUfBM48cS”Œ"1LZzxRww1j23y6xkbWhQcuP4ZjcuTGZMmZ”Œ"1DpWhAnLBiJpfg23zAVKuf2ALUfUETErhv”Œ"1ARxhmCCJH6P7BNTjw6irJoF9hpW1LQvBQ”Œ"3HFL5VcwfVVLxh1X6LYY9Lw2XS4QRqJuWF”Œ"1M7bu8tikhatc9RkdeVPYdswTcWgnYumHK”Œ"1QBSxTiAUWUnWiHJU7BWtszV5bUFE7355e”Œ"38yQrpeVFrutSbHzMB7vmnyqrtFGXGCyk4”Œ"1GEQUSPsZpT6tyAQ8RZV4KYr1KRFtp3qyd”Œ"3PHmByFj5DriUkTfwW8EvgrZdEc92cSi8D”Œ"13Vp8xBWiiySpbopXaZBekcMNijqEnRwjf”Œ"1DEYBNB9bAbrmnDAYCrRzDCw377H9fkhVv”Œ"1FkU4oi5RGTnvMdHyZ1nQCFifbsFEjAKRf”Œ"1GfUaR6yBunodgzaE8h4oQU6HKbLAivGrP”Œ"1KtQJ8cssd5u1zi8sSxQZLHZwrSVdgxPTo”Œ"32m6bMdg5VwGvqrzjZqcvvoHoBaSJfo3Zx”Œ"37PsL53StiXCXZMvFMmuGF3oJprPR4d5WA”Œ"3Hk1sCT9B2npHiwyLxatFtmYtVcJVYL1Vi”Œ"1LVuzzHQHsivJqvHoqbeL2EodMiZUNBcxD”Œ"3PfKvMdap5gtWZHKyUXxp2T8x9F3m1JAKz”Œ"149GzCQJhmbFjWiC5YVz2dXZxLa4hxk5df”Œ"167bvjdiiwB2YZEBhXyCpNkE3ohVzbfXux”Œ"12THYnfyteFpTsvXyrX5NvbDYD9gzfEhqR”Œ"13zccjzRVqLMovcor4CvSVVxzRUj4KZKGJ”Œ"1J4hgPJShai7XuxWRdPajCW8hQrpRiCTTS”Œ"1Jzbga3vZ92nzriSksUYDfWqjHQXhdoV6j”Œ"3CUM4XG6oQF1KHziMGRHiYUQ58GT8frsrd”Œ"12obYLfTTbNZBRWok5LwsihMSLmxzJK5AY”Œ"19jx6yMKMfoTxX3pY1qsDwNmCKs6BjiH2E”Œ"3MEzUqqxUsRpjMWsMqdnE3XtsoaGmSQqW9”Œ"39RwB8D6fg8mA1m7VGAGobKRtZM1vHV99F”Œ"19NZFTp5p8GC8cAs54SLd8vXUaFKcxhBZJ”Œ"1MunBvANTw3PAfGtGLDsFwc7ToJdHHYqok”Œ"3ChedKXRseKyPwufAWWL85MDXtF54BxAY8”Œ"36ZFvPWTsYsuo8NQTNxYPMjutjQVY9KnyL”Œ"1K8rjxR5koU1Gknkn49RSPHWxxXsf4Tzvt”Œ"1LcjSDnCJ2cGeLWYN44tfkPfHVfjWKD5ed”Œ"186LVyMgMjvb2YomNiiF4Tq61pmjBtBnjx”Œ"1Cnehq75kacXKM1msjWMKpiz14hVcUWkin”Œ"1BBPggmstq5nZ4cGpjqEy6SvFaxxZs3G4G”Œ"1KfsX1iUB79kQzMDNc24M38Dc5jx7joJGm”Œ"16X3bw16FZd434s9NjYyTbXXDKnVD3Yxnq”Œ"1Lfb2Dbo4ZkMJ8yuW8jVEX288K1qPdcUWJ”Œ"39VTbyjWyWzQaaDc2otQWfPLP7jww2JbYp”Œ"1E8ggUw2reXiBdXADzkKxiJUeYrkvo2Jz5”Œ"156b9HDc4V6NQUpe3UHFezG7qLLFQbC3SU”Œ"13MmGb2QD7rEeQtMGtNzTuuyM86HSGt2W1”Œ"19NSQT8p35RPW5rnZJvhtsRqN8GVpn3YQt”Œ"1Ahxu6KJoky2tdWjdbDPHydHzTcQLErs3E”Œ"1Ak6ceGf1NsfQRZx9GwfyuzjtupkjGGeMk”Œ"17mDajXn1F7xxxzjaERSdq1W2zGqxqtg2q”Œ"1Jvep2XR1GK4jZBGTC7eAPVLEZWT8LdBdd”Œ"1Bec6efLjLH3Am4ao2KJrv3K75ud71CjAP”Œ"3HXfjGFJc8t2niDot61coaPS7p7e9H6HU1”Œ"35KA5jSZojRxPwKzDaiyaMn9cBUhEiCxJM”Œ"1JrnsrLuR1HavUpMqf2RRh54mGCfnXQJ7k”Œ"3BRmDRKydW49qFagEDTT72uKwDiaHovytj”Œ!1BNbVUU9S1MiSG4iahNXiMKh3RwuW7oFA”Œ"1Dm7vByva7chYtHFGJmzLwDg1zLxwu3WRk”Œ"1L7KsvThH768oMrFbQqMtfUdXyXWMYsGXn”Œ"36pQjvNMRiufycBCbyGsFZDRey48Xpqcgu”Œ"15Z5kYVtoJUhxwxUGXdd5w3PYjiP6rem7w”Œ"3CvimS2pL9dpvLCL6dYKo1Snt4GN1PUUBD”Œ"14ZuRxH1VAnoJZLNmtgmWc4tM1oJo8HThb”Œ"153xLKeLnbqekzW72JYV1tZkusovVQnwQR”Œ"3MEmq6BUzrX6woyWEe4QLi4HvNt2RPTnM6”Œ"1GrAxNQC3TBUhpso4r4AUCs1T4VTGUF5HQ”Œ"17qefcRxQnsz11HMsqYWpmtm376V2FvLMM”Œ"1CyLZGyYrNUEJBv7KY5gWdU3NphsZzCxax”Œ"38TWs42uQPHsL47sv7yShYtPKSYPcHxqyc”Œ"1DxNuw6RVEani7fbJknG1CdHFJBLeBf5iG”Œ"1KUcT4w3CcPf94ECm5NzNZQdecQh9EWaMC”Œ"1PumCqiPAujwDacU7enFDcSHCVLzdbLyso”Œ"3PFwwHbhA9AYgHN4QEsWWyGMH378BPmvFV”Œ"37wpWhbVmzZSbd6wfzQxfxveJLuSg9Uqi7”Œ!1VYoCpTdve6MbKCXui21signEc3q8qPkt”Œ"1FxjrJN8qG1zfHt7s9QgLuivvbft3tj5mK”Œ"17RTmEg1cyDdwcr5KeDd9MebUabjdCexu7”Œ"1NimHi4rUiDQ9FnarEzwwYf35oo7J5SRDp”Œ"1JHx983nkXEYLhd4Zjpa3J5Ny1URVsN8nD”Œ"16AJLNC2fv69RgjJR1N47Z2wDKvJU8L7ow”Œ"1NL7rmYz82mXbyXvcBFZa9Msepm7VEgW9f”Œ"17RLUa9HS2rjV8gXMP2s2vuWSRAP96Y5AX”Œ"14ddoeFjxZY2g9LFdZmLei8aQs4zvYrg2q”Œ"1QDCUTTXPYVMAaKsbtrdcv4zMZ7qrXQN3u”Œ"34X5jesxFhadEWHarCPAjz8qFEH2FMiwDj”Œ"32jPMzNzHKE2f6u6V1Ch9qxXXf456D83F5”Œ"15fj2p9SZzR1nXaX9LwV2pKD3CwdPHkdkc”Œ"3De47DtudoCDuw9q1sd9tD56ArABngr45q”Œ"1EoN2JWGcL77QVaefmZqNsX9VLigRZEb3m”Œ"17uCJzjWMsqwZUQ51XL62AqYE9PCjPmh2m”Œ"1A1TC45cpFhpo5T4zjQGvXJQcovV73k3EC”Œ"1945ZCWbREYkJij7hWkZFjdKRnZFEfwpYE”Œ"1CBrPX5kVByrvo1X7ERYzj7LaPPRJApHV2”Œ!1Ma6AJnN95vcpnyxGY22Y3t7RhhRWBU2G”Œ"35CyWbY5d8NxjeZpTabpjkW5e7ekG9YZXz”Œ"1KBK28cJTWtHQoxpj5ZMFNr6x2GUAdNaKM”Œ"17LpijpcRc24dCuJv3EZjNFQfgzZKFUYQg”Œ"1NPdP3EjwVonUWF258RGGGuzynCjVbc45p”Œ"13UHYfhcnH2nVvDtcF3QZ7QHWFVhvqfa3q”Œ"1Jr8YjyZQN6Gst6HDhktm5V116S1gPkcbV”Œ"19NjzvfoJxmjPiqXE6NWT8Sqtdg62SXjWD”Œ"151cGwpn4BCsKvBUfZLXfXPiQy34uyfZRW”Œ"1FPSyFaokDdqHzy8gHEsV1t125tLD7fL4D”Œ"1HJihUcaKna85YLXGdPhFrNpxyXtZ3DAE8”Œ"1JKPq7hHhYbzNbfaC7CWbYRo5jp7WSciEQ”Œ"17peB7gC49ddytJWTNV7q7mPJhshh9xPnp”Œ"3BipRRTMfwVV4x7xsQfWC2JyZfUv3BPeU1”Œ!1NoJuZiv8dkzJo8Tn5SYHUzYQpnJzR42L”Œ"16EWHcfZ2Mg7MHDRb85A7MRrkMvB8uwhWF”Œ"18ACaeWEdGWY9ksGehMv74JUwVjCyvV9b2”Œ"1GnhHqkghFpssX6uqUYGYcPENfEWaHqFTS”Œ"14vDcojFanpropmwmurg6hLtyDHMn7q5J7”Œ"142NW3Vnf6WeTxhSgye5cegCYZUnePsfHi”Œ!1SkQ9emmGTL4dgzZ7VTnK65akpH6kH9wh”Œ"1H9z3Cm7eY7sWPKdeECJDa7f48ba4eNvJx”Œ"16MW8Ka3uVcdsemnx89TsZ5cP5fKED6PsB”Œ"1BMFvs1Yn5cuQEBQskXJr3CW1uEZnEuqCc”Œ"1F6sN8EFpYkjvYXR81mKTvzZJ7F6LciPBt”Œ"1AruEkiZjomSPQki7afgrrd8eoDabZM1B1”Œ"1DfC2g585yiWQCdFZTBNJdcpDX9CBuKuaV”Œ"1APwYUyN4PzNQa7fsvVUwDTQgXx2WYRjkv”Œ"1PCnLPj7BGHRpQMWUUwLQkoPewnc68RTHF”Œ"3QxcX9PX4zqs34TwyqQWhPUA7V7zKwuh7n”Œ"13YGJFBpdZqp1QYn1wmhBJbyzsek2Ym5cf”Œ"1P9gbMmHXLRbwW6ZUhq2qVnpxvMkktU2wd”Œ"1KPCStyHDZsv4VXkgmdWWWhrXw3jTcFLn4”Œ"1EScUTMV1nSMB4937rngVGb9B8dTy2zQL1”Œ"13gKXyUBw3RYZdDNHUk47tMcAqGtzXP2xY”Œ"1FiCfmFdd1fnDipC1gRkR8LX7HsFWFHgkX”Œ"14eybhhne5R1q5EmoX9wgfwmXmGLhVMERK”Œ"12r6HWVBP37o7c1gC9kEQZ2aySL3WLSGzz”Œ"1EoywkuQXQL8CqJtTv81j4osZPai777JLe”Œ"1CdMsu59kZbEP3KwptZHe8KwECs6qWkNGo”Œ"12HnUoQ3j6Ff1qrsFJfSpdvgGZKBEvGjg4”Œ"1BQFfCuDm27z5APBUCAHQQKNTqZETGKJpH”Œ"1NJhiQTLRDJFtkzRCBKKaFZvLBFRY4kQJd”Œ"1Mdb4PwRVHeuJHyoSwNUywT73wCAeGzuFz”Œ"1MsU8x7P4k45omCsDzqM8ThqBRZ61Yoz95”Œ"3KdNh3JcRSrHcgpyxX1QbVBoPLgNT3CD5s”Œ"19LJHew2XvmutW9L7fSByhfRTLyMPuPns8”Œ"1GYCcpYVkNxDtrYLPf97dSVG1imX2FUcYX”Œ"1P9pxpQ1JtboowhNoGDexhAWdZYar6CFcd”Œ!1ypBvwBuGWaaKGJBfwKsmW7MUGDa4t7Yp”Œ"3NJYMwKTymwpP6Qie73mNxNBn1e5FHn3ok”Œ"15NuSoDUFz8qSy61RQri5Np9moryGWSeep”Œ"14eit1sVGUv3BPoXBMv4TpN6jB2i2pLkTR”Œ"148efrQndKLF3WuWto386icuj1FwqGVEYH”Œ"1JQBM7AUphGRfSNbgTmHc4RoyeYKrLhdZ2”Œ"1JTipWB1qABLWWLrT75pyxdM38fcPPxQWd”Œ"37PMVmY2NzTK3zVQLwVXsyfV4QPEJWYejG”Œ"1PZjSfFxE1AbtE1zis9XrvLiS1Z3SWAPPZ”Œ"13qMH435CSFbKUoNjSqY8uJGYhHkwhDBs6”Œ"3DJFzvwMnHTd8xSrNtRw6KhjtVQH5SShZU”Œ"19UssQyXpSAdaw22wTTXBgLC4aBNMovuA9”Œ"1N9nTfaqSNH2N449Lu12VguHbhbLBn4pmL”Œ"1688qpyj4HPBAh9Tf6nQLcgcUdH9HfJneM”Œ"1EMrTFQXskFYtWaqoycMoJQtt2WVo61k1o”Œ"14ZBeqpL9bWr5Fh4y7fasYoWBRAaKtX2VP”Œ!1rR625mJKvSWSHiiMamKTtmhNU2xAkoLq”Œ"343J3AM5TSoNccY8AxdVTgfvQy9N12Lcoh”Œ"1Pi9KhPJoPwdewcapzwWLfT44WpFnhJF2S”Œ"3FUjg1SpWQvqnNpM5qVXAvU4pyuHb4DNUq”Œ"1FnYj2oSkFsAamyWEkJA4Sv3wkGprYd9yq”Œ"37ovSuMbVpPGptjtCLmzygVEcTauwbNVaB”Œ"37wevbpjJzL7iwC4EFDUeR9k5K3r2rzs1s”Œ"186BW8Cw7MDCqMtKUANYUyBxjzzMqv7EN6”Œ"14ghY7sMsc6UEnLPkaQWbAExnkf6VKooEz”Œ"1E5iv4jN1Yb4CSEtu4YkGYeHRFBzuKeYgC”Œ"15Y3xm4DSm25Sa9k3Sjk54QAFZXC6AXoVS”Œ"1FaJT8wg7f7hyaVLDChGTjqSMJm7iX35Yv”Œ"3GfspTC9hPYT5V9QtHsMR1EUfqesq6iFPA”Œ"1BWwuf38N1kQ9tfWH4y9PoZVYvWD1pwo8D”Œ"3Hu7F9h8n6Bc9txKHJz8iR9CZaoeqEHkcq”Œ"1ELYM3uEN2LMDTf3eADLxYJH9VZK61Fkgb”Œ"1MkMDaDzQDsnexg5WFM8DT41kk5ESXFuAY”Œ"13WZcjfpF8Dga6tDTjXVpF8g8fkMu6NZC8”Œ"1DARb3jXqTVNs9tsZ6x3U57Z7RpgoYc3tk”Œ"3NbrqKWxrm7CuXtbWPzELz4mxH4jJCVYgd”Œ"1MUyZc9GEzdJLmwjGo2bXLYv6Wm2XiRpRK”Œ"19Twx3hSmQXhpDRFTiHbNpTWdYmrUjEPtA”Œ"1EJTJoop5vRo3Fi74wyWc7cWxPw7XfHsoy”Œ"1AE8GVSwj1eUZWpWTTFGiY6cdsHnD2ory9”(Œ"1Pa7YmkWCDFfCJgrtGh2VWARrRPAsLAF9D”Œ"1GXAHDNHGbqxLxzjimnVPFsFkacbiUWpLu”Œ"3Qgzpk4rYFNoNMLDNcfrc3JvTgakKpW8Rn”Œ"1DUiq72acWtNLAxJ8hTdMyiykTj5nMGP54”Œ"13e4SmG9nXg8XsvTBPqtpfJd9s6ffzGwDv”Œ"3GyAVkwUj5DmfyjL44bkue2mYYHwzYiUDR”Œ"3B4uEB2eWxPRvSwg97nU5vmrWvUHg3ddUo”Œ"15e9QRdkRdZuFU7om2pYWgv1Vncc2uPWgK”Œ"3MvYVRLzUuKyoxF7yzRP2bbXdvu3tYN4sW”Œ"33nuExyuR92fmgcs6U3T5XLgKDHbodxTn4”Œ"1AAE19x7XaaQEE1AsQUa2em2AKsdmyFGVu”Œ"1JgviudJk2Bcp3WhnDD2ekwpbt9b5qncfu”Œ"3QDBsWdC6GyjZXsYdDhcoj8X2zw2pXUyzH”Œ"3BTN4ZPKZnGJEGSVe9zPe4bTn8Qh39jkbd”Œ"3QUxEbxs9Nviai7EsuCNCQXivQPPP8gi61”Œ"15DTMyTU5gHYNQzLf2ovohopYvxw6thqHk”Œ"1PwdCdGrzHBLAAeu26rfvxKWntr2VAt7eF”Œ"32MjuyEMJwy2gMd5KqQHCaAkMxhfNbRMeJ”Œ"3361LqVtn7Jc1Kuffx1wTCt2b7cmVXTDzA”Œ"3P6hBMq7vxbEEPiCBZ3NkXpGEfduFfmsCQ”Œ"1FNc4MsnnDWJfVVEX8nxtVxPxhmNRPnwuf”Œ"1PwB7aEzgf781zc9wz1yeFPZAFs5j1vMfV”Œ"1HMHnUhojcsykcpbGgHd13bR1BbCy76oj3”Œ"16fA6DwsCpnwaQHuyJT3UG1A8JWSwJqkDU”Œ"31yvsrM1EpoqPmyi5BBNJitMw1SSjYxSZ1”Œ"1BDTkSZaJ1i2Rb2W3Pm6ZTemAvNGwLXbDw”Œ"1KXMS5AY1vKGKti71yqTZK3mDQp9pBXnx2”Œ"1AhzpH2UhXytim46ELJY6sgncpVTVkqAvJ”Œ"3DsQBGMZQ2tbXaiLo4VGbWg6ZPcVMztREz”Œ"1BUniTt8wsi9seC6UQBQaegvZVgH8Dczye”Œ"1Gfi2SEiHe7L8kbpaKxCFdNc3j65zzLZDk”Œ"128ttLMD5uRmVH8NKz3iSE5YRAP95nLKzF”Œ"1GjAusedhYY358asp1zx1yr2GBWHQmVVe1”Œ"33d5Rp6n6ZCy9YDg4ZhgHhFgjJDp1hqdrL”Œ"19ExBuJW9Rj3Rp6hLxfwQqfpSvz2qJGFrg”Œ"3LqRrbRy7M8FEby773smoEAwHcaBLcN5H5”Œ"1GqpSaCPV8Cc4WFbTirMshjdNqpgtBm2pA”Œ"3JVb82nsDq7J37Fx6J8tbchTqu3RMRHYsK”Œ"34q8Fo2EPiqkNnRxVbhXpaBJXThGBmxMkU”Œ"1PA3ykEAF6nmmqGdaUAEddaR5oZzq9pBPg”Œ"1L6q5mojrhcYvNWCpRedzLanoMkz39u4hd”Œ"1PSz3xpyq7qExDTSjgT6zkkincqbbR2dr8”Œ"16mFmiyQxYoQBN3L1jTCF872UNexpWvsKR”Œ"183yarqs1eNwZ5mXU2kyYveqUwNN7eaCnJ”Œ"1HGLCdy3WGhX1yXq7kYRBueKQv7TFmCEPU”Œ"148SPSa7Ab6xPMhcYyQP5m1GM7QkZ2qhQt”Œ"162vnT9ZCdvkWS3y9CDNsJ6hbWqaEdiHt9”Œ"1MCeLykRhiEw5mWMmsBMX6zsvcfYhB12dN”Œ"1GqUa32eiN8DRMLWM1RVTxprQv4d2hEAbw”Œ"1KUjkZMJZcUBsQ8iCD9kJkg6gWgoDFag7j”Œ"1N34K2vNV936d2UHEZ9pc7PoYtCTVzTKpW”Œ"1M5nmYLQt322sJjezCduySN7fQn1Vx4maR”Œ"1NSwTBTZ8vrktbkCkZ9LZEyYKMMfgzqfVn”Œ"1NrqNbf6s3uMRo6JL8mzqFoeNzULtghv7w”Œ"36z6fXjdsL2NM6f6mkFNWN2uCyErgX9Loa”Œ"1GSiFE6KGRA8CMH3naxTQiCkmz4ChGJXoq”Œ"1KLeSvUQnMMEsMjHcrzwY6Mzjpj8UyfTN3”Œ"151QJx63HVadBFAyQcybeeRsdr1dV4hF7G”Œ"32hpB6G8MuDMd34VRGJ3MHjnw3aeSLv1qe”Œ"196CKpHvQCZAhY5j6We9nshHd923Xccns6”Œ"3JGPSKispw6QYUc9PWftVgujdwt6tufpD9”Œ"152phFHw82HeRSFKZP8QoByBpHfKQ42FY3”Œ"17T2sB2mgVTNa5ojn8xoYHNtoSkSVuThBu”Œ"3DyJXhqMBqz9TtrmMMVxJZ9pdeAzxeanW5”Œ"13xJVSCbkEFUAxdNd6fvP86sPBiPRUh3cw”Œ!1c1ANo9i1R37FddwhRWhBc7GbbDgubDXK”Œ"3Q5yRF9bYBKbyYAFRwrEppZ1GmRgTkXvN4”Œ"1FJYC81X1jSmD3oKy7ueDK9zVWFUx26mc7”Œ"158iDGrmQQSxTq7BCmftGkEgY7vRP9ae4c”Œ"112KWv5BLVpep63U7EN87e9co2jJHoDkJG”Œ"1H76JvKyocPfFeJzJR6vbQhCkqqn96rr8h”Œ"1HsuYcjhiRrdQiCnrLBCYQVCPF7RJAqUdt”Œ"3PvFs9HwwjyXdXW6taZ82YCbwJCNmqKZhH”Œ"1JmKgTzguaUnNckf2384PqSYUmC4wpKgxa”Œ"1HXHRH9QrnWqDhs54jco6HPPrKGP7S4Sds”Œ"1PUgJH4RpsL7mGag2T5bipNoYJWVLWX6vy”Œ"37saouVqBrJzxAq78g6WNYqPbyLjELUjCU”Œ"1DnFGB6rtAkpeAdQ2D1PFV5WWzZdF6SZkE”Œ"1QAHedusZVpFG7dzPMJZQREbcuSEcZDTLv”Œ"35XvsubhK4PYdvpK4Mhqfokc35d8nDPgBq”Œ"13nzgABw6AWXVLD13TNH8emAVLUUSNqUWF”Œ"3N2cT64QCDv4ymU1jJUdRZZrnjUXssDHEY”Œ"3KDiwpp3P7tpSHnxkGFhiREdveSKg2gGdw”Œ"1EM1r2gZmTRAQx3XjgHPJozF7iH8XbZBMc”Œ"1LqX8MPe7p8p2uGKcybRMMtVatD6GfkGKn”Œ"1JgLH7XjfWRM9Mn44YSRUmrhtfWzow4LRv”Œ"1PEeP9tEUMGBaqLXrQWGcNxddwBiKKyjyx”Œ"188frbodKKqFirkB2BwxzYGA9dstU2MhUt”Œ"3MVnF2Vmo7Rmm7iqhBytM9yvDNeSyjw3Se”Œ"1DuNRGu5wqCiEKn4oc517pxomWajSPeeWM”Œ"13MCmuF7bjAGpAk3GRA9ievePXVoRYPGcR”Œ"1F1KF8ejKyYoqEhKceNvN6jCgDrc12a1xm”Œ"38j66bVadKCuQgtGhcwq4UMYE8mqsqvn4e”Œ"1NwDwhngQpm7onC8XZ1mtdsVzywH78NEVx”Œ"14N5x2K3T4N9B44UbFhoFdDfuq8LoT39jr”Œ"1L8qajqs4d6yyQkPYWpPnyzaxV8NMLRqY1”Œ!17GmwnrDmriCV7QcoaNiccB5xDP7wmbrU”Œ"115boyC3qAtvkxXGkXpn9a1JyJKHWa78aU”Œ"1Bg2JdhRdzZEWnxRMPcdsyqPtnerNjEz1J”Œ!1Fdw5DasPVuK3n5tCP4GEK2giH3Xr5BJL”Œ"34H1PJ5yA3PAvfAAUWy5yPfWoiZw7dyY8T”Œ"1MufkVRA6rawhtmNNdE4zkYpMr1X6wF9gW”Œ"1M6f3gejjuP5zpUQw41Cys9fKD1GgDTM42”Œ"3CAB253Zn52sihK2SXZxRsnchYShHcSEiG”Œ"35A4FRPbaCq2M5PAvnxHASyKfsbQwzvcHf”Œ"19fGmNphirvnr2rRdVKHi33zVzimzAeBhN”Œ"14HFGa5PrRdCaTm1d9VMtnp9Hjf8UWpfEk”Œ"1BsSpFqHjDXn6Jm2ebJR5KtVC3conVo2Zp”Œ"3NhkJFeLSYfMu6QerMMXqBNGw4AEFoaEpH”Œ"12W9Kk1G5ujwfKZTZRHNqUYPbyEFrVirz6”Œ"1GeqKNehZyZ8uUqg5MtivnyBnda1Uy7oQR”Œ!1SSBHbaGpAuWR1gA6NBTPM7CH5THy1Uen”Œ"147LiN3eew9yKdPRsPKFewPJj1LQbRorm9”Œ"1LgY452QDCjWRWnKb7qoznDjxRNnTif7F7”Œ"1ERvJFXhkkyimybvLmRu6JfqiyEKKHt972”Œ"1ChNCRP9owWuvWeSy5nKrL6QtNak1wihfR”Œ"1ADsbLZD3a8CkJBKPvMNjnND2WYedGoZSu”Œ"1Pw3xSWqRdKBMQZVbuzCqat3DrSrcKuQW7”Œ"19dEuys7BXfacPL5EiV7odQCg2HBePpYBU”Œ"15NNNgSX3qowhEYLhyPMJfWW9266Wm5UtH”Œ"39U2ErYYQJt9KxGRqMdmQqbLY1tMPDuGi6”Œ"1KcqSaikSwWPR1mSBiocU4zQLRpaaEo6tW”Œ"1AL4Q15c6m3iWT51GXbm4tdFRy14KNdk6C”Œ"1Av7QE6kVyxruLJbQ3CuPtgwY7yY6VMiWW”Œ"14VQGG7By9cKPZk9NwUQaBst6CsjVpbeTp”Œ"3MVgMSrmey9VXnxewU4RzNUThMXfRmwmW1”Œ"1BeneNwCAWaAtJjLyzTvcnyD6KW6912hH6”Œ"13qzHHpeKiS1iGoLkBvTK2wWge47DS6rG5”Œ"1CHsV1HRtW7Kx1u2Dz4nnh8vTJC4HPyDNR”Œ"1PYZ5hXzYjmsPmoW7CHATMDCb1TLEsB7Wk”Œ"1PgbkhdXYs3Yhg4Ku9GTzeBNSkf2m4QpSa”Œ"13GBe7iaNiy1MnVcRpwo5LV5JEH6ptKpjb”Œ"1NC4E5yHsKbwf6y7touJpDzu2jfKaX3DqM”Œ"1MmDz72VktnJuhktaUZrQQYhFb1yDL2cmN”Œ"13Fof5YaKVRoMtk62R7hFmpKqjTogjcx8z”Œ"3JWvhKizLM5mHLE9oPwz1NbnPbM1cjroPn”Œ"1DxCL2jvfMqcgeEQNrdPx95EbKgVBkv1TN”Œ"1Q9c1okWqVpFfQKYa7XXSvGW1Khpb8REhc”Œ"3EajZhxWMSGTuDZEkDwZ3RWCCbh1yzmeuN”Œ"1MiCdfhs1U6Zp5Sb8bfqBaYD11vVsLALcV”Œ"1C1ksZqyXN46mLQJHD5isRFfe8DVitbN6u”Œ"1QJj1NsCTQw36T2uMgMiv3Agqspv9kZ77F”Œ"3NjySXDjAZLb4ZhXP4XB72DEi5GMkqgxCu”Œ"1HA7QLHNmwFUo12aMycdH2Mdzq2WAbkry5”Œ"3P6ncDbGbVy8WHAKb5LRhTH9Kzx5HdhpTK”Œ"14Fi5udHfyepzP7jH9tEMtHvZ9dVZLLFQu”Œ"16kWoRJmy9VfHvjDih25SaknXusZ4Hk28p”Œ"3BRqbDCd9dJ39n77ermjypCCMCPbqdzsq6”Œ"38T72ELBKtdAWsEVxiA84xnxsMnQHSzQjr”Œ"13cbkG7TtzDnLqSfrnB5WLCuxFx75Fc3AR”Œ"1KWFrB5xHggJbUVZvHUxomVagQygyW1i1h”Œ"16wwfoEF2AxbnuJfHf3HHnN3LTvUJ5UzeE”Œ"1MachinaN31UtowPP7yhfng6VAVqWt4F5x”Œ"3C51mxG595QMv1AbRoT7HoRMXRcXYMjSXE”Œ"1PK7JVbd5GGWiEaWFqMS41A4Bp8u7E9G42”Œ"36Ys7jRr6iZpXeSJS5ysMkaHMGULexf86N”Œ"18foRiVBtWJgNMyXcv4BT1rcVHhFw2cmcd”Œ"1NRAoo4VntrhTMabqNEqkWwjZJCphqwHw9”Œ"18WpMb7Fhz4hbRPSofcPk13SpNh2peFCy2”Œ"19WrSSRReTYJV5SCv5jP2HVEYbU8AEGdtX”Œ"1FrG6uuA6T7qQeRCeXNWGm2KAYLDjpD1Nd”Œ"1Kbc4JWqEQYDWWvMxEHGittC5qQLQKWtxX”Œ"1AiqioAepAMSYxLwzB1DLoRf1rdvBz9VGu”Œ"1GJpFQtLBzYM6qM8FG9n2B2oBy6n6PjuBz”Œ"135sooAJmQttLkjyBP1xWjXDSnU6kBRYfb”Œ"15uKWnHzaMZh5oytFuX2TuE8DzhRQsx8LG”Œ"1JbDG3B3PrEugfAUGYNWM44f7NXHwa8i4h”Œ"1PHkmJ4z1aPkU583vuuoCgRE53Jf5tQFLF”Œ"1KsCPR8ZHVSSL2CN1V742izX2WaNHDME44”Œ"16Nub3hQTm33T5VuEtKvE6ueAiWrtKwHwB”Œ"1Q9uPLvhAssKchzDdPG8PxK4LSrsTbMLiP”Œ"1L2emwNEYrD35d2W9ENeTS5PiR2xYUNcrS”Œ"33PnTaY7kCFARxKjefberMSPxKtTAePZCo”Œ"1NBdVW22hkqqJpgSUZStVJkqzVzdhndHpU”Œ"39AtCQ5BMwDXEdFXZchpo3PCf8AM8tKvxr”Œ"17kZURtt49UR2CZoYeWUWzMzHHyriF4AUo”Œ"34hyez3RaQBXTYbZA3fQ5R54Nf49Fy1v3i”Œ"3Bis1z1grgn4ZPo6qVKF1EUVPc1Qxzyvc4”Œ"1GDZJpDB6po7eEWnMWj5JAgEmNkeEP8Ks2”Œ"13BKreJMyV7qmmZSrnfxgBj4TWhpZuvJpz”Œ"33TGMcZkMEraTEfoZhLSKw9BrWmgezc2TA”Œ"38K8fBzr3eH5Vq87Yb33aU5EbXYL4k4WnL”Œ"3KTGZhsoj5kA7VtLwpmqzPJmM5pyMLCdZF”Œ"35NijWZCA8qBTg7gJFqzqexvRGEUFqyKuA”Œ"1EX3DPW8sB9UdUuRhHPrap8bNLbV5HwBSH”Œ"12PbRqFqGCcqnaMQzxGBUbTfRQEp5g8BtJ”Œ"16JacAppstpPDTmyzZbzqwHwHE9sDsbzm7”Œ"14zXxwDkk4UJVZawurbaGEfY6FeaWLFUBK”Œ"15yFMZKHvUhDwH3ezpdJbdKfzc8p9EPQqq”Œ"1A8oNTF5ASYUx12um9pmVg261Bce4ZAiLY”Œ"36SXdY996zE4e3VWoNFFkcY69UhwjFgXDY”Œ"14BigXZesaY1t8AVjyxubqFHew5GWVDuyP”Œ"16shxUk6u6J3ku1nd6cvdziNYRx3iaNayv”Œ"19qK7FkjtT7vMyGmdeqrRBtbxmRzkQw9nR”Œ"1Eh6rnUxDzpa71QZ4xg4TacuhKK3jfixvF”Œ"3A6n5SZWfnMXuJofEhdn63ocmr11bCGFj4”Œ"1PFavxeU8WGZ2yyFNwugNSBeiUywjNGtNE”Œ"1CkU6D4m48vQUdacSPdr14YhD1p6U11zst”Œ"16kG3gmyky4natqzpvRuP7LMZhzQ7BSonn”Œ"3Gje8vhzf1u2qWdPqPf4C3BRjejvtDNNCb”Œ"14UfKHVQUkUnJHGQzqnBZpCKB43ig1wEfh”Œ"1C3azs65UEeMwQRE5JpUq5HrrER6FuuMVC”Œ"12oXDDVyEdCNqPoNNrAs7qxmj4dD2mY6KP”Œ!1m53UTnMVz3BWMLShXGE1uJi89YyHKe2q”Œ"1BidDhkRosfCRZEscFdvPM7NZt3ogHVu8e”Œ"1Dt6s8q8xMcBM3cHV9RWvA9N3nUE99zfF5”Œ"1LDkQtkfiuJDh9Ee4QxWHgikQDjKShE9KJ”Œ"3E4wkWLsDYJFuUzcn5ZRSWbdvjVo9C3o3C”Œ"148P8BqqMMj4hVV9o5xhtVDDNC8kJz2xVe”Œ"1Bp36gtsJi5FCkriFtrGGBjb9fULWXRx7w”Œ"1LuckyY9fRzcJre7aou7ZhWVXktxjjBb9S”Œ"36bcHUHykvNmTh3kaqwejWiQMVyxQAt1GR”Œ"1BmMQjdjJofgz6hAJ5skW9Z2wNadvemfgf”Œ"3MJ7BnwM9cr51kyBdtUcNPzQeVKrevLMbp”Œ"1FTFkR6gT14dHBWi3FLWQDNL7epftgEFjH”Œ!1Gtp8xDqR4nZUWSNAQRwfVvbXm7r1WCUB”Œ"3P5R6EZTT6G2rgkU7LF3YQGRTReEBdDqrE”Œ"34g8ug1QmPW92EFxiHieX2XEc3GygLew9A”Œ"34BQBCwUSAZRkfuATWADkYawoYViVhsWCQ”Œ!1YJ2bggUMM9vHfoyjVkkvYrjo9V9g8XUZ”Œ"1JyL336jXQFn8RqUJqaBPLsELCa5QH2fsU”Œ"1BUi2R7Ys7v4wrpB2cLE5gpC1afS2HtywD”Œ"1DXDuFvDb2gW6DYSfxuxTtEFuNpBhC576k”Œ"1NMJ2iheMxnW5n6SV12XUBv8VFRfqfpZ29”Œ"31vEENHCNMf8gWZ9UPqoJL2D8Q3ZLYYiyJ”Œ"1KrRdePPfaAq5e2ny4nBHcie787qaqAnXm”Œ"18Gykh4jE6fuyARzbC3XEzrALuKWFNHra4”Œ"13mXWzVwgTGFLiizxuFxjMipGxYFXYe9Xn”Œ"1Lnk2aWpJxVW1aTKn9qbfYRm1q7x3Zp7pv”Œ"12rFKNtzcMXpCc1rtAEK1bCbmV6QRsk4ou”Œ"15vpSo4LnxGHnw3eC5geiS6Chy117nWULf”Œ"1MKAPJAAPT1PWy3Nvb1pZgjpizhSaS3zPQ”Œ"1CuWvae15UbF3DcgkQXsjtmZmjoJgj2taB”Œ"1PAKR2gyj5nviM3QmmNHifqJMr9vBoGes4”Œ"1A2PoFdS2ghebiZZnzzgQJ6ybbAhtS5faM”Œ"1781an7BorewWYNsy13DByyWcqC7xeHHk4”Œ"15fzzdoktWUpzXvHXHQRzER6bWUxLzuCzP”Œ"17yUDqEmCc4wikWVz4vYwxTcTqvFkQKnJ3”Œ"1LUu8msMSYkrP6bieDaHtwqTThyP6xyGhD”Œ"3DMytwmpjqVDaRiVxh749mhw7U1x3daJbB”Œ"15GNvj4Ngoaj2xwgQq3YGyBZBcmxvgQr24”Œ"16EU5gnRsJoDgas6afDYZmYNccmfDk3PDj”Œ"1BPp3sEbrMwXuVKPHUR2mZ1fcJTm2yP1T4”Œ"1P9e9ydmeeVsrXZo4od6hxBm5AtF6pCUzK”Œ"1Dm2KBNSBSKB2Wc7MNv7zg4KwbaiH2GTCR”Œ"1Nky51G36hXbvBa1aaVN1Naazt7vVUX6rC”Œ"1Ekvm3HU1DZPWDUTdQp6H19A5CbT4qhPb4”Œ"1K8qYUa4rhJvh5hegtVPiSaokn71xWyDpp”Œ"31qneokFGwpEkoZRksjymxZAiTEmAim3Ab”Œ"3GAmJi7w8fNsn8FUNZfkNj7Yxxc3y3wEtx”Œ"1JZfG4h4Dmiig4JrPTfReVbJzSMRFE2tKb”Œ"1Mxyje8oS9CLSHXjJMvD6sCDwxA9N1ioCB”Œ"181xWv6cYvLLWe1HuywMJr9QUrmHrgeWVm”Œ"168myomuAQ5CC34ocxBqod1EMFRkXcEcNm”Œ"13Th1Ls22zJsi5Cfi8kXK9f5XcBs3rQSWH”Œ"18c57sKFL1gjNMr5eAVnAozUkUqoKPB8w6”Œ"1EBsWon5Drp83EA7iiKYEJEukACGTRYFP2”Œ"1FAFZV9dKnC3abqTDZjoy1fPUvinYtwWv3”Œ"15Lea4Vu6AugH83MtjYaRFHy2KPNqdHbg1”Œ"12fYN7nP2YqmVbBtUrWgTZG3w33Qx7WqLC”Œ"17Kri9FKZEUaiWK5QbnggypVuXAzsfQnau”Œ"12eJA61CbTKAENjsHY5wn9jmPKEBRYvDd1”Œ"198WTYXGQkd4fbUWb123EPokypY4xjoebv”Œ"1Ksb15dh4znkxuGTi2juCdoFLwUiLMivhe”Œ"1HtqDMWgn6186e8t3EesZQiw7gNbaPJfJH”Œ"3GtFonHVCcUPdcFtqd1mRaz4XscyvojMpP”Œ"12cRVykRzEYuu4Rk5opJ9TC9p4mRx2bjAg”Œ"1A62tEff5ZoeW39T9eKq7TgK6htuDEEFaf”Œ"36znCGWsp4q4myX6e2zry8HQryVerC4tu7”Œ"16n6SCRj8HaoGqiLSDNzKSBhaYhbpW9iTU”Œ"18t2S1ax2Xob4LcGmhtLsVZ5Hs6piuZMSH”Œ"1PXwuhMSGdQg1eamKco7jjhLPSemfy6fuR”Œ"17xeGewXvx1uKDoQQen7yrQamBbr89jnQm”Œ"13jiT9VM7Fb4iMd9FZFip1dPRYVnckGZaW”Œ!1oXeBMWZrWWxpEgheaNVGVfBWV2n7YmAb”Œ"3M5co2bqPQR5HUgJuQf8o5P2eVgibPTECY”Œ"133VjQmpSKqDUg3wgYE9PBT1ahnnZTS5He”Œ"18Rb6hZjVCBCnkQND5cMRVDaTBsiBFi9tF”Œ"1DhfDp1UhYXYcH3ajzLNncQrKoND8aTFVr”Œ"1DkRNVGggRKhX2U4FaFwaBB5EQjKQxs9ou”Œ"1A36UrLHxeg9ABoS4zPsRUegyCWTWER2kF”Œ"1EjnQzMJngw6LJfhsRqkPubWWc14MczfF9”Œ"34oG3VDtUSEPaLTAgucs6anuV1wUpG8Dsd”Œ"36Q2k9PvwcmK4zNBYeMTFaudQRg5zfX7Uc”Œ"36M6a1DjEbY52KaDd6hBV7rXZZJmH7yU3m”Œ"1KHQyQMZQhVJmv8wCMRueoUwoEYfHyMR1w”Œ"1CAxeEpw1DToJ8886YTA8W7GTWxoLXLe5J”Œ"1PybSvHzYsfdGoz3TJ1csbzWF3FEDyhr4C”Œ"16QNo1SRC4N3YRMhEGzaZw7oRG2ArEvxHY”Œ"16oLf5eMeo9T4z6qf2jxY1LWbErMox4fAp”Œ"1F3T3b1tszqc217EmQtJH7133Am3AMixLh”Œ"1EoCzPpdnLqDPj17e3ENXBbEwUUj6KEguw”Œ"1NxaBCFQwejSZbQfWcYNwgqML5wWoE3rK4”Œ"1P6gpqvcvP7SdyATYpez67NXEdmy2Z7weS”Œ"3Daeb14jAHuWf8sf4zFVwTrd64nSjm7rZP”Œ"1HiQqCxtP7bT21ZpFt1wzaFgJpchCqB3dA”Œ"1GfS12FBeTqx4a7aXuPd7RQq9SWnPdoZHX”Œ"1KkwmcEjSBLtSaVMWParwUy89YZkYGZwh9”Œ"1Lnt32vFqwz7WkqS1KXN2xwX2DZ4Nj4j1m”Œ"12TZB4NFDBypXHdoR7L5jLqhPXbxRRKNyq”Œ"13FSi92oJYdCj82i5SZybL9U6DgGkDJAC9”Œ"1AikP5ibSFhqXnVB9vTezHPAE7GdLsnBJM”Œ"1A8sP2DKh2gm1RXpDwEhjfbFpXdSZwHVVj”Œ"3NDiPBnsK3iES67RMYFew3SoL3QYmEMccp”Œ"1DTaoZVZx4LV7uaohtWB8Spd2xYSHkNdYp”Œ"15TDwcXbX1CFtgdxpm15exAfcp1E8FJycj”Œ"12Tt2hRb77t5RvGuiaf4RRiQQAChXMTgdS”Œ"1Mfhvy35twkernPHEV7Z7BLz2XmjyyUfp2”Œ"355gubKZeMnn9aF3pVP95rzj3uTDxwXyXr”Œ"115erL2BYDeyS5p9fVdUJUYu7BvNJSH3tP”Œ"1DZq5ZAmpGnEbHtHxgBA8ZqYtaUVHPzBMj”Œ"1579PEqQUqgZYZSMvmtndPXUXs44uZ7quC”Œ"1JiHPGZ1oMG8DQFcjcU67nwgr8CWcZd1sf”Œ"1KaHEtFFnL7N287YDXmwks3fDjAQMPnkse”Œ"1ND4yPJwpEVn989z9WjAjYVW3STD8G2Tii”Œ"1CnyoDHBG2p2A3vah3Tm8EVMckuStyqmC6”Œ"3LUSWtCaYY8mocUUT2kmLvuX7v9ZnQyx35”Œ"1BMWBmZKtFL8CXGG71qsVvsXxX8UEczs5i”Œ"3AYNrV1h85Rk3ZnyBnmuYD2pS25H5tRdCj”Œ"1FHKRvVafWbBq1kSLjfUURM1kuxJT91iAo”Œ"15p7JWJv4BJSWf1NAspydsBPncbUTLPbuU”Œ"19iq37UCvBRdycZvEuL4DXKnYB4Cy6hBCy”Œ"1DbPAzg3pphrTvQcuLTtg4aGQGUVvnrUnP”Œ"1BC9L3RXrJP6QQVCAeHRpzCmcKccE9sHg7”Œ"3JCBTKDZveNBHJoL5rxGpqAeEJs6ktg3UH”Œ"3EAcZsrzipDZxziB6SodqvXtdm2haP5mPA”Œ"1FgcLVCXSV2XqMYrNK8yKbDDh3pG6jNNex”Œ"1AKdp9QzjZJGkjgZ9JNgTcYBdcUe9URnSC”Œ"12Zvde9RsiJ9V2Pv2BWgc2y2wFaf9QbN8v”Œ"1Mp5wHimKgqUB38dfiKShULBVr2pSGS5Vz”Œ"1MpBazNE1uiZFchdXWBx18qYcrY1sP2AJx”Œ"37QH3EYhddmaH8DyuPGQZ5g2JojkfVNT3D”Œ"3NKxsi4yUxMPPd6T5d2ip1bvcALeBAtjpE”Œ"1GHnwx1DnVpAxQJNxxcudCSei6Q3AHjL4m”Œ"33KDyRig2uEquCgF5mJDzuACDh4MpqvJD5”Œ"3MsdeRdfp8nasLFJiHrb3UBz14526k5WQx”Œ!1T4EnVGe4FdhCScgP3P78qWwSmJtVDj9Z”Œ"17XjWZdnxciR9tJup6NpreGijG3DZPbCMd”Œ"1H3NMCJkbE1dP8tZTEpitkKm9CbDfraPtp”Œ"19dvjmhyxtVG9gAnXgcsb5yW3KvUEXigqy”Œ"1G3vMPpS9GAnVn8w6V4dKtmoX3SUA1ARTK”Œ"1MYwi3gJurejBsHkXp2DbUAoubsiukQwbJ”Œ"17DtNd3SbivwXoNxRqyMxvrMcyA8uWY9Pf”Œ"18Sg2HnKkxhQeCUKq3j9cYq3w67h6ds29i”Œ"12csxmrxN5iuJDtNHSNC2f1Ts7AXcoKUXm”Œ"12cn7Fq95Uk7MaDBxg6W5qtXD5UWdDGgrD”Œ!1LUeoCRwa1avGEjEirppweC7wJzLHqnYY”Œ"14ruhZDfKPsp5ioEFhSrbyooRhrf5Rb6VP”Œ"1BitmixerEiyyp3eTLaCpgBbhYERs48qza”Œ"1FZjMEbzN24k46EJnW2cY9JuYt7FbRny72”Œ"1LbCUpa7B4h4DyVomQZQ5EEMQx7nz3yD17”Œ"1NuhaoWCeRqwrxtrpqpjhTSPuFbD8dujjJ”Œ"17vqtdBYERuZRQpQYuY7jvSrxLbUpVkrik”Œ"1ETa8z8rEJd4sFEAS7GFpSGTDYFKFiYTup”Œ"32yJ2Gn58HvyytDLywqcKLR5wVj77nptPU”Œ"3BJcQoe3QqTmozrmPhj4HHBgdKEWq6sf5P”Œ"1JyF6g6Ucar8hAUhWWQpENeAypHW2d14b5”Œ"1C2kCX4x3oA8xHrhEdCj3q4incYzBJXhpX”Œ"113HhhZ2sXEEzYFW17NGkeeVAhpbbP7i1G”Œ"1Gr9vNf7ZFttCiHzm8T3hLodpgTcVgUyJN”Œ"1P2e49P7cbVqXx3BPA9BRbQBP84TYsvojM”Œ"1GAMTecVa5pb8hFmsbV2G1ZJAi4yR5cSc9”Œ"17rNXmw4kJhPXNzrnRJ75JtBNrGXXyJJ4J”Œ"39J5kjRFxBPsm7mZs6gm5mZYJLtRAJ2vrT”Œ"1N6bzHvdiSbBzqTKGAfgjCsN4HRfZWGc5j”Œ"16MPEMhgB7DuMjEUsAuE3vhB48ArHAhBPe”Œ"14CDhLGVDbVTVnku4EkZxmRxQwbqm22n7r”Œ"1NND1bhE65BSB9cmwcUKGzeWoNT6woMyDc”Œ"1EzrWDsQqKuUq3kHwcrN9fSv3kW1tzEnpz”Œ"1PgZLujni7q3PyacroeiZAPUMqf8ydxDQe”Œ"1KWZxk2a4qk1xuEFXCm6WB3y2xciGnHiez”Œ"12ZwGXRAF7a936k8wwS2UoDv1MTopG9v8B”Œ"1Fkaw3LBAp3QUFXTsgJkokFSvxc7jnhuNk”Œ"1DJQw2t6GkvitZ3e9TfESV39FkSXZQYdLA”Œ"3FxNPjJmcQsw5czeheoLMub7sveWW1BNFK”Œ"1CwJg31HXCgz9Fsjc84As4ZmQQvd8MgYb4”Œ"15pRMvJtWSdMia9gwZF7iFoJhTB1EyH3es”Œ"1MPVT2T1dbwnK8GqPNvgF8MoFnT3QujVVk”Œ"1Bs9He5qWtbjNRfSCV4jWAEXrHjPeL1Fqt”Œ"1Dw6oa1cWaNk3QH8yMrL8H3LQb4AgdZUL2”Œ"38G4ML7vc2hkdrjjXbwY2FkRqkdkJKMp9Y”Œ"31z8QRwygtrttn16UaBAeSXVwXtwft8ydx”Œ"12VVpEWqMi6v9Ez4HnMwPfa7kHnNTocqzm”Œ"1KCWTcDBqhy8cChMBBax5J8egRwhoWHUsM”Œ"1C5K1VxwXCskQcT3UGGMT2Wfd4k7gyxcVB”Œ"1M6vA316ZEwBFFY1XZfiofDKbHqNTPJxM1”Œ"19ssCztwpovMFBwRPcDx3EcdQyVJMEN59L”Œ"1ETzwhNCg98toZBdpdR7x4hkn5AktQ5oVS”Œ"1DjiQRhVWRvmMDEf8DhLppHB1n9chKRUzV”Œ"1JGjanxquzEvwAs9GAd367X5AhzuAEmzDB”Œ"1Mccjd8rD9LqUjghNvsJdyV4QNY5BikSQU”Œ"17XKhJAZaA2mKZwqGUR5yCKxa1XnyTx9Gw”Œ"19yycmX6DXYNb4uirYJrg77zGRCkuo2cNs”Œ"1HY2vuUfyChxPHmRkTua3u5tSokrmuNdyR”Œ"1PhmMsdwamJA6soKw5mNMXxzGomHEHWY5P”Œ"3Lf5bKjXo4GrtCaGqZU8WDsxS136x9PxDJ”Œ"1GugJ7moAj6yyjCvqk8ZTwjYCTGGdawsN2”Œ"1LRfxkbVevdweEEzDUjuDtTeURUkAAmyRA”Œ"1FBCSLjojYD6WcfNm8md7XBFTWQz2w2Gx5”Œ"1DMHXvMsi41tAR9MKUfCQLTQRD4idB93h3”Œ"3DutUHBs7korXqdvvybtH5br52B6k1Gp34”Œ"175N3Et8n1epm5YkYgn77GDdVAemekvXkk”Œ"14ix1qCqCSS2jSkXoNqCR9S1Jz81kWkhRA”Œ"1MWggh65RWJVRkUdGNCVZwYxB4zkJnLTLu”Œ"1NYccwXtLfQ5GbcNaPM4n4TtsNVDjzUsyo”Œ"1GxNrf91VFkgHuYYnUwBi13BEtSSKW2zoy”Œ"3HJQr6GaH6CFRxLtAedSr447RHsrRMCaVD”Œ"1Hqf4gugJuyqwUGEKeJ5v4G1iVVQ8PxzQ6”Œ"1LCRvKi7F1YYwr99jfWi3FMHo8a5i1inPD”Œ"1P5L7yKBHcPMCoWxh9PYAkNbBqdzKUXuvf”Œ"3K2pxWPyR3zF4u2URKsanKESYvx2j6uV7X”Œ"16veFS6pvvofTmcXKzE36bmnQsGx1wsUmW”Œ"1F6rexMt8aVe47hncm9QqfyfWDjyCMMbeb”Œ"13joZnPSfdqF7RfsRrYFtykbCDo4ao3NHV”Œ"3HyBfUJdb9XaY9AJ5Je3LCBUBzWNJiSudf”Œ"1HviZSB3ucw5A93T2rKTyyAa3Zxhv9pEJN”Œ"1Gmpp7g3vzSZPTPvAbabexWSKpMZotcAbj”Œ"14M3jxSNatffhEZWfHDEfqYrGth53jmAfU”Œ!1RHzvnT9u3j6rxtqxk84CrB3kkBGV2XdB”Œ"1KhZg2yyvQBoPHGJNS4jfZVh1NP6z5Dgyt”Œ"35RfcnXFfoZoxbRps9KXPH8wv9cL8jSDwF”Œ"3BUiafszFT2cLmDbJSDnkEBy7ifneYdUqi”Œ"1Kj3KW6GpSbGqhXWUkhQ719Wmk1QxtVb3V”Œ"3A3bj8a6JsA2fTFWoqtvXgnuUXzakFpqVN”Œ"3BzD2w92HhGbLosNNURR3QeN1YrrJfAXse”Œ"12zRPuDDD9pigvkKFEpryJDaUG1etujgD1”Œ"145z31Cato5JNk8sPsDYAcSAGa31u7DNxN”Œ"19DYWEFM3n84JYJM6Yyh7jcsxXzXLVZ5nL”Œ"18x9cWJb5T3dCqr7jRUAh4Gxs2x1iUE1ck”Œ"1GmdzKJ8jivUGtw4mjygPioZ4uR8FC8MaQ”Œ"1AMwUpnqMYW27MyHqMgCDaXd6gMCRJShXt”Œ"1P3rHUBemVfBLmhfYj27QHUS9D1i9tRbC6”Œ"3Pepvnnu7n35tb6f2HLJP2WxLGSaNuWok4”Œ"1P2W37HYtJr6KHtSaTWY5N2M2F5j5wa3UX”Œ"1Pv69W2DzefrsCK5JfsUe5aSBmeL9NC9qe”Œ"16Ktc8CSyhKkUW2G7QyfqoQ3yUW4voHita”Œ"1KkWKvguaoNcED7LayFQgm1nnjuiR6ftMg”Œ"38AQ9ggkgwyYRYd3q1iepN3QsRrb9YQgqK”Œ"19PuStHXqmv6x2jGoiaRR8hMdm7djYKAFG”Œ"1G1Ky4KP3JRbCxoDwdWxonWtsT8huXD95Q”Œ"1HHx83AWKTaa3fiZTXzJnx9aBWFu5ex7TJ”Œ"1LXCjyaCzqNMN3BTfA4B31tu19d4B2XHmi”Œ"3DCd95xWX6X6pNxHTizC38XZUEDrRhkUhh”Œ"14psee6VkVKfGX4pJX9BiD2R3QoEWDpX69”Œ"1DAiydhYHdncdKCWQPLKLfzUrLMRoFTSUD”Œ"14PBV5nxqUWKFBxahVFCaaiRNqxyEBXmoJ”Œ"154pCbWwv8MHRLD79o4YrnifYfKp7d4cYi”Œ"1A7uS1VMFFjUQi5VehRmn7D7X6wnoikWpE”Œ"34aeeabKYwK5gzhq8vAoxqkpJ6DKuQYDZa”Œ"3GC1YQGiYT6vhwoVemgMpMMSFGRcR9mPoQ”Œ"1G6ertSWGkwqrKVc5Qqej4NPrRzpgX8Rth”Œ"3ExMerCEiJ837zY9hd6Pmiiwc9GJNfyY7f”Œ"13Woq6UJVTnxpStsaYxZhtfiZwbt463XwB”Œ"1GLMLhdVShacmXCusKmBCnCHHxr3TiQPSN”Œ"1Bgt2VjzQHrCby3MDoVcAcArSsbkWLWUJy”Œ"1NYr2H4p4f5kYyNEm9ezVsJt3Bf1CWXNfd”Œ"14U7JGThuhUhRJiVAUTjqbj1CUnMEhyNTh”Œ"17gJd6imAN3VeByHBjCMVRcyXKwZKWNJKN”Œ"1FAv42GaDuQixSzEzSbx6aP1Kf4WVWpQUY”Œ"13xfufZWTSmvJiKZ63NgtVoXdsLU5fYLhQ”Œ"19G8nXGug2ZdbA923nRUoGM7GpWFn5X2E1”Œ"3D5gYABwfkjohGdvzg3btMbv6yT9YAFvTX”Œ"1E7TANdjYYWnrDcVKticWQCFJSi5nRs624”Œ"35n7EX1HSr2ndGXzZyReBXZAH2KYC4PCGE”Œ"169SrQn27xTypZykVVzHgKRRU4T85QHRDE”Œ"16cdqDpL8gs1NKSmUm5YZevp69mKvmrbEh”Œ"14DDSXe5fripmgz9eHym6SH4HhkMhNzEcH”Œ"1DcW8tjMKiPzQXaTa9D1mUEeFDxNvFpjTj”Œ"1JxyPUgDTte5NusgTFJkrcX2A45aTuo23w”Œ"1B8Kn3M4gUPJAa2T97TcKSnx1Sq3kkaPg2”Œ"17HS74Vbhxuk7VRUHcWiP5vwvsybbDYDWX”Œ"17qPgmd1FYY6MAAoyXs1Ji7a5kfLj58ANX”Œ"3K3ycrvXRWQCkEkqZZwDWUiiuimckgAhit”Œ"38mvVNqYvUPrThWyjLw7L5HyncambvNX4P”Œ"1GQZjvKnw51dtCC2UqCunBwbFxaKz2T5rK”Œ"1GcSVNeMAmDhbhVMf8jMEmdjwRViFf6XNV”Œ"167r8ZweKXrrVPzN78CFrYitzengvpEjE4”Œ"1BVEUHpAyuFW7q7U1jYcoQyzfkQsHTQRex”Œ"352XdzaMRfdn2kcdiDJbqQj6AeKQPefMec”Œ"13nX3G3LvhJ4augPKMWqmsAoaTivBngzsS”Œ"1BbsvNxQWXW2zgRmNbsPKEQBPfVtWdTrEb”Œ"34W8SrLzMiw229RCKhu96im9Z2VxoFcvnb”Œ"15gwCNa5LmFwjkURsFT8aEo6hMCm5QciTL”Œ"3PWKtYhFsq5zvM8BHBcB2VfwEN5i4Md8kX”Œ"134EG9zKaaMHrfTTsBiEc96vgrVHfAhEEo”Œ"1AJXdEYTaiT2mGg4c2fXtDkBjbVJxsvuPG”Œ"1HMbfY2og7DFbnkytNEyGDhz6FtSWQWT2T”Œ"16j4rWHpg1q8ZrwMzAKFz7uUHCUrQM51JF”Œ"1BgJc1H3VA1qvw4deMdKtoFH68vvnZgwBy”Œ"16YWwfGdU6jF7ekmpBuRS7QfXQ37zbwdMw”Œ"14EV1p1osJ3ckibo84hfNUGnY6DJxVLtZz”Œ"17wjwZremxwZv19zJoWWB8xPJjuerrbwHF”Œ"1NbCKUweazuhFG55dZSzaJjYqRLZjsHPGV”Œ"3Pw5Qpv2NNowph4i3MGSCWhzdnffxwb1ic”Œ"16gUyDswznpstoZXbinW8iL9ABbrqiXjC2”Œ"35Nuiz8MVchnV276svn8PBxVE35xfycExA”Œ"16JGbyQxNNg4pyTaEK6KR4jWNLSPFBx7rm”Œ"187hi6k6NMubeczLx4y1mALPEFvw9VS1AT”Œ"1HJfUN1CHZHD9PyBAv9HERbcQQCX8Di4sa”Œ"1HnDpGt881URWdCPD4q3tMshNsuhXJfprj”Œ"1MFkzhHtGo441CKqSESvAaXKGR898P4DxA”Œ"18mC3SrR58RjMfDvtMewKM6JtCEtgh79Je”Œ"1Ge31pv1zKmWX1ArxQYVzHo6BPovXoANaZ”Œ"16J5vGJNUi2Yr8nSpG8GuVRvXhikCtrUfW”Œ"33X4S5sPGVakUzXuL31hKZME6CKooc9z3d”Œ"1PJNZnn47GvDhr7o2RGYEVBeqTBFUxJpUZ”Œ"17PuA9GQnPbJ3ZteG45Tjs1g6Ah5pKKZvb”Œ"14oFgFihWZ6cYPpw5GUf32qYRpTFoMEUWo”Œ"17UEL1LwibWQ7YR5nXjoRZB8kPuUiJgr7M”Œ"1HT2WGhDbEStgCWb4pGz4BX9Q4LpJq33k4”Œ"3MVWX3GHAV9z8z6B7ynedDk3gPU6eJZMEt”Œ"179Q1mkE7hA7urmT5fbW8KqaTwZ96Ydoxt”Œ"3GmkBWyE1ximtnaAEWXTE3HeJyRwkUttr2”Œ"14pZeHxs1Rxpav1KmRpC74V4Ep93F296nr”Œ"1GjCmaKxz55PnQMc5Jr14UGeYWoqaDi3PV”Œ"1NZkceu9wjws9VbfipfiaWru3t3z4YwYwm”Œ"149jCQRrnqgsJ412WiR1RUH55CGuorye7H”Œ"1DBpM1kzMXVrm6C7Qz3wf1t4D4xj1GVy1x”Œ"14WBeDQnajPxLo1K4pgGZgcDJ2z71PgP9t”Œ"1GKA2EaHQw1wQRQykuLAnZeKM9ZWpqjqJh”Œ"1Nx2MUtqLLDwwmNP62x1DQ7eoKyLkaQdj5”Œ"1AXmsgTdS2y7Nn6L1HScXj93jaxKSTrmes”Œ"1AmQGNFJ1uuoGvAWedD3VyfzjhGxDZkMCH”Œ"1J8ryAmZPQXLL6rfUC4S5vqcsynqv6X1Wt”Œ"1D4wkbyb3BsPYLwb9gZmpfrimgthbjdyKN”Œ"1EFDGkgCfnFaFA8gQ7Dmv6aDG7Tov2Cpd7”Œ"33BAkP5dAr7qjpdgabZd2ufj1BYdJQQumB”Œ"1AMTxK8XKtXsoE2mGgmmT8jSDcFCxLHScU”Œ"1MoEj8CAD4qT3C6ysNNCX7MsbXvzujJvEU”Œ"3BZPAXwb1a85CYjgbaGtVGzLmBHJRw7ZgN”Œ!1KYScrQsuogyLxrhXFviJDxhHiUKBmwLe”Œ"13y1zYYDsRBX9TZ8XqR1Fo12vgwBUnfAkd”Œ"1CpBK745STxPKSY2KDZeuRAV3CKTvAQH26”Œ"14nzH5Ym3KZLUZ727VTXb5DbG2m7uEQpaU”Œ"32pARFNyB64fwRtQueHT6aQnx4pkkNLirr”Œ"1B45pUtXXZZoqd8FG2893u7BztKpwWw6sK”Œ"3QqTUZuGhRinjjpaMMpnbHD2Uie2Tz64FT”Œ"3LYP2eDFLGT95Bon6v4T1LZ9j6KAu9RoRh”Œ"1KBm7RPECEbynDsXUyC2D7HeihfW5d3MJR”Œ"3H5p6kZnnKYR7GpmpaNUjNbeDTF4PvotAL”Œ"35aGoQDFPa9KjDAS1G8mD6WjDF8W9DJ6Ru”Œ"1Ktxi4uzH2xMW5sByRXMJN8oLZi7P3FH2J”Œ"1DWsnerzYWivGkbHkhWK9C5zV4TxVJRC3t”Œ!1iQmaRqSqvs7VGe42PgKKeQSYZWdKiVAn”Œ"1EZKCUf3dZkSVjEtbC2pdxSb34E7BfTgJj”Œ"1DfgDRqeiBsih7D6DK2pKNRzXZKoepucrE”•”      Œ"1BG3ufhSskaDbGy2xcP46aLVog1yUgwLjf”Œ"3BojiAaRk4QvaBXutua63wG5uMNtcy7nCz”Œ"139RENd5Vxcf6yJNcC5aD6GtWEjGbEQkdN”Œ"1GoCWQmTKW51mEZes4qiN7rbVX4Jk3svn1”Œ"1GFS979DijcAqLwePrbi9DFjxkwGq3uiuL”Œ"1DKujn5J72ucTqWnTqaBnFsF2makYrn4ih”Œ"1E1e2LqrkBzwxC6Lfg68aQop3fhppzJ38C”Œ"16VnxePVfs4HHqFye4nd9nprSZ9uVYTMK4”Œ"1NGvkXCPYeh5Q2dER3g5DehRMWK5n48EGh”Œ"39pvSqfNcUosc8RGVWxyzKM3ny96a3uSkW”Œ"3GqaS2QLG9NwJNescoUvJ2ir34xq3hzckn”Œ"3NrkRdtYY3N6hjyhWSLScGYtShqwphpoe7”Œ"1Cv89fLnaEJx5AviawzDhknN2sC7rvLR5A”Œ"181qoTmGtuXgdc11U868dMS7PsLoVD7anX”Œ"13XVMytkzHQZrcARHKkARp8EPTTCrSWy2z”Œ"16B3N4dqogBieVcbYcGt6CgqVwZgEpip79”Œ"1G4UN7hiSuY96my6egrpbCnm2zumPABSyw”Œ"16ugfwGEHZmbNYAWxddednNsJS5Et1v1up”Œ"1Avtzjr2AiU5qz5AqvhFGUv3WmjpeqZpHy”Œ"37hhwprFbsNMpidB1caGMX1fktnGmi3eCE”Œ"39rRSK86FA2kjcLFajdnH32mm2dVGfxwfz”Œ"37X5mtn38nHj9ZtDeyZKfvwWjjuguNdvgK”Œ"3EGMFh6zZzSrUUsBHVuf2DWavDP4PVreVi”Œ"1AePYhb9zknvhWK5C7HxLw17Nta35x9jsU”Œ"1JqwVDJEt56DFLPXbA6o8LwJa1egsfCfAT”Œ"1DwnJLT3MWUmTd2unYAH6hmgEFXgKg6wKa”Œ"1NLYsPR9NoiLi4zqKHXfDTifPoNYqs6Us9”Œ"38EzU6AuMrF63Vt26dnY6YQWNAfQuy6Q2E”Œ"1DhTbq5dYyWC9yBeo8A9ofzbY6a6kbvSVB”Œ"1FqicaTcspfj6AtfMZrYAqZabdfQbsJWQV”Œ"17sTZiPLbcm9fmNeG51q5UoksZKfLHBUvb”Œ"1LQpPVp4xKYExsz2nksyRquZ14315M4RJY”Œ"33Nx7Tib1trQBXUzGhhnWVC8DqYmpfFAgB”Œ"19BoL5g6SKmMeeyV81GShTWwUVMzkVToX8”Œ"1DBqQFTtyhsHYSeWzcPaA5qu76kpuuiyZ1”Œ"3N4xQcDmvDtaSuqgMv8TRe4P25anJGv8Nx”Œ"1828ctUSqinwftx9zR7cSaK3GvYR5ejKN2”Œ"1BDtBeAjRfANZdTQy8UYghWhvuHybZe57k”Œ"1CCxkfLS9myyy35CA4Cnk3xwXdxGCFHvbd”Œ"3GfKtc76GGeW6y6rHNbfi8Dw2eC6NVWfZV”Œ"3GUWYWwk8xs99CJTuFpUPvpu5H8SSPatTv”Œ"1JGoneE1RbscK2aYnGHygEeGfak2Aihuay”Œ"37ss1hHwh3hWjXid2dm58AvFwX1w61iGVu”Œ"1MqR9axrTfWfrbuEGPCyKNiytH5Ydcu554”Œ"1CzczumQoD3TsnnyDtbYCahmewjMcV2hUf”Œ"1HKXyrXLJAbFL15F6ufZBCQBghjbZtgp4M”Œ"3FDrxFyv1MtkABw5rdLGFFhntvxvMWZj2q”Œ"16DLNXMZk9bYGfgayKZKNQneGcBJBGFMPm”Œ"1ERq6gSk6i9La3MjZAKTCnZXLmHUZ4btYS”Œ"1Ey1ZXE13rrWEeX9YpsQ6oEHAoMu8ccFRq”Œ"12vYhxtmK7vtSXLhVCGWZaaMFaSpG3Cgd9”Œ"1J7mhbWwjwQDKucZHJyqGCSsJ6mcmwhxUw”Œ"1LgEGpPdFH1NDzSXEBEZJw7a4ybgFFhi6s”Œ"3HYJQjrvDX98fQR39JJRpa2JmLwAfWaqRw”Œ"3JmdHRuTDuFTqxk7bFazY6GooNgwohZQwa”Œ"3HQApLsGSuzcpLVUAiEBgkH54Q7KkDkccu”Œ"1F7BSYV4pfWXrEUZrrtytz5dFuqg28dzFZ”Œ"1FUazbxA4hcLisgYbysQnAzPfRgAmFkkCC”Œ"3DwsTTsDxk73r3V6CWWJTeE9dLRKo2jJof”Œ"3FJisHBiouHA4KJvxNt2RhQmRshTiMqPWL”Œ"19cjibzy4BUQEXyVMpA4LQ8dDZQAyyr7RF”Œ!19yGY7fjmyKZ4upCEtqrCnZPu9kJeMdDL”Œ"18HnhDg6KjTJX5WpBN8Z9VBsf4aU8x9pRr”Œ"357vNpkjrCJCgJuo5PFoGrP96p6PCZ7heA”Œ"14xqsGpTwriE7dZXAnG3q7ihisbpTCSJRe”Œ"1LQWSQXeHvTxQfxCbnFagVCmSYKbSX26LG”Œ"3CduKyoGVbBcuGK8SVjAsJcH9WnAXdJsDB”Œ"39k9sXRLT4w4TS9LP3Aa8qpShLR4badx2P”Œ"1DvTRUPvaKC3PVMpaSa6b24AioctoLTamF”Œ"3A8HmmuHGQ3ZpYR8Ux9uAiAGpVWpSCUvFs”Œ"3MEs1nUej5i3qzVsCp9W7ycRBVf6JmoCHJ”Œ"1NZcEBVZPorqQBE3Voksnef7uELv4ooHLE”Œ"381VgnySuuJExkquphdJCkCjXqxqRsBqLa”Œ"129Lm6DuKnNWgd5TJvbjZ9tjjVEahLfpF4”Œ"19p688BNjc9h6FfrtdSxHjCsb9Duasgnmn”Œ"1BxUsWQci8XCTCUR3BANDHtVi1CTrJkCoF”Œ!1wkHHwJyhKCqT1JA8UNbgXNpQdJ9QLRvf”Œ"1FwMPvqXhYav84K83VnUgJPJiDcMMYVzVF”Œ"1M7EAj1h7UkfuWS51FEtuPHn8kx2mjnGan”Œ"14wayXUcVeFby3Becrjv5Gf5Hcu4cVrzmV”Œ"1FzeSdfCnmKv4wZxxzsSBSmQRxEY6wRq1c”Œ"15hBWYvbNf5zzpFrFzdY2vA1Yos2KZSXVG”Œ"1GaryDxaacUkHHBxbTfHFaVKxf9qodHT5o”Œ"3CfwqbGRph14bedNxfQophp97opAzSBcRd”Œ"12GWJHj7Fv2pggUYTTTzhsdSKt71wbieig”Œ"1EyWoPNt1Ywyv3LvBCCCUFUHgsqTAdZgLR”Œ"3BWtiseGYEJJmRvr3qJ5qXNXbeWTkZc7Bj”Œ"18gGDpcqfRjSqsLTyUUxjtN8Au5aDDAajm”Œ"1CTNFSZtdge4KTxfaoU7hiuveP9frdP2jz”Œ"1GMuXdqdr6NxBjrbwbXrM9LCgdkHzb8i3Q”Œ"17TGLyfy5bAN7yVDPxhLypEPJg1SgQHobv”Œ"12tYSANNqeyEE2CmUX4aUAxQWYGKpJPAcT”Œ"3DqN3oBsRkTWPcdB3cCrvmHYboJAzbQCy1”Œ"1GqX39eb9b6H2uNytuhDCyqwikkL4z8zDB”Œ"3GSKdQjNtpCy8CFgR12XSH9CaYm5qbZGPj”Œ"3Lniav6LvuA119d3eXP2FvArY3NbFRrjis”Œ"1BNEXGDC1yuKi77gJBC32cVvdyK7GNUnMF”Œ"1HJoSdBRJNW2A8d73U8vtA9WQpeAZqvgwK”Œ"13RzjuhahhVMCdf2TZb2nS1LQHiP5kCs5u”Œ"1JXQBy26GdkF3WeogEHn2aoT2g57MmtLR8”Œ"1JT6ZEwBkr66kcvTQZNxqMqPnfntJCQFeP”Œ"19KqtuFmqspLc7jkujP1yWqE782ut8NreJ”Œ"3Ea2YUibydrQjiVwTrF9jHe4ZyLYqL8vwU”Œ"18SZJ41SPwJKUkJkS7fQG84qajXjHSUdkm”Œ"14GUvNNaLpzbBscrDPtnZzDNjUZBc7qiVm”Œ"16kT9mbBJMDnTotDHutfJh5B2XUQdKeYVd”Œ"1KfcTcUd19aJ9pgcfbhgDdJjUK5FaomPdi”Œ"1PeRPqjrdstC7QHqoievvsm5L4NGWXvtPz”Œ"1PAve1jm56Yup5Qqa3xoipADKDYTvSiTz5”Œ"1QG1cjyuwFwF3mPNK3QXcAgncocpdycrmi”Œ"1Pe5wYirAVRXmYJ3aL8PMxoK1kyed1Ucv1”Œ"1QKeAwjQHgxepWyLWwrEwLZXer13EXXp8w”Œ"1Kfan6Go7GmFDptbf2mPfbzc8w2yhj5Kcz”Œ"1CDDYFAiiAB73ofGUGZppPmgEaFb2vr4QX”Œ"18kACC9YyqJeDRQ3UMFD4jLiu4zVowh4ag”Œ"3GGNPqfT2Z53ziLevw4aQxs2fvbAhXW3gz”Œ"3E4jUttXLzsw2ry75PrbkgmhRkqMFLMk1s”Œ"12M1JWqGbGT8tt4jhjD5tAHVJFDSTwKczc”Œ"35UJSeWEUCeRhTc49MbQq7h2sYdS9pE3PG”Œ"1K2R6TqA84dUs3HgfpGTPqyGKUn8eAJuhQ”Œ"19EZoWGdH2Kmvn44FXJmJJMaLj5pjp7PEv”Œ"3MFb3tSioddJ8VnVj1Xw4BhZ1MRWWXb2j9”Œ"1Htcf45tm7mMn6ciq3PNyK5HgMeiHJbZC4”Œ"18XwBmRa6o8ciMa7WXJBgSFyZY7dsWgPfL”Œ"16hJXYWfGfTXwvv2u68qnEoYmifQy1Ckmx”Œ"3FY5E4vAybEknWQSWZef18L1f3Uf6MPy8G”Œ"1AkArpZK6cAssGocRwvYzF2YXiphKoKsqW”Œ"3Fw4syd82duCV3TsCVmUkDdLoeQqeyH3MC”Œ"1K1probTpnFEtcGWoaAvowxvXPXQ9j29hx”Œ"1GneHFmt3VarkVxRjML5uxQLcXAn1HR2Xb”Œ"1JiEGZ47QWzovSFiZHaaUmWWBbquCtDRaw”Œ"3E5AzATWMiaNSzkS7ThVGtb7hrYd6HG4Qm”Œ"1Hdo8Yn75rw6hR2tDUvfqbfcyQBNNNzXsj”Œ"16rAxGvpMa99n6Egnfvwk2APUA4m95Je5J”Œ"3KgrspKZyQUgqDa8Tck34HhumaF4A5xjP5”Œ"15LWuRVTAT1WRUL1dWbugLwEbrybwV5hp5”Œ"3368Pu5KKm7UqqiCQMSSP5GKSLwgvFh9bi”Œ"1AGxVWjyxrTGcDFNn5SQ2mHcALPVrQvPS9”Œ"1Jb8kztW9Q83KN2SSWJYj6sBo9uYU3zyn8”Œ"1Gdi5Em6DdRQ1SuLhckkepW6F7oxjEkfdg”Œ"1KYdCx2TB6WDX1tYxSThfFksFM1C8KzGpr”Œ"3Ks62y2fxGgjg4iJgWbVh7L1z1KReL7WDi”Œ"1MZKCLK8ZeJjuiZfmhmTLZk4SNSKDxMeBD”Œ"1FmUi67Vj4bZQSK4yQEfgv6eFtzeSryU1k”Œ"1Eqm37qKr1You7z57ei9DYLPATnmSmMK7o”Œ"1PvNoVbbczFk4JRWT38geN15zzkmT9Mwmy”Œ"1HYp6NfTHAc4kiEPuD4gfqARARQiyfhdC3”Œ"1DPMtdkcjVM4xWBfdFx54Mrmz8jMUPAxv4”Œ"1JZhJf7NsLoFUrKTY8a5WDy4RJq81LshhB”Œ"1J8tZqyXUJmQC9RyeBPTXcQVc4nLLVL55n”Œ!1VTCecbJLLeXXfCMJ2SfRQ6sphwNFqEWr”Œ"3KZgdEeXiebvzBZq9ZrfoT8M8vs5D9BXJA”Œ"1JQZsKWo4CMLu5ukzPxaUVfQNVQF1oMh8W”Œ"1QFcWuqLqEzTGZDASUDrTYkRmeQpttbXh5”Œ"18BU4TeXzjEkaLg4m9ufdvtzkdbC9Y1Vtq”Œ"12zdF1mN3Fpze575nUiCZh8G6MRf69RLzh”Œ"1J57n3QHTvSeJERjHpEoJMfRcKemdYayjn”Œ"1LZEpbTiVaMJRbH4QC2RfwMPLWvtrhzKtJ”Œ"19g1fBWoPJe7emiBd5B32J5JxT7M36DQrQ”Œ"32GHqkCEsm8wQvUP8JUGkAkkGqxju7XX6F”Œ"13fcdK8sdEeSDTCWRMVtJMqk5m6aZoC7De”Œ"379ek6Hc9CpR5KB1Nc5HqH7W9s9HYrVdKn”Œ"15MXSAP5vjRhdgow3HPuu3CbEANxKwXSs8”Œ"1MMQsz8p2Ehqq5HCZBmTLrXTB2X6zhC9Ls”Œ"19reDZcXVVTUNaGiMHPwcwQ53FfoB1HMCk”Œ"16PdWxCbNLBuFFANEVE73LD4bD2sioRb9q”Œ"1EssjtxE9Z5oRw1H7nnbUqYaph5ZUWQnbd”Œ"1AVKgZ8d2uyBpnHRzfA2CK6tPzbwJRBKvL”Œ"1H4FKHwFDQGh2YsydsdJL8Um6QWSgVZATX”Œ"32N5ryMKUeDoh46m8zsv6W1JsnM9bFjjK3”Œ"32y12aHPWQtqr8S6Ey4QPVYHdih1t38R35”Œ"186VumaToHxqhKF74p1mncHBdjQtm3j5Yx”Œ"3LgWsNgiLgrtTQeiu3snATonRXxB7ADKPx”Œ"1QBS3W11dputu1qJnsxxSUyfiCMPhWZyqT”Œ"1FArNi6QpYgbXwMy7ymgWCzN3CeTJj6bQ4”Œ"137i9LiH32DAkwQ1posXKpXZRn6hxs5kex”Œ"3L57aghdVH3aWeqS8r4dFm89sc9RSgxfZZ”Œ"1EzP1jdQ78KszeYypYeyQsC2MNvzCgTqfd”Œ"1EQP2b9KNbq6A3FKC5QBjtTW1mH1boqopw”Œ"13TUR1UsKADB8F9smjZiYBJshXAotoVQRt”Œ"19iKuZgtt4XjUWrqj2Et8ahdz5Zw5VddrC”Œ"1MiC42pvwdqiqQCD2FpihVJuJHxxCCKnt5”Œ!1iDsAgAoesBnouGWQjKh5tLAU4BcQQdQD”Œ"3P2tXKiiegdHHAMBdKfJwUCfYaD6udLpd4”Œ"35HRFLLHWKu2KbYtUxNkxfQKgcoCQynHXN”Œ"3JM6T32Aon1PSXPq7PxQuKsp7h1DLqGwom”Œ"18YJxvZR9AEciGKVmBkZY25eMcnGLeZxjw”Œ"1M8wnU986EVdCyRJPLsmv1ka7sYRQcRt1h”Œ"1FXnUwFCeD5rLbzhuxmjSPYXKV7XrGZeEw”Œ"33Rm7zz1GpN7RxgiP7pZDTnbedavTZ6uN3”Œ"1MpRGCoBehJtPW92tQKXDX23trdYpjH2zi”Œ"3GDcETNZYybfWNozWrGkfGny5rPwwiAm9y”Œ"35akEUyPwrieuJzv7NbBK2c4D5HSWaBECA”Œ"1KB3KuSSqdyoDnQR3sx4VGUrKhpQqasF4a”Œ"1MNnggWWnQD7Ndjv54wePUM8pFDDpaJ3Gx”Œ"1BpwuJcBWKYcCHdy8HzYMBLtKCSGYzxo3m”Œ"1LvNGh1j7Ri7Q8s22J4gBwd9VF38cDefm5”Œ"1AT1Z5WJiPGFLrYQ4dQB3ypCuFEKTuH2bD”Œ"1NrVGuPWJMorWpmkavZYY5S3M6VHdSmFuE”Œ"1MspJL4XBELgcKPsUJJ44KLTg6kxevCeND”Œ"1NLSpz7LmhnCYErHPqGHXSJjNKGFJN5i5k”Œ"3GYY3jAoESTjjwNjActqjo2razNcZFTzNQ”Œ"1KiGhdhqdrRUUphwzy3m1PKuP3VyuqtVrA”Œ"1J1on73Qr6wH5XqSkFaYpByf4yQfSWqJLH”Œ"3BgeZNmzFGA2YpZP9WTnitdB4VySYEFrgq”Œ"1Pe3LpRSR8uyxUUPsWMNrEnychDE67Xt9t”Œ"15etye4D5Zbk7rmzJnyDEFvaCrPHmpbZkp”Œ"18QsSzmrYvrbitvu3ycqWpqKkBLhWJii23”Œ"16K4AAYVMmhp1EHg5sre5KQXeLovb2mhjB”Œ"1PXc64fyMUzSDAxSTzmomHBWMN6RwmbX9z”Œ"16xigYphLrDpeJg5L6o3npSDdw4BhNo87Q”Œ"18WX29cenfZtF7Rx6xPGqoE5YFV7uwPk3M”Œ"19KpFFny6e2eG6KR7YDtZqeX3kLsZLXthv”Œ"1Hqv634RfziRv2imWWbUMcjcdL3iQHoTXp”Œ"12fbCHMDPRepEahvCiPmD9YXnWBDPHnKpd”Œ"17q4KqKiXt4usSL8ycwuETpSLm9x9444VB”Œ"34Bd4W3uVZZcWqx9qm7MKyxnHErERvxPUZ”Œ"1A51Q3MHmUu1heWtx77fqSbfXt4vyjAGMP”Œ"1KqwFYVXdg1rQfCw1ztMUBcFNdcChxvTpR”Œ"14VoCpeTcAp63kQUFeB84aT62i81ZEy4hY”Œ"1Lh5AGfhEoDdWVT7m4cEJBxEXpcM6A7VC3”Œ"1AZtG5pc23XQUCsmcPanZM3BfrqnSvmGoy”Œ"1D8F53D6VmA1zJM8wYf56zA25ca1CQHrub”Œ"1C7WQiSt1fL7EUpVoWaS1KiXyMAEf1KZDz”Œ"1JvFZMP1uXe8CSFiokmTxKebabUDYw7RHG”Œ"3FXp4PCPf4LgiQ5WRsngLpAvkeazUSg2dU”Œ"13V9abpt4X6N8MmGwuHXJUSNqHVcbEagGr”Œ"1DHM2WQTMf55AzpYzYdQ6R6B6FmhkcTBaC”Œ!1XS7uUtWAcbwiWwQZmTqnfmXg546uZHEQ”Œ"3FJpAdNKy3QcXxAWyanXe1gU6WjsLuAdz8”Œ"3NQLcbAZkZNx9JinQvmhd6pz9NzMGv3bxk”Œ"3QijF8LY636tnwKKAgNCh6T71Gvqu3ctkr”Œ"14FtscKNHjsAv5eLAPwAT1wZkHRGZn7wzz”Œ"18LGiSwGpWQVMU2G2ddp9rEWnCGWegT1UF”Œ"15bExiratGLS1S3hSPBmTRe99QpPnMAPbL”Œ"1CMLEdGThc1f6CS3FfX8SXsXECXTdkYFYe”Œ"1EMqygJa9XDDgU9WnV4QQpefN46SeqLiFy”Œ"1NKhxEL1Vs6uJJ3goJ74FZE9jtE2SoTHnQ”Œ"1MFZZV6sx4X4G67emYX177V9u8V9oxR9fd”Œ"1Djr99RzJjVHWqKAvRxc78qJbwuTPvs5Ze”Œ"1HaPE73W1yoRsrvvHtHzS3qDJQ9XRCz5Cq”Œ"1PfwxNbK13CmgNGu2oFqEnftJ14BkBwEQN”Œ"1BTuDCSkx8HAXiit1bdLQiqVqAyUwTDgPn”Œ"35d3cM6rdUxcp1sAPpGoqKuedBDYfoVpnj”Œ"3AEYg7yxdvu7EKcvt6WBuMviyFiWEnCNMG”Œ"18zCp91788TRjRrnNdaXRojRN2iDWbSEQi”Œ"1D174sN51kWcSS69kMv8HN43NdBGisDGc9”Œ"1LGE1eeUpnZX7coQ3iw7CPzthFE2U5QxXb”Œ"1MYn7wxcCvBcX8dGqipwnrU68RZ82txpqP”Œ"1JbQuhu8mnx6SCZeFpEKPMo4SNe3wDwhyB”Œ"1GvawELC5tFSgjNW7nDX4cJEP42b61d7LQ”Œ"16KVrVGv8zQKeFwi4bonGRySK34AFrFc9x”Œ"1LdzAH1VJCGi2wMtz9c1QYd5c2eFy4NE1M”Œ"3DvLyKMJNzSzCKHos8ff7m7w9ScEfwfqgD”Œ"1GPH3hKrtxb1yFZUQsfvceASWfLuoSuLE8”Œ"19aHqBhQauKTfhhT4cSXSMstqhSrLKz6mh”Œ"3AFCfdH5wNqCxXKPgVXdU7VmxWyqsjVQFf”Œ"15Y8duh3UeGjkmkakqrRbUT7P9RScjPbVy”Œ"1BxzendPMJ5oCTX39TLLkUdtNHEqLucsXu”Œ"1LdJRqCHbS8aqpfPy6tNUky7QezThrxatN”Œ"1AaAupsZazwmfft9NXrLKaPrcEfpkgeE5z”Œ"19m3RoMP3kz96Y9oLoSL1A1rmajrjk5TEq”Œ"1K9sGP6JUYcuuZVVi5j3Ygc6fzg3tZ96o7”Œ"3DnEKCUPjD4egtT5ihbuvE3Zrnjw5jaFss”Œ"3PxZcbTwedQKjL2qXGP8oUg9gNXvDrFhuA”Œ"1FNj6ciu5h6hBDwJJrFaUDG6bgP7h8h6av”Œ"1H6ZZpRmMnrw8ytepV3BYwMjYYnEkWDqVP”Œ"1BJGgwNXGKMybhttEbNt7ySWdC5HXXiseS”Œ"1GwVGfHRi6Vh7vvTNUGfuE4wkmXQyHujo6”Œ"18HCqwdnFuAcb5biGgqfczZ3biAjtZzDkG”Œ"3FPUFN4mrdF5SyVK87vQUmkV1Ywg739WdY”Œ"18XspMr22wkZYHUSe4PFEczFDpVg2VwkXQ”Œ"1EAR93quMfF2gxwM2ngnBwhvc2Aduu6upr”Œ"15uVHu3NtxFCHGTXvXiTKPcdRvWxtkkwsY”Œ"1LCCYBtAWzMgqZ8oDf1nyoTtWMpwBmciSz”Œ"1Hyh5Fk46NQq9uW2sAgkcbEmsGErqzJF4R”Œ"1NaEzJa4pqN37k6tSTrXcUVPoDsXMzTFq8”Œ!1zH1LASHGFxfgY5qqZ4yzoGxt32T9AR58”Œ"1F6tHgrJocrxPUpDLGqjERoM3i2nBDCYUj”Œ"1HrC3BvpMCVsQq2skJwgzYEmPGShB2x6w2”Œ"1NP2rP4wLULASmfgxRF2XnBckr88WU5smA”Œ"3Epi5Li5YdZk5vsP4hozzifqhXHdkWRFr1”Œ"1Na3qwgZ2aYxA2413LhMj6aGkb7wPeRhza”Œ"1KJvAdZpNqJ6W8Gno8V9VyFQ413aw2DoEd”Œ"3BkU2LoddFqggqbB2cbFt9EdGmLjqkzaRX”Œ"1EewsAi1UmEiXHdVF5omnmJByiFZJJ4S3v”Œ"14zaovwEmvBHyR9vLH4ZWuyo4vUcT5cUcr”Œ"14mav4hRFHFtgndMu27UBHEo7CGbVWp7cj”Œ"1AefSJWTusjSbTEBzrthKGKzm1yEeuwSyR”Œ"1BGQ5Nppxxyt6q39HoDfRBV29yjVm9pRvG”Œ"1Ea3XdHp6pSJSfPZEEn7BJCbNtYbsfyU8m”Œ"1B9LL9RpntcAyPkNBzCWYWdWMauV9mUrYR”Œ"3Fjg1ZXTndeJi4SFBE7LHefmqXT11ucDVU”Œ"17FyzJ5PytQtgdHumpxawdRqX7VANR5EwY”Œ"19zSZPrY3VqDxq5qLrRZu5bytUyreptYbw”Œ"1DBoztiFjW39L6medN2bjNzZSjkXkjkpwo”Œ"1FdFpHwkhq1huysW9zcdZk3vFZAVxjuoH8”Œ"327LPw5QhNvrkEm1uXUhBSdd8rzA6KhW8F”Œ"36cq4FKJRtEvoaZJsxUnT9v9FRUHKexCb7”Œ"37YmQ38kSRYKjeiaEgroEo4b4hMvJfJiBM”Œ"1AXi4eNp58QfMoHhqqyoanMUUwnTCCmxNw”Œ"1DvQ6NqqFKYiqLU7DL9HaD7C5NtFHQRhe5”Œ"1ECM2yA7CmHx1Q9aBqDpQUMiAWZNvitS9Q”Œ"19K9LxNiBmsx3j51HAhpKFMRYH3fCLYKwS”Œ"12B96vjP5wD3iJU2nsSYVSUPcqtrtmokS7”Œ"1EQxGVenyck2e6Wo1MhFNudiGQjPrmJJT2”Œ"123WEmFKUHaeUPjjhh1JsZeUbj4xaDJaX4”Œ"1JqK4bqcexgCmuqmW4hGf6WYdGjoAieD9c”Œ"33i9bvsirdeXUVQgLw7Cd3v3Mxhwoa1BtP”Œ"1PdyGRVqZmawT9W8Cn374sXNwyKGTfXR35”Œ"1NoRW5ZgbzoCtBTN9QrV7NPzBumis1qmJn”Œ"3E3Md4dZmBgLqccyEBkYGyrRmA3qpJyoiV”Œ"1LZEWfLV5NBUex4vJujxmX1KJBa9sgCVba”Œ"37zfKL5EqsokYztEj6yrJxeJah52CgJixG”Œ"1AnRgsbGtfBbjCtAAgtr38vZvDBLdCqaCR”Œ"114VRSr2HDdTuJJ2Krdf4ezcSzysHsg5rz”Œ"3G5wDYB4E5tWn7Jic5YWE1aMfUFdSXN9QM”Œ"3Lv58VT5T9jpBP7mAabSuuYNYWzFt95ZYZ”Œ"1HAUver9on7bhUihUdBqXLRyCExUk1XBak”Œ"1JBmWD8F77iEQ7b7txTr7uiCJhvZap8R7v”Œ"1NeaEKUxkbUUupCV5SCfF41wq3Eh2My3qh”Œ"191b7PJHaw74opgiMp882ARnvPtYejhL7g”Œ"1FG6JNj2U1M9yrMCRRMYGTWwr6CpHVas2G”Œ"1Q7re1rqggzjAXTrpzCLH3LHDDKD5wH29S”Œ"1PnLdEdP4DcyDKLfZzD5LXNx8GHvCj2Sdh”Œ"3NcgvA5LKg7G4FP4r719E9U6wCCGFVq4T2”Œ"3Q7NB5WxbcscFZdbW6z7PkxRKN9wjnYnMQ”Œ"159wobWHF6uAy5BngmLDYDPVzcA4b87pCH”Œ"1GhWnRjyhaHp7cz7tpUHqQDs9pJ8Dv2YNe”Œ"31tZHLgfQyCuCxGBZXaPo8Hqirq3m3w92g”Œ"13Tso2f53TDpWZH3g3y9ukkCqdBrcy6x5S”Œ"17y7WvprFSSynXEVsxovXEbMFpJS7871a1”Œ"1LZaWcU9ECyoFUT3CYpquGFSmLU3Jy7cr5”Œ"3Fc9pbWbbTKhtntqHRuEMemmPv7b32s73z”Œ"1L8Yb4ftgVmDjMHwcpAJAm3T4EKW2oaKGv”Œ"1JwEKZ3wmbnvza4J5WSJdutsL7gkRWn8md”Œ"1KshiGuRB6dJkxyPpjr7bvvuQuT46GQ8mu”Œ"1E8VfYP1kd1TCqJVFxtBP5bQPGK43v31BW”Œ"1HDYituHiqMkLC7vCng3Uwxk33bqyfyRRp”Œ"1PbCAG91kds63i2Rk6osFgQb2tsvdXGKpC”Œ"3KbubWMbHVZ4qrMCZDcKpwdXyzQxkfwqhV”Œ"18dFkCHhGpFTFfzmGnM4c2yPAKaqbLJwid”Œ"1LPcEbsWWLPq2b28yL2biYbZR9DyBhenEq”Œ"1QGLs9oMhspaxphiZXYJDZTwPYdUyzvcHU”Œ"3MgR76MP8d4MqxhrzASHoSnn5iLRb34A7t”Œ"1Pk8eivuiQxCrT5Has5iH65MBdXgAyF51G”Œ"3JhoM3bLGiwTjLJ5BBBnRxcCZNZrh6uBMs”Œ"122oNA84wNPkNUNNXjJ3LtBGtRJXnhigoj”Œ"1E95NaEAMm4JiLxi7mcwKUY2MbkGudqve8”Œ"3Ek6PC8eeieRck4yvWi4GxDQPL4oRtsBjk”Œ"12zmPuqpFV64cMaTfb5XRq9BPd8v3w8mgA”Œ"17aCtkUoXZcsdTLoe77NsnYLCY1VXUtztd”Œ"32eDHtiQdAF6XLAHuf2YbgGqZi4HKg7hsC”Œ"19rbPC36KLtdNcH2uHu5p1rkjTk5odDP43”Œ"3CMU8w6Eu8D3uGa48PapTCMwqCYQwLXQg9”Œ"19icjC43V4X87hx9C5xW3vtxM9WvFhqQmW”Œ"3P25yVGHcd3WQpkeofFJ9JgrMKMbZMH2bT”Œ"1EVYjU4NAbXGti58VFALPadUsa6vupYAoi”Œ"13QpzYFNzHFNvtcvTX3tpjKK4dX8PsGcW6”Œ"1NBmDReEzxYAFxs4zK1KsF6heyZ1Mk7bxn”Œ"1DDJtMw1vmUtrgTo93gfJ4gqqBLREeqxQH”Œ"1PDfKfFTWRj2zQMo63HpgmXs1GYbqLWCHJ”Œ"3B9WzSegQzXoBYyUYgWybjKczUe4hknZ6k”Œ"3CYfJFSyZSmzJekyoHHYMHKQnFfyB7HRNZ”Œ"167WjpKrbjBEPGQgLM5KAHGQt6mE912trK”Œ"1CrsqqATCce6dSgfm22HgHdbh8P8QjAtQ2”Œ"1NYeEX47Ntnnmvw3K8RS2krE4B5nHFZr9k”Œ"1A6NoAXS62fHH2KAKK6B7gZzoRxyrjDqPA”Œ"19MCFyVmyEhFjYNS8aKJT454jm4YZQjbqm”Œ"1DM6b2dv478zVGdW4QowgYdR4nYHQjZ7nR”Œ"336Ka4riKSN4ZaNGNW7hBZ6Cu2NHxUEKoj”Œ"338RCCMBRN8aqcr26F6znfkVQtwDC8ANo5”Œ"3Dcm3y3VdLb5Mc2Qhn7KcAkhBDs1BrYpmw”Œ"1DoRcYJj4hbpLkn4YNreHscVGhhHv1B5xu”Œ"17sg6xBVNmq9h3Lmw6xR9sniyrwZic77oA”Œ"33W357APamsYFysy9tk8F5FsgbmyX8WyNu”Œ"14tCmx9HZswNDwcHujXyVkwfjePGbLPsYT”Œ"1B1XPo2hedK5arqGy2yraK8N3Gn9CMn8NQ”Œ"17VqtfiFmMFdK74fNgvCaBebqunNkjs4nq”Œ"17zGB1cddcG6ioCbqNpdYvXyRZN4vpsvKo”Œ"18QzjbSVrQm1wPPwtK2SpEaV9uRr9v6E4g”Œ"1BNy8nGCCxmjvjZmdqhfHrBk5iVsPrSfa3”Œ"12Ub7aV8RfERU7nJsRsGLg3gx9RJVG2pAt”Œ!1TjzsvPK29XrvPsPjeKw9H3vVjLDSduM6”Œ"3L1tkJ2H3SX9DBzSNEe1mgHvxExoHb9HgH”Œ"1DM7GBRxQjuY8SgnxzPKx2A2qnQQjEnvNo”Œ"1JfcLtJPf9GLCMJtWHEWEQe4bYf4NK6tQX”Œ"12Xr5gDQJws7tRxawf2C9Uc7ZtsbSrrq2G”Œ"1C5V2RCr1PiW1jGp1EGSZ3v83F7eAATJoN”Œ"1E94ZGLKdY4h3rNVJyxYztQMyGdwDaewcX”Œ"3ELXRUeefncZCtyEBdQPN1qJNq8wgptf8t”Œ"1CsZUNWR8pCGX6adA7kedT38HjpiNTRR5P”Œ"1DpCCziWbRf8PHTJhiXiC9TbzCeStcjAER”Œ"32MWEVrKrJUsrLUUwD27w3qNysja4F49bu”Œ"1Lpua75bYTfeVUaEYDEkzY1vxwaVFvUNNr”Œ"1ETUHogT3HoMDS5uk6eeSEsLPFLV8ye1an”Œ"1BLcqris8oKtybHVkwb5JtvxCf3SYAN2h6”Œ!1x46uVs6NQFnN8yUGpg1A8aVVdnmcVUKH”Œ"19iDAYVySFpXktDtj8pUaQ3FqwtSY2k9d2”Œ"1KEyvenDoR2pPGsioybSZhcMKHKvi7JsMk”Œ"3HGvyUto1fH4yFcMyDbYAaxSaFhM5B9fv1”Œ"1NtdWQ5xL69a61bubjigyVBL5grWVMCRMt”Œ"1FwEyX2Lq7wvXcosYSgbPzVkWxXdDY7pwV”Œ"18p9Ftp3m4435tdpZTvoBsm3yjUgkvTF2b”Œ"19JHxSiLnCWFCys9t72f5WQkW1Uxz8cumi”Œ"1GG7Jjag6qv4D6oxnPAR5sNn2AjU5kTJMh”Œ"125D3Rj9VtpStd4hBJa2bu7o7EXxwhGg9v”Œ"131xAfXH44dJiSLXm2LQsRhfF4xQKyu9GX”Œ"3Jjfrry3kj71v3mi3yybrNFHKTQkmBfZPx”Œ"1NBXkTW5k5EtTsntypCa1muXMPGfoH6nBA”Œ"1BbChv3njDATxSW8tBAoyFpSX6VXsgn7oo”Œ"16nWGj18QSGVS21BkCjw5UYwrSHiiPhLog”Œ"1JPQgFC8ydb5ksfb9vRRB2yfztQ2jejoVH”Œ"1JjvtKsuQZbFZTyw1RL5abzUiETsJbxig3”Œ"1CzD5398f4vhsvfn98ZGQ4anzAkefsc4wm”Œ"18uTuSCiZdzKMtTX1CGC3YHwYHrRzvQCh9”Œ"1Nx29bSWw3ZhcKVnffqbfp2WoFaJrwaWuk”Œ"32e2e9CFVLuZGmtb8p5kNjW2qANSKiCJnh”Œ"1EyecpHnBTnAUNstHnqb6gE3bW5PGjzkYQ”Œ"1HtYFdGcN2upBcHgQHWPNWnvFYTNeFS5qn”Œ"18hvvhZnw9W7dkqkAnSaRFk583JAtJcf7p”Œ"1JWuF4aeKhhhYYUi6xbHUUvAdztw8KKCgw”Œ"31siR2nhqm19TkvBLVn4vFyXgteh6CqZkT”Œ"37f1DWoAW2CPgwjZmL8bGsgVH7yD2L8hPg”Œ"1GSXQvvXD9CY5KFh1nH7ZTTAbH6EVJxJRa”Œ"1CuA8Y8XQy6JP7vsVpecokcUfdNWFqyrXz”Œ"1MxXFfcyy2FqkUBxZSMgSJkmWtzPhVd1dX”Œ"1MYHbRKPUUbwzEY35o63szmK3JDqj5wvNC”Œ"37833e8C1maomCxsXpUNPLkcHeDsyFfe2o”Œ"162xeophpqw9PefvBN3VGisvUERCZt4Tih”Œ"1Cc3dQJ7bvRrRkHNogFsRyZrc7iWS6vt8M”Œ"1EiTPxzhJLYae9KgnEXQS98aS8yBnb9zz2”Œ"32AMMTNPXfjvAy5B5Mee7V5FyF7V84XUom”Œ!1HYXiu4moYspyo2peEjDty7NUMpyYJR4u”Œ"1EqRc7pMJptC4nqc5u6e1hsoskkByhnD7U”Œ"18uMiWu7LLWNkwhLrkhzu4ov6u2HcZuWNR”Œ"1H1x2gEKD44HKhY27r2EbWt7XvgqNYZFMf”Œ"1HrqG7VZhCHrqUx3R4P93bQNQS9PKs6hZh”Œ"365sCd7jouTZtgzvyjzNFex4cWXPHx6fX2”Œ"1C8YisxVB3YMZmx8fX8BW6wCULg2RUAfxw”Œ"15Yai3rhSmCu1rDVnkkdoWWmgHdXELxB9x”Œ"12szsiHucum82n4knbnPweiLCcdPUP4rer”Œ"19JWeRnkcXDJK17FNRvUCZyh7Nb3Y3sk9T”Œ"3GZUecUdEgYUjpe2KZcUKUSf9PcvbspK8P”Œ"145A2Boaf7yHRq3xgvSb7oDYmZvDGWfX14”Œ"3HSZ8NHkzfkbFaHa8TrQJvZmjcsQ8dvggn”Œ"1PHMCEF8a4hS4BcUg3P1tupW9ShoZkmUFJ”Œ"1JHiFHg8oLpQd4e71SJYXdXPCTGxA9KQ1t”Œ"16EM9cEkuFCWvxgWvpuKoYeRBVjZ4FJF4N”Œ"1MeK9hdeepHbcif6ayGxDMkGQQ4rYg6MQc”Œ"1Put1epHahzbWoCBbs62oCjzWwZSMbwQYF”Œ"1CH72r68yGBgKi3THpD5x4WvPHk1pr1i7c”Œ"37njgbvsMdr3r2ksPN4DRMSgbygkspZVdQ”Œ"3QBrnXm1h27pX9LpXZMTX3kpRP1pTa8rxW”(Œ"1Eo4uqSDjEFRpiwq5uGY1RADu2JuuF5E6Q”Œ"12vMA6EeTu3b2nsSrgtcaMebEqKDdj14G3”Œ"3HNJda2ijNwYA3vZvunshxtRKaJ6UozYQp”Œ"12TPLPoJLUqsyuMmpeEcNsmnyqE93PwFSL”Œ"1PqcKAEURJpBzDRhDN898ih9Z8zycC2aa1”Œ"1GeNfX39U2WzbUzWodmtJyuZLNEesH7fWZ”Œ"1HNZdFAS42Njie3PP7hCT5hrsvbVufwx4F”Œ"1HmvMr7iqHXwYsrHr7Zis7turDExjpE7oX”Œ"3MYi3U2ztNLZphwTtAtZhwxH6ri92QrXbC”Œ"16YvB7vE9cdivqTt2WgJk7LXrvPGWuZ8wV”Œ"16hv4SkKq3D8Q65KfN1scbPA5VRazx9Vw9”Œ"18nQFGc8AJFChLb2kHmXBfwmbZWXKdLoyv”Œ"193QEqsV6fpBqczJgNgU4YMwEtWmeeTMsj”Œ"1Bf5B16mjKJS1XT9wr9WyKSWwMq4XhyvrG”Œ"1JAYLLUZaSY2QfvR6NsiBt4suf7GFAFoNH”Œ"3He1a5b6BRiMm5FumauuDQivJqwwh2WPMh”Œ"1A9zYTgk1VA2swSYpKnT7yLhmmFvdyzZZF”Œ"1KXVr36PPKFckQy9vfipvm132uRTNFRPg8”Œ"1NmnP9NqXHjdbmbCwpXBrLEzBTQL7mgSXf”Œ"1AvDwALNKUo9ZMgyyZSUVaAuiJTBShMsX6”Œ"1B8AC2acXL47wK6T5v92EhAoEG7FpSwn4z”Œ"1HBmcnzQ4WwgxfGsNkVhMN7QkL9Ce46Zxa”Œ"1C66hZKLR5WbFFCKtq9juEZLqdrRgYxGMh”Œ"1AzijypwLGCfRhN1AgMbbM5sW3YZBDbFDZ”Œ"1KjJo2nU85bvP1kaRR6Uaww8yNB2wLn8tL”Œ"15tzCz2B9niW7nwj7LLMea9ptEmwgCp8St”Œ"1JUS1azqaAmwNRWX7h6c6QtATnb16XZEos”Œ"132mRBknhoYpV2FUw9j9V7KoRxEb43QAVa”Œ"1HCKz3qEofoDvMJbxLVQJwh5VtV26sUvLX”Œ"37ojzvb4Ep4xx6ZsPCnfykASEnAp7osonS”Œ"15jDAsiDyhkfFLGBjtp1oj5s2LJvTcFFGa”Œ"16vcTw2EWWep9TaReZZ2RJ8xWF9YEB94pP”Œ"18Z7UKMDsjgZVUtxfwJEWwp1SpYagho8Ec”Œ"15iPS6PYmujA8R9EhPsWDa3WALSKGLHM2y”Œ"1Cb3svdCPZERjTXjXMPfv3yYbmUYtC2A4N”Œ"1F3Vs8PcutX5YnVfbfdyf9BriPxGLyQkty”Œ"1FtvvmZVE8tdngzFAyG1FK8Vkebj6AtpJZ”Œ"17H8jKm1XrH3vf5AYy2vp3YPgkQhnqNtjM”Œ"1Nj6NTovqB7RTEo3JRiMiJWr3HrJVNsVnh”Œ"34MNgVu3WTcM6vSgFo9aBq8GXqu31eV4oi”Œ"34ZzYw5xB8JTFcECJrFo12sCEGK9St11bU”Œ"39N4Bn34NnQ1aeWQtmZEi17fWdgEs7bhHg”Œ"1NLtxM84fdPPETZnW8Yr34UjoFfM4AD58p”Œ"1Dj7pgHTEzufovuHCdvuypWXECHStkhnjq”Œ"3HthHWBKyfdesKHSJaBgUArvNWG1TTCP4p”Œ"3Q9qc7MbhxteFVpv2TGMmeYbMJ9xvaHBPi”Œ"12d4QYfvdWLtLsc4Rm1ssSMJqXSiXPLiY6”Œ"19AVhLC9bnevmMzTZBfuEw6twVxFvC8byi”Œ"18dxb53hkM7sKY9L7TVid6X7xXxfamywHg”Œ"1ADsgdPqTYXjZS2YRcU3X1vF6kfL76XdjK”Œ"1MAZeFLD1SADpcWv9MpAoQVTAHDzqhYrKe”Œ"16xbkC786n7vCnhsM7Vk9WLaUiz9rYimE7”Œ"13U3KZzC6mGPcu7tBx9McD33CXwwciRy2x”Œ"1Lvh7PgLknSWnsoXQAPy6aZRmmrM8DMoov”Œ"12xojTD29o2d44jq5qBj2QDQds16hHmGrn”Œ"19Bc8m24s3vfFgoqMJbjst2VGxChkGa1YA”Œ"32iwgA7sZWdKooaxi44pB6ifKUdyW9VN5V”Œ"3Pwp5u7PwgrMw3gAAyLAkDKYKRrFuFkneG”Œ"1GiTRvFYXxRQSB8p48eAPvEXWTjQfpLMbP”Œ"1HvP21Ziu8KfxwHJjNvPPjBqFC4WbZq19Q”Œ"1JSmYe6ExpecrbVnL6y6BqQsdVg4Dnoydd”Œ"33DGP1oK9E8HQBzHDhNvKtRJya4MD3NiQ3”Œ"3QbS5TFHKygpWcRtXyhqupYxrYwCZj1MNr”Œ"19ifDdruHTvbHQmrejx5TTcWsz1oCk5aGT”Œ"1KbQj3cbqDpm1GqpDjyvWqwuP9XY2Xy9je”Œ"14rjLoZF7SL8GsBAUm6Wkh5PbnecvKxCNj”Œ"17wCrg8WSKNPs8767JRtK7dAmLeXfHhAeu”Œ"1B6mVqgU2DA4FYWnkHGZpjHQ7KYeQtH2fQ”Œ"1JxfcfN55FNxTdwT3afTNJd16wNqZQmmMf”Œ"1LMCL2DLndgREwGftES5iXjoLa1fStnxY4”Œ"1Cph1zHHYjYJhZZjFk1nd6MGJU68DuBZoq”Œ"198Wfv5CbCiQGhwPSf3ZaSLdLhJnKan4DU”Œ"13Kmwg7n7jAPuSMqTXTFYxZmLTRD59598c”Œ"13YRshxKeAt2VWxTCLipLU3HbA4rCTg5RK”Œ"1F6PKgL27qhE98kzD77pZiN8pNKWqm6od7”Œ"1PDmkoZe4H348XdnWPQkLmqsB5geKtqbRr”Œ"1E8ngVP9CHpeY5sAu6VEgscci1m5CpYpZg”Œ"1E1CEM3nMMX9CfC7BQNmuLhvakUx6JNAqm”Œ"1KrkddPeLoYgu78cwh9ZBWu4zAhMX49E5D”Œ!1txZrQyabxEbaHRMp9vszgA6yKBgEae6r”Œ"19oLxCQx1wAPbwxW3eEYRD2tLySNmnYPVg”Œ"17ftYDV8umdKzSMv1EWKN1g9iLaCDwYMnJ”Œ"1K6QqjzhQ5PDjMj5vyPjf59CUnKALxsF4u”Œ"16hsxxDGWjXHRnNTAdq9XLc5CJCaFUtn7z”Œ"1Giy2iABDs6BXMjzCahkJ8yU5SbqXGu4vP”Œ"1PAciXcqRhsaSnv7SfMuna2EPbzB5TkpVq”Œ"1BHbxc3T8XQ2nuJFS8njwNtPJVoVY3wJpb”Œ"17aYEDpdgq6mur9skyQfBsHmBGvB8HkKNh”Œ"33NAfZsViwpfhgTYA5SHmyRxnUL69sGrAy”Œ"1HpDsyLedqhNDM6dxB1J3tpSTNFWrA9KkP”Œ"1FvuJ9pFnvaauzSHSijrZQcQKQe9fxaFnu”Œ"1NSuEAGTuhhEyFtgdgnsayuuLRGBsx2YSb”Œ"3AXAA2dNs3M98WLGUeHzqY6UnXnmXftwp4”Œ"12abb8AjjYKQYDxqoLc2Wt4cjdXnNRtJnf”Œ"1Pm75i68P4jAQtFQK66rRpdKNJmKnbajCv”Œ"1BcK8MvpkTQ21BKTQ9EjwZCnwJnF4ukngS”Œ"1EJn2o993zie8oT8ghKCEtugRnCSnoWfCm”Œ"1GxUBzVEEU1HCMPUKWJcqs1MnMtCnokPmH”Œ"1Jxbs5Dikchcc8L76LPM4eF2QMeBgVTQUu”Œ"1K1ERAk1FdKACDevfU27tZRNX6uP57Vjmc”Œ"3ELopZjJRcRkPvNZyKXinX5Swy8VXVLM1N”Œ"1P7oVsdGN4tuo3CNDBx3nRDfVAqFsCx1Uq”Œ"171Uufz4j7M3QcVvyu7sFLSzzZA8v3YaQY”Œ"1KNkaqHV8DHg26TsBUADgzKE1vNLvStrrr”Œ"1CvcjHvXvifEfJB9JZdKnzxXombXAmqMdk”Œ"34T7ZkdpMzYdhNz4hdxaFMCAbndkZHE7i5”Œ"37o4g5Bw9MsL7uzaYUTt6NawYsot773MbK”Œ"13Ve3Bb6F5PNKeQNxUuae14Zg3pSxFMqzt”Œ"3MH6gdU5NB6WaSh3jFJ7ZEqwaBgAAEvsdn”Œ"1Lx4HANC5jrsaZrK1CnoFY4MaGnacwbFq4”Œ"3Gdk2AFyEWbgNvR8JFDiuV1uXv3SySnyiJ”Œ"1LcYJVUB7MigjvPZin33eJWg8baSVSMSNv”Œ"32o2KpTm3Bcqt9i65ixvRShjjixXwnupme”Œ"16ALB3PCPHUhe6M3jygTyPpH1kxCRZ8PhT”Œ"1D8mfZnWs5zoAp4qBSM1nZW36SDkJctN98”Œ"1K6y59vD4sXWtbFD5pLJcpFC5ibFLp93t1”Œ"15mAE8bZ3XeSZ23bcpPqFzfQ4Y8YBE69pd”Œ"1Lkw8UbfbeymfUNoobwJEceT2W5xzngF3A”Œ"1KuFeA6TMiPxGFXPPYQYRsav185hTG4W5c”Œ"1C68QsKhgLtBvW5wLX9BNmVxSpi9pxPHvK”Œ"1D1cfwYg9iPTL2uHEaPHHxGP6QyvCbmsq8”Œ"1EDhsmy4oTLheShZpMKbAcoVNMxjuNcX9w”Œ"3B4rkcZuyugmY8Q4vycfE52BJuGdSoniSJ”Œ"14ifkpwKyxaMC64DyFfJTdQhF9fHdnzYvL”Œ"1KXS8LZKEMMUGTUQxwAcPLUrwrK7kEC9cq”Œ"1CCizJhyNdRx8KZkVMHSt7yVKthGsC6UZF”Œ"16jJ2NSd71ZmR8aGgyZFrdSt2VPpia5Zw2”Œ"1KrR7mgioU2AAu5u6om98RTNZDahNj63Ev”Œ"3QCrbyUSQ3MzFMPjWAR6593nZnA2HeXSw5”Œ"1EA9uQBn9j5RrGzbAYUgP47rfcnhNsTtkS”Œ"14jCGpL7cPoyUJd4cvdWLV6PGdHod1konb”Œ!1W2KYCRzfktMCpjcJSGLmUF6Ex2JsEdCs”Œ"1P4vCoVCc5nBP6Bkg7GmzBUviL9qc4VQ7S”Œ"1Lc44SrVZSotN6pFQ4FicQMHFqAMavKupL”Œ"1Egm7v2wKK9L6KorB2GFJGS1kyEzZ28TqH”Œ"13CGyQwsc3ZP9V7Cy5kENQnUXk2o4zxKpb”Œ"1Kb7rah9vhYL9sM9hoQC6phLtgpcB4LvAg”Œ"32W2GWb1dTfnFnSw4Uqqe6qCTZbcVbXAfW”Œ"18SAs5M9GrwQ7s9GRSuh28oqsGprRUwk1A”Œ"37P9iTfADe6jkj9x1jdg8cRU4CYQtm7LHG”Œ"1CWDUWt9vKE5YiL5XKXyaD7jEMsz9ArLsv”Œ"15ZWwmTS3x9ttvYBkRLaEHBrGqnczmGon5”Œ"1AkXrqN5Y5eAsYzkm5h6cVuH76zrQD6GQm”Œ"1JSgJdtMjTihfpagqzGUE4weaQKEbs5V1J”Œ"1LurPidpSyBBgYgQMvXvW3h5DZUh4UV6D8”Œ"33xVHuVBUSwcGfqwP9nDfyCcw9aTJYHmiV”Œ"1GwusA8JijjnCmQZaVLkN2qHNpn4Z4nLvX”Œ"1NHUCZpq2pwxzjs3LxgbpmNndp74xrh2uZ”Œ"1AFCkb8afqHsma21ZEB55zJW9w3zsqY9EN”Œ"1EiLhYk7Dar92jqo7xuPSWSxCvMhPPZFnU”Œ"1GwT3h39UgFMTRMBLscVnimSFoVsGw15ok”Œ"1LBkVzxuBsgvHTY6iuqn7NLvq1DcpRGAb4”Œ"1FMVgRatrh1ZQW5xEDgdGf7mXJQLw49aeW”Œ"1DrrYDDRPTqzy7SjhQaJ6ganP7yv237wSW”Œ"1JKapT6nkuzZEq52xQHDcPnRHPcwA2H1Pe”Œ"1FYjQupJiikN8zqunJrHAsU39vnQP5U7UY”Œ"1K8stRX5EBapsFsVcCTQLkue8JZR3G6gMY”Œ"13MUhWfmDtXz6CXEp31QmTGzuaKJp7U2Dz”Œ"1Q5pgwRmPxR9Kf7bqnyVn7rHvaN5xhuEzT”Œ"1QHcxkhmmVt5Yajwwo2j9753uqSEtU3WEQ”Œ"3PTqo5GANvTivGKa53sTwjGiMiTqutQXqq”Œ"3PnY5iNKBDcT4P7FzajE8d6ssJ84EnqorZ”Œ"1L4ZCpSH5Tkcb2S7GojhymADvxnYZiajN5”Œ"1NPYNDLCyJE3C4osYWdNzgsJikXeSGhT11”Œ"18x2EH3my8i1NzCEgP3BGb9oX65RtnkKhC”Œ"1DfC3AbfBfCi251xcUuDuoSuWDRHFMMvW9”Œ"1DoWHySUD2HjMHj7WMeWY5qqjrcrvf8TxF”Œ"19TEkVh9p6GpGNnrdy5G6wxyVNMTXzu8LM”Œ"1PNfbDskDeZRyfZ3UqwwyeUsyk75rpnqDf”Œ"3DzvWmKE5Z4HaPLti16UgUVsWZ7Y86v5tB”Œ"15RsW8EMdcRHmKgeCAKFWd2Lg75rWhSk7y”Œ"3EbXGbZLZgD9XGgiUJdz9rHfTmQfLi49jW”Œ"156dQ5wD97z8M4jL1DZnLouRXKwhJGW1zJ”Œ"1BmC1MR4VV5dJHaPtkPfeEWMVEHRewxRHz”Œ"1DVZJw5E5H7x4F5ynh9KKpctpe4LdvLHms”Œ"1GrLsXV1we2HaNrmoUDxBAWdrhbvdGK5ek”Œ!1VtZtwf1hN9hU2rKrb4VASRFr5m9YBmRT”Œ"1L4tduLKNwg5NawSiXiETQChCnuFFViHqy”Œ"19PAXWzPDSJiTRud37fvKpbZx3JbEGSQBA”Œ"33ypD5GrSKsgD1RCLQVVEvdFbmJTBq3aeV”Œ"17t5ERwTHKGTHLinochkNpFB29dqm2qpai”Œ"16LtYSPsb1CWptwrWWVJvSMXcrGYL64yiM”Œ"1DA2i9KLm2dHoKaiWty2DhjxJpa9sZUYHP”Œ"3NLvaGqNfqJ88zfXhYbihDoaaKaracKCf3”Œ"14tNM872rUuMyvcmtNBTiYdeSoazSsUoZr”Œ"3L4xZN7yP8tVCWwTnw6RF5VCK6zB1wXjyg”Œ"1BE8pKRER1vMi84BNZGpxXJWvXWmDZySo9”Œ"1Fm3E1yLcwiSrbnDRQHdPgY2bREmHJPJFk”Œ"1PjLG1Z9uGh5XKUBjpjCyqorhDFBEhF7PR”Œ"1AUVA1WVJPN4rpayqiDLZJdyywHCVuyQRa”Œ"1J9Gm2BCEzPKnit5HotTKBjTyZYZ2GkVWf”Œ"1FAzjQuikMoqSQ1AfkVeePuHNNam1nApPc”Œ"1383c5hw5AytYBMYMGvMkBrd8LLvJ8Hses”Œ"18GB7aeg6rvUR42tjLtpi42CgJ2iJK1Baf”Œ"1LtLHPU9qveR8zKByT3vgzPHgq8sdMm1sQ”Œ"1AiC49w3Rx3smg158qg4hLgSDR2M1kSAQB”Œ"1Des45n1xCdWYrT5oZwkoBkSqj2qAzrhZq”Œ"1FFTiuy2jupY8YY5EMLGcLJAnCecupTnRr”Œ"1DHDyErP9EWqyYP57Vgv65F9QYy2cRPuah”Œ"1LiZxKeBrSgH4ga9uWpM63dfkv9cTaRLit”Œ"1CtkjBzZ3gTCsgmnDaSMaJ1Dqf82QS57Mk”Œ"1MYuW8om4muh6Wa6nY49YmLodZnXUTQg5E”Œ"1Jh8GnNKXjfuBJkWnaA795DaQq3WZ7ison”Œ"3Ek5fr7sSLBe42tFRScQQAKZpSeQWPDm7e”Œ"3PQp8y5zpRPPe8kWEM5BRvQ1nGt73Gso65”Œ"14UPkoQFH3LkkhcgA9GVgKq8YVyEMQ7PqF”Œ"1CrmrxtJLhFbHeb8NUEHKKJ2oa1fsnN8cL”Œ"12yf4RGNWQ2UvLHNnDzbKZ73t64ERkrv4H”Œ"3QnMdVze1zWcaqq4BvM4Zy55cUATZhr7dB”Œ"1PJ1XoMRja2yFnMwAy93StyBmUiVU7vMp2”Œ"112TkpcVKzcVRw2rWU3uE816VWgjgixbLH”Œ"1JtWDgJ7SXMhvGF6kbaMUczqMZrCMgTJt5”Œ"3LWUnDz3gVdnNghGWmybqyT2uXEyUT4DuS”Œ"1HWrh1vuBB2YFLzfEfWtZyth55ZgRxMHum”Œ"1G6tkPyz9FnhyBwa6ua8jKFfNH4fF3cryv”Œ"1LpwnDCzHLAWCoh6hzyyh1BgRFYSpYNcdR”Œ"1GW99WJqJJUSJTFuvoP6AjQHb3WzG4sJeS”Œ"1Jyk78QLGTi4e1DbYqgBizQujAPskxtsU3”Œ"19VoDPRPdFJ44rwYF8SkquMaPSKGFC8MRh”Œ"3MvKdWmrWpKbP4pcEvcyhW1myfvR3pZhBC”Œ"16iRf2yqtP1dD1Jr2mRSxBV2BuPgV7A4z1”Œ"3CrwCTa3DiFLNgm8iKvemoHZYFQxm9dCtn”Œ"145SsRT279w2R74yr2FE3jaKXUdD4RrfLC”Œ"15QFgMciF7DM3iLU42rzk4nWJ5wLH4HaKk”Œ"343Ag6URZKrAL2ZJcifEdkhRhb2v8gCGEf”Œ"3KeA781qGKLmoz93ugcWdXDsUEejLGjZa8”Œ"148PpbrFRvh66QysGEDhL7dT9SkpLSwMPK”Œ"16Ht7oujZYmi2fcPAetVafCYjTQkKrRuWa”Œ"1HVg4tjfMWfLw3VceTtefiLEDbga6D9Nfc”Œ"1GEDXAfhqwr9xkXwko1ToChmA4798ivVR3”Œ"1AvRnTX2y21k3i9P1PtkiMn1eC5ivatpEj”Œ"1B6Mv7J7u2iLeT4R96KBpDEA8NVLjqUWpX”Œ"1Lg4EtHDCVebAavtfVmHM5eNFhHg9JYgKK”Œ"17SdsgtGCUvMCKiyVFdRQUakMu3N2QWJAC”Œ"1KUEJQMrytDehHr6jMTTnzz98FhySXknQt”Œ"1Gx6KQS9uFV19S1Azy88oKrerg7YeKgc1q”Œ"1HgeeALkuGbnt7JMiNr6HnKpeLavxstRmc”Œ"12Z1KQnDGEVFqpRWdKWHKgoXxpvjZXTgV9”Œ"16z5UDZKCCvUum3xnq138ZJg2ymqFCGBrT”Œ"1NczVBkM4DsUxAg7Y2cGS7NfU4Mx9UEZ7m”Œ"17V4BNm9zKHZi33GQJuGtMVaXHGassEt3v”Œ"3FAyJdho46dUKztexBVrbwWP9fv74vJ9i2”Œ"12TvcdwPisBe3dghdNc46WuQtFAy4qHui5”Œ"1BpxaXNdxmYYTcy79idtHrq8UgPXoD4zFV”Œ"1BgFN1s8XT93sB52sdP8cL6bCL3wFbh2dT”Œ"1JFLAy7rPXXkrCJupsE1yeQ8Dp9AkytrDy”Œ"1L4XKWHB3UWtGQTbTtZjZCV4aCJE6y9QQ8”Œ"16wfmQQKi7tWt7wDNdaiBQSDkuQiS7nTY2”Œ"1FyfmQ2fy9GQNDseV1ih3q3xjj3DvCzLJX”Œ"1ArAth5cAUUYgEnUtVDhuN788r6fSpdsc7”Œ"17etKtDVGAL5rYjwZGbu76heKhe3MnJT3r”Œ"1JJSNGCqQhaVjLWTLyDaCQeAo2PiJAKqaa”Œ"1JXRyWz2ekTq58ya26mbSV7MhntQ9dctpg”Œ"1KZgwiaMyfSiv8AxxU1qCrZ5bi5Ga3ePQR”Œ"3Ggf3fgRZqBCnXXMmEP22MXGsJ7arYbeff”Œ"1CXRw5xxYnFCmcPdosoWdU6bX9AHAAiHxL”Œ"128AW7f3Kw8XEbCipz1sMqcCDT5jeeZKvG”Œ"1FC6g8koxsaMFFDjAMAA2X7UjpPP2pcBav”Œ"1DmRncZkKAav6c5LdL5j7MHGTvh8SiWziv”Œ"15xEVbv9T6gCD1jYdyA5WoBTa7KEpcU7L4”Œ"1CsxzarbG12kYjbU3nTgDvWdh1PyeYxPZx”Œ!1o257rGU9XBpPVyHH3HAWPazDSUMVMJKT”Œ"1EsR3egWaTgLzof41DavzBhmS8V4cUJhV8”Œ"13f5Y72MoJH48B4wTwphsdRRq3zGZQ8NtX”Œ"1Hi1vwKZBvJXhkS4hvsRyq5ha7x8dVdpZC”Œ"3QNZkTekZgoRhYPnBraivneKsQyJJy57JW”Œ"13ZJWX7cBL1x9BEqVpTd1SjbwsDaj4VQKA”Œ"3GFYxS7TYD25DNQEP37DZanRr5T5ASnMhB”Œ"1BgygZU3ZGySYomducE7Q595uBKhRbAgjc”Œ"1Kkbd1i9boUMFD9opANGwCHKtpQWHAnR35”Œ"1EA3vRWM5yeARgwgqxiVuBvvVhTEhzsVdu”Œ"1AdSJkQea8JrZ7GqfaDDw8MLLsWyQebRUE”Œ"1BwxnBywszvuTpQ4uMYmoqxofm4ivGrCFg”Œ"15f7VFxNrBT8oZy2E2Ch21izTnoSo3Dvju”Œ"167zhmXT5UT1DCKko4tcVqD8sFPaA5yDQZ”Œ"1PUGvnRcvVjWd1fVAakTigwDT5vcBMaYVm”Œ"1ARf1Q2tGTPcUba5TP3F2kB9UUSR1NPEsL”Œ"17yzWAkZYCyFw1qFo5MgY6Ff97rkPJ1XXC”Œ"1G5T1v1HwWfxWtgsQZDqFYJWeDnCAqF1dv”Œ"1JnYNfpy4TntLUNZvgLFwXguH4sHnNHpJu”Œ"34pGuwvjwZvAtoumYYSncMeyc2uKMmjUfd”Œ"14WR2vzANd94CTXaYutJRuo3BtE9qJKn1G”Œ"13rnLdkYMSstXMyXocvEWLfg7AeSJBoXFY”Œ"3AQULuD8qDdDa6EQXRgUyeFtpDq3LJQP2x”Œ"17VUMwaCSwWQPfv29jEe6YUgGyrqwYkEzH”Œ"1JUckCGckRYrf2YFuSN96aousKqzpGgQf2”Œ"3LHQxtW14gueCXZbrhaxxRXhtWbystP4x8”Œ"1FUbWaE6UM9EVee7rNZJYTHJoF1yJEQu7D”Œ"1Lc179ndaVGSMZVcf7M3EY4rxXZ7oYNmaR”Œ"341tuFDvSX6GAF8QU2WKQL4uaQGxwP29BQ”Œ"18Lk2GbRC2mZJttnULg3E1iF9WAmuyvFwA”Œ"38ZBxydcSpyR3xDnyk1pKtf6GNArjB6PYB”Œ"3G3VEVqpxkdeqR4JTkF7urUZMArSWX59wP”Œ"12fuidvHBXUToPcuyDdA7V7BiLKK1hGXnR”Œ"1FmqLHupFxM2ZR3pNQbTXEcgTcySaVE3op”Œ"1MkEcvQHruoUq91HAXL9R65T1cN41qvCwd”Œ"3B73Epdp36jaCZP4346VuEMu69ofbyfept”Œ"3QSaqYnLayBVpQQFNYh2nqyj7hBQivH9Tb”Œ"18eeeVtKRv9MPmwheESeH7W6FW4hFrEFiU”Œ"1GiRo5JWhyr5qAXgg739UaC6QX3WgCcE33”Œ"3CG2jDXws9VsJKFuYcnYngezrskYJFBUEH”Œ"1AnHYyuJxKdFvpzA1R8Ak62K1q7VrcUvpc”Œ"1EqaD5YquN61sdtx4wtrA9DQJvakBWS4GE”Œ"1CQgedWG7WpQDKdNqdgdLanyQm3K9h3dLh”Œ"17AiuwxM5drjogatuHihqLS4xurbMJiyh4”Œ"19berohh7AvC23rrgdYQcayttmXPwK8mCF”Œ"1LzBaqdj3XHk7WXKtbRKQdvPcX5j9rHrV6”Œ"1CVwsXr5cuMvq4Fc3AMrg5BZWk67Ly6Wf7”Œ"12k7kXAo7JjcHTnoPxpYsYQRVDAZaM4Jfq”Œ"1NSL5s6cod2c2CytuG7Pjj7gN94PxSefRt”Œ"39516xBDpTa9jip3oGv4j3ck94Brs2i6ie”Œ"3A9s9Ap9eneUnfSaUpGuMCv3ybV2Jifp5r”Œ"13BxTZjGj1yA4AEFW4mV7kqMvMRQ7ZnXTz”Œ"358GLHRbCJ6S9Y7JeqPX2PGim68RnbKYQR”Œ"1N8TzGJjyd5sjitAifbBviFEfY5wZ2EmNv”Œ"1KJREAi8d4cEPeipgTPVHBTetBoeo2xGvi”Œ"1BmvG3Bb9RUzCTtBxZrB8rWkvFfrByrtaM”Œ"33LkREFaxmevRBAh5rqzkFZYStfHCFyVCR”Œ"38VpEpVEKKkMB79BuTLQEz95ZBcFNR5FG7”Œ"17PABeDVcSixWVWXAdEyZoi4i6Uue7WBWy”Œ"1HJY8set1VqV69TrNiMJ6a7WzjqGTgv3zU”Œ"3AhpkBp2xR2h1JLnob71aZvyT8TN5tFxjt”Œ"3NdF6qYktEvQTSfSMhjhsB5Z2Paibps4rA”Œ"1CoW6JdZuHUqhps3fwWwscfz2rk9QZN2UK”Œ"18Z2KEnBG9sTRh2ZiUVieuDnWvVe3Ae9SZ”Œ"1GgYtgibYFbSty6ZQMnQuivC9vAMyoxWcn”Œ"3AWoqYAcaZFTF3F84c9wvadnobvKnEgcx6”Œ"3B9YYxC6dehZfQffrCQT3LtVJWVtWvrjwL”Œ"1N1SBK869tTf7F2A5tW1mk8NwnhutJB15A”Œ"3FVkWoWhMRyWRmdEEb84C11xcLHqvwrg5D”Œ"1E9K4ZsTRtudLNFwUVyGaoZuRy6Apj8nn2”Œ"1CJzoGkphYpSFS1SKzJyPp1miYqResLbaA”Œ"1LXFznaw4bFRPStzHkhk4Q7rZD3MBXoLeM”Œ"1CXdCss5cbHx7CAonS6ndabx8zNrbkwD1F”Œ"3JFNxTEqEUdrnoPdxQW6HvH3bybxge7pey”Œ"1P4uxJCeKRXjKViinkosbfULQfPLsLtRAt”Œ"1Nk3KmZM4oTCqG2sC1y5Bmr53xfc3DqEX8”Œ"1226AQRBGqJTp4G3nZyExwRdwapEe7Beav”Œ"3QtRqoh4nonp3K21WWfSbEKpP7wihLigFD”Œ"1DTG8Es79dPtKg4kjEnkLrcXEWV2fKN4mg”Œ"1GPVYU5y69heLXfwv8Vn78fcXLk4jn8dC8”Œ"122HNqouEzBc3Tpe95skcFR6mabC8zsCAy”Œ"1N9iXS7QjxZoU2uBNHZcWNvTbFi2JT5g1K”Œ"36mUoQfEDA7aZ1rf8q6C3iJnyecJihu9go”Œ"3MAbCh4wNLC4ndV8p9tEVtoK59ACR7a8BC”Œ!1x5oHD4afKrYonpuRAK9GUx4hWhkn9LUE”Œ"1Dvv9DKWJZJVKqANTbE9hLKxqG4xHiuXkJ”Œ"1PtN6ySdxUMZnNsqTk62x3n3X2CZUhpwTG”Œ"198EZHBiuFs1fX5RpM8ttBMDkBnhxRMFvp”Œ"1D2sVeXGUdw5Kvjsx49uocbNA98VrNY7rH”Œ"1Eqe97nj3VmpWjFL9kTTqpuBQdFhazboRq”Œ!1EfQDQoGJiBVEQ8QT63XbCeZHX4nR9SqS”Œ"13Bzea4vTeEEYivmGu9JT2ShZKeNszs7xz”Œ"1KMhjGpjWLdCQZrbP8AZpaUxfA4WGacJcP”Œ"3Mg6DAaucXTBXc3TBwdBLNKMDRzSLAwRyy”Œ"14RKGndXbRwd8tMTQFXZ4SG5h825kxip6V”Œ"33KsPwdpBqAwJEBY1YaXYkJqmT5rR34rex”Œ"12CGFzh35ggZuHSiXC3pt32Mn9LcaFb79z”Œ"3Kyivhhixi5UDEHj3g5Aet2N87hQ9V21p3”Œ"1MzrFkZsqqCBpPMYu5E8Q29qe3A5QHvFJf”Œ"13dEVnM8pLtZ6EwrfB43GC6h5QHBLZPKWW”Œ"1DFQruutXThiy6H2qY6Aub2JuPbYXhVjCD”Œ"1Eys7HBwoLwSyim8mD3v8cF6PPFy1wVUVA”Œ"3BgUgRx7y9FtP9wVJKXKBBJEpFNtxGCK9A”Œ"32MWU5RbXG4YeFzs7KBdVf25zG7DwY3pJy”Œ"1KYmWgxw5cKokwYeyRwMBbc37Es4oNLyde”Œ"1C1Vpg3ruyzdKbn7AuMgmV3paazRRs9S9U”Œ"154gPpS26MctGXfURJb8bR1Kn3jC2TKTL8”Œ"1Fkv723jSvLvDd5HcGPfYPHNxWjMfLxyR4”Œ"1JBgPfcSMtD7Gy9fe9z23VMFPj8skopNqm”Œ"1DrweXpZoVzCu3hTed58xd2TjQnXrqEsEv”Œ"1NA3ucR5FJSAzyxPqKJxnD2vsfhBGqRPuy”Œ"15jap1MDwhCQBxg5o848cdr32sgc4jgw6X”Œ"1BNZZFcvjYUv7PqeduZFKJsHLnrS64w3cF”Œ"1JjEdC9n53N9DdhtoGVCWTcDzVWJkPBTMn”Œ"1KmhRfxVhMSa8q2kNEKS8Quf4mXoSscrKu”Œ"32GkWwa3DjB5kv8qCjifB11hkGhfPaEr4D”Œ"14karJ8QCo8VURbaTHGjfNTKTNW9Q7qim2”Œ"1LynGZNwdHCjMf1FPaAfYWGSu5fzHE7eDH”Œ"1BoSETaY5bNmChfqpR6icongU7DEtNwgXM”Œ"13HmtYg81p3GdFF1uzUkPkJsbu49b1uRUq”Œ"1LWqQyCoCe6CtkJi89zn2pitEWXSuE9ZX2”Œ"1G4YdfYrzEsiZyvGRknKusxGfvDLR2BhhX”Œ"1A86tQCnaeagVLDUhgroe3DYxkXeQhqPg5”Œ"3HZF3TsbZ2rxdoUmc6e3zVB2HaaZsnvv9q”Œ"3Fuxgkd2oyEJp61L67Vhb6TXqk83xQgqWa”Œ"16XCzTioAmw7wXFRy5DRnHbAWmyvJk9nE8”Œ"1Q1UWZr9b9yZEvnUrUJyTN4bdrZr3RAnWK”Œ"14do9QCp6UKzt8d1eyUZbr6nsQHjQX65EW”Œ"17PN1qKwXs48oFAHT1XN1Q1M1yCqpcy9Ra”Œ"1J78aQ5AhDWujrrVfN64Fj5LUB5bewK3R6”Œ"14kviuvFiJ7aGSLZnHhYFp6e7JFJaLcz9S”Œ"3N6erYb4bweuJgr4UHBuSh8MD83UXwnaxT”Œ"1LM8aapHBhfprneGsun5TbdvjjpZKiwkkA”Œ"3LUaZRC2moh989hGHyarunE2UKWYafS1HX”Œ"15Q1YiEXmrbQtSY5FjWcnoK2V49VK8AfnW”Œ"1D8o2TfTJD6J5cSUEThaLYG8cfqLgXznLh”Œ"3HqACBXP1sag5Va8FtZJ1MCPeWywztY7vi”Œ"3ApjGqeCLzn65vtSXSfiNQWLg7kTVLrHry”Œ"1725HX5YuNEkuCSJCj1deqexqkDwFBF3xw”Œ"1PPcZZF4oQrhL9axb3ztM6bjcmwCDqwZTb”Œ"18q4b2ozixCN6ixwta4PpUjK8KJ5QvoHEa”Œ"1AXGC8esRfgj6DffsGW8cQV1VMpgCXyeGB”Œ"1MA9WqCXTxgSriuWY7go2bZqkhvKbNRWXu”Œ"3FX7omJkyzzMtXZMXHg7kwhGn3cQTXkU9R”Œ"1LdzpkhsFQLEC7ysNHBTzcZ5mYE8Nea6f4”Œ"33PWeRsrM3zG5LXycNCq6DFq3EeQezX1ss”Œ"1CL8wzFdKPUqoBrkR3QJ7Eck3h4ACrLaaN”Œ"17yqPA4HHA6PA277WZf7FfDw18EURrTsBw”Œ"36B7qE5GFU24qUiV5qLeze5e6KjzBYfVLj”Œ"34MZHSr9h2BB8CobJis6n5capfdvRrzBJx”Œ"3P7kSJPPfNvwsp3ZC5oSkFaXnHcRUC2toY”Œ"16KXy4T1Z1fcxsXJ9aPnNQNHuTk2EJN1pE”Œ"1HZkBC8qKrH7h21nByFf41nkG9ZLWwiMVc”Œ"37LBbnjXGanTudxYXkrwbx3JFdF8Fn1iWr”Œ"1ERUUoohQE4YuDU9MQKD99a3w63LmYz2Fq”Œ"1JBthh1zMBu74Xbe4EYeZX2GAE5D5h2eoN”Œ"1GoCBqrfggkzghYwTJnYSQqavwiLG9RfFn”Œ"3MqzbPfJkJstg9ag5TGN7FWBKhv75zU5yC”Œ"3MP43e6gPtjv5WDF11tY2qVp9gvdUgxah6”Œ"1L4q6hCDtaPHq64rMNKCtMy21CpGtzKC8W”Œ"139CpBRnx2YzyUCzT4LMD2mMs8Bh4xywKo”Œ"1KPeJBj2BgMLKRZvW2LUpKcM2d4S3bsHxY”Œ"1NztGqHJxSyUHqx1xniuAw9UNkmiLTFXKZ”Œ"1Co5oRVWjXHAqRC5cy1xH2SwRGeb5cF7nK”Œ"1PiVtEQwjNxMccVC3q4Mgq1CqLJyNnnqu8”Œ"17iXnWz3QCEDLMp82V8ijJjxTxQEemUUmM”Œ"14W8Vh1WX9RwUnvkXgg85u8CdqAE3xozp4”Œ"1GvxjfMhE1PPSg7L6pCbTLXFgQ2FA3tzzN'

In [16]:
for i in ['”Œ!', '”\x90(Œ"', '”•\x0b \x01     Œ"', '”•”\x7f      Œ"']:
    output_labels = output_labels.replace(i, '”Œ"')

output_fraud = output_labels.split('”Œ"')
output_fraud

['34xLKAZwB5DXLrTi4BXksuKGL6psA4Z4jN',
 '12KyRXPfPwDmRy4y1K1Tbw7ib6eMHxVBuQ',
 '1D5sGvM9tRt9rx8CwSCcNAZtDFNhtYgRmZ',
 '1dD47b63nBV9ABks6SQU1PC52BaVi9Lix',
 '1PfWy4BCyRwxp8wYajbpbQC18szdKYCq9q',
 '13DruKxiK6v4MGARkVKDFs7Wpj6u2WoAA5',
 '1B98ZUhDg5h8GB8n3pNHb4p1w1r6YcCcQx',
 '1ChjNHDxkT6tQeoxVbtHbtJr6bbACt5y2y',
 '35vxmN1162cfnuab3DVhaMdmfa3jXL6njJ',
 '18UMFna2bMzP1G8it5WiXpHzJVMrruXRib',
 '18hNKMTQrUWzdtDs9MEp7FKp3Sfs66Qwhd',
 '1PBmfaUui7ataQm1mh8E4pitrdoAMChWrM',
 '1NrmFsmLzVN8YBW1ZWapYBPbLLYM7WvAQj',
 '1b1y2ns1Ht4K9y4jA1c3L6T3sQuWY4JN5',
 '1y6tuJiuYeRuFE4JbJXGPtnNmdqZAUDvt',
 '1Ha9DmRDWznzrnBdURGFmyU2PzbNSihwb5',
 '1E6DNBa1hVV58ffmf5a8a5R49g97J5eLsS',
 '15sM2vWoMWW3UzTZm8mxLXKSjwXPnp8bZL',
 '1GtvGLPasDq14xS4kTHfmVpEwMEHZcpWNz',
 '3A9au92PaNf2t4Wrfsg6w3dEkNcxGrNq2X',
 '3FUzoFg2uTZEFuvgYqia6gFFaGZMSDrYa3',
 '1KFYAVFduGqLXcDx3zEyGnTxm71eLgR363',
 '3BBdZ9Zjom17GTgmq8RX3f84P2Nf6L7SnV',
 '1MP8AgRq3E1z7V3vemursb7Rsc3rTCbTbh',
 '3MKaTNDSnbzqqYPEF6ebGQ6f9PnNLMAoQQ',
 '1PQGnnmtseXfqBxagyTH7RKkEu

In [17]:
print('Total Number of fraudulent/high-risk output addresses : ' + str(len(output_fraud)))

Total Number of fraudulent/high-risk output addresses : 4429


Find common addresses between the 2 lsits

In [18]:
print('Number of common fraudulent/high-risk addresses betweeen the 2 lists : ' + str(len(list(set(input_fraud).intersection(output_fraud)))))

Number of common fraudulent/high-risk addresses betweeen the 2 lists : 7


In [19]:
del output_labels

### Transactions Data

Combining 1705 blocks into one DataFrame for ease of use. The resulting DataFrame leads to a CSV file of approximately 2.6GB.

In [20]:
path = r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Data\repo_clone\bse_clovrlabs_btc_fraud\data\pannel'
block_files = glob.glob(os.path.join(path , "*.csv"))

blocks_data_list = []

for file in block_files:
    block = pd.read_csv(file)
    blocks_data_list.append(block)

trans_3w = pd.concat(blocks_data_list, axis=0, ignore_index=True)
print('DataFrame Size: ', trans_3w.shape)
trans_3w.head()

DataFrame Size:  (14033260, 7)


,txn_hash,input_address,output_address,ammount,fees,block_index,block_time
0,bd36f2ca16e2a2c73c807b7d1569657b30de8453450cd2...,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,24200000.0,320000.0,453318,2017-02-16 12:05:04 +0000 UTC
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2503.647687,3.314845,453318,2017-02-16 12:05:04 +0000 UTC
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,635940.180556,841.988605,453318,2017-02-16 12:05:04 +0000 UTC
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,590236.946809,781.47725,453318,2017-02-16 12:05:04 +0000 UTC
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,101303.224947,134.126076,453318,2017-02-16 12:05:04 +0000 UTC


In [21]:
trans_3w.dtypes

txn_hash          object
input_address     object
output_address    object
ammount           object
fees              object
block_index       object
block_time        object
dtype: object

In [22]:
trans_3w['txn_hash'] = trans_3w['txn_hash'].astype(str)
trans_3w['input_address'] = trans_3w['input_address'].astype(str)
trans_3w['output_address'] = trans_3w['output_address'].astype(str)
trans_3w['ammount'] = pd.to_numeric(trans_3w['ammount'])
trans_3w['fees'] = pd.to_numeric(trans_3w['fees'])
trans_3w['block_index'] = pd.to_numeric(trans_3w['block_index'])
trans_3w['block_time'] = pd.to_datetime(trans_3w['block_time'], format="%Y-%m-%d %H:%M:%S +0000 UTC")

In [23]:
# Number of unique transactions
trans_3w['txn_hash'].nunique() # Shows considerable number of group transactions

2235580

In [24]:
# Number of unique input addresses
trans_3w['input_address'].nunique()

2905358

In [25]:
trans_3w['input_address'].value_counts() # Shows large discrepancy in activity between accounts

3PUuiYu5cFMsagkffArrKZzQFtWdHttU3x          87190
3CD1QW6fjgTwKq3Pj97nty28WZAVkziNom          69304
97333c48ad0f0d6d33efc6cbf153966b2139aec5    58000
19JRz9seZ7DG5LVa9HkXC1bkMzxvnuhsAy          57574
b78f207cb338c16dce1ff785a940d6f878a70f3a    51887
                                            ...  
1569TQzfNeh7ZM6jAHj6fPKAF4opkhPdFG              1
15nrwkdA7zNGpXJzs5QvAKuJ9VKMabNPkq              1
1BahGBGWUFupzoVLWZZCkbGirreMvqqcY5              1
1Q78Q7k8BDK8LBs9hwcnFiPW3DuZgs4YST              1
13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR              1
Name: input_address, Length: 2905358, dtype: int64

In [26]:
# Number of unique output addresses
trans_3w['output_address'].nunique()

3267729

In [27]:
trans_3w['output_address'].value_counts() # Smaller range than input_address but still shows large discrepancy in activity

97333c48ad0f0d6d33efc6cbf153966b2139aec5    58250
b78f207cb338c16dce1ff785a940d6f878a70f3a    52000
3PUuiYu5cFMsagkffArrKZzQFtWdHttU3x          47728
f3a17c35fd995f7a3aee089703ab81bcb4ef0b28    43700
0499c77cab212f94c796955355557c40b1f472ee    35350
                                            ...  
1DFwhbqotMc3SADPQBq2GbmDsVqgSVm1zo              1
1K7kVkxuXFL1zhETH3oKBk97j6uGT9S9NN              1
14YFLKf6pYyJndKUmCAZFXChv4hGT11Pcv              1
16Lnq73Z5xgFZJsnnguhbo7CRwAnGxpkP6              1
1NpSHwsjyt4hE8RjjUNm5CSrF9Cv7x62W1              1
Name: output_address, Length: 3267729, dtype: int64

In [28]:
trans_3w.describe() # very high levels of standard deviation

,ammount,fees,block_index
count,1.403312e+07,1.403312e+07,1.403326e+07
mean,3.648195e+07,1.099449e+04,4.546637e+05
std,6.394097e+08,4.283267e+04,9.884161e+02
min,-3.035458e+09,0.000000e+00,4.533180e+05
25%,7.080253e+03,3.790049e+01,4.536240e+05
50%,1.944997e+05,7.989566e+02,4.546760e+05
75%,2.986816e+06,1.199397e+04,4.555070e+05
max,2.753992e+11,3.925864e+07,4.564370e+05


An exploration of negative amount values highlighted the exisentce of self-loops (where the input and output addresses are the same).

In [29]:
# Isolating Self-Loops
trans_3w_self = trans_3w[trans_3w.input_address == trans_3w.output_address]
print('Number of Self-Loops: ', trans_3w_self.shape[0])

Number of Self-Loops:  1454171


We note that there are a significant number of self-loops.

In [30]:
# Excluding Self-Loops
trans_3w_cl = trans_3w[trans_3w.input_address != trans_3w.output_address]
print('Size of DataFrame without Self-Loops: ', trans_3w_cl.shape)
trans_3w_cl.head()

Size of DataFrame without Self-Loops:  (12579089, 7)


,txn_hash,input_address,output_address,ammount,fees,block_index,block_time
0,bd36f2ca16e2a2c73c807b7d1569657b30de8453450cd2...,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,2.420000e+07,320000.000000,453318,2017-02-16 12:05:04
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2.503648e+03,3.314845,453318,2017-02-16 12:05:04
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,6.359402e+05,841.988605,453318,2017-02-16 12:05:04
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5.902369e+05,781.477250,453318,2017-02-16 12:05:04
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,1.013032e+05,134.126076,453318,2017-02-16 12:05:04


In [31]:
# Number of unique timestamps
trans_3w['block_time'].nunique() # Some blocks were minted at the same time

1592

In [32]:
# Features with default 0
add_feat = ['input_flag', 'output_flag']
trans_3w = trans_3w.assign(**dict.fromkeys(add_feat, 0))

# Feature with default 1
trans_3w = trans_3w.assign(**dict.fromkeys(['group_trans'], 1))

trans_3w.head()

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time,input_flag,output_flag,group_trans
0,bd36f2ca16e2a2c73c807b7d1569657b30de8453450cd2...,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,2.420000e+07,320000.000000,453318,2017-02-16 12:05:04,0,0,1
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2.503648e+03,3.314845,453318,2017-02-16 12:05:04,0,0,1
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,6.359402e+05,841.988605,453318,2017-02-16 12:05:04,0,0,1
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5.902369e+05,781.477250,453318,2017-02-16 12:05:04,0,0,1
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,1.013032e+05,134.126076,453318,2017-02-16 12:05:04,0,0,1


In [33]:
# Filter through transaction ids to find those with multiple sellers and buyers

txn_counts = trans_3w['txn_hash'].value_counts().loc[lambda x : x>1]

# converting to df and assigning new names to the columns
txn_counts = pd.DataFrame(txn_counts)
txn_counts = txn_counts.reset_index()
txn_counts.columns = ['txn_hash', 'txn_counts']
print('Number of Group Transactions: ', txn_counts.shape[0]) # size is too large to reasonably iterate through

# Transaction id counts inspection
txn_counts.head() # The first is a distinct outlier

Number of Group Transactions:  1947068


,txn_hash,txn_counts
0,4a7b8c0b2eb30207c48a857d79ed40bb532d93a3e62e17...,80427
1,5e549c35fc9e48b6e8f543e7682d34eeae17fc59d97c10...,38064
2,8f4e7cb0ad80daf4d6694cf22a89cd882dab8e1cb3f831...,31545
3,01370e462f9eb440cdfdd0cfa092783686ed0320c22038...,29452
4,18f19b8b8c2156f224e0c5544605deb775728ebc1deb31...,21684


In [34]:
# Obtain 1:1 transactions only
txn_uniq = trans_3w['txn_hash'].value_counts().loc[lambda x : x==1]

# converting to df and assigning new names to the columns
txn_uniq = pd.DataFrame(txn_uniq)
txn_uniq = txn_uniq.reset_index()
txn_uniq.columns = ['txn_hash', 'txn_counts']

# Get list of transactions that appear once
one_txn = txn_uniq['txn_hash'].tolist()
print('Number of 1:1 Transactions: ', len(one_txn))

Number of 1:1 Transactions:  288512


In [35]:
# Create flag for transactions with multiple buyers and sellers

#for i in progressbar(range(len(one_txn))):
#    for i in one_txn:
#        trans_3w.loc[trans_3w.txn_hash == i, 'group_trans'] = 0
#    trans_3w['group_trans'].value_counts() # Estimated 3/4 days

Transaction flag is too computationally intensive to run for the full DataFrame.

In [36]:
# Create flag for fraudulent/high-risk input address
for i in input_fraud:
    trans_3w.loc[trans_3w.input_address == i, 'input_flag'] = 1

trans_3w['input_flag'].value_counts()

0    14011487
1       21773
Name: input_flag, dtype: int64

In [37]:
# Create flag for fraudulent/high-risk output address
for i in output_fraud:
    trans_3w.loc[trans_3w.output_address == i, 'output_flag'] = 1

trans_3w['output_flag'].value_counts()

0    13480805
1      552455
Name: output_flag, dtype: int64

In [38]:
# Save dataframe as csv for ease of reuse

trans_3w_adj = trans_3w.drop(['group_trans'], axis=1) # was unable to populate column 
#trans_3w_adj.to_csv("trans_3w.csv", index = False)

DataFrame added to Google Drive Folder

In [39]:
del trans_3w_self, txn_counts, txn_uniq, trans_3w

*Self-Loops Excluded*

In [40]:
# Add Features with default 0
add_feat = ['input_flag', 'output_flag']
trans_3w_cl = trans_3w_cl.assign(**dict.fromkeys(add_feat, 0))

In [41]:
# Create flag for fraudulent/high-risk input address
for i in input_fraud:
    trans_3w_cl.loc[trans_3w_cl.input_address == i, 'input_flag'] = 1

trans_3w_cl['input_flag'].value_counts()

0    12557450
1       21639
Name: input_flag, dtype: int64

In [42]:
# Create flag for fraudulent/high-risk output address
for i in output_fraud:
    trans_3w_cl.loc[trans_3w_cl.output_address == i, 'output_flag'] = 1

trans_3w_cl['output_flag'].value_counts()

0    12276383
1      302706
Name: output_flag, dtype: int64

In [43]:
# Save dataframe as csv for ease of reuse

#trans_3w_cl.to_csv("trans_3w_cl.csv", index = False)

Removing Self-Loops is shown to decrease the sample size of both licit and illicit observations, but the sample remains large enough to have sufficient variation in transactions, time stamps, etc.

In [44]:
trans_3w_cl.describe()

,ammount,fees,block_index,input_flag,output_flag
count,1.257895e+07,1.257895e+07,1.257909e+07,1.257909e+07,1.257909e+07
mean,2.636878e+07,1.044633e+04,4.546534e+05,1.720236e-03,2.406422e-02
std,3.031525e+08,4.359917e+04,1.016369e+03,4.144004e-02,1.532486e-01
min,-3.035069e+09,0.000000e+00,4.533180e+05,0.000000e+00,0.000000e+00
25%,1.214776e+04,6.144486e+01,4.535920e+05,0.000000e+00,0.000000e+00
50%,2.352923e+05,9.262710e+02,4.546340e+05,0.000000e+00,0.000000e+00
75%,2.871007e+06,1.137892e+04,4.555450e+05,0.000000e+00,0.000000e+00
max,2.753992e+11,3.925864e+07,4.564370e+05,1.000000e+00,1.000000e+00


Note: Negative transaction amounts remain after self-loops are removed.

### Graph Representation

In [20]:
# Restart and run from here
trans_3w_adj = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\trans_3w.csv')
trans_3w_cl = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\trans_3w_cl.csv')

*Self-Loops Included*

In [21]:
# Obtain full network NetworkX Graph
G_full = nx.from_pandas_edgelist(trans_3w_adj, 'input_address', 'output_address', create_using=nx.DiGraph())

print('Is Graph Directed? ', nx.is_directed(G_full))

print(f"The full network has {G_full.number_of_nodes()} nodes.")
print(f"The full network has {G_full.number_of_edges()} edges.")

Is Graph Directed?  True
The full network has 4667173 nodes.
The full network has 10545282 edges.


In [4]:
# Node count check
in_ls = trans_3w_adj['input_address'].tolist()
out_ls = trans_3w_adj['output_address'].tolist()

node_ls = in_ls + out_ls

len(set(node_ls)) # match to number of nodes confirmed

4667173

In [5]:
# input_address fraud subset
input_fraud = trans_3w_adj[trans_3w_adj['input_flag'] == 1]

In [6]:
# Obtain input fraud nodes NetworkX Graph
G_infraud = nx.from_pandas_edgelist(input_fraud, 'input_address', 'output_address', create_using=nx.DiGraph())

print(f"The fraudulent input address network has {G_infraud.number_of_nodes()} nodes.")
print(f"The fraudulent input address network has {G_infraud.number_of_edges()} edges.")

The fraudulent input address network has 10813 nodes.
The fraudulent input address network has 15677 edges.


In [7]:
# output_address fraud subset
output_fraud = trans_3w_adj[trans_3w_adj['output_flag'] == 1]

In [8]:
# Obtain output fraud nodes NetworkX Graph
G_outfraud = nx.from_pandas_edgelist(output_fraud, 'input_address', 'output_address', create_using=nx.DiGraph())

print(f"The fraudulent input address network has {G_outfraud.number_of_nodes()} nodes.")
print(f"The fraudulent input address network has {G_outfraud.number_of_edges()} edges.")

The fraudulent input address network has 156599 nodes.
The fraudulent input address network has 198957 edges.


Note: The generated graphs were too large too plot.

In [9]:
del node_ls, in_ls, out_ls, G_infraud, G_outfraud

*Self-Loops Excluded*

In [10]:
# Obtain full network NetworkX Graph excluding self-loops
G_cl = nx.from_pandas_edgelist(trans_3w_cl, 'input_address', 'output_address', create_using=nx.DiGraph())

print(f"The full network excludind self-loops has {G_cl.number_of_nodes()} nodes.")
print(f"The full network excludind self-loops has {G_cl.number_of_edges()} edges.")

The full network excludind self-loops has 4666873 nodes.
The full network excludind self-loops has 10487158 edges.


Note: The number of nodes is smaller then the previous network. This indicates the existence of some isolated nodes.

In [11]:
# input_address fraud subset
input_fr_cl = trans_3w_cl[trans_3w_cl['input_flag'] == 1]

# Obtain input fraud nodes NetworkX Graph
G_infr_cl = nx.from_pandas_edgelist(input_fr_cl, 'input_address', 'output_address', create_using=nx.DiGraph())

print(f"The fraudulent input address network excludind self-loops has {G_infr_cl.number_of_nodes()} nodes.")
print(f"The fraudulent input address network excludind self-loops has {G_infr_cl.number_of_edges()} edges.")

The fraudulent input address network excludind self-loops has 10813 nodes.
The fraudulent input address network excludind self-loops has 15671 edges.


In [12]:
# output_address fraud subset
output_fr_cl = trans_3w_cl[trans_3w_cl['output_flag'] == 1]

# Obtain output fraud nodes NetworkX Graph
G_outfr_cl = nx.from_pandas_edgelist(output_fr_cl, 'input_address', 'output_address', create_using=nx.DiGraph())

print(f"The fraudulent input address network excludind self-loops has {G_outfr_cl.number_of_nodes()} nodes.")
print(f"The fraudulent input address network excludind self-loops has {G_outfr_cl.number_of_edges()} edges.")

The fraudulent input address network excludind self-loops has 156598 nodes.
The fraudulent input address network excludind self-loops has 198575 edges.


Note: Differences in the counts of nodes in the fraudulent networks are negligible. 

In [13]:
del G_infr_cl, G_outfr_cl

### Node Attributes

*Full Network - Including Self-Loops*

In [14]:
node_properties = pd.DataFrame()

#### Degree

In [15]:
degree = [val for (node, val) in G_full.degree()]
in_degree = [val for (node, val) in G_full.in_degree()]
out_degree = [val for (node, val) in G_full.out_degree()]
address = [node for (node, val) in G_full.degree()]

In [16]:
node_properties['address'] = address
node_properties['degree'] = degree
node_properties['in_degree'] = in_degree
node_properties['out_degree'] = out_degree

#### Centrality

In [17]:
# Eigenvector Centrality

eigen_centrality = nx.eigenvector_centrality(G_full)
eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [18]:
node_properties['eigen_centrality'] = eigen_centrality

It would be insightful to also add Closeness and Betweenness centrality measures. However, these calculations are based on the calculation of the number of shortest paths that pass through each nodes. This is too computationally intensive with the given size of the network (created computer crashes).

#### Summary of Node Properties

In [19]:
# Add flag

to_flag = set(input_fraud['input_address'].tolist() + output_fraud['output_address'].tolist())

node_properties = node_properties.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in to_flag:
    node_properties.loc[node_properties.address == i, 'fraud_flag'] = 1
    
node_properties.head()

,address,degree,in_degree,out_degree,eigen_centrality,fraud_flag
0,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,1,0,1,7.556986e-20,0
1,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,3,1,2,7.556986e-19,1
2,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,2,0,2,7.556986e-20,0
3,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5,3,2,2.115956e-18,1
4,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,2,0,2,7.556986e-20,0


In [20]:
# Describe properties overall

node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.667173e+06,4.667173e+06,4.667173e+06,4.667173e+06
mean,4.518916e+00,2.259458e+00,2.259458e+00,3.964992e-05
std,3.936702e+01,1.998728e+01,2.868514e+01,4.611837e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,7.556986e-20
25%,1.000000e+00,0.000000e+00,0.000000e+00,7.556986e-20
50%,2.000000e+00,1.000000e+00,1.000000e+00,3.476214e-18
75%,3.000000e+00,1.000000e+00,2.000000e+00,3.369872e-14
max,2.791900e+04,1.211600e+04,2.533600e+04,1.726987e-01


In [21]:
# Describe Properties of Fraudulent Nodes

node_properties_fraud = node_properties[node_properties['fraud_flag'] == 1]

node_properties_fraud[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4604.000000,4604.000000,4604.000000,4.604000e+03
mean,103.652476,45.297567,58.354909,6.445558e-04
std,965.689619,385.821550,709.624568,4.936353e-03
min,1.000000,0.000000,0.000000,7.556986e-20
25%,3.000000,1.000000,1.000000,7.556986e-19
50%,4.000000,1.000000,2.000000,5.837772e-17
75%,13.000000,6.000000,5.000000,2.775400e-06
max,27919.000000,11178.000000,25336.000000,1.726987e-01


In [22]:
# Describe Properties of Non-Fraudulent Nodes

node_properties_licit = node_properties[node_properties['fraud_flag'] == 0]

node_properties_licit[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.662569e+06,4.662569e+06,4.662569e+06,4.662569e+06
mean,4.421028e+00,2.216961e+00,2.204067e+00,3.905261e-05
std,2.491860e+01,1.584609e+01,1.798349e+01,4.341457e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,7.556986e-20
25%,1.000000e+00,0.000000e+00,0.000000e+00,7.556986e-20
50%,2.000000e+00,1.000000e+00,1.000000e+00,3.476214e-18
75%,3.000000e+00,1.000000e+00,2.000000e+00,3.202809e-14
max,1.564500e+04,1.211600e+04,1.564200e+04,1.107319e-01


In [23]:
#node_properties.to_csv("node_properties.csv", index = False)

DataFrame uploaded to Google Drive Folder

In [24]:
del node_properties, node_properties_fraud, node_properties_licit

*Check of Node Attributes for Full Network - Excluding Self-Loops*

In [25]:
node_properties_cl = pd.DataFrame()

In [26]:
# Degree

degree_cl = [val for (node, val) in G_cl.degree()]
in_degree_cl = [val for (node, val) in G_cl.in_degree()]
out_degree_cl = [val for (node, val) in G_cl.out_degree()]
address_cl = [node for (node, val) in G_cl.degree()]

In [27]:
# Eigenvector Centrality

eigen_centrality_cl = nx.eigenvector_centrality(G_cl)
eigen_centrality_cl = [eigen_centrality_cl[node] for node in eigen_centrality_cl]

In [28]:
# Add results to dataframe

node_properties_cl['address'] = address_cl
node_properties_cl['degree'] = degree_cl
node_properties_cl['in_degree'] = in_degree_cl
node_properties_cl['out_degree'] = out_degree_cl
node_properties_cl['eigen_centrality'] = eigen_centrality_cl

In [29]:
# Add flag

to_flag = set(input_fr_cl['input_address'].tolist() + output_fr_cl['output_address'].tolist())

node_properties_cl = node_properties_cl.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in to_flag:
    node_properties_cl.loc[node_properties_cl.address == i, 'fraud_flag'] = 1
    
node_properties_cl.head()

,address,degree,in_degree,out_degree,eigen_centrality,fraud_flag
0,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,1,0,1,1.193322e-21,0
1,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,3,1,2,1.312654e-20,1
2,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,2,0,2,1.193322e-21,0
3,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5,3,2,3.699298e-20,1
4,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,2,0,2,1.193322e-21,0


In [30]:
# Describe properties overall

node_properties_cl[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [31]:
# Describe Properties of Fraudulent Nodes

node_properties_fraud_cl = node_properties_cl[node_properties_cl['fraud_flag'] == 1]

node_properties_fraud_cl[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4563.000000,4563.000000,4563.000000,4.563000e+03
mean,104.323691,45.620206,58.703485,6.415253e-04
std,969.849548,387.480354,712.726363,4.749921e-03
min,1.000000,0.000000,0.000000,1.193322e-21
25%,3.000000,1.000000,1.000000,1.312654e-20
50%,4.000000,1.000000,2.000000,7.291198e-19
75%,13.000000,6.000000,4.000000,2.963829e-06
max,27917.000000,11178.000000,25335.000000,1.642653e-01


In [32]:
# Describe Properties of Non-Fraudulent Nodes

node_properties_licit_cl = node_properties_cl[node_properties_cl['fraud_flag'] == 0]

node_properties_licit_cl[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.662310e+06,4.662310e+06,4.662310e+06,4.662310e+06
mean,4.396595e+00,2.204700e+00,2.191895e+00,3.911174e-05
std,2.491044e+01,1.584305e+01,1.798105e+01,4.364369e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,3.699298e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.396723e-14
max,1.564300e+04,1.211600e+04,1.564100e+04,1.049846e-01


The overall node properties from the dataframe excluding self-loops are near identical to the full dataframe.

In [33]:
#node_properties_cl.to_csv("node_properties_cl.csv", index = False)

In [34]:
del node_properties_cl, node_properties_fraud_cl, node_properties_licit_cl

### Graph Attributes

#### Communities

In [35]:
# Louvain Communities

#louvain_generator = louvain_communities(G_full, seed=42)
#com_louvain = next(louvain_generator)
#print('Number of Louvain Communities: ' + str(len(com_louvain)))

In [36]:
# K Cliques

#k_clique_generator = k_clique_communities(G_full.to_undirected(), k=10)
#com_k = next(k_clique_generator)
#print('Number of K-Cliques: ' + str(len(com_k)))

Greedy Modularity, Girvan Newman and K Cliques methods were also considered, but were too computationally intensive for the size of the graph.

#### Density

In [37]:
# Full Network
nx.density(G_full)

4.841171772770945e-07

Density figure is close to zero, indicating that a very low portion of potential connections/transactions occur.

####  Network Diameter

In [38]:
# Obtain Longest Shortest Path - Network Diameter

#for i in progressbar(range(100)):
#    nx.diameter(G_full.to_undirected()) -> Too Computationally intensive

## Subset of the Data

#### Giant Component

In [22]:
# Generator for giant component

giant = max(nx.connected_components(G_full.to_undirected()))

In [ ]:
# Get sub-graph of Giant Component

G_giant = G_full.subgraph(giant)
G_giant = G_giant.to_directed()
print(nx.info(G_giant))

*Node Attributes of Giant Component*

In [ ]:
node_properties_giant = pd.DataFrame()

In [ ]:
address = [node for (node, val) in G_giant.degree()]
degree = [val for (node, val) in G_giant.degree()]
in_degree = [val for (node, val) in G_giant.in_degree()]
out_degree = [val for (node, val) in G_giant.out_degree()]

eigen_centrality = nx.eigenvector_centrality(G_giant)
eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [ ]:
node_properties_giant['address'] = address
node_properties_giant['degree'] = degree
node_properties_giant['in_degree'] = in_degree
node_properties_giant['out_degree'] = out_degree
node_properties_giant['eigen_centrality'] = eigen_centrality

In [ ]:
# Add flag

#to_flag = set(input_fraud['input_address'].tolist() + output_fraud['output_address'].tolist())
to_flag = set(input_fraud + output_fraud)

node_properties_giant = node_properties_giant.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in to_flag:
    node_properties_giant.loc[node_properties_giant.address == i, 'fraud_flag'] = 1

In [ ]:
# Assess Presence of Fraudulent Nodes

node_properties_giant['fraud_flag'].value_counts() 

Note: Number of fraudulent nodes didn't change by much.

In [ ]:
# Describe properties overall

node_properties_giant[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

In [ ]:
# Describe Properties of Fraudulent Nodes

node_properties_fraud_giant = node_properties_giant[node_properties_giant['fraud_flag'] == 1]

node_properties_fraud_giant[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

In [ ]:
# Describe Properties of Licit Nodes

node_properties_licit_giant = node_properties_giant[node_properties_giant['fraud_flag'] == 0]

node_properties_licit_giant[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

Note: Node attributes kept the same properties overall. The mean degree and eigen vector centrality slightly increased for in the 3 node properties dataframes.

In [ ]:
del node_properties_giant, node_properties_fraud_giant, node_properties_licit_giant

*Network Properties of Giant Component*

In [ ]:
# Density of Giant Component

nx.density(G_giant)

The density of the graph increased significantly when only taking the giant component. Hence, the giant component is more strongly connected then the overall graph. From this result we would expect the number of communities to decrease.

In [ ]:
# Louvain Communities

#louvain_generator = louvain_communities(G_giant, seed=42)
#com_louvain = next(louvain_generator)
#print('Number of Louvain Communities in the Giant Component: ' + str(len(com_louvain)))

In [ ]:
# K Cliques

k_clique_generator = k_clique_communities(G_giant.to_undirected(), k=10)
com_k_clique = next(k_clique_generator)
print('Number of K Cliques in the Giant Component: ' + str(len(com_k_clique)))